In [1]:
import os
import shutil
import re

import dbf
import pandas as pd
from bs4 import BeautifulSoup
from dbfread import DBF
import numpy as np

from pandas.io.excel import ExcelWriter

In [2]:
path1 = r"c:\ProgramData\AVEVA Plant SCADA 2023\User"
folders = [e for e in os.listdir(path1) if os.path.isdir(os.path.join(path1, e))]

In [4]:
os.path.isdir(os.listdir(path1)[0])

False

In [8]:
os.path.join(path1, 'ff')

'c:\\ProgramData\\AVEVA Plant SCADA 2020 R2\\User\\\\ff'

array([['VO2GGIO98', 'Ind/t', 'STRING', 'ST',
        'Диференційний захист введено (час)'],
       ['VO2GGIO98', 'Ind/stVal', 'STRING', 'ST',
        'Диференційний захист введено (режим)'],
       ['VO3GGIO99', 'Ind/stVal', 'STRING', 'ST',
        'Диференційний захист виведено (режим)'],
       ['VO3GGIO99', 'Ind/t', 'STRING', 'ST',
        'Диференційний захист виведено (час)'],
       ['LO20GGIO96', 'Ind/t', 'STRING', 'ST',
        '1 ступінь газового захисту (час)'],
       ['LO20GGIO96', 'Ind/stVal', 'STRING', 'ST',
        '1 ступінь газового захисту (режим)'],
       ['LO19GGIO95', 'Ind/t', 'STRING', 'ST',
        '2 ступінь газового захисту (відключення) (час)'],
       ['LO19GGIO95', 'Ind/stVal', 'STRING', 'ST',
        '2 ступінь газового захисту (відключення) (режим)'],
       ['LO18GGIO94', 'Ind/t', 'STRING', 'ST',
        '2 ступінь газового захисту (сигнал) (час)'],
       ['LO18GGIO94', 'Ind/stVal', 'STRING', 'ST',
        '2 ступінь газового захисту (сигнал) (режим)']

In [2]:
class ProjectBases:

    def __init__(self, path_project):
        self.path_project = path_project
        self.base_equip = BaseProject(os.path.join(self.path_project, 'equip.DBF'))
        self.base_eqtype = BaseProject(os.path.join(self.path_project, 'eqtype.DBF'))
        self.base_io = BaseProject(os.path.join(self.path_project, 'units.DBF'))
        self.base_tag = BaseProject(os.path.join(self.path_project, 'variable.DBF'))
        self.base_advalm = BaseProject(os.path.join(self.path_project, 'advalm.DBF'))
        self.base_pgdynobj = BaseProject(os.path.join(self.path_project, 'pgdynobj.DBF'))

    def get_dbf(self, name_file):
        db = None
        with dbf.Table(os.path.join(self.path_project, name_file + '.DBF'), codepage='cp1251') as tb:
            db = tb.open(mode=dbf.READ_ONLY)
        return db

    def dataframe(self, name_file):
        return pd.DataFrame(DBF(os.path.join(self.path_project, name_file + '.DBF'), encoding='cp1251'))


class BaseProject:

    def __init__(self, file_name):
        self.filename = file_name

    def get_dbf(self):
        db = None
        with dbf.Table(self.filename, codepage='cp1251') as tb:
            db = tb.open(mode=dbf.READ_ONLY)
        return db

    def to_dbf(self, params_list):
        with dbf.Table(self.filename, codepage='cp1251') as tb:
            tb.open(mode=dbf.READ_WRITE)
            for params in params_list:
                tb.append(params)

    def to_dataframe_dbf(self, df):
        params_list = df.to_dict('records')
        with dbf.Table(self.filename, codepage='cp1251') as tb:
            tb.open(mode=dbf.READ_WRITE)
            for params in params_list:
                tb.append(params)

    def clear_dbf(self):
        with dbf.Table(self.filename, codepage='cp1251') as tb:
            tb.open(mode=dbf.READ_WRITE)
            tb.zap()

    def dataframe(self):
        return pd.DataFrame(DBF(self.filename, encoding='cp1251'))


class Project:
    def __init__(self, name_project, name_cluster='Cluster1', name_server='IOServer1'):
        self.path_projects = r"c:\ProgramData\AVEVA Plant SCADA 2023\User"
        self.name_projects = name_project
        self.path_project = os.path.join(self.path_projects, self.name_projects)

        self.config_setProject_xls = self.get_path_file('setProject.xlsx')
        self.config_libDevice_xls = self.get_path_file('libDevice.xlsx')

        self.type_device_current = ''
        self.cluster = name_cluster
        self.server = name_server
        self.base = ProjectBases(self.path_project)

        self.set_df_xls('df_io', self.config_setProject_xls, 'IO')
        self.set_df_xls('df_equips', self.config_setProject_xls, 'LD')

    def set_df_xls(self, name_df, config_xls, sheet_name):
        self.__dict__[name_df] = None
        try:
            df = pd.read_excel(config_xls, sheet_name=sheet_name)
            df = df.fillna('')
            df = df.astype(str)
            self.__dict__[name_df] = df
            return self.__dict__[name_df]
        except:
            pass

    def create_dbf_from_xls(self, ls_name_file):
        for name_file, key_col in ls_name_file:
            df = self.set_df_xls(f'df_{name_file}', self.config_setProject_xls, name_file)
            df_exist = self.base.dataframe(name_file)

            names = []
            if len(df_exist) != 0:
                names = list(df_exist[key_col].values)

            if df is not None:
                params_list = []
                columns = self.base.get_dbf(name_file).field_names

                for i in range(len(df)):
                    row = df.iloc[i]
                    values = {}
                    for column in columns:
                        val = ''
                        if column in df.columns:
                            val = str(row[column])
                            if val == 'nan':
                                val = ''
                        values[column] = val

                    if values[key_col] not in names:
                        params_list.append(values)
                        names.append(values[key_col])

                with dbf.Table(self.get_path_file(f'{name_file}.DBF'), codepage='cp1251') as tb:
                    tb.open(mode=dbf.READ_WRITE)
                    for params in params_list:
                        tb.append(params)

    def copy_libfile(self, ls_file):
        ls_file = ['libDevice.xlsx', 'setProject.xlsx']

        for name_file in ls_file:
            try:
                shutil.copy(
                    os.path.join('C:', 'libCitectScada', name_file),
                    self.path_project
                )
            except:
                pass

    def create_eqtype(self):
        self.create_dbf_from_xls([('eqtype', 'NAME')])

        template_dev = self.base.dataframe('eqtype')['TEMPLATE'].values
        template_dev
        for dev in template_dev:
            try:
                shutil.copy(
                    os.path.join('C:', 'libCitectScada', dev),
                    self.path_project
                )
            except:
                pass

    def create_io(self):

        if self.df_io is not None:

            # self.df_io['port'] = 'PORT1_BOARD1'
            addr = ''

            params_list = []

            df_tmp = self.base.base_io.dataframe()
            try:
                numb = int(df_tmp.iloc[-1]['NUMBER'])
            except:
                numb = 0

            df_io_exist = self.base.base_io.dataframe()

            names = []
            if len(df_io_exist) != 0:
                names = list(df_io_exist['NAME'].values)
            else:
                numb = 0

            for i in range(len(self.df_io)):
                io = self.df_io.iloc[i]

                if io['name'] not in names:
                    numb += 1

                if io['protocol'] == 'IEC61850N':
                    name_file_iec61850 = f"conf61850_{io['name']}.xml"
                    addr = f"[RUN]:{name_file_iec61850}"
                    self.create_config61850(name_file_iec61850, isLDs=True)

                values = {'SERVER': self.server,
                          'NAME': io['name'],
                          'NUMBER': f'{numb}',
                          'ADDR': addr,
                          'PROTOCOL': io['protocol'],
                          'PORT': io['port'],
                          'MODE': 'Primary',
                          'LOGWRITE': '',
                          'LOGREAD': '',
                          'CACHE': '',
                          'CACHETIME': '',
                          'COMMENT': '',
                          'PROTOCOLID': '',
                          'LINKED': '',
                          'EXTERNDB': '',
                          'DRIVERID': '',
                          'CONNSTRING': '',
                          'REFRESH': '',
                          'TAGPREFIX': '',
                          'LASTUPDATE': '',
                          'REMOTE': '',
                          'REMOTEWRIT': '',
                          'TIME': '',
                          'PERIOD': '',
                          'PHONE': '',
                          'CALLERID': '',
                          'ONCONNECT': '',
                          'ONHANGUP': '',
                          'ONBROWSE': '',
                          'LIVEUPDATE': '',
                          'LASTVARMOD': '',
                          'MEMORY': str(io['memory']),
                          'PERSIST': '',
                          'TAGGEN': '',
                          'TAGGENTEMP': '',
                          'PRIORITY': '1',
                          'BGPOLL': '',
                          'BGPOLLRATE': '',
                          'PERSFREQ': '',
                          'PERSPATH': '',
                          'MINUPDATE': '',
                          'STALEPERIO': '',
                          'READONLY': '',
                          'EXCLUSIVE': ''}

                if values['NAME'] not in names:
                    params_list.append(values)
                    names.append(values['NAME'])

            self.base.base_io.to_dbf(params_list)

    def create_equip(self, isCreateTag=False, typeCreateTag='simple'):
        if self.df_equips is not None:

            # self.df_equips['io'] = self.df_io.iloc[0]['name']  #TODO

            params_list = []

            df_equip_exist = self.base.base_equip.dataframe()

            names = []
            if len(df_equip_exist) != 0:
                names = list(df_equip_exist['NAME'].values)

            for i in range(len(self.df_equips)):
                equip = self.df_equips.iloc[i]

                name_LD = f"{equip['name']}"
                if equip['number'].strip() != '': name_LD += f"_{equip['number']}"
                name_LD1 = name_LD;
                if equip['spec'].strip() != '': name_LD += f"_{equip['spec']}"

                name_equip = name_LD
                if equip['device'].strip() != '': name_equip += f"_{equip['device']}"

                values = {'NAME': name_equip,
                          'CLUSTER': self.cluster,
                          'TYPE': equip['device'],
                          'AREA': '',
                          'LOCATION': '',
                          'COMMENT': equip['comment'],
                          'CUSTOM1': f"{name_LD}\\",
                          # 'CUSTOM1': f"{equip['io']}\\",
                          'CUSTOM2': '',
                          'CUSTOM3': '',
                          'CUSTOM4': '',
                          'CUSTOM5': '',
                          'CUSTOM6': '',
                          'CUSTOM7': '',
                          'CUSTOM8': '',
                          'IODEVICE': equip['io'],
                          'PAGE': '',
                          'HELP': '',
                          'DEFSTATE': '',
                          'SCHEDULED': '',
                          'TAGPREFIX': f"{name_LD}/",
                          'TAGGENLINK': '',
                          'LINKED': '',
                          'EDITCODE': '',
                          'PARAM': '',
                          'COMPOSITE': '',
                          'REFERENCE': '',
                          'DEVSCHED': '',
                          'SCHEDID': '',
                          'ALIAS': '',
                          'CONTENT': '',
                          'HIDDEN': ''}

                if values['NAME'] not in names:
                    params_list.append(values)
                    names.append(values['NAME'])

                if isCreateTag:
                    sheet_name = equip['device'] + '_61850'
                    df_dev = pd.read_excel(self.config_libDevice_xls, sheet_name=sheet_name)
                    df_dev['io'] = equip['io']
                    df_dev['LD'] = name_LD
                    df_dev['LD1'] = name_LD1
                    df_dev['equip'] = values['NAME']
                    self.create_tag(df_dev, type=typeCreateTag)
                    # self.create_tag(df_dev, type='IEC61850')

            self.base.base_equip.to_dbf(params_list)

    def create_tag(self, df, type='simple', replace=None):
        '''

        :param df:
        :param type: 'simple', 'IEC61850'
        :param replace:
        :return:
        '''
        df_tags_exist = self.base.base_tag.dataframe()

        names = []
        if len(df_tags_exist) != 0:
            names = list(df_tags_exist['NAME'].values)

        df_tags = pd.DataFrame()
        if replace and len(df_tags_exist) != 0:
            df_tags = df_tags_exist

        if replace:
            self.base.base_tag.clear_dbf()

        for i in range(len(df)):
            dev = df.iloc[i]
            values = self.create_tag_values(dev, type=type)

            if values['NAME'] not in names:
                df_tags = pd.concat([df_tags, pd.DataFrame([values])], ignore_index=True)
                names.append(values['NAME'])
            else:
                if replace:
                    df_tags.loc[df_tags['NAME'] == values['NAME'], :] = list(values.values())

        self.base.base_tag.to_dataframe_dbf(df_tags)

    def create_tag_values(self, df, type='simple'):
        '''

        :param df:
        :param type: 'simple', 'IEC61850'
        :return:
        '''
        dev = df.fillna('')

        values = {}

        if type == 'simple':
            values = {'NAME': dev['name'],
                      'TYPE': dev['type'],
                      'UNIT': dev['io'],
                      'ADDR': "",
                      'RAW_ZERO': '',
                      'RAW_FULL': '',
                      'ENG_ZERO': '',
                      'ENG_FULL': '',
                      'ENG_UNITS': '',
                      'FORMAT': '',
                      'COMMENT': dev['comment'],
                      'EDITCODE': '',
                      'LINKED': '',
                      'OID': '',
                      'REF1': '',
                      'REF2': '',
                      'DEADBAND': '',
                      'CUSTOM': '',
                      'TAGGENLINK': '',
                      'CLUSTER': self.cluster,
                      'HISTORIAN': 'TRUE',
                      'EQUIP': '',
                      'CUSTOM1': '',
                      'CUSTOM2': '',
                      'CUSTOM3': '',
                      'CUSTOM4': '',
                      'CUSTOM5': '',
                      'CUSTOM6': '',
                      'CUSTOM7': '',
                      'CUSTOM8': '',
                      'ITEM': ''}

        if type == 'IEC61850_104':
            attribs = dev['attribs'].split('/')
            name_tag_attribs = '\\'.join(attribs)
            addr = f"{dev['LD']}/{dev['node']}${dev['fc']}${'$'.join(attribs)}"
            unit = dev['io']
            if dev['attribs'] == '':
                addr = ''
                unit = 'Internal'
            values = {'NAME': f"{dev['LD1']}_{dev['name104']}",
                      'TYPE': dev['type'],
                      'UNIT': unit,
                      'ADDR': '',
                      'RAW_ZERO': f"{dev['RAW_ZERO']}",
                      'RAW_FULL': f"{dev['RAW_FULL']}",
                      'ENG_ZERO': f"{dev['ENG_ZERO']}",
                      'ENG_FULL': f"{dev['ENG_FULL']}",
                      'ENG_UNITS': f"{dev['ENG_UNITS']}",
                      'FORMAT': rf"{dev['FORMAT']}",
                      # 'FORMAT': "###.##",
                      'COMMENT': dev['comment'],
                      'EDITCODE': '',
                      'LINKED': '',
                      'OID': '',
                      'REF1': '',
                      'REF2': '',
                      'DEADBAND': '',
                      'CUSTOM': '',
                      'TAGGENLINK': '',
                      'CLUSTER': self.cluster,
                      'HISTORIAN': 'TRUE',
                      'EQUIP': dev['equip'],
                      'CUSTOM1': '',
                      'CUSTOM2': '',
                      'CUSTOM3': '',
                      'CUSTOM4': '',
                      'CUSTOM5': '',
                      'CUSTOM6': '',
                      'CUSTOM7': '',
                      'CUSTOM8': '',
                      'ITEM': f"{dev['node']}\\{name_tag_attribs}"}

        if type == 'IEC61850':
            attribs = dev['attribs'].split('/')
            name_tag_attribs = '\\'.join(attribs)
            addr = f"{dev['LD']}/{dev['node']}${dev['fc']}${'$'.join(attribs)}"
            unit = dev['io']
            if dev['attribs'] == '':
                addr = ''
                unit = 'Internal'
            values = {'NAME': f"{dev['LD']}\\{dev['node']}\\{name_tag_attribs}",
                      'TYPE': dev['type'],
                      'UNIT': unit,
                      'ADDR': addr,
                      'RAW_ZERO': '',
                      'RAW_FULL': '',
                      'ENG_ZERO': '',
                      'ENG_FULL': '',
                      'ENG_UNITS': '',
                      'FORMAT': '',
                      'COMMENT': dev['comment'],
                      'EDITCODE': '',
                      'LINKED': '',
                      'OID': '',
                      'REF1': '',
                      'REF2': '',
                      'DEADBAND': '',
                      'CUSTOM': '',
                      'TAGGENLINK': '',
                      'CLUSTER': self.cluster,
                      'HISTORIAN': 'TRUE',
                      'EQUIP': dev['equip'],
                      'CUSTOM1': '',
                      'CUSTOM2': '',
                      'CUSTOM3': '',
                      'CUSTOM4': '',
                      'CUSTOM5': '',
                      'CUSTOM6': '',
                      'CUSTOM7': '',
                      'CUSTOM8': '',
                      'ITEM': f"{dev['node']}\\{name_tag_attribs}"}

        return values

    def create_config61850(self, name_file, isLDs=False):

        s1 = ""

        if isLDs:
            lds = []
            if self.df_equips is not None:
                for i in range(len(self.df_equips)):
                    equip = self.df_equips.iloc[i]

                    name_LD = f"{equip['name']}"
                    if equip['number'].strip() != '': name_LD += f"_{equip['number']}"
                    if equip['spec'].strip() != '': name_LD += f"_{equip['spec']}"
                    lds.append(name_LD)
                    s1 += f"""
                            <LogicalDevice Name="{name_LD}">
                                  <BRCB>brcbEV1</BRCB>
                                  <BRCB>brcbEV2</BRCB>
                                  <URCB>urcbEV1</URCB>
                               </LogicalDevice>

                        """
            else:
                f"""
                    <LogicalDevice Name="Relay">
                          <BRCB>brcbEV1</BRCB>
                          <BRCB>brcbEV2</BRCB>
                          <URCB>urcbEV1</URCB>
                       </LogicalDevice>
                """

        xml_doc = f"""
            <ScadaDevice xmlns="http://www.schneider-electric.com/SCADA/Drivers/IEC61850/DeviceConfig/v1/">
               <IPConnection>
                  <IP>127.0.0.1</IP>
                  <Port>102</Port>
               </IPConnection>
               <IED>TEMPLATE</IED>
               {s1}
            </ScadaDevice>
            """

        soup = BeautifulSoup(xml_doc, 'lxml-xml')
        xml = soup.prettify()
        with open(self.get_path_file(name_file), "w", encoding='utf-8') as file:
            file.write(xml)

    def get_path_file(self, name_file):
        return os.path.join(self.path_project, name_file)

# project = Project('test61850_2')
# project.copy_libfile(['libDevice.xlsx', 'setProject.xlsx'])
# project.create_dbf_from_xls([('boards', 'NAME'), ('ports', 'NAME')])
# 
# project.create_eqtype()
# project.create_io()

In [3]:
project = Project('FabrichnayaDSP_main')

In [4]:
df = project.base.base_advalm.dataframe()
df

,TAG,NAME,DESC,EXPR,CATEGORY,HELP,PRIV,AREA,COMMENT,SEQUENCE,...,CUSTOM8,CLUSTER,PAGING,PAGINGGRP,EDITCODE,LINKED,TAGGENLINK,EQUIP,ITEM,HISTORIAN
0,A0001,Стрiлка 1/3,Видана команда на переведення в ПЛЮС,"TagBitF(502, 0)",4,,,,,,...,,,,,,,,,,
1,A0002,Стрiлка 1/3,Видана команда на переведення в МIНУС,"TagBitF(502, 1)",4,,,,,,...,,,,,,,,,,
2,A0003,Стрiлка 1/3,Видана команда на допомiжне переведення,"TagBitF(502, 2)",4,,,,,,...,,,,,,,,,,
3,A0004,Стрiлка 1/3,Стрiлка переведена в ПЛЮС,"TagBitF(502, 3)",50,,,,,,...,,,,,,,,,,
4,A0005,Стрiлка 1/3,Стрiлка переведена в МIНУС,"TagBitF(502, 4)",50,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3285,A3318,Предвх. св. ЧЗ109А,Несправнiсть в ланцюзi жовтого вогня усунена,"RegGetValueMidBit(RegGetValueF(995), 0, 8) <> 4",10,,,,,,...,,,,,,,,,,
3286,A3319,Предвх. св. ЧЗ109А,Несправнiсть в ланцюзi зеленого вогня,"RegGetValueMidBit(RegGetValueF(995), 0, 8) = 5",10,,,,,,...,,,,,,,,,,
3287,A3320,Предвх. св. ЧЗ109А,Несправнiсть в ланцюзi зеленого вогня усунена,"RegGetValueMidBit(RegGetValueF(995), 0, 8) <> 5",10,,,,,,...,,,,,,,,,,
3288,A3321,Предвх. св. ЧЗ109А,Включений жовтий вогонь (при несправностi свiт...,"RegGetValueMidBit(RegGetValueF(995), 0, 8) = 6",10,,,,,,...,,,,,,,,,,


In [ ]:
vc = df['NAME'].value_counts()

In [ ]:
df1 = df[df['NAME'].str.startswith('Стрiлка')]
df1

In [ ]:
st1 = df1['NAME'].unique()
st1

In [ ]:
st12 = df1['NAME'].apply(lambda x: x[8:])
ls_str = st12.unique().tolist()
ls_str

In [ ]:
ls_names = ['Стрiлка', 'Вих. св.', 'Предвх. св.', 'Ман. св.', 'Стр. дiл.', 'Безстр. дiл.', 'Шлях', 'Тупик']
ds = {}

for s in ls_names:
    df1 = df[df['NAME'].str.startswith(s)]
    st1 = df1['NAME'].unique()
    st2 = df1['NAME'].apply(lambda x: x[len(s) + 1:])
    ls_str = st2.unique().tolist()
    ds[s] = ls_str


In [ ]:
s1 = 'RegGetValueMidBit(RegGetValueF(994), 0, 8) = 1'
s2 = 'AlmSpFree(814, 8) = 0'
s3 = 'TagBitF(628, 3)'


In [5]:
def f1(row):
    s1 = row['EXPR']
    if s1.startswith('RegGetValueMidBit'):
        return s1[31:].split(')')[0]
    if 'TagBit' in s1:
        return s1.split('(')[1][:-1].split(',')[0]
    if s1.startswith('Alm'):
        ss2 = s1.split('(')
        ss3 = ss2[1].split(')')
        return ss3[0].split(',')[0]

In [6]:
df2= df[['NAME', 'DESC', 'EXPR']].copy()
df2

,NAME,DESC,EXPR
0,Стрiлка 1/3,Видана команда на переведення в ПЛЮС,"TagBitF(502, 0)"
1,Стрiлка 1/3,Видана команда на переведення в МIНУС,"TagBitF(502, 1)"
2,Стрiлка 1/3,Видана команда на допомiжне переведення,"TagBitF(502, 2)"
3,Стрiлка 1/3,Стрiлка переведена в ПЛЮС,"TagBitF(502, 3)"
4,Стрiлка 1/3,Стрiлка переведена в МIНУС,"TagBitF(502, 4)"
...,...,...,...
3285,Предвх. св. ЧЗ109А,Несправнiсть в ланцюзi жовтого вогня усунена,"RegGetValueMidBit(RegGetValueF(995), 0, 8) <> 4"
3286,Предвх. св. ЧЗ109А,Несправнiсть в ланцюзi зеленого вогня,"RegGetValueMidBit(RegGetValueF(995), 0, 8) = 5"
3287,Предвх. св. ЧЗ109А,Несправнiсть в ланцюзi зеленого вогня усунена,"RegGetValueMidBit(RegGetValueF(995), 0, 8) <> 5"
3288,Предвх. св. ЧЗ109А,Включений жовтий вогонь (при несправностi свiт...,"RegGetValueMidBit(RegGetValueF(995), 0, 8) = 6"


In [7]:
df2['reg'] = None

In [8]:
df2['reg'] = df2.apply(f1, axis=1)

In [9]:
df2

,NAME,DESC,EXPR,reg
0,Стрiлка 1/3,Видана команда на переведення в ПЛЮС,"TagBitF(502, 0)",502
1,Стрiлка 1/3,Видана команда на переведення в МIНУС,"TagBitF(502, 1)",502
2,Стрiлка 1/3,Видана команда на допомiжне переведення,"TagBitF(502, 2)",502
3,Стрiлка 1/3,Стрiлка переведена в ПЛЮС,"TagBitF(502, 3)",502
4,Стрiлка 1/3,Стрiлка переведена в МIНУС,"TagBitF(502, 4)",502
...,...,...,...,...
3285,Предвх. св. ЧЗ109А,Несправнiсть в ланцюзi жовтого вогня усунена,"RegGetValueMidBit(RegGetValueF(995), 0, 8) <> 4",995
3286,Предвх. св. ЧЗ109А,Несправнiсть в ланцюзi зеленого вогня,"RegGetValueMidBit(RegGetValueF(995), 0, 8) = 5",995
3287,Предвх. св. ЧЗ109А,Несправнiсть в ланцюзi зеленого вогня усунена,"RegGetValueMidBit(RegGetValueF(995), 0, 8) <> 5",995
3288,Предвх. св. ЧЗ109А,Включений жовтий вогонь (при несправностi свiт...,"RegGetValueMidBit(RegGetValueF(995), 0, 8) = 6",995


In [10]:
df2.to_excel('reg_dev_fabr1.xlsx', index=False)

In [ ]:
df3 = df2.iloc[df2['NAME'].drop_duplicates().index]
df4 = df3[['NAME', 'reg']]
df4

In [ ]:
# writer = pd.ExcelWriter('reg_dev_fabr.xlsx', engine='xlsxwriter')
df4.to_excel('reg_dev_fabr.xlsx', index=False)

In [13]:
s1 = '''
<variables name="A_1_3MAK_KN" typeName="BOOL"/>
		<variables name="A_1_3MAK_M_IM" typeName="BOOL"/>
		<variables name="A_1_3MAK_P_IM" typeName="BOOL"/>
		<variables name="A_1_3MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_1_3MK_K" typeName="EBOOL"/>
		<variables name="A_1_3MKStr" typeName="BOOL"/>
		<variables name="A_1_3MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_1_3MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_1_3MU_U" typeName="EBOOL"/>
		<variables name="A_1_3MU_U_OBR" typeName="BOOL"/>
		<variables name="A_1_3MU_U_PR" typeName="BOOL"/>
		<variables name="A_1_3MU_UK" typeName="EBOOL"/>
		<variables name="A_1_3NPS_U" typeName="EBOOL"/>
		<variables name="A_1_3PK_K" typeName="EBOOL"/>
		<variables name="A_1_3PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_1_3PU_U" typeName="EBOOL"/>
		<variables name="A_1_3PU_U_OBR" typeName="BOOL"/>
		<variables name="A_1_3PU_U_PR" typeName="BOOL"/>
		<variables name="A_1_3PU_UK" typeName="EBOOL"/>
		<variables name="A_1_3VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_1_3VK_U" typeName="EBOOL"/>
		<variables name="A_1_3VK_U_OBR" typeName="BOOL"/>
		<variables name="A_1_3VK_U_PR" typeName="BOOL"/>
		<variables name="A_1_3VK_UK" typeName="EBOOL"/>
		<variables name="A_1_3VZ" typeName="BOOL"/>
		<variables name="A_1_3VZREZ_IM" typeName="BOOL"/>
		<variables name="A_1_3ZSTR" typeName="BOOL"/>
		<variables name="A_1_3ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_1_15IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_1_15KS" typeName="BOOL"/>
		<variables name="A_1_15P_IND" typeName="BYTE"/>
		<variables name="A_1_15P_K" typeName="EBOOL"/>
		<variables name="A_1_15RI" typeName="BOOL"/>
		<variables name="A_1_15Z" typeName="BOOL"/>
		<variables name="A_1AIR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_1AKS" typeName="BOOL"/>
		<variables name="A_1AP_IND" typeName="BYTE"/>
		<variables name="A_1AP_K" typeName="EBOOL"/>
		<variables name="A_1ARI" typeName="BOOL"/>
		<variables name="A_1AZ" typeName="BOOL"/>
		<variables name="A_1F_ACTIV_IM" typeName="BOOL"/>
		<variables name="A_1F_JK" typeName="BOOL"/>
		<variables name="A_1F_JL" typeName="BOOL"/>
		<variables name="A_1F_KL" typeName="BOOL"/>
		<variables name="A_1F_NEISPR_IM" typeName="BOOL"/>
		<variables name="A_1FA_K" typeName="EBOOL">
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_1FB_K" typeName="EBOOL">
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_1FC_K" typeName="EBOOL">
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_1FK_K" typeName="BOOL"/>
		<variables name="A_1GIR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_1GKS" typeName="BOOL"/>
		<variables name="A_1GP_IND" typeName="BYTE"/>
		<variables name="A_1GP_K" typeName="EBOOL"/>
		<variables name="A_1GRI" typeName="BOOL"/>
		<variables name="A_1GZ" typeName="BOOL"/>
		<variables name="A_1IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_1P1MS_U" typeName="EBOOL"/>
		<variables name="A_1P1MS_U_OBR" typeName="BOOL"/>
		<variables name="A_1P1MS_U_PR" typeName="BOOL"/>
		<variables name="A_1P1MS_UK" typeName="EBOOL"/>
		<variables name="A_1P2MS_U" typeName="EBOOL"/>
		<variables name="A_1P2MS_U_OBR" typeName="BOOL"/>
		<variables name="A_1P2MS_U_PR" typeName="BOOL"/>
		<variables name="A_1P2MS_UK" typeName="EBOOL"/>
		<variables name="A_1RI" typeName="BOOL"/>
		<variables name="A_1SP_IND" typeName="BYTE"/>
		<variables name="A_1SP_K" typeName="EBOOL"/>
		<variables name="A_1SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_1SP_MK_IND" typeName="BOOL"/>
		<variables name="A_1SP_PK_IND" typeName="BOOL"/>
		<variables name="A_1VZREZ_IM" typeName="BOOL"/>
		<variables name="A_1Z" typeName="BOOL"/>
		<variables name="A_2_4MAK_KN" typeName="BOOL"/>
		<variables name="A_2AIR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_2AKS" typeName="BOOL"/>
		<variables name="A_2AP_IND" typeName="BYTE"/>
		<variables name="A_2AP_K" typeName="EBOOL"/>
		<variables name="A_2ARI" typeName="BOOL"/>
		<variables name="A_2AZ" typeName="BOOL"/>
		<variables name="A_2F_ACTIV_IM" typeName="BOOL"/>
		<variables name="A_2F_JK" typeName="BOOL"/>
		<variables name="A_2F_JL" typeName="BOOL"/>
		<variables name="A_2F_KL" typeName="BOOL"/>
		<variables name="A_2F_NEISPR_IM" typeName="BOOL"/>
		<variables name="A_2FA_K" typeName="EBOOL">
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_2FB_K" typeName="EBOOL">
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_2FC_K" typeName="EBOOL">
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_2FK_K" typeName="BOOL"/>
		<variables name="A_2GIR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_2GKS" typeName="BOOL"/>
		<variables name="A_2GP_IND" typeName="BYTE"/>
		<variables name="A_2GP_K" typeName="EBOOL"/>
		<variables name="A_2GRI" typeName="BOOL"/>
		<variables name="A_2GZ" typeName="BOOL"/>
		<variables name="A_2P1MS_U" typeName="EBOOL"/>
		<variables name="A_2P1MS_U_OBR" typeName="BOOL"/>
		<variables name="A_2P1MS_U_PR" typeName="BOOL"/>
		<variables name="A_2P1MS_UK" typeName="EBOOL"/>
		<variables name="A_2P2MS_U" typeName="EBOOL"/>
		<variables name="A_2P2MS_U_OBR" typeName="BOOL"/>
		<variables name="A_2P2MS_U_PR" typeName="BOOL"/>
		<variables name="A_2P2MS_UK" typeName="EBOOL"/>
		<variables name="A_3_5IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_3_5RI" typeName="BOOL"/>
		<variables name="A_3_5SP_3STR_IND" typeName="BYTE"/>
		<variables name="A_3_5SP_5STR_IND" typeName="BYTE"/>
		<variables name="A_3_5SP_K" typeName="EBOOL"/>
		<variables name="A_3_5Z" typeName="BOOL"/>
		<variables name="A_3SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_3SP_MK_IND" typeName="BOOL"/>
		<variables name="A_3SP_PK_IND" typeName="BOOL"/>
		<variables name="A_3VZREZ_IM" typeName="BOOL"/>
		<variables name="A_4_84IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_4_84RI" typeName="BOOL"/>
		<variables name="A_4_84SP_4STR_IND" typeName="BYTE"/>
		<variables name="A_4_84SP_84STR_IND" typeName="BYTE"/>
		<variables name="A_4_84SP_K" typeName="BOOL"/>
		<variables name="A_4_84Z" typeName="BOOL"/>
		<variables name="A_4_124IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_4_124KS" typeName="BOOL"/>
		<variables name="A_4_124P_IND" typeName="BYTE"/>
		<variables name="A_4_124P_K" typeName="EBOOL"/>
		<variables name="A_4_124RI" typeName="BOOL"/>
		<variables name="A_4_124Z" typeName="BOOL"/>
		<variables name="A_4MAK_KN" typeName="BOOL"/>
		<variables name="A_4MAK_M_IM" typeName="BOOL"/>
		<variables name="A_4MAK_P_IM" typeName="BOOL"/>
		<variables name="A_4MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_4MK_K" typeName="EBOOL"/>
		<variables name="A_4MKStr" typeName="BOOL"/>
		<variables name="A_4MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_4MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_4MU_U" typeName="EBOOL"/>
		<variables name="A_4MU_U_OBR" typeName="BOOL"/>
		<variables name="A_4MU_U_PR" typeName="BOOL"/>
		<variables name="A_4MU_UK" typeName="EBOOL"/>
		<variables name="A_4NPS_U" typeName="EBOOL"/>
		<variables name="A_4PK_K" typeName="EBOOL"/>
		<variables name="A_4PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_4PU_U" typeName="EBOOL"/>
		<variables name="A_4PU_U_OBR" typeName="BOOL"/>
		<variables name="A_4PU_U_PR" typeName="BOOL"/>
		<variables name="A_4PU_UK" typeName="EBOOL"/>
		<variables name="A_4SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_4SP_MK_IND" typeName="BOOL"/>
		<variables name="A_4SP_PK_IND" typeName="BOOL"/>
		<variables name="A_4VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_4VK_U" typeName="EBOOL"/>
		<variables name="A_4VK_U_OBR" typeName="BOOL"/>
		<variables name="A_4VK_U_PR" typeName="BOOL"/>
		<variables name="A_4VK_UK" typeName="EBOOL"/>
		<variables name="A_4VZ" typeName="BOOL"/>
		<variables name="A_4VZREZ_IM" typeName="BOOL"/>
		<variables name="A_4ZSTR" typeName="BOOL"/>
		<variables name="A_4ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_5_7MAK_KN" typeName="BOOL"/>
		<variables name="A_5_9IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_5_9KS" typeName="BOOL"/>
		<variables name="A_5_9P_IND" typeName="BYTE"/>
		<variables name="A_5_9P_K" typeName="EBOOL"/>
		<variables name="A_5_9RI" typeName="BOOL"/>
		<variables name="A_5_9Z" typeName="BOOL"/>
		<variables name="A_5_201IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_5_201KS" typeName="BOOL"/>
		<variables name="A_5_201P_IND" typeName="BYTE"/>
		<variables name="A_5_201P_K" typeName="EBOOL"/>
		<variables name="A_5_201RI" typeName="BOOL"/>
		<variables name="A_5_201Z" typeName="BOOL"/>
		<variables name="A_5MAK_KN" typeName="BOOL"/>
		<variables name="A_5MAK_M_IM" typeName="BOOL"/>
		<variables name="A_5MAK_P_IM" typeName="BOOL"/>
		<variables name="A_5MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_5MK_K" typeName="EBOOL"/>
		<variables name="A_5MKStr" typeName="BOOL"/>
		<variables name="A_5MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_5MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_5MU_U" typeName="EBOOL"/>
		<variables name="A_5MU_U_OBR" typeName="BOOL"/>
		<variables name="A_5MU_U_PR" typeName="BOOL"/>
		<variables name="A_5MU_UK" typeName="EBOOL"/>
		<variables name="A_5NPS_U" typeName="EBOOL"/>
		<variables name="A_5P1MS_U" typeName="EBOOL"/>
		<variables name="A_5P1MS_U_OBR" typeName="BOOL"/>
		<variables name="A_5P1MS_U_PR" typeName="BOOL"/>
		<variables name="A_5P1MS_UK" typeName="EBOOL"/>
		<variables name="A_5P2MS_U" typeName="EBOOL"/>
		<variables name="A_5P2MS_U_OBR" typeName="BOOL"/>
		<variables name="A_5P2MS_U_PR" typeName="BOOL"/>
		<variables name="A_5P2MS_UK" typeName="EBOOL"/>
		<variables name="A_5PK_K" typeName="EBOOL"/>
		<variables name="A_5PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_5PU_U" typeName="EBOOL"/>
		<variables name="A_5PU_U_OBR" typeName="BOOL"/>
		<variables name="A_5PU_U_PR" typeName="BOOL"/>
		<variables name="A_5PU_UK" typeName="EBOOL"/>
		<variables name="A_5SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_5SP_MK_IND" typeName="BOOL"/>
		<variables name="A_5SP_PK_IND" typeName="BOOL"/>
		<variables name="A_5VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_5VK_U" typeName="EBOOL"/>
		<variables name="A_5VK_U_OBR" typeName="BOOL"/>
		<variables name="A_5VK_U_PR" typeName="BOOL"/>
		<variables name="A_5VK_UK" typeName="EBOOL"/>
		<variables name="A_5VZ" typeName="BOOL"/>
		<variables name="A_5VZREZ_IM" typeName="BOOL"/>
		<variables name="A_5ZSTR" typeName="BOOL"/>
		<variables name="A_5ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_6_8IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_6_8KS" typeName="BOOL"/>
		<variables name="A_6_8MAK_KN" typeName="BOOL"/>
		<variables name="A_6_8P_IND" typeName="BYTE"/>
		<variables name="A_6_8P_K" typeName="EBOOL"/>
		<variables name="A_6_8RI" typeName="BOOL"/>
		<variables name="A_6_8Z" typeName="BOOL"/>
		<variables name="A_6_84IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_6_84KS" typeName="BOOL"/>
		<variables name="A_6_84P_IND" typeName="BYTE"/>
		<variables name="A_6_84P_K" typeName="EBOOL"/>
		<variables name="A_6_84RI" typeName="BOOL"/>
		<variables name="A_6_84Z" typeName="BOOL"/>
		<variables name="A_6IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_6MAK_KN" typeName="BOOL"/>
		<variables name="A_6MAK_M_IM" typeName="BOOL"/>
		<variables name="A_6MAK_P_IM" typeName="BOOL"/>
		<variables name="A_6MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_6MK_K" typeName="EBOOL"/>
		<variables name="A_6MKStr" typeName="BOOL"/>
		<variables name="A_6MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_6MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_6MU_U" typeName="EBOOL"/>
		<variables name="A_6MU_U_OBR" typeName="BOOL"/>
		<variables name="A_6MU_U_PR" typeName="BOOL"/>
		<variables name="A_6MU_UK" typeName="EBOOL"/>
		<variables name="A_6NPS_U" typeName="EBOOL"/>
		<variables name="A_6P1MS_U" typeName="EBOOL"/>
		<variables name="A_6P1MS_U_OBR" typeName="BOOL"/>
		<variables name="A_6P1MS_U_PR" typeName="BOOL"/>
		<variables name="A_6P1MS_UK" typeName="EBOOL"/>
		<variables name="A_6P2MS_U" typeName="EBOOL"/>
		<variables name="A_6P2MS_U_OBR" typeName="BOOL"/>
		<variables name="A_6P2MS_U_PR" typeName="BOOL"/>
		<variables name="A_6P2MS_UK" typeName="EBOOL"/>
		<variables name="A_6P_IND" typeName="BYTE"/>
		<variables name="A_6P_K" typeName="EBOOL"/>
		<variables name="A_6PIR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_6PK_K" typeName="EBOOL"/>
		<variables name="A_6PKS" typeName="BOOL"/>
		<variables name="A_6PRI" typeName="BOOL"/>
		<variables name="A_6PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_6PU_U" typeName="EBOOL"/>
		<variables name="A_6PU_U_OBR" typeName="BOOL"/>
		<variables name="A_6PU_U_PR" typeName="BOOL"/>
		<variables name="A_6PU_UK" typeName="EBOOL"/>
		<variables name="A_6PZ" typeName="BOOL"/>
		<variables name="A_6RI" typeName="BOOL"/>
		<variables name="A_6SP_IND" typeName="BYTE"/>
		<variables name="A_6SP_K" typeName="EBOOL"/>
		<variables name="A_6SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_6SP_MK_IND" typeName="BOOL"/>
		<variables name="A_6SP_PK_IND" typeName="BOOL"/>
		<variables name="A_6VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_6VK_U" typeName="EBOOL"/>
		<variables name="A_6VK_U_OBR" typeName="BOOL"/>
		<variables name="A_6VK_U_PR" typeName="BOOL"/>
		<variables name="A_6VK_UK" typeName="EBOOL"/>
		<variables name="A_6VZ" typeName="BOOL"/>
		<variables name="A_6VZREZ_IM" typeName="BOOL"/>
		<variables name="A_6Z" typeName="BOOL"/>
		<variables name="A_6ZSTR" typeName="BOOL"/>
		<variables name="A_6ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_7AIR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_7AKS" typeName="BOOL"/>
		<variables name="A_7AP_IND" typeName="BYTE"/>
		<variables name="A_7AP_K" typeName="EBOOL"/>
		<variables name="A_7ARI" typeName="BOOL"/>
		<variables name="A_7AZ" typeName="BOOL"/>
		<variables name="A_7IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_7KS" typeName="BOOL"/>
		<variables name="A_7P_IND" typeName="BYTE"/>
		<variables name="A_7P_K" typeName="EBOOL"/>
		<variables name="A_7RI" typeName="BOOL"/>
		<variables name="A_7Z" typeName="BOOL"/>
		<variables name="A_8_16IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_8_16KS" typeName="BOOL"/>
		<variables name="A_8_16P_IND" typeName="BYTE"/>
		<variables name="A_8_16P_K" typeName="EBOOL"/>
		<variables name="A_8_16RI" typeName="BOOL"/>
		<variables name="A_8_16Z" typeName="BOOL"/>
		<variables name="A_8_84Z" typeName="BOOL"/>
		<variables name="A_8CHIR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_8CHKS" typeName="BOOL"/>
		<variables name="A_8CHP_IND" typeName="BYTE"/>
		<variables name="A_8CHP_K" typeName="EBOOL"/>
		<variables name="A_8CHRI" typeName="BOOL"/>
		<variables name="A_8CHZ" typeName="BOOL"/>
		<variables name="A_8IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_8MAK_KN" typeName="BOOL"/>
		<variables name="A_8MAK_M_IM" typeName="BOOL"/>
		<variables name="A_8MAK_P_IM" typeName="BOOL"/>
		<variables name="A_8MAK_UST_IM" typeName="BOOL"/>
		<variables name="a_8MK_K" typeName="EBOOL"/>
		<variables name="A_8MKStr" typeName="BOOL"/>
		<variables name="A_8MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_8MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_8MU_U" typeName="EBOOL"/>
		<variables name="A_8MU_U_OBR" typeName="BOOL"/>
		<variables name="A_8MU_U_PR" typeName="BOOL"/>
		<variables name="A_8MU_UK" typeName="EBOOL"/>
		<variables name="A_8NIR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_8NKS" typeName="BOOL"/>
		<variables name="A_8NP_IND" typeName="BYTE"/>
		<variables name="A_8NP_K" typeName="EBOOL"/>
		<variables name="A_8NPS_U" typeName="EBOOL"/>
		<variables name="A_8NRI" typeName="BOOL"/>
		<variables name="A_8NZ" typeName="BOOL"/>
		<variables name="A_8P1MS_U" typeName="EBOOL"/>
		<variables name="A_8P1MS_U_OBR" typeName="BOOL"/>
		<variables name="A_8P1MS_U_PR" typeName="BOOL"/>
		<variables name="A_8P1MS_UK" typeName="EBOOL"/>
		<variables name="A_8P2MS_U" typeName="EBOOL"/>
		<variables name="A_8P2MS_U_OBR" typeName="BOOL"/>
		<variables name="A_8P2MS_U_PR" typeName="BOOL"/>
		<variables name="A_8P2MS_UK" typeName="EBOOL"/>
		<variables name="A_8PK_K" typeName="EBOOL"/>
		<variables name="A_8PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_8PU_U" typeName="EBOOL"/>
		<variables name="A_8PU_U_OBR" typeName="BOOL"/>
		<variables name="A_8PU_U_PR" typeName="BOOL"/>
		<variables name="A_8PU_UK" typeName="EBOOL"/>
		<variables name="A_8RI" typeName="BOOL"/>
		<variables name="A_8SP_IND" typeName="BYTE"/>
		<variables name="A_8SP_K" typeName="EBOOL"/>
		<variables name="A_8SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_8SP_MK_IND" typeName="BOOL"/>
		<variables name="A_8SP_PK_IND" typeName="BOOL"/>
		<variables name="A_8VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_8VK_U" typeName="EBOOL"/>
		<variables name="A_8VK_U_OBR" typeName="BOOL"/>
		<variables name="A_8VK_U_PR" typeName="BOOL"/>
		<variables name="A_8VK_UK" typeName="EBOOL"/>
		<variables name="A_8VZ" typeName="BOOL"/>
		<variables name="A_8VZREZ_IM" typeName="BOOL"/>
		<variables name="A_8Z" typeName="BOOL"/>
		<variables name="A_8ZSTR" typeName="BOOL"/>
		<variables name="A_8ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_9IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_9MAK_KN" typeName="BOOL"/>
		<variables name="A_9MAK_M_IM" typeName="BOOL"/>
		<variables name="A_9MAK_P_IM" typeName="BOOL"/>
		<variables name="A_9MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_9MK_K" typeName="EBOOL"/>
		<variables name="A_9MKStr" typeName="BOOL"/>
		<variables name="A_9MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_9MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_9MU_U" typeName="EBOOL"/>
		<variables name="A_9MU_U_OBR" typeName="BOOL"/>
		<variables name="A_9MU_U_PR" typeName="BOOL"/>
		<variables name="A_9MU_UK" typeName="EBOOL"/>
		<variables name="A_9NPS_U" typeName="EBOOL"/>
		<variables name="A_9P1MS_K" typeName="EBOOL"/>
		<variables name="A_9P1MS_U" typeName="EBOOL"/>
		<variables name="A_9P1MS_U_OBR" typeName="BOOL"/>
		<variables name="A_9P1MS_U_PR" typeName="BOOL"/>
		<variables name="A_9P1MS_UK" typeName="EBOOL"/>
		<variables name="A_9P2MS_K" typeName="EBOOL"/>
		<variables name="A_9P2MS_U" typeName="EBOOL"/>
		<variables name="A_9P2MS_U_OBR" typeName="BOOL"/>
		<variables name="A_9P2MS_U_PR" typeName="BOOL"/>
		<variables name="A_9P2MS_UK" typeName="EBOOL"/>
		<variables name="A_9PK_K" typeName="EBOOL"/>
		<variables name="A_9PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_9PU_U" typeName="EBOOL"/>
		<variables name="A_9PU_U_OBR" typeName="BOOL"/>
		<variables name="A_9PU_U_PR" typeName="BOOL"/>
		<variables name="A_9PU_UK" typeName="EBOOL"/>
		<variables name="A_9RI" typeName="BOOL"/>
		<variables name="A_9SP_IND" typeName="BYTE"/>
		<variables name="A_9SP_K" typeName="EBOOL"/>
		<variables name="A_9SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_9SP_MK_IND" typeName="BOOL"/>
		<variables name="A_9SP_PK_IND" typeName="BOOL"/>
		<variables name="A_9VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_9VK_U" typeName="EBOOL"/>
		<variables name="A_9VK_U_OBR" typeName="BOOL"/>
		<variables name="A_9VK_U_PR" typeName="BOOL"/>
		<variables name="A_9VK_UK" typeName="EBOOL"/>
		<variables name="A_9VZ" typeName="BOOL"/>
		<variables name="A_9VZREZ_IM" typeName="BOOL"/>
		<variables name="A_9Z" typeName="BOOL"/>
		<variables name="A_9ZSTR" typeName="BOOL"/>
		<variables name="A_9ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_10_12IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_10_12KS" typeName="BOOL"/>
		<variables name="A_10_12P_IND" typeName="BYTE"/>
		<variables name="A_10_12P_K" typeName="EBOOL"/>
		<variables name="A_10_12RI" typeName="BOOL"/>
		<variables name="A_10_12Z" typeName="BOOL"/>
		<variables name="A_10_49Z" typeName="BOOL"/>
		<variables name="A_10MAK_KN" typeName="BOOL"/>
		<variables name="A_11_13MAK_KN" typeName="BOOL"/>
		<variables name="A_11_13MAK_M_IM" typeName="BOOL"/>
		<variables name="A_11_13MAK_P_IM" typeName="BOOL"/>
		<variables name="A_11_13MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_11_13MK_K" typeName="EBOOL"/>
		<variables name="A_11_13MKStr" typeName="BOOL"/>
		<variables name="A_11_13MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_11_13MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_11_13MU_U" typeName="EBOOL"/>
		<variables name="A_11_13MU_U_OBR" typeName="BOOL"/>
		<variables name="A_11_13MU_U_PR" typeName="BOOL"/>
		<variables name="A_11_13MU_UK" typeName="EBOOL"/>
		<variables name="A_11_13NPS_U" typeName="EBOOL"/>
		<variables name="A_11_13PK_K" typeName="EBOOL"/>
		<variables name="A_11_13PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_11_13PU_U" typeName="EBOOL"/>
		<variables name="A_11_13PU_U_OBR" typeName="BOOL"/>
		<variables name="A_11_13PU_U_PR" typeName="BOOL"/>
		<variables name="A_11_13PU_UK" typeName="EBOOL"/>
		<variables name="A_11_13VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_11_13VK_U" typeName="EBOOL"/>
		<variables name="A_11_13VK_U_OBR" typeName="BOOL"/>
		<variables name="A_11_13VK_U_PR" typeName="BOOL"/>
		<variables name="A_11_13VK_UK" typeName="EBOOL"/>
		<variables name="A_11_13VZ" typeName="BOOL"/>
		<variables name="A_11_13VZREZ_IM" typeName="BOOL"/>
		<variables name="A_11_13ZSTR" typeName="BOOL"/>
		<variables name="A_11_13ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_11_19IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_11_19RI" typeName="BOOL"/>
		<variables name="A_11_19SP_11STRT_IND" typeName="BYTE"/>
		<variables name="A_11_19SP_17STR_IND" typeName="BYTE"/>
		<variables name="A_11_19SP_19STR_IND" typeName="BYTE"/>
		<variables name="A_11_19SP_K" typeName="EBOOL"/>
		<variables name="A_11_19Z" typeName="BOOL"/>
		<variables name="A_11MAK_KN" typeName="BOOL"/>
		<variables name="A_11SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_11SP_MK_IND" typeName="BOOL"/>
		<variables name="A_11SP_PK_IND" typeName="BOOL"/>
		<variables name="A_11VZREZ_IM" typeName="BOOL"/>
		<variables name="A_12_14IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_12_14KS" typeName="BOOL"/>
		<variables name="A_12_14P_IND" typeName="BYTE"/>
		<variables name="A_12_14P_K" typeName="EBOOL"/>
		<variables name="A_12_14RI" typeName="BOOL"/>
		<variables name="A_12_14Z" typeName="BOOL"/>
		<variables name="A_12_302IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_12_302KS" typeName="BOOL"/>
		<variables name="A_12_302P_IND" typeName="BYTE"/>
		<variables name="A_12_302P_K" typeName="EBOOL"/>
		<variables name="A_12_302RI" typeName="BOOL"/>
		<variables name="A_12_302Z" typeName="BOOL"/>
		<variables name="A_12IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_12MAK_KN" typeName="BOOL"/>
		<variables name="A_12MAK_M_IM" typeName="BOOL"/>
		<variables name="A_12MAK_P_IM" typeName="BOOL"/>
		<variables name="A_12MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_12MK_K" typeName="EBOOL"/>
		<variables name="A_12MKStr" typeName="BOOL"/>
		<variables name="A_12MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_12MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_12MU_U" typeName="EBOOL"/>
		<variables name="A_12MU_U_OBR" typeName="BOOL"/>
		<variables name="A_12MU_U_PR" typeName="BOOL"/>
		<variables name="A_12MU_UK" typeName="EBOOL"/>
		<variables name="A_12NPS_U" typeName="EBOOL"/>
		<variables name="A_12PK_K" typeName="EBOOL"/>
		<variables name="A_12PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_12PU_U" typeName="EBOOL"/>
		<variables name="A_12PU_U_OBR" typeName="BOOL"/>
		<variables name="A_12PU_U_PR" typeName="BOOL"/>
		<variables name="A_12PU_UK" typeName="EBOOL"/>
		<variables name="A_12RI" typeName="BOOL"/>
		<variables name="A_12SP_IND" typeName="BYTE"/>
		<variables name="A_12SP_K" typeName="EBOOL"/>
		<variables name="A_12SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_12SP_MK_IND" typeName="BOOL"/>
		<variables name="A_12SP_PK_IND" typeName="BOOL"/>
		<variables name="A_12VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_12VK_U" typeName="EBOOL"/>
		<variables name="A_12VK_U_OBR" typeName="BOOL"/>
		<variables name="A_12VK_U_PR" typeName="BOOL"/>
		<variables name="A_12VK_UK" typeName="EBOOL"/>
		<variables name="A_12VZ" typeName="BOOL"/>
		<variables name="A_12VZREZ_IM" typeName="BOOL"/>
		<variables name="A_12Z" typeName="BOOL"/>
		<variables name="A_12ZSTR" typeName="BOOL"/>
		<variables name="A_12ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_13IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_13MAK_KN" typeName="BOOL"/>
		<variables name="A_13RI" typeName="BOOL"/>
		<variables name="A_13SP_IND" typeName="BYTE"/>
		<variables name="A_13SP_K" typeName="EBOOL"/>
		<variables name="A_13SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_13SP_MK_IND" typeName="BOOL"/>
		<variables name="A_13SP_PK_IND" typeName="BOOL"/>
		<variables name="A_13VZREZ_IM" typeName="BOOL"/>
		<variables name="A_13Z" typeName="BOOL"/>
		<variables name="A_14_16MAK_KN" typeName="BOOL"/>
		<variables name="A_14_16MAK_M_IM" typeName="BOOL"/>
		<variables name="A_14_16MAK_P_IM" typeName="BOOL"/>
		<variables name="A_14_16MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_14_16MK_K" typeName="EBOOL"/>
		<variables name="A_14_16MKStr" typeName="BOOL"/>
		<variables name="A_14_16MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_14_16MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_14_16MU_U" typeName="EBOOL"/>
		<variables name="A_14_16MU_U_OBR" typeName="BOOL"/>
		<variables name="A_14_16MU_U_PR" typeName="BOOL"/>
		<variables name="A_14_16MU_UK" typeName="EBOOL"/>
		<variables name="A_14_16NPS_U" typeName="EBOOL"/>
		<variables name="A_14_16PK_K" typeName="EBOOL"/>
		<variables name="A_14_16PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_14_16PU_U" typeName="EBOOL"/>
		<variables name="A_14_16PU_U_OBR" typeName="BOOL"/>
		<variables name="A_14_16PU_U_PR" typeName="BOOL"/>
		<variables name="A_14_16PU_UK" typeName="EBOOL"/>
		<variables name="A_14_16VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_14_16VK_U" typeName="EBOOL"/>
		<variables name="A_14_16VK_U_OBR" typeName="BOOL"/>
		<variables name="A_14_16VK_U_PR" typeName="BOOL"/>
		<variables name="A_14_16VK_UK" typeName="EBOOL"/>
		<variables name="A_14_16VZ" typeName="BOOL"/>
		<variables name="A_14_16VZREZ_IM" typeName="BOOL"/>
		<variables name="A_14_16ZSTR" typeName="BOOL"/>
		<variables name="A_14_16ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_14IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_14MAK_KN" typeName="BOOL"/>
		<variables name="A_14RI" typeName="BOOL"/>
		<variables name="A_14SP_IND" typeName="BYTE"/>
		<variables name="A_14SP_K" typeName="EBOOL"/>
		<variables name="A_14SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_14SP_MK_IND" typeName="BOOL"/>
		<variables name="A_14SP_PK_IND" typeName="BOOL"/>
		<variables name="A_14VZREZ_IM" typeName="BOOL"/>
		<variables name="A_14Z" typeName="BOOL"/>
		<variables name="A_15_117IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_15_117KS" typeName="BOOL"/>
		<variables name="A_15_117P_IND" typeName="BYTE"/>
		<variables name="A_15_117P_K" typeName="EBOOL"/>
		<variables name="A_15_117RI" typeName="BOOL"/>
		<variables name="A_15_117Z" typeName="BOOL"/>
		<variables name="A_15IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_15MAK_KN" typeName="BOOL"/>
		<variables name="A_15MAK_M_IM" typeName="BOOL"/>
		<variables name="A_15MAK_P_IM" typeName="BOOL"/>
		<variables name="A_15MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_15MK_K" typeName="EBOOL"/>
		<variables name="A_15MKStr" typeName="BOOL"/>
		<variables name="A_15MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_15MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_15MU_U" typeName="EBOOL"/>
		<variables name="A_15MU_U_OBR" typeName="BOOL"/>
		<variables name="A_15MU_U_PR" typeName="BOOL"/>
		<variables name="A_15MU_UK" typeName="EBOOL"/>
		<variables name="A_15NPS_U" typeName="EBOOL"/>
		<variables name="A_15PK_K" typeName="EBOOL"/>
		<variables name="A_15PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_15PU_U" typeName="EBOOL"/>
		<variables name="A_15PU_U_OBR" typeName="BOOL"/>
		<variables name="A_15PU_U_PR" typeName="BOOL"/>
		<variables name="A_15PU_UK" typeName="EBOOL"/>
		<variables name="A_15RI" typeName="BOOL"/>
		<variables name="A_15SP_IND" typeName="BYTE"/>
		<variables name="A_15SP_K" typeName="EBOOL"/>
		<variables name="A_15SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_15SP_MK_IND" typeName="BOOL"/>
		<variables name="A_15SP_PK_IND" typeName="BOOL"/>
		<variables name="A_15VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_15VK_U" typeName="EBOOL"/>
		<variables name="A_15VK_U_OBR" typeName="BOOL"/>
		<variables name="A_15VK_U_PR" typeName="BOOL"/>
		<variables name="A_15VK_UK" typeName="EBOOL"/>
		<variables name="A_15VZ" typeName="BOOL"/>
		<variables name="A_15VZREZ_IM" typeName="BOOL"/>
		<variables name="A_15Z" typeName="BOOL"/>
		<variables name="A_15ZSTR" typeName="BOOL"/>
		<variables name="A_15ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_16IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_16MAK_KN" typeName="BOOL"/>
		<variables name="A_16P1MS_U" typeName="EBOOL"/>
		<variables name="A_16P1MS_U_OBR" typeName="BOOL"/>
		<variables name="A_16P1MS_U_PR" typeName="BOOL"/>
		<variables name="A_16P1MS_UK" typeName="EBOOL"/>
		<variables name="A_16P2MS_U" typeName="EBOOL"/>
		<variables name="A_16P2MS_U_OBR" typeName="BOOL"/>
		<variables name="A_16P2MS_U_PR" typeName="BOOL"/>
		<variables name="A_16P2MS_UK" typeName="EBOOL"/>
		<variables name="A_16RI" typeName="BOOL"/>
		<variables name="A_16SP_IND" typeName="BYTE"/>
		<variables name="A_16SP_K" typeName="EBOOL"/>
		<variables name="A_16SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_16SP_MK_IND" typeName="BOOL"/>
		<variables name="A_16SP_PK_IND" typeName="BOOL"/>
		<variables name="A_16VZREZ_IM" typeName="BOOL"/>
		<variables name="A_16Z" typeName="BOOL"/>
		<variables name="A_17_19BRI" typeName="BOOL"/>
		<variables name="A_17MAK_KN" typeName="BOOL"/>
		<variables name="A_17MAK_M_IM" typeName="BOOL"/>
		<variables name="A_17MAK_P_IM" typeName="BOOL"/>
		<variables name="A_17MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_17MK_K" typeName="EBOOL"/>
		<variables name="A_17MKStr" typeName="BOOL"/>
		<variables name="A_17MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_17MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_17MU_U" typeName="EBOOL"/>
		<variables name="A_17MU_U_OBR" typeName="BOOL"/>
		<variables name="A_17MU_U_PR" typeName="BOOL"/>
		<variables name="A_17MU_UK" typeName="EBOOL"/>
		<variables name="A_17NPS_U" typeName="EBOOL"/>
		<variables name="A_17PK_K" typeName="EBOOL"/>
		<variables name="A_17PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_17PU_U" typeName="EBOOL"/>
		<variables name="A_17PU_U_OBR" typeName="BOOL"/>
		<variables name="A_17PU_U_PR" typeName="BOOL"/>
		<variables name="A_17PU_UK" typeName="EBOOL"/>
		<variables name="A_17SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_17SP_MK_IND" typeName="BOOL"/>
		<variables name="A_17SP_PK_IND" typeName="BOOL"/>
		<variables name="A_17VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_17VK_U" typeName="EBOOL"/>
		<variables name="A_17VK_U_OBR" typeName="BOOL"/>
		<variables name="A_17VK_U_PR" typeName="BOOL"/>
		<variables name="A_17VK_UK" typeName="EBOOL"/>
		<variables name="A_17VZ" typeName="BOOL"/>
		<variables name="A_17VZREZ_IM" typeName="BOOL"/>
		<variables name="A_17ZSTR" typeName="BOOL"/>
		<variables name="A_17ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_18_26IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_18_26RI" typeName="BOOL"/>
		<variables name="A_18_26SP_18STR_IND" typeName="BYTE"/>
		<variables name="A_18_26SP_26STR_IND" typeName="BYTE"/>
		<variables name="A_18_26SP_K" typeName="EBOOL"/>
		<variables name="A_18_26Z" typeName="BOOL"/>
		<variables name="A_18MAK_KN" typeName="BOOL"/>
		<variables name="A_18MAK_M_IM" typeName="BOOL"/>
		<variables name="A_18MAK_P_IM" typeName="BOOL"/>
		<variables name="A_18MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_18MK_K" typeName="EBOOL"/>
		<variables name="A_18MKStr" typeName="BOOL"/>
		<variables name="A_18MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_18MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_18MU_U" typeName="EBOOL"/>
		<variables name="A_18MU_U_OBR" typeName="BOOL"/>
		<variables name="A_18MU_U_PR" typeName="BOOL"/>
		<variables name="A_18MU_UK" typeName="EBOOL"/>
		<variables name="A_18NPS_U" typeName="EBOOL"/>
		<variables name="A_18PK_K" typeName="EBOOL"/>
		<variables name="A_18PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_18PU_U" typeName="EBOOL"/>
		<variables name="A_18PU_U_OBR" typeName="BOOL"/>
		<variables name="A_18PU_U_PR" typeName="BOOL"/>
		<variables name="A_18PU_UK" typeName="EBOOL"/>
		<variables name="A_18SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_18SP_MK_IND" typeName="BOOL"/>
		<variables name="A_18SP_PK_IND" typeName="BOOL"/>
		<variables name="A_18VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_18VK_U" typeName="EBOOL"/>
		<variables name="A_18VK_U_OBR" typeName="BOOL"/>
		<variables name="A_18VK_U_PR" typeName="BOOL"/>
		<variables name="A_18VK_UK" typeName="EBOOL"/>
		<variables name="A_18VZ" typeName="BOOL"/>
		<variables name="A_18VZREZ_IM" typeName="BOOL"/>
		<variables name="A_18ZSTR" typeName="BOOL"/>
		<variables name="A_18ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_19MAK_KN" typeName="BOOL"/>
		<variables name="A_19MAK_M_IM" typeName="BOOL"/>
		<variables name="A_19MAK_P_IM" typeName="BOOL"/>
		<variables name="A_19MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_19MK_K" typeName="EBOOL"/>
		<variables name="A_19MKStr" typeName="BOOL"/>
		<variables name="A_19MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_19MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_19MU_U" typeName="EBOOL"/>
		<variables name="A_19MU_U_OBR" typeName="BOOL"/>
		<variables name="A_19MU_U_PR" typeName="BOOL"/>
		<variables name="A_19MU_UK" typeName="EBOOL"/>
		<variables name="A_19NPS_U" typeName="EBOOL"/>
		<variables name="A_19PK_K" typeName="EBOOL"/>
		<variables name="A_19PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_19PU_U" typeName="EBOOL"/>
		<variables name="A_19PU_U_OBR" typeName="BOOL"/>
		<variables name="A_19PU_U_PR" typeName="BOOL"/>
		<variables name="A_19PU_UK" typeName="EBOOL"/>
		<variables name="A_19SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_19SP_MK_IND" typeName="BOOL"/>
		<variables name="A_19SP_PK_IND" typeName="BOOL"/>
		<variables name="A_19VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_19VK_U" typeName="EBOOL"/>
		<variables name="A_19VK_U_OBR" typeName="BOOL"/>
		<variables name="A_19VK_U_PR" typeName="BOOL"/>
		<variables name="A_19VK_UK" typeName="EBOOL"/>
		<variables name="A_19VZ" typeName="BOOL"/>
		<variables name="A_19VZREZ_IM" typeName="BOOL"/>
		<variables name="A_19ZSTR" typeName="BOOL"/>
		<variables name="A_19ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_20_22IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_20_22RI" typeName="BOOL"/>
		<variables name="A_20_22SP_20STR_IND" typeName="BYTE"/>
		<variables name="A_20_22SP_22STR_IND" typeName="BYTE"/>
		<variables name="A_20_22SP_K" typeName="EBOOL"/>
		<variables name="A_20_22Z" typeName="BOOL"/>
		<variables name="A_20MAK_KN" typeName="BOOL"/>
		<variables name="A_20MAK_M_IM" typeName="BOOL"/>
		<variables name="A_20MAK_P_IM" typeName="BOOL"/>
		<variables name="A_20MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_20MK_K" typeName="EBOOL"/>
		<variables name="A_20MKStr" typeName="BOOL"/>
		<variables name="A_20MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_20MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_20MU_U" typeName="EBOOL"/>
		<variables name="A_20MU_U_OBR" typeName="BOOL"/>
		<variables name="A_20MU_U_PR" typeName="BOOL"/>
		<variables name="A_20MU_UK" typeName="EBOOL"/>
		<variables name="A_20NPS_U" typeName="EBOOL"/>
		<variables name="A_20PK_K" typeName="EBOOL"/>
		<variables name="A_20PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_20PU_U" typeName="EBOOL"/>
		<variables name="A_20PU_U_OBR" typeName="BOOL"/>
		<variables name="A_20PU_U_PR" typeName="BOOL"/>
		<variables name="A_20PU_UK" typeName="EBOOL"/>
		<variables name="A_20SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_20SP_MK_IND" typeName="BOOL"/>
		<variables name="A_20SP_PK_IND" typeName="BOOL"/>
		<variables name="A_20VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_20VK_U" typeName="EBOOL"/>
		<variables name="A_20VK_U_OBR" typeName="BOOL"/>
		<variables name="A_20VK_U_PR" typeName="BOOL"/>
		<variables name="A_20VK_UK" typeName="EBOOL"/>
		<variables name="A_20VZ" typeName="BOOL"/>
		<variables name="A_20VZREZ_IM" typeName="BOOL"/>
		<variables name="A_20ZSTR" typeName="BOOL"/>
		<variables name="A_20ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_21MAK_KN" typeName="BOOL"/>
		<variables name="A_22_24MAK_KN" typeName="BOOL"/>
		<variables name="A_22_24MAK_M_IM" typeName="BOOL"/>
		<variables name="A_22_24MAK_P_IM" typeName="BOOL"/>
		<variables name="A_22_24MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_22_24MK_K" typeName="EBOOL"/>
		<variables name="A_22_24MKStr" typeName="BOOL"/>
		<variables name="A_22_24MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_22_24MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_22_24MU_U" typeName="EBOOL"/>
		<variables name="A_22_24MU_U_OBR" typeName="BOOL"/>
		<variables name="A_22_24MU_U_PR" typeName="BOOL"/>
		<variables name="A_22_24MU_UK" typeName="EBOOL"/>
		<variables name="A_22_24NPS_U" typeName="EBOOL"/>
		<variables name="A_22_24PK_K" typeName="EBOOL"/>
		<variables name="A_22_24PM_K" typeName="EBOOL"/>
		<variables name="A_22_24PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_22_24PU_U" typeName="EBOOL"/>
		<variables name="A_22_24PU_U_OBR" typeName="BOOL"/>
		<variables name="A_22_24PU_U_PR" typeName="BOOL"/>
		<variables name="A_22_24PU_UK" typeName="EBOOL"/>
		<variables name="A_22_24VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_22_24VK_U" typeName="EBOOL"/>
		<variables name="A_22_24VK_U_OBR" typeName="BOOL"/>
		<variables name="A_22_24VK_U_PR" typeName="BOOL"/>
		<variables name="A_22_24VK_UK" typeName="EBOOL"/>
		<variables name="A_22_24VZ" typeName="BOOL"/>
		<variables name="A_22_24VZREZ_IM" typeName="BOOL"/>
		<variables name="A_22_24ZSTR" typeName="BOOL"/>
		<variables name="A_22_24ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_22_32IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_22_32KS" typeName="BOOL"/>
		<variables name="A_22_32P_IND" typeName="BYTE"/>
		<variables name="A_22_32P_K" typeName="EBOOL"/>
		<variables name="A_22_32RI" typeName="BOOL"/>
		<variables name="A_22_32Z" typeName="BOOL"/>
		<variables name="A_22MAK_KN" typeName="BOOL"/>
		<variables name="A_22SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_22SP_MK_IND" typeName="BOOL"/>
		<variables name="A_22SP_PK_IND" typeName="BOOL"/>
		<variables name="A_22VZREZ_IM" typeName="BOOL"/>
		<variables name="A_23_25MAK_KN" typeName="BOOL"/>
		<variables name="A_23_29IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_23_29KS" typeName="BOOL"/>
		<variables name="A_23_29P_IND" typeName="BYTE"/>
		<variables name="A_23_29P_K" typeName="EBOOL"/>
		<variables name="A_23_29RI" typeName="BOOL"/>
		<variables name="A_23_29Z" typeName="BOOL"/>
		<variables name="A_23IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_23KS" typeName="BOOL"/>
		<variables name="A_23RI" typeName="BOOL"/>
		<variables name="A_23SP_IND" typeName="BYTE"/>
		<variables name="A_23SP_K" typeName="EBOOL"/>
		<variables name="A_23Z" typeName="BOOL"/>
		<variables name="A_24_42AKS" typeName="BOOL"/>
		<variables name="A_24_42AP_IND" typeName="BYTE"/>
		<variables name="A_24_42AP_K" typeName="EBOOL"/>
		<variables name="A_24_42ARI" typeName="BOOL"/>
		<variables name="A_24_42AZ" typeName="BOOL"/>
		<variables name="A_24_42IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_24_42KS" typeName="BOOL"/>
		<variables name="A_24_42P_IND" typeName="BYTE"/>
		<variables name="A_24_42P_K" typeName="EBOOL"/>
		<variables name="A_24_42RI" typeName="BOOL"/>
		<variables name="A_24_42Z" typeName="BOOL"/>
		<variables name="A_24IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_24MAK_KN" typeName="BOOL"/>
		<variables name="A_24RI" typeName="BOOL"/>
		<variables name="A_24SP_IND" typeName="BYTE"/>
		<variables name="A_24SP_K" typeName="EBOOL"/>
		<variables name="A_24SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_24SP_MK_IND" typeName="BOOL"/>
		<variables name="A_24SP_PK_IND" typeName="BOOL"/>
		<variables name="A_24VZREZ_IM" typeName="BOOL"/>
		<variables name="A_24Z" typeName="BOOL"/>
		<variables name="A_25_27IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_25_27RI" typeName="BOOL"/>
		<variables name="A_25_27SP_25STR_IND" typeName="BYTE"/>
		<variables name="A_25_27SP_27STR_IND" typeName="BYTE"/>
		<variables name="A_25_27SP_K" typeName="EBOOL"/>
		<variables name="A_25_27Z" typeName="BOOL"/>
		<variables name="A_25_31IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_25_31KS" typeName="BOOL"/>
		<variables name="A_25_31P_IND" typeName="BYTE"/>
		<variables name="A_25_31P_K" typeName="EBOOL"/>
		<variables name="A_25_31RI" typeName="BOOL"/>
		<variables name="A_25_31Z" typeName="BOOL"/>
		<variables name="A_25MAK_KN" typeName="BOOL"/>
		<variables name="A_25MAK_M_IM" typeName="BOOL"/>
		<variables name="A_25MAK_P_IM" typeName="BOOL"/>
		<variables name="A_25MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_25MK_K" typeName="EBOOL"/>
		<variables name="A_25MKStr" typeName="BOOL"/>
		<variables name="A_25MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_25MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_25MU_U" typeName="EBOOL"/>
		<variables name="A_25MU_U_OBR" typeName="BOOL"/>
		<variables name="A_25MU_U_PR" typeName="BOOL"/>
		<variables name="A_25MU_UK" typeName="EBOOL"/>
		<variables name="A_25NPS_U" typeName="EBOOL"/>
		<variables name="A_25PK_K" typeName="EBOOL"/>
		<variables name="A_25PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_25PU_U" typeName="EBOOL"/>
		<variables name="A_25PU_U_OBR" typeName="BOOL"/>
		<variables name="A_25PU_U_PR" typeName="BOOL"/>
		<variables name="A_25PU_UK" typeName="EBOOL"/>
		<variables name="A_25SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_25SP_MK_IND" typeName="BOOL"/>
		<variables name="A_25SP_PK_IND" typeName="BOOL"/>
		<variables name="A_25VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_25VK_U" typeName="EBOOL"/>
		<variables name="A_25VK_U_OBR" typeName="BOOL"/>
		<variables name="A_25VK_U_PR" typeName="BOOL"/>
		<variables name="A_25VK_UK" typeName="EBOOL"/>
		<variables name="A_25VZ" typeName="BOOL"/>
		<variables name="A_25VZREZ_IM" typeName="BOOL"/>
		<variables name="A_25ZSTR" typeName="BOOL"/>
		<variables name="A_25ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_26_28MAK_KN" typeName="BOOL"/>
		<variables name="A_26_28MAK_M_IM" typeName="BOOL"/>
		<variables name="A_26_28MAK_P_IM" typeName="BOOL"/>
		<variables name="A_26_28MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_26_28MK_K" typeName="EBOOL"/>
		<variables name="A_26_28MKStr" typeName="BOOL"/>
		<variables name="A_26_28MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_26_28MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_26_28MU_U" typeName="EBOOL"/>
		<variables name="A_26_28MU_U_OBR" typeName="BOOL"/>
		<variables name="A_26_28MU_U_PR" typeName="BOOL"/>
		<variables name="A_26_28MU_UK" typeName="EBOOL"/>
		<variables name="A_26_28NPS_U" typeName="BOOL"/>
		<variables name="A_26_28PK_K" typeName="EBOOL"/>
		<variables name="A_26_28PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_26_28PU_U" typeName="EBOOL"/>
		<variables name="A_26_28PU_U_OBR" typeName="BOOL"/>
		<variables name="A_26_28PU_U_PR" typeName="BOOL"/>
		<variables name="A_26_28PU_UK" typeName="EBOOL"/>
		<variables name="A_26_28VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_26_28VK_U" typeName="EBOOL"/>
		<variables name="A_26_28VK_U_OBR" typeName="BOOL"/>
		<variables name="A_26_28VK_U_PR" typeName="BOOL"/>
		<variables name="A_26_28VK_UK" typeName="EBOOL"/>
		<variables name="A_26_28VZ" typeName="BOOL"/>
		<variables name="A_26_28VZREZ_IM" typeName="BOOL"/>
		<variables name="A_26_28ZSTR" typeName="BOOL"/>
		<variables name="A_26_28ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_26_40AIR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_26_40AKS" typeName="BOOL"/>
		<variables name="A_26_40AP_IND" typeName="BYTE"/>
		<variables name="A_26_40AP_K" typeName="EBOOL"/>
		<variables name="A_26_40ARI" typeName="BOOL"/>
		<variables name="A_26_40AZ" typeName="BOOL"/>
		<variables name="A_26_40BIR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_26_40BKS" typeName="BOOL"/>
		<variables name="A_26_40BP_IND" typeName="BYTE"/>
		<variables name="A_26_40BP_K" typeName="EBOOL"/>
		<variables name="A_26_40BRI" typeName="BOOL"/>
		<variables name="A_26_40BZ" typeName="BOOL"/>
		<variables name="A_26MAK_KN" typeName="BOOL"/>
		<variables name="A_26SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_26SP_MK_IND" typeName="BOOL"/>
		<variables name="A_26SP_PK_IND" typeName="BOOL"/>
		<variables name="A_26VZREZ_IM" typeName="BOOL"/>
		<variables name="A_27_101IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_27_101KS" typeName="BOOL"/>
		<variables name="A_27_101P_IND" typeName="BYTE"/>
		<variables name="A_27_101P_K" typeName="EBOOL"/>
		<variables name="A_27_101RI" typeName="BOOL"/>
		<variables name="A_27_101Z" typeName="BOOL"/>
		<variables name="A_27MAK_KN" typeName="BOOL"/>
		<variables name="A_27MAK_M_IM" typeName="BOOL"/>
		<variables name="A_27MAK_P_IM" typeName="BOOL"/>
		<variables name="A_27MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_27MK_K" typeName="EBOOL"/>
		<variables name="A_27MKStr" typeName="BOOL"/>
		<variables name="A_27MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_27MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_27MU_U" typeName="EBOOL"/>
		<variables name="A_27MU_U_OBR" typeName="BOOL"/>
		<variables name="A_27MU_U_PR" typeName="BOOL"/>
		<variables name="A_27MU_UK" typeName="EBOOL"/>
		<variables name="A_27NPS_U" typeName="EBOOL"/>
		<variables name="A_27PK_K" typeName="EBOOL"/>
		<variables name="A_27PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_27PU_U" typeName="EBOOL"/>
		<variables name="A_27PU_U_OBR" typeName="BOOL"/>
		<variables name="A_27PU_U_PR" typeName="BOOL"/>
		<variables name="A_27PU_UK" typeName="EBOOL"/>
		<variables name="A_27SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_27SP_MK_IND" typeName="BOOL"/>
		<variables name="A_27SP_PK_IND" typeName="BOOL"/>
		<variables name="A_27VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_27VK_U" typeName="EBOOL"/>
		<variables name="A_27VK_U_OBR" typeName="BOOL"/>
		<variables name="A_27VK_U_PR" typeName="BOOL"/>
		<variables name="A_27VK_UK" typeName="EBOOL"/>
		<variables name="A_27VZ" typeName="BOOL"/>
		<variables name="A_27VZREZ_IM" typeName="BOOL"/>
		<variables name="A_27ZSTR" typeName="BOOL"/>
		<variables name="A_27ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_28_302IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_28_302KS" typeName="BOOL"/>
		<variables name="A_28_302P_IND" typeName="BYTE"/>
		<variables name="A_28_302P_K" typeName="EBOOL"/>
		<variables name="A_28_302RI" typeName="BOOL"/>
		<variables name="A_28_302Z" typeName="BOOL"/>
		<variables name="A_28IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_28MAK_KN" typeName="BOOL"/>
		<variables name="A_28RI" typeName="BOOL"/>
		<variables name="A_28SP_IND" typeName="BYTE"/>
		<variables name="A_28SP_K" typeName="EBOOL"/>
		<variables name="A_28SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_28SP_MK_IND" typeName="BOOL"/>
		<variables name="A_28SP_PK_IND" typeName="BOOL"/>
		<variables name="A_28VZREZ_IM" typeName="BOOL"/>
		<variables name="A_28Z" typeName="BOOL"/>
		<variables name="A_29_31MAK_KN" typeName="BOOL"/>
		<variables name="A_29_31MAK_M_IM" typeName="BOOL"/>
		<variables name="A_29_31MAK_P_IM" typeName="BOOL"/>
		<variables name="A_29_31MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_29_31MK_K" typeName="EBOOL"/>
		<variables name="A_29_31MKStr" typeName="BOOL"/>
		<variables name="A_29_31MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_29_31MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_29_31MU_U" typeName="EBOOL"/>
		<variables name="A_29_31MU_U_OBR" typeName="BOOL"/>
		<variables name="A_29_31MU_U_PR" typeName="BOOL"/>
		<variables name="A_29_31MU_UK" typeName="EBOOL"/>
		<variables name="A_29_31NPS_U" typeName="EBOOL"/>
		<variables name="A_29_31PK_K" typeName="EBOOL"/>
		<variables name="A_29_31PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_29_31PU_U" typeName="EBOOL"/>
		<variables name="A_29_31PU_U_OBR" typeName="BOOL"/>
		<variables name="A_29_31PU_U_PR" typeName="BOOL"/>
		<variables name="A_29_31PU_UK" typeName="EBOOL"/>
		<variables name="A_29_31VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_29_31VK_U" typeName="EBOOL"/>
		<variables name="A_29_31VK_U_OBR" typeName="BOOL"/>
		<variables name="A_29_31VK_U_PR" typeName="BOOL"/>
		<variables name="A_29_31VK_UK" typeName="EBOOL"/>
		<variables name="A_29_31VZ" typeName="BOOL"/>
		<variables name="A_29_31VZREZ_IM" typeName="BOOL"/>
		<variables name="A_29_31ZSTR" typeName="BOOL"/>
		<variables name="A_29_31ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_29IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_29MAK_KN" typeName="BOOL"/>
		<variables name="A_29RI" typeName="BOOL"/>
		<variables name="A_29SP_IND" typeName="BYTE"/>
		<variables name="A_29SP_K" typeName="EBOOL"/>
		<variables name="A_29SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_29SP_MK_IND" typeName="BOOL"/>
		<variables name="A_29SP_PK_IND" typeName="BOOL"/>
		<variables name="A_29VZREZ_IM" typeName="BOOL"/>
		<variables name="A_29Z" typeName="BOOL"/>
		<variables name="A_30_32MAK_KN" typeName="BOOL"/>
		<variables name="A_30_34MAK_KN" typeName="BOOL"/>
		<variables name="A_30_34MAK_M_IM" typeName="BOOL"/>
		<variables name="A_30_34MAK_P_IM" typeName="BOOL"/>
		<variables name="A_30_34MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_30_34MK_K" typeName="EBOOL"/>
		<variables name="A_30_34MKStr" typeName="BOOL"/>
		<variables name="A_30_34MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_30_34MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_30_34MU_U" typeName="EBOOL"/>
		<variables name="A_30_34MU_U_OBR" typeName="BOOL"/>
		<variables name="A_30_34MU_U_PR" typeName="BOOL"/>
		<variables name="A_30_34MU_UK" typeName="EBOOL"/>
		<variables name="A_30_34NPS_U" typeName="EBOOL"/>
		<variables name="A_30_34PK_K" typeName="EBOOL"/>
		<variables name="A_30_34PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_30_34PU_U" typeName="EBOOL"/>
		<variables name="A_30_34PU_U_OBR" typeName="BOOL"/>
		<variables name="A_30_34PU_U_PR" typeName="BOOL"/>
		<variables name="A_30_34PU_UK" typeName="EBOOL"/>
		<variables name="A_30_34VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_30_34VK_U" typeName="EBOOL"/>
		<variables name="A_30_34VK_U_OBR" typeName="BOOL"/>
		<variables name="A_30_34VK_U_PR" typeName="BOOL"/>
		<variables name="A_30_34VK_UK" typeName="EBOOL"/>
		<variables name="A_30_34VZ" typeName="BOOL"/>
		<variables name="A_30_34VZREZ_IM" typeName="BOOL"/>
		<variables name="A_30_34ZSTR" typeName="BOOL"/>
		<variables name="A_30_34ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_30IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_30RI" typeName="BOOL"/>
		<variables name="A_30SP_IND" typeName="BYTE"/>
		<variables name="A_30SP_K" typeName="BOOL"/>
		<variables name="A_30SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_30SP_MK_IND" typeName="BOOL"/>
		<variables name="A_30SP_PK_IND" typeName="BOOL"/>
		<variables name="A_30VZREZ_IM" typeName="BOOL"/>
		<variables name="A_30Z" typeName="BOOL"/>
		<variables name="A_31IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_31MAK_KN" typeName="BOOL"/>
		<variables name="A_31RI" typeName="BOOL"/>
		<variables name="A_31SP_IND" typeName="BYTE"/>
		<variables name="A_31SP_K" typeName="EBOOL"/>
		<variables name="A_31SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_31SP_MK_IND" typeName="BOOL"/>
		<variables name="A_31SP_PK_IND" typeName="BOOL"/>
		<variables name="A_31VZREZ_IM" typeName="BOOL"/>
		<variables name="A_31Z" typeName="BOOL"/>
		<variables name="A_32_34IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_32_34KS" typeName="BOOL"/>
		<variables name="A_32_34P_IND" typeName="BYTE"/>
		<variables name="A_32_34P_K" typeName="EBOOL"/>
		<variables name="A_32_34RI" typeName="BOOL"/>
		<variables name="A_32_34Z" typeName="BOOL"/>
		<variables name="A_32IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_32MAK_KN" typeName="BOOL"/>
		<variables name="A_32MAK_M_IM" typeName="BOOL"/>
		<variables name="A_32MAK_P_IM" typeName="BOOL"/>
		<variables name="A_32MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_32MK_K" typeName="EBOOL"/>
		<variables name="A_32MKStr" typeName="BOOL"/>
		<variables name="A_32MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_32MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_32MU_U" typeName="EBOOL"/>
		<variables name="A_32MU_U_OBR" typeName="BOOL"/>
		<variables name="A_32MU_U_PR" typeName="BOOL"/>
		<variables name="A_32MU_UK" typeName="EBOOL"/>
		<variables name="A_32NPS_U" typeName="EBOOL"/>
		<variables name="A_32PK_K" typeName="EBOOL"/>
		<variables name="A_32PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_32PU_U" typeName="EBOOL"/>
		<variables name="A_32PU_U_OBR" typeName="BOOL"/>
		<variables name="A_32PU_U_PR" typeName="BOOL"/>
		<variables name="A_32PU_UK" typeName="EBOOL"/>
		<variables name="A_32RI" typeName="BOOL"/>
		<variables name="A_32SP_IND" typeName="BYTE"/>
		<variables name="A_32SP_K" typeName="EBOOL"/>
		<variables name="A_32SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_32SP_MK_IND" typeName="BOOL"/>
		<variables name="A_32SP_PK_IND" typeName="BOOL"/>
		<variables name="A_32VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_32VK_U" typeName="EBOOL"/>
		<variables name="A_32VK_U_OBR" typeName="BOOL"/>
		<variables name="A_32VK_U_PR" typeName="BOOL"/>
		<variables name="A_32VK_UK" typeName="EBOOL"/>
		<variables name="A_32VZ" typeName="BOOL"/>
		<variables name="A_32VZREZ_IM" typeName="BOOL"/>
		<variables name="A_32Z" typeName="BOOL"/>
		<variables name="A_32ZSTR" typeName="BOOL"/>
		<variables name="A_32ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_33_35MAK_KN" typeName="BOOL"/>
		<variables name="A_33_35MAK_M_IM" typeName="BOOL"/>
		<variables name="A_33_35MAK_P_IM" typeName="BOOL"/>
		<variables name="A_33_35MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_33_35MK_K" typeName="EBOOL"/>
		<variables name="A_33_35MKStr" typeName="BOOL"/>
		<variables name="A_33_35MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_33_35MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_33_35MU_U" typeName="EBOOL"/>
		<variables name="A_33_35MU_U_OBR" typeName="BOOL"/>
		<variables name="A_33_35MU_U_PR" typeName="BOOL"/>
		<variables name="A_33_35MU_UK" typeName="EBOOL"/>
		<variables name="A_33_35NPS_U" typeName="EBOOL"/>
		<variables name="A_33_35PK_K" typeName="EBOOL"/>
		<variables name="A_33_35PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_33_35PU_U" typeName="EBOOL"/>
		<variables name="A_33_35PU_U_OBR" typeName="BOOL"/>
		<variables name="A_33_35PU_U_PR" typeName="BOOL"/>
		<variables name="A_33_35PU_UK" typeName="EBOOL"/>
		<variables name="A_33_35VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_33_35VK_U" typeName="EBOOL"/>
		<variables name="A_33_35VK_U_OBR" typeName="BOOL"/>
		<variables name="A_33_35VK_U_PR" typeName="BOOL"/>
		<variables name="A_33_35VK_UK" typeName="EBOOL"/>
		<variables name="A_33_35VZ" typeName="BOOL"/>
		<variables name="A_33_35VZREZ_IM" typeName="BOOL"/>
		<variables name="A_33_35ZSTR" typeName="BOOL"/>
		<variables name="A_33_35ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_33_43IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_33_43RI" typeName="BOOL"/>
		<variables name="A_33_43SP_33STR_IND" typeName="BYTE"/>
		<variables name="A_33_43SP_41STR_IND" typeName="BYTE"/>
		<variables name="A_33_43SP_43STR_IND" typeName="BYTE"/>
		<variables name="A_33_43SP_K" typeName="EBOOL"/>
		<variables name="A_33_43Z" typeName="BOOL"/>
		<variables name="A_33MAK_KN" typeName="BOOL"/>
		<variables name="A_33SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_33SP_MK_IND" typeName="BOOL"/>
		<variables name="A_33SP_PK_IND" typeName="BOOL"/>
		<variables name="A_33VZREZ_IM" typeName="BOOL"/>
		<variables name="A_34_36IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_34_36MAK_KN" typeName="BOOL"/>
		<variables name="A_34_36RI" typeName="BOOL"/>
		<variables name="A_34_36SP_34STR_IND" typeName="BYTE"/>
		<variables name="A_34_36SP_36STR_IND" typeName="BYTE"/>
		<variables name="A_34_36SP_K" typeName="EBOOL"/>
		<variables name="A_34_36Z" typeName="BOOL"/>
		<variables name="A_34SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_34SP_MK_IND" typeName="BOOL"/>
		<variables name="A_34SP_PK_IND" typeName="BOOL"/>
		<variables name="A_34VZREZ_IM" typeName="BOOL"/>
		<variables name="A_35IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_35MAK_KN" typeName="BOOL"/>
		<variables name="A_35RI" typeName="BOOL"/>
		<variables name="A_35SP_IND" typeName="BYTE"/>
		<variables name="A_35SP_K" typeName="EBOOL"/>
		<variables name="A_35SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_35SP_MK_IND" typeName="BOOL"/>
		<variables name="A_35SP_PK_IND" typeName="BOOL"/>
		<variables name="A_35VZREZ_IM" typeName="BOOL"/>
		<variables name="A_35Z" typeName="BOOL"/>
		<variables name="A_36MAK_KN" typeName="BOOL"/>
		<variables name="A_36MAK_M_IM" typeName="BOOL"/>
		<variables name="A_36MAK_P_IM" typeName="BOOL"/>
		<variables name="A_36MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_36MK_K" typeName="EBOOL"/>
		<variables name="A_36MKStr" typeName="BOOL"/>
		<variables name="A_36MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_36MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_36MU_U" typeName="EBOOL"/>
		<variables name="A_36MU_U_OBR" typeName="BOOL"/>
		<variables name="A_36MU_U_PR" typeName="BOOL"/>
		<variables name="A_36MU_UK" typeName="EBOOL"/>
		<variables name="A_36NPS_U" typeName="EBOOL"/>
		<variables name="A_36PK_K" typeName="EBOOL"/>
		<variables name="A_36PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_36PU_U" typeName="EBOOL"/>
		<variables name="A_36PU_U_OBR" typeName="BOOL"/>
		<variables name="A_36PU_U_PR" typeName="BOOL"/>
		<variables name="A_36PU_UK" typeName="EBOOL"/>
		<variables name="A_36SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_36SP_MK_IND" typeName="BOOL"/>
		<variables name="A_36SP_PK_IND" typeName="BOOL"/>
		<variables name="A_36VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_36VK_U" typeName="EBOOL"/>
		<variables name="A_36VK_U_OBR" typeName="BOOL"/>
		<variables name="A_36VK_U_PR" typeName="BOOL"/>
		<variables name="A_36VK_UK" typeName="EBOOL"/>
		<variables name="A_36VZ" typeName="BOOL"/>
		<variables name="A_36VZREZ_IM" typeName="BOOL"/>
		<variables name="A_36ZSTR" typeName="BOOL"/>
		<variables name="A_36ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_37_39IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_37_39RI" typeName="BOOL"/>
		<variables name="A_37_39SP_37STR_IND" typeName="BYTE"/>
		<variables name="A_37_39SP_39STR_IND" typeName="BYTE"/>
		<variables name="A_37_39SP_K" typeName="EBOOL"/>
		<variables name="A_37_39Z" typeName="BOOL"/>
		<variables name="A_37MAK_KN" typeName="BOOL"/>
		<variables name="A_37MAK_M_IM" typeName="BOOL"/>
		<variables name="A_37MAK_P_IM" typeName="BOOL"/>
		<variables name="A_37MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_37MK_K" typeName="EBOOL"/>
		<variables name="A_37MKStr" typeName="BOOL"/>
		<variables name="A_37MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_37MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_37MU_U" typeName="EBOOL"/>
		<variables name="A_37MU_U_OBR" typeName="BOOL"/>
		<variables name="A_37MU_U_PR" typeName="BOOL"/>
		<variables name="A_37MU_UK" typeName="EBOOL"/>
		<variables name="A_37NPS_U" typeName="EBOOL"/>
		<variables name="A_37PK_K" typeName="EBOOL"/>
		<variables name="A_37PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_37PU_U" typeName="EBOOL"/>
		<variables name="A_37PU_U_OBR" typeName="BOOL"/>
		<variables name="A_37PU_U_PR" typeName="BOOL"/>
		<variables name="A_37PU_UK" typeName="EBOOL"/>
		<variables name="A_37SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_37SP_MK_IND" typeName="BOOL"/>
		<variables name="A_37SP_PK_IND" typeName="BOOL"/>
		<variables name="A_37VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_37VK_U" typeName="EBOOL"/>
		<variables name="A_37VK_U_OBR" typeName="BOOL"/>
		<variables name="A_37VK_U_PR" typeName="BOOL"/>
		<variables name="A_37VK_UK" typeName="EBOOL"/>
		<variables name="A_37VZ" typeName="BOOL"/>
		<variables name="A_37VZREZ_IM" typeName="BOOL"/>
		<variables name="A_37ZSTR" typeName="BOOL"/>
		<variables name="A_37ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_38AP1MS_U" typeName="EBOOL"/>
		<variables name="A_38AP1MS_U_OBR" typeName="BOOL"/>
		<variables name="A_38AP1MS_U_PR" typeName="BOOL"/>
		<variables name="A_38AP1MS_UK" typeName="EBOOL"/>
		<variables name="A_38AP2MS_U" typeName="EBOOL"/>
		<variables name="A_38AP2MS_U_OBR" typeName="BOOL"/>
		<variables name="A_38AP2MS_U_PR" typeName="BOOL"/>
		<variables name="A_38AP2MS_UK" typeName="EBOOL"/>
		<variables name="A_38BP1MS_U" typeName="EBOOL"/>
		<variables name="A_38BP1MS_U_OBR" typeName="BOOL"/>
		<variables name="A_38BP1MS_U_PR" typeName="BOOL"/>
		<variables name="A_38BP1MS_UK" typeName="EBOOL"/>
		<variables name="A_38BP2MS_U" typeName="EBOOL"/>
		<variables name="A_38BP2MS_U_OBR" typeName="BOOL"/>
		<variables name="A_38BP2MS_U_PR" typeName="BOOL"/>
		<variables name="A_38BP2MS_UK" typeName="EBOOL"/>
		<variables name="A_38IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_38MAK_KN" typeName="BOOL"/>
		<variables name="A_38MAK_M_IM" typeName="BOOL"/>
		<variables name="A_38MAK_P_IM" typeName="BOOL"/>
		<variables name="A_38MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_38MK_K" typeName="EBOOL"/>
		<variables name="A_38MKStr" typeName="BOOL"/>
		<variables name="A_38MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_38MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_38MU_U" typeName="EBOOL"/>
		<variables name="A_38MU_U_OBR" typeName="BOOL"/>
		<variables name="A_38MU_U_PR" typeName="BOOL"/>
		<variables name="A_38MU_UK" typeName="EBOOL"/>
		<variables name="A_38NPS_U" typeName="EBOOL"/>
		<variables name="A_38PK_K" typeName="EBOOL"/>
		<variables name="A_38PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_38PU_U" typeName="EBOOL"/>
		<variables name="A_38PU_U_OBR" typeName="BOOL"/>
		<variables name="A_38PU_U_PR" typeName="BOOL"/>
		<variables name="A_38PU_UK" typeName="EBOOL"/>
		<variables name="A_38RI" typeName="BOOL"/>
		<variables name="A_38SP_IND" typeName="BYTE"/>
		<variables name="A_38SP_K" typeName="EBOOL"/>
		<variables name="A_38SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_38SP_MK_IND" typeName="BOOL"/>
		<variables name="A_38SP_PK_IND" typeName="BOOL"/>
		<variables name="A_38VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_38VK_U" typeName="EBOOL"/>
		<variables name="A_38VK_U_OBR" typeName="BOOL"/>
		<variables name="A_38VK_U_PR" typeName="BOOL"/>
		<variables name="A_38VK_UK" typeName="EBOOL"/>
		<variables name="A_38VZ" typeName="BOOL"/>
		<variables name="A_38VZREZ_IM" typeName="BOOL"/>
		<variables name="A_38Z" typeName="BOOL"/>
		<variables name="A_38ZSTR" typeName="BOOL"/>
		<variables name="A_38ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_39_51IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_39_51KS" typeName="BOOL"/>
		<variables name="A_39_51P_IND" typeName="BYTE"/>
		<variables name="A_39_51P_K" typeName="EBOOL"/>
		<variables name="A_39_51RI" typeName="BOOL"/>
		<variables name="A_39_51Z" typeName="BOOL"/>
		<variables name="A_39AP1MS_U" typeName="EBOOL"/>
		<variables name="A_39AP1MS_U_OBR" typeName="BOOL"/>
		<variables name="A_39AP1MS_U_PR" typeName="BOOL"/>
		<variables name="A_39AP1MS_UK" typeName="EBOOL"/>
		<variables name="A_39AP2MS_U" typeName="EBOOL"/>
		<variables name="A_39AP2MS_U_OBR" typeName="BOOL"/>
		<variables name="A_39AP2MS_U_PR" typeName="BOOL"/>
		<variables name="A_39AP2MS_UK" typeName="EBOOL"/>
		<variables name="A_39BP1MS_U" typeName="EBOOL"/>
		<variables name="A_39BP1MS_U_OBR" typeName="BOOL"/>
		<variables name="A_39BP1MS_U_PR" typeName="BOOL"/>
		<variables name="A_39BP1MS_UK" typeName="EBOOL"/>
		<variables name="A_39BP2MS_U" typeName="EBOOL"/>
		<variables name="A_39BP2MS_U_OBR" typeName="BOOL"/>
		<variables name="A_39BP2MS_U_PR" typeName="BOOL"/>
		<variables name="A_39BP2MS_UK" typeName="EBOOL"/>
		<variables name="A_39MAK_KN" typeName="BOOL"/>
		<variables name="A_39MAK_M_IM" typeName="BOOL"/>
		<variables name="A_39MAK_P_IM" typeName="BOOL"/>
		<variables name="A_39MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_39MK_K" typeName="EBOOL"/>
		<variables name="A_39MKStr" typeName="BOOL"/>
		<variables name="A_39MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_39MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_39MU_U" typeName="EBOOL"/>
		<variables name="A_39MU_U_OBR" typeName="BOOL"/>
		<variables name="A_39MU_U_PR" typeName="BOOL"/>
		<variables name="A_39MU_UK" typeName="EBOOL"/>
		<variables name="A_39NPS_U" typeName="EBOOL"/>
		<variables name="A_39PK_K" typeName="EBOOL"/>
		<variables name="A_39PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_39PU_U" typeName="EBOOL"/>
		<variables name="A_39PU_U_OBR" typeName="BOOL"/>
		<variables name="A_39PU_U_PR" typeName="BOOL"/>
		<variables name="A_39PU_UK" typeName="EBOOL"/>
		<variables name="A_39SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_39SP_MK_IND" typeName="BOOL"/>
		<variables name="A_39SP_PK_IND" typeName="BOOL"/>
		<variables name="A_39VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_39VK_U" typeName="EBOOL"/>
		<variables name="A_39VK_U_OBR" typeName="BOOL"/>
		<variables name="A_39VK_U_PR" typeName="BOOL"/>
		<variables name="A_39VK_UK" typeName="EBOOL"/>
		<variables name="A_39VZ" typeName="BOOL"/>
		<variables name="A_39VZREZ_IM" typeName="BOOL"/>
		<variables name="A_39ZSTR" typeName="BOOL"/>
		<variables name="A_39ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_40MAK_KN" typeName="BOOL"/>
		<variables name="A_41MAK_KN" typeName="BOOL"/>
		<variables name="A_41MAK_M_IM" typeName="BOOL"/>
		<variables name="A_41MAK_P_IM" typeName="BOOL"/>
		<variables name="A_41MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_41MK_K" typeName="EBOOL"/>
		<variables name="A_41MKStr" typeName="BOOL"/>
		<variables name="A_41MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_41MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_41MU_U" typeName="EBOOL"/>
		<variables name="A_41MU_U_OBR" typeName="BOOL"/>
		<variables name="A_41MU_U_PR" typeName="BOOL"/>
		<variables name="A_41MU_UK" typeName="EBOOL"/>
		<variables name="A_41NPS_U" typeName="EBOOL"/>
		<variables name="A_41PK_K" typeName="EBOOL"/>
		<variables name="A_41PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_41PU_U" typeName="EBOOL"/>
		<variables name="A_41PU_U_OBR" typeName="BOOL"/>
		<variables name="A_41PU_U_PR" typeName="BOOL"/>
		<variables name="A_41PU_UK" typeName="EBOOL"/>
		<variables name="A_41SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_41SP_MK_IND" typeName="BOOL"/>
		<variables name="A_41SP_PK_IND" typeName="BOOL"/>
		<variables name="A_41VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_41VK_U" typeName="EBOOL"/>
		<variables name="A_41VK_U_OBR" typeName="BOOL"/>
		<variables name="A_41VK_U_PR" typeName="BOOL"/>
		<variables name="A_41VK_UK" typeName="EBOOL"/>
		<variables name="A_41VZ" typeName="BOOL"/>
		<variables name="A_41VZREZ_IM" typeName="BOOL"/>
		<variables name="A_41ZSTR" typeName="BOOL"/>
		<variables name="A_41ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_42_44MAK_KN" typeName="BOOL"/>
		<variables name="A_43_32IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_43_32P_IND" typeName="BYTE"/>
		<variables name="A_43_32P_K" typeName="BOOL"/>
		<variables name="A_43_32RI" typeName="BOOL"/>
		<variables name="A_43_32Z" typeName="BOOL"/>
		<variables name="A_43_35KS" typeName="BOOL"/>
		<variables name="A_43MAK_KN" typeName="BOOL"/>
		<variables name="A_43MAK_M_IM" typeName="BOOL"/>
		<variables name="A_43MAK_P_IM" typeName="BOOL"/>
		<variables name="A_43MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_43MK_K" typeName="EBOOL"/>
		<variables name="A_43MKStr" typeName="BOOL"/>
		<variables name="A_43MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_43MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_43MU_U" typeName="EBOOL"/>
		<variables name="A_43MU_U_OBR" typeName="BOOL"/>
		<variables name="A_43MU_U_PR" typeName="BOOL"/>
		<variables name="A_43MU_UK" typeName="EBOOL"/>
		<variables name="A_43NPS_U" typeName="EBOOL"/>
		<variables name="A_43PK_K" typeName="EBOOL"/>
		<variables name="A_43PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_43PU_U" typeName="EBOOL"/>
		<variables name="A_43PU_U_OBR" typeName="BOOL"/>
		<variables name="A_43PU_U_PR" typeName="BOOL"/>
		<variables name="A_43PU_UK" typeName="EBOOL"/>
		<variables name="A_43SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_43SP_MK_IND" typeName="BOOL"/>
		<variables name="A_43SP_PK_IND" typeName="BOOL"/>
		<variables name="A_43VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_43VK_U" typeName="EBOOL"/>
		<variables name="A_43VK_U_OBR" typeName="BOOL"/>
		<variables name="A_43VK_U_PR" typeName="BOOL"/>
		<variables name="A_43VK_UK" typeName="EBOOL"/>
		<variables name="A_43VZ" typeName="BOOL"/>
		<variables name="A_43VZREZ_IM" typeName="BOOL"/>
		<variables name="A_43ZSTR" typeName="BOOL"/>
		<variables name="A_43ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_46MAK_KN" typeName="BOOL"/>
		<variables name="A_47_49IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_47_49RI" typeName="BOOL"/>
		<variables name="A_47_49SP_47STR_IND" typeName="BYTE"/>
		<variables name="A_47_49SP_49STR_IND" typeName="BYTE"/>
		<variables name="A_47_49SP_K" typeName="EBOOL"/>
		<variables name="A_47_49Z" typeName="BOOL"/>
		<variables name="A_47MAK_KN" typeName="BOOL"/>
		<variables name="A_47MAK_M_IM" typeName="BOOL"/>
		<variables name="A_47MAK_P_IM" typeName="BOOL"/>
		<variables name="A_47MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_47MK_K" typeName="EBOOL"/>
		<variables name="A_47MKStr" typeName="BOOL"/>
		<variables name="A_47MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_47MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_47MU_U" typeName="EBOOL"/>
		<variables name="A_47MU_U_OBR" typeName="BOOL"/>
		<variables name="A_47MU_U_PR" typeName="BOOL"/>
		<variables name="A_47MU_UK" typeName="EBOOL"/>
		<variables name="A_47NPS_U" typeName="EBOOL"/>
		<variables name="A_47PK_K" typeName="EBOOL"/>
		<variables name="A_47PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_47PU_U" typeName="EBOOL"/>
		<variables name="A_47PU_U_OBR" typeName="BOOL"/>
		<variables name="A_47PU_U_PR" typeName="BOOL"/>
		<variables name="A_47PU_UK" typeName="EBOOL"/>
		<variables name="A_47SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_47SP_MK_IND" typeName="BOOL"/>
		<variables name="A_47SP_PK_IND" typeName="BOOL"/>
		<variables name="A_47VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_47VK_U" typeName="EBOOL"/>
		<variables name="A_47VK_U_OBR" typeName="BOOL"/>
		<variables name="A_47VK_U_PR" typeName="BOOL"/>
		<variables name="A_47VK_UK" typeName="EBOOL"/>
		<variables name="A_47VZ" typeName="BOOL"/>
		<variables name="A_47VZREZ_IM" typeName="BOOL"/>
		<variables name="A_47ZSTR" typeName="BOOL"/>
		<variables name="A_47ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_48MAK_KN" typeName="BOOL"/>
		<variables name="A_49MAK_KN" typeName="BOOL"/>
		<variables name="A_49MAK_M_IM" typeName="BOOL"/>
		<variables name="A_49MAK_P_IM" typeName="BOOL"/>
		<variables name="A_49MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_49MK_K" typeName="EBOOL"/>
		<variables name="A_49MKStr" typeName="BOOL"/>
		<variables name="A_49MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_49MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_49MU_U" typeName="EBOOL"/>
		<variables name="A_49MU_U_OBR" typeName="BOOL"/>
		<variables name="A_49MU_U_PR" typeName="BOOL"/>
		<variables name="A_49MU_UK" typeName="EBOOL"/>
		<variables name="A_49NPS_U" typeName="EBOOL"/>
		<variables name="A_49PK_K" typeName="EBOOL"/>
		<variables name="A_49PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_49PU_U" typeName="EBOOL"/>
		<variables name="A_49PU_U_OBR" typeName="BOOL"/>
		<variables name="A_49PU_U_PR" typeName="BOOL"/>
		<variables name="A_49PU_UK" typeName="EBOOL"/>
		<variables name="A_49SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_49SP_MK_IND" typeName="BOOL"/>
		<variables name="A_49SP_PK_IND" typeName="BOOL"/>
		<variables name="A_49VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_49VK_U" typeName="EBOOL"/>
		<variables name="A_49VK_U_OBR" typeName="BOOL"/>
		<variables name="A_49VK_U_PR" typeName="BOOL"/>
		<variables name="A_49VK_UK" typeName="EBOOL"/>
		<variables name="A_49VZ" typeName="BOOL"/>
		<variables name="A_49VZREZ_IM" typeName="BOOL"/>
		<variables name="A_49ZSTR" typeName="BOOL"/>
		<variables name="A_49ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_51IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_51MAK_KN" typeName="BOOL"/>
		<variables name="A_51MAK_M_IM" typeName="BOOL"/>
		<variables name="A_51MAK_P_IM" typeName="BOOL"/>
		<variables name="A_51MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_51MK_K" typeName="EBOOL"/>
		<variables name="A_51MKStr" typeName="BOOL"/>
		<variables name="A_51MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_51MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_51MU_U" typeName="EBOOL"/>
		<variables name="A_51MU_U_OBR" typeName="BOOL"/>
		<variables name="A_51MU_U_PR" typeName="BOOL"/>
		<variables name="A_51MU_UK" typeName="EBOOL"/>
		<variables name="A_51NPS_U" typeName="EBOOL"/>
		<variables name="A_51PK_K" typeName="EBOOL"/>
		<variables name="A_51PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_51PU_U" typeName="EBOOL"/>
		<variables name="A_51PU_U_OBR" typeName="BOOL"/>
		<variables name="A_51PU_U_PR" typeName="BOOL"/>
		<variables name="A_51PU_UK" typeName="EBOOL"/>
		<variables name="A_51RI" typeName="BOOL"/>
		<variables name="A_51SP_IND" typeName="BYTE"/>
		<variables name="A_51SP_K" typeName="EBOOL"/>
		<variables name="A_51SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_51SP_MK_IND" typeName="BOOL"/>
		<variables name="A_51SP_PK_IND" typeName="BOOL"/>
		<variables name="A_51VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_51VK_U" typeName="EBOOL"/>
		<variables name="A_51VK_U_OBR" typeName="BOOL"/>
		<variables name="A_51VK_U_PR" typeName="BOOL"/>
		<variables name="A_51VK_UK" typeName="EBOOL"/>
		<variables name="A_51VZ" typeName="BOOL"/>
		<variables name="A_51VZREZ_IM" typeName="BOOL"/>
		<variables name="A_51Z" typeName="BOOL"/>
		<variables name="A_51ZSTR" typeName="BOOL"/>
		<variables name="A_51ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_53IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_53MAK_KN" typeName="BOOL"/>
		<variables name="A_53MAK_M_IM" typeName="BOOL"/>
		<variables name="A_53MAK_P_IM" typeName="BOOL"/>
		<variables name="A_53MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_53MK_K" typeName="EBOOL"/>
		<variables name="A_53MKStr" typeName="BOOL"/>
		<variables name="A_53MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_53MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_53MU_U" typeName="EBOOL"/>
		<variables name="A_53MU_U_OBR" typeName="BOOL"/>
		<variables name="A_53MU_U_PR" typeName="BOOL"/>
		<variables name="A_53MU_UK" typeName="EBOOL"/>
		<variables name="A_53NPS_U" typeName="EBOOL"/>
		<variables name="A_53PK_K" typeName="EBOOL"/>
		<variables name="A_53PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_53PU_U" typeName="EBOOL"/>
		<variables name="A_53PU_U_OBR" typeName="BOOL"/>
		<variables name="A_53PU_U_PR" typeName="BOOL"/>
		<variables name="A_53PU_UK" typeName="EBOOL"/>
		<variables name="A_53RI" typeName="BOOL"/>
		<variables name="A_53SP_IND" typeName="BYTE"/>
		<variables name="A_53SP_K" typeName="EBOOL"/>
		<variables name="A_53SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_53SP_MK_IND" typeName="BOOL"/>
		<variables name="A_53SP_PK_IND" typeName="BOOL"/>
		<variables name="A_53VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_53VK_U" typeName="EBOOL"/>
		<variables name="A_53VK_U_OBR" typeName="BOOL"/>
		<variables name="A_53VK_U_PR" typeName="BOOL"/>
		<variables name="A_53VK_UK" typeName="EBOOL"/>
		<variables name="A_53VZ" typeName="BOOL"/>
		<variables name="A_53VZREZ_IM" typeName="BOOL"/>
		<variables name="A_53Z" typeName="BOOL"/>
		<variables name="A_53ZSTR" typeName="BOOL"/>
		<variables name="A_53ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_55MAK_KN" typeName="BOOL"/>
		<variables name="A_61MAK_KN" typeName="BOOL"/>
		<variables name="A_63MAK_KN" typeName="BOOL"/>
		<variables name="A_71_73MAK_KN" typeName="BOOL"/>
		<variables name="A_72MAK_KN" typeName="BOOL"/>
		<variables name="A_74MAK_KN" typeName="BOOL"/>
		<variables name="A_75MAK_KN" typeName="BOOL"/>
		<variables name="A_76MAK_KN" typeName="BOOL"/>
		<variables name="A_77MAK_KN" typeName="BOOL"/>
		<variables name="A_79MAK_KN" typeName="BOOL"/>
		<variables name="A_81MAK_KN" typeName="BOOL"/>
		<variables name="A_83MAK_KN" typeName="BOOL"/>
		<variables name="A_84MAK_KN" typeName="BOOL"/>
		<variables name="A_84MAK_M_IM" typeName="BOOL"/>
		<variables name="A_84MAK_P_IM" typeName="BOOL"/>
		<variables name="A_84MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_84MK_K" typeName="EBOOL"/>
		<variables name="A_84MKStr" typeName="BOOL"/>
		<variables name="A_84MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_84MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_84MU_U" typeName="EBOOL"/>
		<variables name="A_84MU_U_OBR" typeName="BOOL"/>
		<variables name="A_84MU_U_PR" typeName="BOOL"/>
		<variables name="A_84MU_UK" typeName="EBOOL"/>
		<variables name="A_84NPS_U" typeName="EBOOL"/>
		<variables name="A_84PK_K" typeName="EBOOL"/>
		<variables name="A_84PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_84PU_U" typeName="EBOOL"/>
		<variables name="A_84PU_U_OBR" typeName="BOOL"/>
		<variables name="A_84PU_U_PR" typeName="BOOL"/>
		<variables name="A_84PU_UK" typeName="EBOOL"/>
		<variables name="A_84SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_84SP_MK_IND" typeName="BOOL"/>
		<variables name="A_84SP_PK_IND" typeName="BOOL"/>
		<variables name="A_84VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_84VK_U" typeName="EBOOL"/>
		<variables name="A_84VK_U_OBR" typeName="BOOL"/>
		<variables name="A_84VK_U_PR" typeName="BOOL"/>
		<variables name="A_84VK_UK" typeName="EBOOL"/>
		<variables name="A_84VZ" typeName="BOOL"/>
		<variables name="A_84VZREZ_IM" typeName="BOOL"/>
		<variables name="A_84ZSTR" typeName="BOOL"/>
		<variables name="A_84ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_85MAK_KN" typeName="BOOL"/>
		<variables name="A_87_89MAK_KN" typeName="BOOL"/>
		<variables name="A_91_93MAK_KN" typeName="BOOL"/>
		<variables name="A_98_104IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_98_104RI" typeName="BOOL"/>
		<variables name="A_98_104SP_98STR_IND" typeName="BYTE"/>
		<variables name="A_98_104SP_104STR_IND" typeName="BYTE"/>
		<variables name="A_98_104SP_K" typeName="EBOOL"/>
		<variables name="A_98_104Z" typeName="BOOL"/>
		<variables name="A_98_122IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_98_122KS" typeName="BOOL"/>
		<variables name="A_98_122P_IND" typeName="BYTE"/>
		<variables name="A_98_122P_K" typeName="EBOOL"/>
		<variables name="A_98_122RI" typeName="BOOL"/>
		<variables name="A_98_122Z" typeName="BOOL"/>
		<variables name="A_98MAK_KN" typeName="BOOL"/>
		<variables name="A_98MAK_M_IM" typeName="BOOL"/>
		<variables name="A_98MAK_P_IM" typeName="BOOL"/>
		<variables name="A_98MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_98MK_K" typeName="EBOOL"/>
		<variables name="A_98MKStr" typeName="BOOL"/>
		<variables name="A_98MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_98MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_98MU_U" typeName="EBOOL"/>
		<variables name="A_98MU_U_OBR" typeName="BOOL"/>
		<variables name="A_98MU_U_PR" typeName="BOOL"/>
		<variables name="A_98MU_UK" typeName="EBOOL"/>
		<variables name="A_98NPS_U" typeName="EBOOL"/>
		<variables name="A_98PK_K" typeName="EBOOL"/>
		<variables name="A_98PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_98PU_U" typeName="EBOOL"/>
		<variables name="A_98PU_U_OBR" typeName="BOOL"/>
		<variables name="A_98PU_U_PR" typeName="BOOL"/>
		<variables name="A_98PU_UK" typeName="EBOOL"/>
		<variables name="A_98SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_98SP_MK_IND" typeName="BOOL"/>
		<variables name="A_98SP_PK_IND" typeName="BOOL"/>
		<variables name="A_98VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_98VK_U" typeName="EBOOL"/>
		<variables name="A_98VK_U_OBR" typeName="BOOL"/>
		<variables name="A_98VK_U_PR" typeName="BOOL"/>
		<variables name="A_98VK_UK" typeName="EBOOL"/>
		<variables name="A_98VZ" typeName="BOOL"/>
		<variables name="A_98VZREZ_IM" typeName="BOOL"/>
		<variables name="A_98ZSTR" typeName="BOOL"/>
		<variables name="A_98ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_100IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_100MAK_KN" typeName="BOOL"/>
		<variables name="A_100MAK_M_IM" typeName="BOOL"/>
		<variables name="A_100MAK_P_IM" typeName="BOOL"/>
		<variables name="A_100MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_100MK_K" typeName="EBOOL"/>
		<variables name="A_100MKStr" typeName="BOOL"/>
		<variables name="A_100MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_100MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_100MU_U" typeName="EBOOL"/>
		<variables name="A_100MU_U_OBR" typeName="BOOL"/>
		<variables name="A_100MU_U_PR" typeName="BOOL"/>
		<variables name="A_100MU_UK" typeName="EBOOL"/>
		<variables name="A_100NPS_U" typeName="EBOOL"/>
		<variables name="A_100PK_K" typeName="EBOOL"/>
		<variables name="A_100PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_100PU_U" typeName="EBOOL"/>
		<variables name="A_100PU_U_OBR" typeName="BOOL"/>
		<variables name="A_100PU_U_PR" typeName="BOOL"/>
		<variables name="A_100PU_UK" typeName="EBOOL"/>
		<variables name="A_100RI" typeName="BOOL"/>
		<variables name="A_100SP_IND" typeName="BYTE"/>
		<variables name="A_100SP_K" typeName="EBOOL"/>
		<variables name="A_100SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_100SP_MK_IND" typeName="BOOL"/>
		<variables name="A_100SP_PK_IND" typeName="BOOL"/>
		<variables name="A_100VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_100VK_U" typeName="EBOOL"/>
		<variables name="A_100VK_U_OBR" typeName="BOOL"/>
		<variables name="A_100VK_U_PR" typeName="BOOL"/>
		<variables name="A_100VK_UK" typeName="EBOOL"/>
		<variables name="A_100VZ" typeName="BOOL"/>
		<variables name="A_100VZREZ_IM" typeName="BOOL"/>
		<variables name="A_100Z" typeName="BOOL"/>
		<variables name="A_100ZSTR" typeName="BOOL"/>
		<variables name="A_100ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_101_103IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_101_103RI" typeName="BOOL"/>
		<variables name="A_101_103SP_101STR_IND" typeName="BYTE"/>
		<variables name="A_101_103SP_103STR_IND" typeName="BYTE"/>
		<variables name="A_101_103SP_K" typeName="EBOOL"/>
		<variables name="A_101_103Z" typeName="BOOL"/>
		<variables name="A_101MAK_KN" typeName="BOOL"/>
		<variables name="A_101MAK_M_IM" typeName="BOOL"/>
		<variables name="A_101MAK_P_IM" typeName="BOOL"/>
		<variables name="A_101MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_101MK_K" typeName="EBOOL"/>
		<variables name="A_101MKStr" typeName="BOOL"/>
		<variables name="A_101MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_101MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_101MU_U" typeName="EBOOL"/>
		<variables name="A_101MU_U_OBR" typeName="BOOL"/>
		<variables name="A_101MU_U_PR" typeName="BOOL"/>
		<variables name="A_101MU_UK" typeName="EBOOL"/>
		<variables name="A_101NPS_U" typeName="EBOOL"/>
		<variables name="A_101P_IND" typeName="BYTE"/>
		<variables name="A_101P_K" typeName="EBOOL"/>
		<variables name="A_101PCHKM" typeName="BOOL"/>
		<variables name="A_101PK_K" typeName="EBOOL"/>
		<variables name="A_101PNKM" typeName="BOOL"/>
		<variables name="A_101PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_101PU_U" typeName="EBOOL"/>
		<variables name="A_101PU_U_OBR" typeName="BOOL"/>
		<variables name="A_101PU_U_PR" typeName="BOOL"/>
		<variables name="A_101PU_UK" typeName="EBOOL"/>
		<variables name="A_101RI" typeName="BOOL"/>
		<variables name="A_101SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_101SP_MK_IND" typeName="BOOL"/>
		<variables name="A_101SP_PK_IND" typeName="BOOL"/>
		<variables name="A_101VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_101VK_U" typeName="EBOOL"/>
		<variables name="A_101VK_U_OBR" typeName="BOOL"/>
		<variables name="A_101VK_U_PR" typeName="BOOL"/>
		<variables name="A_101VK_UK" typeName="EBOOL"/>
		<variables name="A_101VZ" typeName="BOOL"/>
		<variables name="A_101VZREZ_IM" typeName="BOOL"/>
		<variables name="A_101ZSTR" typeName="BOOL"/>
		<variables name="A_101ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_102_120IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_102_120RI" typeName="BOOL"/>
		<variables name="A_102_120SP_102STR_IND" typeName="BYTE"/>
		<variables name="A_102_120SP_120STR_IND" typeName="BYTE"/>
		<variables name="A_102_120SP_K" typeName="EBOOL"/>
		<variables name="A_102_120Z" typeName="BOOL"/>
		<variables name="A_102_140IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_102_140KS" typeName="BOOL"/>
		<variables name="A_102_140P_IND" typeName="BYTE"/>
		<variables name="A_102_140P_K" typeName="EBOOL"/>
		<variables name="A_102_140RI" typeName="BOOL"/>
		<variables name="A_102_140Z" typeName="BOOL"/>
		<variables name="A_102MAK_KN" typeName="BOOL"/>
		<variables name="A_102MAK_M_IM" typeName="BOOL"/>
		<variables name="A_102MAK_P_IM" typeName="BOOL"/>
		<variables name="A_102MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_102MK_K" typeName="EBOOL"/>
		<variables name="A_102MKStr" typeName="BOOL"/>
		<variables name="A_102MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_102MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_102MU_U" typeName="EBOOL"/>
		<variables name="A_102MU_U_OBR" typeName="BOOL"/>
		<variables name="A_102MU_U_PR" typeName="BOOL"/>
		<variables name="A_102MU_UK" typeName="EBOOL"/>
		<variables name="A_102NPS_U" typeName="EBOOL"/>
		<variables name="A_102OZ_K" typeName="EBOOL"/>
		<variables name="A_102P_IND" typeName="BYTE"/>
		<variables name="A_102P_K" typeName="EBOOL"/>
		<variables name="A_102PCHKM" typeName="BOOL"/>
		<variables name="A_102PK_K" typeName="EBOOL"/>
		<variables name="A_102PL_K" typeName="EBOOL"/>
		<variables name="A_102PL_U" typeName="EBOOL"/>
		<variables name="A_102PL_U_OBR" typeName="BOOL"/>
		<variables name="A_102PL_U_PR" typeName="BOOL"/>
		<variables name="A_102PL_UK" typeName="EBOOL"/>
		<variables name="A_102PNKM" typeName="BOOL"/>
		<variables name="A_102PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_102PU_U" typeName="EBOOL"/>
		<variables name="A_102PU_U_OBR" typeName="BOOL"/>
		<variables name="A_102PU_U_PR" typeName="BOOL"/>
		<variables name="A_102PU_UK" typeName="EBOOL"/>
		<variables name="A_102SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_102SP_MK_IND" typeName="BOOL"/>
		<variables name="A_102SP_PK_IND" typeName="BOOL"/>
		<variables name="A_102VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_102VK_U" typeName="EBOOL"/>
		<variables name="A_102VK_U_OBR" typeName="BOOL"/>
		<variables name="A_102VK_U_PR" typeName="BOOL"/>
		<variables name="A_102VK_UK" typeName="EBOOL"/>
		<variables name="A_102VZ" typeName="BOOL"/>
		<variables name="A_102VZREZ_IM" typeName="BOOL"/>
		<variables name="A_102ZSTR" typeName="BOOL"/>
		<variables name="A_102ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_103MAK_KN" typeName="BOOL"/>
		<variables name="A_103MAK_M_IM" typeName="BOOL"/>
		<variables name="A_103MAK_P_IM" typeName="BOOL"/>
		<variables name="A_103MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_103MK_K" typeName="EBOOL"/>
		<variables name="A_103MKStr" typeName="BOOL"/>
		<variables name="A_103MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_103MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_103MU_U" typeName="EBOOL"/>
		<variables name="A_103MU_U_OBR" typeName="BOOL"/>
		<variables name="A_103MU_U_PR" typeName="BOOL"/>
		<variables name="A_103MU_UK" typeName="EBOOL"/>
		<variables name="A_103NPS_U" typeName="EBOOL"/>
		<variables name="A_103OZ_K" typeName="EBOOL"/>
		<variables name="A_103P_IND" typeName="BYTE"/>
		<variables name="A_103P_K" typeName="EBOOL"/>
		<variables name="A_103PCHKM" typeName="BOOL"/>
		<variables name="A_103PK_K" typeName="EBOOL"/>
		<variables name="A_103PL_K" typeName="EBOOL"/>
		<variables name="A_103PL_U" typeName="EBOOL"/>
		<variables name="A_103PL_U_OBR" typeName="BOOL"/>
		<variables name="A_103PL_U_PR" typeName="BOOL"/>
		<variables name="A_103PL_UK" typeName="EBOOL"/>
		<variables name="A_103PNKM" typeName="BOOL"/>
		<variables name="A_103PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_103PU_U" typeName="EBOOL"/>
		<variables name="A_103PU_U_OBR" typeName="BOOL"/>
		<variables name="A_103PU_U_PR" typeName="BOOL"/>
		<variables name="A_103PU_UK" typeName="EBOOL"/>
		<variables name="A_103SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_103SP_MK_IND" typeName="BOOL"/>
		<variables name="A_103SP_PK_IND" typeName="BOOL"/>
		<variables name="A_103VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_103VK_U" typeName="EBOOL"/>
		<variables name="A_103VK_U_OBR" typeName="BOOL"/>
		<variables name="A_103VK_U_PR" typeName="BOOL"/>
		<variables name="A_103VK_UK" typeName="EBOOL"/>
		<variables name="A_103VZ" typeName="BOOL"/>
		<variables name="A_103VZREZ_IM" typeName="BOOL"/>
		<variables name="A_103ZSTR" typeName="BOOL"/>
		<variables name="A_103ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_104MAK_KN" typeName="BOOL"/>
		<variables name="A_104MAK_M_IM" typeName="BOOL"/>
		<variables name="A_104MAK_P_IM" typeName="BOOL"/>
		<variables name="A_104MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_104MK_K" typeName="EBOOL"/>
		<variables name="A_104MKStr" typeName="BOOL"/>
		<variables name="A_104MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_104MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_104MU_U" typeName="EBOOL"/>
		<variables name="A_104MU_U_OBR" typeName="BOOL"/>
		<variables name="A_104MU_U_PR" typeName="BOOL"/>
		<variables name="A_104MU_UK" typeName="EBOOL"/>
		<variables name="A_104NPS_U" typeName="EBOOL"/>
		<variables name="A_104OZ_K" typeName="EBOOL"/>
		<variables name="A_104P_IND" typeName="BYTE"/>
		<variables name="A_104P_K" typeName="EBOOL"/>
		<variables name="A_104PCHKM" typeName="BOOL"/>
		<variables name="A_104PK_K" typeName="EBOOL"/>
		<variables name="A_104PL_K" typeName="EBOOL"/>
		<variables name="A_104PL_U" typeName="EBOOL"/>
		<variables name="A_104PL_U_OBR" typeName="BOOL"/>
		<variables name="A_104PL_U_PR" typeName="BOOL"/>
		<variables name="A_104PL_UK" typeName="EBOOL"/>
		<variables name="A_104PNKM" typeName="BOOL"/>
		<variables name="A_104PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_104PU_U" typeName="EBOOL"/>
		<variables name="A_104PU_U_OBR" typeName="BOOL"/>
		<variables name="A_104PU_U_PR" typeName="BOOL"/>
		<variables name="A_104PU_UK" typeName="EBOOL"/>
		<variables name="A_104SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_104SP_MK_IND" typeName="BOOL"/>
		<variables name="A_104SP_PK_IND" typeName="BOOL"/>
		<variables name="A_104VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_104VK_U" typeName="EBOOL"/>
		<variables name="A_104VK_U_OBR" typeName="BOOL"/>
		<variables name="A_104VK_U_PR" typeName="BOOL"/>
		<variables name="A_104VK_UK" typeName="EBOOL"/>
		<variables name="A_104VZ" typeName="BOOL"/>
		<variables name="A_104VZREZ_IM" typeName="BOOL"/>
		<variables name="A_104ZSTR" typeName="BOOL"/>
		<variables name="A_104ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_105AIR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_105AKS" typeName="BOOL"/>
		<variables name="A_105AP_IND" typeName="BYTE"/>
		<variables name="A_105AP_K" typeName="EBOOL"/>
		<variables name="A_105ARI" typeName="BOOL"/>
		<variables name="A_105AZ" typeName="BOOL"/>
		<variables name="A_105IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_105MAK_KN" typeName="BOOL"/>
		<variables name="A_105MAK_M_IM" typeName="BOOL"/>
		<variables name="A_105MAK_P_IM" typeName="BOOL"/>
		<variables name="A_105MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_105MK_K" typeName="EBOOL"/>
		<variables name="A_105MKStr" typeName="BOOL"/>
		<variables name="A_105MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_105MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_105MU_U" typeName="EBOOL"/>
		<variables name="A_105MU_U_OBR" typeName="BOOL"/>
		<variables name="A_105MU_U_PR" typeName="BOOL"/>
		<variables name="A_105MU_UK" typeName="EBOOL"/>
		<variables name="A_105NPS_U" typeName="EBOOL"/>
		<variables name="A_105OZ_K" typeName="EBOOL"/>
		<variables name="A_105P_IND" typeName="BYTE"/>
		<variables name="A_105P_K" typeName="EBOOL"/>
		<variables name="A_105PCHKM" typeName="BOOL"/>
		<variables name="A_105PK_K" typeName="EBOOL"/>
		<variables name="A_105PL_K" typeName="EBOOL"/>
		<variables name="A_105PL_U" typeName="EBOOL"/>
		<variables name="A_105PL_U_OBR" typeName="BOOL"/>
		<variables name="A_105PL_U_PR" typeName="BOOL"/>
		<variables name="A_105PL_UK" typeName="EBOOL"/>
		<variables name="A_105PNKM" typeName="BOOL"/>
		<variables name="A_105PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_105PU_U" typeName="EBOOL"/>
		<variables name="A_105PU_U_OBR" typeName="BOOL"/>
		<variables name="A_105PU_U_PR" typeName="BOOL"/>
		<variables name="A_105PU_UK" typeName="EBOOL"/>
		<variables name="A_105RI" typeName="BOOL"/>
		<variables name="A_105SP_IND" typeName="BYTE"/>
		<variables name="A_105SP_K" typeName="EBOOL"/>
		<variables name="A_105SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_105SP_MK_IND" typeName="BOOL"/>
		<variables name="A_105SP_PK_IND" typeName="BOOL"/>
		<variables name="A_105VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_105VK_U" typeName="EBOOL"/>
		<variables name="A_105VK_U_OBR" typeName="BOOL"/>
		<variables name="A_105VK_U_PR" typeName="BOOL"/>
		<variables name="A_105VK_UK" typeName="EBOOL"/>
		<variables name="A_105VZ" typeName="BOOL"/>
		<variables name="A_105VZREZ_IM" typeName="BOOL"/>
		<variables name="A_105Z" typeName="BOOL"/>
		<variables name="A_105ZSTR" typeName="BOOL"/>
		<variables name="A_105ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_106_108IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_106_108RI" typeName="BOOL"/>
		<variables name="A_106_108SP_106STR_IND" typeName="BYTE"/>
		<variables name="A_106_108SP_108STR_IND" typeName="BYTE"/>
		<variables name="A_106_108SP_K" typeName="EBOOL"/>
		<variables name="A_106_108Z" typeName="BOOL"/>
		<variables name="A_106AIR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_106AKS" typeName="BOOL"/>
		<variables name="A_106AP_IND" typeName="BYTE"/>
		<variables name="A_106AP_K" typeName="EBOOL"/>
		<variables name="A_106ARI" typeName="BOOL"/>
		<variables name="A_106AZ" typeName="BOOL"/>
		<variables name="A_106MAK_KN" typeName="BOOL"/>
		<variables name="A_106MAK_M_IM" typeName="BOOL"/>
		<variables name="A_106MAK_P_IM" typeName="BOOL"/>
		<variables name="A_106MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_106MK_K" typeName="EBOOL"/>
		<variables name="A_106MKStr" typeName="BOOL"/>
		<variables name="A_106MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_106MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_106MU_U" typeName="EBOOL"/>
		<variables name="A_106MU_U_OBR" typeName="BOOL"/>
		<variables name="A_106MU_U_PR" typeName="BOOL"/>
		<variables name="A_106MU_UK" typeName="EBOOL"/>
		<variables name="A_106NPS_U" typeName="EBOOL"/>
		<variables name="A_106P_IND" typeName="BYTE"/>
		<variables name="A_106P_K" typeName="EBOOL"/>
		<variables name="A_106PCHKM" typeName="BOOL"/>
		<variables name="A_106PK_K" typeName="EBOOL"/>
		<variables name="A_106PNKM" typeName="BOOL"/>
		<variables name="A_106PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_106PU_U" typeName="EBOOL"/>
		<variables name="A_106PU_U_OBR" typeName="BOOL"/>
		<variables name="A_106PU_U_PR" typeName="BOOL"/>
		<variables name="A_106PU_UK" typeName="EBOOL"/>
		<variables name="A_106SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_106SP_MK_IND" typeName="BOOL"/>
		<variables name="A_106SP_PK_IND" typeName="BOOL"/>
		<variables name="A_106VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_106VK_U" typeName="EBOOL"/>
		<variables name="A_106VK_U_OBR" typeName="BOOL"/>
		<variables name="A_106VK_U_PR" typeName="BOOL"/>
		<variables name="A_106VK_UK" typeName="EBOOL"/>
		<variables name="A_106VZ" typeName="BOOL"/>
		<variables name="A_106VZREZ_IM" typeName="BOOL"/>
		<variables name="A_106ZSTR" typeName="BOOL"/>
		<variables name="A_106ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_107_121IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_107_121RI" typeName="BOOL"/>
		<variables name="A_107_121SP_107STR_IND" typeName="BYTE"/>
		<variables name="A_107_121SP_121STR_IND" typeName="BYTE"/>
		<variables name="A_107_121SP_K" typeName="EBOOL"/>
		<variables name="A_107_121Z" typeName="BOOL"/>
		<variables name="A_107MAK_KN" typeName="BOOL"/>
		<variables name="A_107MAK_M_IM" typeName="BOOL"/>
		<variables name="A_107MAK_P_IM" typeName="BOOL"/>
		<variables name="A_107MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_107MK_K" typeName="EBOOL"/>
		<variables name="A_107MKStr" typeName="BOOL"/>
		<variables name="A_107MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_107MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_107MU_U" typeName="EBOOL"/>
		<variables name="A_107MU_U_OBR" typeName="BOOL"/>
		<variables name="A_107MU_U_PR" typeName="BOOL"/>
		<variables name="A_107MU_UK" typeName="EBOOL"/>
		<variables name="A_107NPS_U" typeName="EBOOL"/>
		<variables name="A_107P_IND" typeName="BYTE"/>
		<variables name="A_107P_K" typeName="EBOOL"/>
		<variables name="A_107PCHKM" typeName="BOOL"/>
		<variables name="A_107PK_K" typeName="EBOOL"/>
		<variables name="A_107PNKM" typeName="BOOL"/>
		<variables name="A_107PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_107PU_U" typeName="EBOOL"/>
		<variables name="A_107PU_U_OBR" typeName="BOOL"/>
		<variables name="A_107PU_U_PR" typeName="BOOL"/>
		<variables name="A_107PU_UK" typeName="EBOOL"/>
		<variables name="A_107SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_107SP_MK_IND" typeName="BOOL"/>
		<variables name="A_107SP_PK_IND" typeName="BOOL"/>
		<variables name="A_107VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_107VK_U" typeName="EBOOL"/>
		<variables name="A_107VK_U_OBR" typeName="BOOL"/>
		<variables name="A_107VK_U_PR" typeName="BOOL"/>
		<variables name="A_107VK_UK" typeName="EBOOL"/>
		<variables name="A_107VZ" typeName="BOOL"/>
		<variables name="A_107VZREZ_IM" typeName="BOOL"/>
		<variables name="A_107ZSTR" typeName="BOOL"/>
		<variables name="A_107ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_108_108AOZ_K" typeName="EBOOL"/>
		<variables name="A_108_108APL_K" typeName="EBOOL"/>
		<variables name="A_108_108APL_U" typeName="EBOOL"/>
		<variables name="A_108_108APL_U_OBR" typeName="BOOL"/>
		<variables name="A_108_108APL_U_PR" typeName="BOOL"/>
		<variables name="A_108_108APL_UK" typeName="EBOOL"/>
		<variables name="A_108_109AOZ_K" typeName="EBOOL"/>
		<variables name="A_108_109APL_K" typeName="EBOOL"/>
		<variables name="A_108_109APL_U" typeName="EBOOL"/>
		<variables name="A_108_109APL_U_OBR" typeName="BOOL"/>
		<variables name="A_108_109APL_U_PR" typeName="BOOL"/>
		<variables name="A_108_109APL_UK" typeName="EBOOL"/>
		<variables name="A_108AIR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_108AKS" typeName="BOOL"/>
		<variables name="A_108AOZ_K" typeName="EBOOL"/>
		<variables name="A_108AP_IND" typeName="BYTE"/>
		<variables name="A_108AP_K" typeName="EBOOL"/>
		<variables name="A_108APL_K" typeName="EBOOL"/>
		<variables name="A_108APL_U" typeName="EBOOL"/>
		<variables name="A_108APL_U_OBR" typeName="BOOL"/>
		<variables name="A_108APL_U_PR" typeName="BOOL"/>
		<variables name="A_108APL_UK" typeName="EBOOL"/>
		<variables name="A_108ARI" typeName="BOOL"/>
		<variables name="A_108AZ" typeName="BOOL"/>
		<variables name="A_108MAK_KN" typeName="BOOL"/>
		<variables name="A_108MAK_M_IM" typeName="BOOL"/>
		<variables name="A_108MAK_P_IM" typeName="BOOL"/>
		<variables name="A_108MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_108MK_K" typeName="EBOOL"/>
		<variables name="A_108MKStr" typeName="BOOL"/>
		<variables name="A_108MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_108MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_108MU_U" typeName="EBOOL"/>
		<variables name="A_108MU_U_OBR" typeName="BOOL"/>
		<variables name="A_108MU_U_PR" typeName="BOOL"/>
		<variables name="A_108MU_UK" typeName="EBOOL"/>
		<variables name="A_108NPS_U" typeName="EBOOL"/>
		<variables name="A_108P_IND" typeName="BYTE"/>
		<variables name="A_108P_K" typeName="EBOOL"/>
		<variables name="A_108PCHKM" typeName="BOOL"/>
		<variables name="A_108PK_K" typeName="EBOOL"/>
		<variables name="A_108PNKM" typeName="BOOL"/>
		<variables name="A_108PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_108PU_U" typeName="EBOOL"/>
		<variables name="A_108PU_U_OBR" typeName="BOOL"/>
		<variables name="A_108PU_U_PR" typeName="BOOL"/>
		<variables name="A_108PU_UK" typeName="EBOOL"/>
		<variables name="A_108SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_108SP_MK_IND" typeName="BOOL"/>
		<variables name="A_108SP_PK_IND" typeName="BOOL"/>
		<variables name="A_108VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_108VK_U" typeName="EBOOL"/>
		<variables name="A_108VK_U_OBR" typeName="BOOL"/>
		<variables name="A_108VK_U_PR" typeName="BOOL"/>
		<variables name="A_108VK_UK" typeName="EBOOL"/>
		<variables name="A_108VZ" typeName="BOOL"/>
		<variables name="A_108VZREZ_IM" typeName="BOOL"/>
		<variables name="A_108ZSTR" typeName="BOOL"/>
		<variables name="A_108ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_109AIR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_109AKS" typeName="BOOL"/>
		<variables name="A_109AOZ_K" typeName="EBOOL"/>
		<variables name="A_109AP_IND" typeName="BYTE"/>
		<variables name="A_109AP_K" typeName="EBOOL"/>
		<variables name="A_109APL" typeName="EBOOL"/>
		<variables name="A_109APL_U" typeName="EBOOL"/>
		<variables name="A_109APL_U_OBR" typeName="BOOL"/>
		<variables name="A_109APL_U_PR" typeName="BOOL"/>
		<variables name="A_109APL_UK" typeName="EBOOL"/>
		<variables name="A_109ARI" typeName="BOOL"/>
		<variables name="A_109AZ" typeName="BOOL"/>
		<variables name="A_109IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_109MAK_KN" typeName="BOOL"/>
		<variables name="A_109MAK_M_IM" typeName="BOOL"/>
		<variables name="A_109MAK_P_IM" typeName="BOOL"/>
		<variables name="A_109MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_109MK_K" typeName="EBOOL"/>
		<variables name="A_109MKStr" typeName="BOOL"/>
		<variables name="A_109MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_109MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_109MU_U" typeName="EBOOL"/>
		<variables name="A_109MU_U_OBR" typeName="BOOL"/>
		<variables name="A_109MU_U_PR" typeName="BOOL"/>
		<variables name="A_109MU_UK" typeName="EBOOL"/>
		<variables name="A_109NPS_U" typeName="EBOOL"/>
		<variables name="A_109PK_K" typeName="EBOOL"/>
		<variables name="A_109PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_109PU_U" typeName="EBOOL"/>
		<variables name="A_109PU_U_OBR" typeName="BOOL"/>
		<variables name="A_109PU_U_PR" typeName="BOOL"/>
		<variables name="A_109PU_UK" typeName="EBOOL"/>
		<variables name="A_109RI" typeName="BOOL"/>
		<variables name="A_109SP_IND" typeName="BYTE"/>
		<variables name="A_109SP_K" typeName="EBOOL"/>
		<variables name="A_109SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_109SP_MK_IND" typeName="BOOL"/>
		<variables name="A_109SP_PK_IND" typeName="BOOL"/>
		<variables name="A_109VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_109VK_U" typeName="EBOOL"/>
		<variables name="A_109VK_U_OBR" typeName="BOOL"/>
		<variables name="A_109VK_U_PR" typeName="BOOL"/>
		<variables name="A_109VK_UK" typeName="EBOOL"/>
		<variables name="A_109VZ" typeName="BOOL"/>
		<variables name="A_109VZREZ_IM" typeName="BOOL"/>
		<variables name="A_109Z" typeName="BOOL"/>
		<variables name="A_109ZSTR" typeName="BOOL"/>
		<variables name="A_109ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_110_112IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_110_112RI" typeName="BOOL"/>
		<variables name="A_110_112SP_110STR_IND" typeName="BYTE"/>
		<variables name="A_110_112SP_112STR_IND" typeName="BYTE"/>
		<variables name="A_110_112SP_K" typeName="EBOOL"/>
		<variables name="A_110_112Z" typeName="BOOL"/>
		<variables name="A_110MAK_KN" typeName="BOOL"/>
		<variables name="A_110MAK_M_IM" typeName="BOOL"/>
		<variables name="A_110MAK_P_IM" typeName="BOOL"/>
		<variables name="A_110MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_110MK_K" typeName="EBOOL"/>
		<variables name="A_110MKStr" typeName="BOOL"/>
		<variables name="A_110MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_110MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_110MU_U" typeName="EBOOL"/>
		<variables name="A_110MU_U_OBR" typeName="BOOL"/>
		<variables name="A_110MU_U_PR" typeName="BOOL"/>
		<variables name="A_110MU_UK" typeName="EBOOL"/>
		<variables name="A_110NPS_U" typeName="EBOOL"/>
		<variables name="A_110PK_K" typeName="EBOOL"/>
		<variables name="A_110PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_110PU_U" typeName="EBOOL"/>
		<variables name="A_110PU_U_OBR" typeName="BOOL"/>
		<variables name="A_110PU_U_PR" typeName="BOOL"/>
		<variables name="A_110PU_UK" typeName="EBOOL"/>
		<variables name="A_110SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_110SP_MK_IND" typeName="BOOL"/>
		<variables name="A_110SP_PK_IND" typeName="BOOL"/>
		<variables name="A_110VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_110VK_U" typeName="EBOOL"/>
		<variables name="A_110VK_U_OBR" typeName="BOOL"/>
		<variables name="A_110VK_U_PR" typeName="BOOL"/>
		<variables name="A_110VK_UK" typeName="EBOOL"/>
		<variables name="A_110VZ" typeName="BOOL"/>
		<variables name="A_110VZREZ_IM" typeName="BOOL"/>
		<variables name="A_110ZSTR" typeName="BOOL"/>
		<variables name="A_110ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_111_113IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_111_113RI" typeName="BOOL"/>
		<variables name="A_111_113SP_111STR_IND" typeName="BYTE"/>
		<variables name="A_111_113SP_113STR_IND" typeName="BYTE"/>
		<variables name="A_111_113SP_K" typeName="EBOOL"/>
		<variables name="A_111_113Z" typeName="BOOL"/>
		<variables name="A_111MAK_KN" typeName="BOOL"/>
		<variables name="A_111MAK_M_IM" typeName="BOOL"/>
		<variables name="A_111MAK_P_IM" typeName="BOOL"/>
		<variables name="A_111MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_111MK_K" typeName="EBOOL"/>
		<variables name="A_111MKStr" typeName="BOOL"/>
		<variables name="A_111MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_111MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_111MU_U" typeName="EBOOL"/>
		<variables name="A_111MU_U_OBR" typeName="BOOL"/>
		<variables name="A_111MU_U_PR" typeName="BOOL"/>
		<variables name="A_111MU_UK" typeName="EBOOL"/>
		<variables name="A_111NPS_U" typeName="EBOOL"/>
		<variables name="A_111PK_K" typeName="EBOOL"/>
		<variables name="A_111PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_111PU_U" typeName="EBOOL"/>
		<variables name="A_111PU_U_OBR" typeName="BOOL"/>
		<variables name="A_111PU_U_PR" typeName="BOOL"/>
		<variables name="A_111PU_UK" typeName="EBOOL"/>
		<variables name="A_111SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_111SP_MK_IND" typeName="BOOL"/>
		<variables name="A_111SP_PK_IND" typeName="BOOL"/>
		<variables name="A_111VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_111VK_U" typeName="EBOOL"/>
		<variables name="A_111VK_U_OBR" typeName="BOOL"/>
		<variables name="A_111VK_U_PR" typeName="BOOL"/>
		<variables name="A_111VK_UK" typeName="EBOOL"/>
		<variables name="A_111VZ" typeName="BOOL"/>
		<variables name="A_111VZREZ_IM" typeName="BOOL"/>
		<variables name="A_111ZSTR" typeName="BOOL"/>
		<variables name="A_111ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_112MAK_KN" typeName="BOOL"/>
		<variables name="A_112MAK_M_IM" typeName="BOOL"/>
		<variables name="A_112MAK_P_IM" typeName="BOOL"/>
		<variables name="A_112MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_112MK_K" typeName="EBOOL"/>
		<variables name="A_112MKStr" typeName="BOOL"/>
		<variables name="A_112MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_112MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_112MU_U" typeName="EBOOL"/>
		<variables name="A_112MU_U_OBR" typeName="BOOL"/>
		<variables name="A_112MU_U_PR" typeName="BOOL"/>
		<variables name="A_112MU_UK" typeName="EBOOL"/>
		<variables name="A_112NPS_U" typeName="EBOOL"/>
		<variables name="A_112PK_K" typeName="EBOOL"/>
		<variables name="A_112PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_112PU_U" typeName="EBOOL"/>
		<variables name="A_112PU_U_OBR" typeName="BOOL"/>
		<variables name="A_112PU_U_PR" typeName="BOOL"/>
		<variables name="A_112PU_UK" typeName="EBOOL"/>
		<variables name="A_112SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_112SP_MK_IND" typeName="BOOL"/>
		<variables name="A_112SP_PK_IND" typeName="BOOL"/>
		<variables name="A_112VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_112VK_U" typeName="EBOOL"/>
		<variables name="A_112VK_U_OBR" typeName="BOOL"/>
		<variables name="A_112VK_U_PR" typeName="BOOL"/>
		<variables name="A_112VK_UK" typeName="EBOOL"/>
		<variables name="A_112VZ" typeName="BOOL"/>
		<variables name="A_112VZREZ_IM" typeName="BOOL"/>
		<variables name="A_112ZSTR" typeName="BOOL"/>
		<variables name="A_112ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_113MAK_KN" typeName="BOOL"/>
		<variables name="A_113MAK_M_IM" typeName="BOOL"/>
		<variables name="A_113MAK_P_IM" typeName="BOOL"/>
		<variables name="A_113MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_113MK_K" typeName="EBOOL"/>
		<variables name="A_113MKStr" typeName="BOOL"/>
		<variables name="A_113MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_113MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_113MU_U" typeName="EBOOL"/>
		<variables name="A_113MU_U_OBR" typeName="BOOL"/>
		<variables name="A_113MU_U_PR" typeName="BOOL"/>
		<variables name="A_113MU_UK" typeName="EBOOL"/>
		<variables name="A_113NPS_U" typeName="EBOOL"/>
		<variables name="A_113PK_K" typeName="EBOOL"/>
		<variables name="A_113PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_113PU_U" typeName="EBOOL"/>
		<variables name="A_113PU_U_OBR" typeName="BOOL"/>
		<variables name="A_113PU_U_PR" typeName="BOOL"/>
		<variables name="A_113PU_UK" typeName="EBOOL"/>
		<variables name="A_113SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_113SP_MK_IND" typeName="BOOL"/>
		<variables name="A_113SP_PK_IND" typeName="BOOL"/>
		<variables name="A_113VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_113VK_U" typeName="EBOOL"/>
		<variables name="A_113VK_U_OBR" typeName="BOOL"/>
		<variables name="A_113VK_U_PR" typeName="BOOL"/>
		<variables name="A_113VK_UK" typeName="EBOOL"/>
		<variables name="A_113VZ" typeName="BOOL"/>
		<variables name="A_113VZREZ_IM" typeName="BOOL"/>
		<variables name="A_113ZSTR" typeName="BOOL"/>
		<variables name="A_113ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_114IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_114MAK_KN" typeName="BOOL"/>
		<variables name="A_114MAK_M_IM" typeName="BOOL"/>
		<variables name="A_114MAK_P_IM" typeName="BOOL"/>
		<variables name="A_114MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_114MK_K" typeName="EBOOL"/>
		<variables name="A_114MKStr" typeName="BOOL"/>
		<variables name="A_114MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_114MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_114MU_U" typeName="EBOOL"/>
		<variables name="A_114MU_U_OBR" typeName="BOOL"/>
		<variables name="A_114MU_U_PR" typeName="BOOL"/>
		<variables name="A_114MU_UK" typeName="EBOOL"/>
		<variables name="A_114NPS_U" typeName="EBOOL"/>
		<variables name="A_114PK_K" typeName="EBOOL"/>
		<variables name="A_114PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_114PU_U" typeName="EBOOL"/>
		<variables name="A_114PU_U_OBR" typeName="BOOL"/>
		<variables name="A_114PU_U_PR" typeName="BOOL"/>
		<variables name="A_114PU_UK" typeName="EBOOL"/>
		<variables name="A_114RI" typeName="BOOL"/>
		<variables name="A_114SP_IND" typeName="BYTE"/>
		<variables name="A_114SP_K" typeName="EBOOL"/>
		<variables name="A_114SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_114SP_MK_IND" typeName="BOOL"/>
		<variables name="A_114SP_PK_IND" typeName="BOOL"/>
		<variables name="A_114VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_114VK_U" typeName="EBOOL"/>
		<variables name="A_114VK_U_OBR" typeName="BOOL"/>
		<variables name="A_114VK_U_PR" typeName="BOOL"/>
		<variables name="A_114VK_UK" typeName="EBOOL"/>
		<variables name="A_114VZ" typeName="BOOL"/>
		<variables name="A_114VZREZ_IM" typeName="BOOL"/>
		<variables name="A_114Z" typeName="BOOL"/>
		<variables name="A_114ZSTR" typeName="BOOL"/>
		<variables name="A_114ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_115IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_115MAK_KN" typeName="BOOL"/>
		<variables name="A_115MAK_M_IM" typeName="BOOL"/>
		<variables name="A_115MAK_P_IM" typeName="BOOL"/>
		<variables name="A_115MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_115MK_K" typeName="EBOOL"/>
		<variables name="A_115MKStr" typeName="BOOL"/>
		<variables name="A_115MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_115MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_115MU_U" typeName="EBOOL"/>
		<variables name="A_115MU_U_OBR" typeName="BOOL"/>
		<variables name="A_115MU_U_PR" typeName="BOOL"/>
		<variables name="A_115MU_UK" typeName="EBOOL"/>
		<variables name="A_115NPS_U" typeName="EBOOL"/>
		<variables name="A_115PK_K" typeName="EBOOL"/>
		<variables name="A_115PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_115PU_U" typeName="EBOOL"/>
		<variables name="A_115PU_U_OBR" typeName="BOOL"/>
		<variables name="A_115PU_U_PR" typeName="BOOL"/>
		<variables name="A_115PU_UK" typeName="EBOOL"/>
		<variables name="A_115RI" typeName="BOOL"/>
		<variables name="A_115SP_IND" typeName="BYTE"/>
		<variables name="A_115SP_K" typeName="EBOOL"/>
		<variables name="A_115SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_115SP_MK_IND" typeName="BOOL"/>
		<variables name="A_115SP_PK_IND" typeName="BOOL"/>
		<variables name="A_115VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_115VK_U" typeName="EBOOL"/>
		<variables name="A_115VK_U_OBR" typeName="BOOL"/>
		<variables name="A_115VK_U_PR" typeName="BOOL"/>
		<variables name="A_115VK_UK" typeName="EBOOL"/>
		<variables name="A_115VZ" typeName="BOOL"/>
		<variables name="A_115VZREZ_IM" typeName="BOOL"/>
		<variables name="A_115Z" typeName="BOOL"/>
		<variables name="A_115ZSTR" typeName="BOOL"/>
		<variables name="A_115ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_116_124IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_116_124KS" typeName="BOOL"/>
		<variables name="A_116_124P_IND" typeName="BYTE"/>
		<variables name="A_116_124P_K" typeName="EBOOL"/>
		<variables name="A_116_124RI" typeName="BOOL"/>
		<variables name="A_116_124Z" typeName="BOOL"/>
		<variables name="A_116IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_116MAK_KN" typeName="BOOL"/>
		<variables name="A_116MAK_M_IM" typeName="BOOL"/>
		<variables name="A_116MAK_P_IM" typeName="BOOL"/>
		<variables name="A_116MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_116MK_K" typeName="EBOOL"/>
		<variables name="A_116MKStr" typeName="BOOL"/>
		<variables name="A_116MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_116MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_116MU_U" typeName="EBOOL"/>
		<variables name="A_116MU_U_OBR" typeName="BOOL"/>
		<variables name="A_116MU_U_PR" typeName="BOOL"/>
		<variables name="A_116MU_UK" typeName="EBOOL"/>
		<variables name="A_116NPS_U" typeName="EBOOL"/>
		<variables name="A_116PK_K" typeName="EBOOL"/>
		<variables name="A_116PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_116PU_U" typeName="EBOOL"/>
		<variables name="A_116PU_U_OBR" typeName="BOOL"/>
		<variables name="A_116PU_U_PR" typeName="BOOL"/>
		<variables name="A_116PU_UK" typeName="EBOOL"/>
		<variables name="A_116RI" typeName="BOOL"/>
		<variables name="A_116SP_IND" typeName="BYTE"/>
		<variables name="A_116SP_K" typeName="EBOOL"/>
		<variables name="A_116SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_116SP_MK_IND" typeName="BOOL"/>
		<variables name="A_116SP_PK_IND" typeName="BOOL"/>
		<variables name="A_116VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_116VK_U" typeName="EBOOL"/>
		<variables name="A_116VK_U_OBR" typeName="BOOL"/>
		<variables name="A_116VK_U_PR" typeName="BOOL"/>
		<variables name="A_116VK_UK" typeName="EBOOL"/>
		<variables name="A_116VZ" typeName="BOOL"/>
		<variables name="A_116VZREZ_IM" typeName="BOOL"/>
		<variables name="A_116Z" typeName="BOOL"/>
		<variables name="A_116ZSTR" typeName="BOOL"/>
		<variables name="A_116ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_117_119MAK_KN" typeName="BOOL"/>
		<variables name="A_117_119MAK_M_IM" typeName="BOOL"/>
		<variables name="A_117_119MAK_P_IM" typeName="BOOL"/>
		<variables name="A_117_119MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_117_119MK_K" typeName="EBOOL"/>
		<variables name="A_117_119MKSTR" typeName="BOOL"/>
		<variables name="A_117_119MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_117_119MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_117_119MU_U" typeName="EBOOL"/>
		<variables name="A_117_119MU_U_OBR" typeName="BOOL"/>
		<variables name="A_117_119MU_U_PR" typeName="BOOL"/>
		<variables name="A_117_119MU_UK" typeName="EBOOL"/>
		<variables name="A_117_119NPS_U" typeName="EBOOL"/>
		<variables name="A_117_119PK_K" typeName="EBOOL"/>
		<variables name="A_117_119PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_117_119PU_U" typeName="EBOOL"/>
		<variables name="A_117_119PU_U_OBR" typeName="BOOL"/>
		<variables name="A_117_119PU_U_PR" typeName="BOOL"/>
		<variables name="A_117_119PU_UK" typeName="EBOOL"/>
		<variables name="A_117_119VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_117_119VK_U" typeName="EBOOL"/>
		<variables name="A_117_119VK_U_OBR" typeName="BOOL"/>
		<variables name="A_117_119VK_U_PR" typeName="BOOL"/>
		<variables name="A_117_119VK_UK" typeName="EBOOL"/>
		<variables name="A_117_119VZ" typeName="BOOL"/>
		<variables name="A_117_119VZREZ_IM" typeName="BOOL"/>
		<variables name="A_117_119ZSTR" typeName="BOOL"/>
		<variables name="A_117_119ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_117AIR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_117AKS" typeName="BOOL"/>
		<variables name="A_117AP_IND" typeName="BYTE"/>
		<variables name="A_117AP_K" typeName="EBOOL"/>
		<variables name="A_117ARI" typeName="BOOL"/>
		<variables name="A_117AZ" typeName="BOOL"/>
		<variables name="A_117BIR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_117BKS" typeName="BOOL"/>
		<variables name="A_117BP_IND" typeName="BYTE"/>
		<variables name="A_117BP_K" typeName="EBOOL"/>
		<variables name="A_117BRI" typeName="BOOL"/>
		<variables name="A_117BZ" typeName="BOOL"/>
		<variables name="A_117IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_117P_IND" typeName="BYTE"/>
		<variables name="A_117P_K" typeName="EBOOL"/>
		<variables name="A_117PIR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_117PKS" typeName="BOOL"/>
		<variables name="A_117PRI" typeName="BOOL"/>
		<variables name="A_117PZ" typeName="BOOL"/>
		<variables name="A_117RI" typeName="BOOL"/>
		<variables name="A_117SP_IND" typeName="BYTE"/>
		<variables name="A_117SP_K" typeName="EBOOL"/>
		<variables name="A_117SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_117SP_MK_IND" typeName="BOOL"/>
		<variables name="A_117SP_PK_IND" typeName="BOOL"/>
		<variables name="A_117VZREZ_IM" typeName="BOOL"/>
		<variables name="A_117Z" typeName="BOOL"/>
		<variables name="A_119_121IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_119_121KS" typeName="BOOL"/>
		<variables name="A_119_121P_IND" typeName="BYTE"/>
		<variables name="A_119_121P_K" typeName="EBOOL"/>
		<variables name="A_119_121RI" typeName="BOOL"/>
		<variables name="A_119_121Z" typeName="BOOL"/>
		<variables name="A_119IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_119RI" typeName="BOOL"/>
		<variables name="A_119SP_IND" typeName="BYTE"/>
		<variables name="A_119SP_K" typeName="EBOOL"/>
		<variables name="A_119SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_119SP_MK_IND" typeName="BOOL"/>
		<variables name="A_119SP_PK_IND" typeName="BOOL"/>
		<variables name="A_119VZREZ_IM" typeName="BOOL"/>
		<variables name="A_119Z" typeName="BOOL"/>
		<variables name="A_120MAK_KN" typeName="BOOL"/>
		<variables name="A_120MAK_M_IM" typeName="BOOL"/>
		<variables name="A_120MAK_P_IM" typeName="BOOL"/>
		<variables name="A_120MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_120MK_K" typeName="EBOOL"/>
		<variables name="A_120MKStr" typeName="BOOL"/>
		<variables name="A_120MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_120MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_120MU_U" typeName="EBOOL"/>
		<variables name="A_120MU_U_OBR" typeName="BOOL"/>
		<variables name="A_120MU_U_PR" typeName="BOOL"/>
		<variables name="A_120MU_UK" typeName="EBOOL"/>
		<variables name="A_120NPS_U" typeName="EBOOL"/>
		<variables name="A_120PK_K" typeName="EBOOL"/>
		<variables name="A_120PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_120PU_U" typeName="EBOOL"/>
		<variables name="A_120PU_U_OBR" typeName="BOOL"/>
		<variables name="A_120PU_U_PR" typeName="BOOL"/>
		<variables name="A_120PU_UK" typeName="EBOOL"/>
		<variables name="A_120SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_120SP_MK_IND" typeName="BOOL"/>
		<variables name="A_120SP_PK_IND" typeName="BOOL"/>
		<variables name="A_120VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_120VK_U" typeName="EBOOL"/>
		<variables name="A_120VK_U_OBR" typeName="BOOL"/>
		<variables name="A_120VK_U_PR" typeName="BOOL"/>
		<variables name="A_120VK_UK" typeName="EBOOL"/>
		<variables name="A_120VZ" typeName="BOOL"/>
		<variables name="A_120VZREZ_IM" typeName="BOOL"/>
		<variables name="A_120ZSTR" typeName="BOOL"/>
		<variables name="A_120ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_121MAK_KN" typeName="BOOL"/>
		<variables name="A_121MAK_M_IM" typeName="BOOL"/>
		<variables name="A_121MAK_P_IM" typeName="BOOL"/>
		<variables name="A_121MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_121MK_K" typeName="EBOOL"/>
		<variables name="A_121MKStr" typeName="BOOL"/>
		<variables name="A_121MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_121MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_121MU_U" typeName="EBOOL"/>
		<variables name="A_121MU_U_OBR" typeName="BOOL"/>
		<variables name="A_121MU_U_PR" typeName="BOOL"/>
		<variables name="A_121MU_UK" typeName="EBOOL"/>
		<variables name="A_121NPS_U" typeName="EBOOL"/>
		<variables name="A_121PK_K" typeName="EBOOL"/>
		<variables name="A_121PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_121PU_U" typeName="EBOOL"/>
		<variables name="A_121PU_U_OBR" typeName="BOOL"/>
		<variables name="A_121PU_U_PR" typeName="BOOL"/>
		<variables name="A_121PU_UK" typeName="EBOOL"/>
		<variables name="A_121SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_121SP_MK_IND" typeName="BOOL"/>
		<variables name="A_121SP_PK_IND" typeName="BOOL"/>
		<variables name="A_121VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_121VK_U" typeName="EBOOL"/>
		<variables name="A_121VK_U_OBR" typeName="BOOL"/>
		<variables name="A_121VK_U_PR" typeName="BOOL"/>
		<variables name="A_121VK_UK" typeName="EBOOL"/>
		<variables name="A_121VZ" typeName="BOOL"/>
		<variables name="A_121VZREZ_IM" typeName="BOOL"/>
		<variables name="A_121ZSTR" typeName="BOOL"/>
		<variables name="A_121ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_122_160IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_122_160KS" typeName="BOOL"/>
		<variables name="A_122_160P_IND" typeName="BYTE"/>
		<variables name="A_122_160P_K" typeName="EBOOL"/>
		<variables name="A_122_160RI" typeName="BOOL"/>
		<variables name="A_122_160Z" typeName="BOOL"/>
		<variables name="A_122IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_122MAK_KN" typeName="BOOL"/>
		<variables name="A_122MAK_M_IM" typeName="BOOL"/>
		<variables name="A_122MAK_P_IM" typeName="BOOL"/>
		<variables name="A_122MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_122MK_K" typeName="EBOOL"/>
		<variables name="A_122MKStr" typeName="BOOL"/>
		<variables name="A_122MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_122MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_122MU_U" typeName="EBOOL"/>
		<variables name="A_122MU_U_OBR" typeName="BOOL"/>
		<variables name="A_122MU_U_PR" typeName="BOOL"/>
		<variables name="A_122MU_UK" typeName="EBOOL"/>
		<variables name="A_122NPS_U" typeName="EBOOL"/>
		<variables name="A_122PK_K" typeName="EBOOL"/>
		<variables name="A_122PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_122PU_U" typeName="EBOOL"/>
		<variables name="A_122PU_U_OBR" typeName="BOOL"/>
		<variables name="A_122PU_U_PR" typeName="BOOL"/>
		<variables name="A_122PU_UK" typeName="EBOOL"/>
		<variables name="A_122RI" typeName="BOOL"/>
		<variables name="A_122SP_IND" typeName="BYTE"/>
		<variables name="A_122SP_K" typeName="EBOOL"/>
		<variables name="A_122SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_122SP_MK_IND" typeName="BOOL"/>
		<variables name="A_122SP_PK_IND" typeName="BOOL"/>
		<variables name="A_122VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_122VK_U" typeName="EBOOL"/>
		<variables name="A_122VK_U_OBR" typeName="BOOL"/>
		<variables name="A_122VK_U_PR" typeName="BOOL"/>
		<variables name="A_122VK_UK" typeName="EBOOL"/>
		<variables name="A_122VZ" typeName="BOOL"/>
		<variables name="A_122VZREZ_IM" typeName="BOOL"/>
		<variables name="A_122Z" typeName="BOOL"/>
		<variables name="A_122ZSTR" typeName="BOOL"/>
		<variables name="A_122ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_124_138IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_124_138RI" typeName="BOOL"/>
		<variables name="A_124_138SP_124STR_IND" typeName="BYTE"/>
		<variables name="A_124_138SP_138STR_IND" typeName="BYTE"/>
		<variables name="A_124_138SP_K" typeName="EBOOL"/>
		<variables name="A_124_138Z" typeName="BOOL"/>
		<variables name="A_124MAK_KN" typeName="BOOL"/>
		<variables name="A_124MAK_M_IM" typeName="BOOL"/>
		<variables name="A_124MAK_P_IM" typeName="BOOL"/>
		<variables name="A_124MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_124MK_K" typeName="EBOOL"/>
		<variables name="A_124MKStr" typeName="BOOL"/>
		<variables name="A_124MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_124MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_124MU_U" typeName="EBOOL"/>
		<variables name="A_124MU_U_OBR" typeName="BOOL"/>
		<variables name="A_124MU_U_PR" typeName="BOOL"/>
		<variables name="A_124MU_UK" typeName="EBOOL"/>
		<variables name="A_124NPS_U" typeName="EBOOL"/>
		<variables name="A_124PK_K" typeName="EBOOL"/>
		<variables name="A_124PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_124PU_U" typeName="EBOOL"/>
		<variables name="A_124PU_U_OBR" typeName="BOOL"/>
		<variables name="A_124PU_U_PR" typeName="BOOL"/>
		<variables name="A_124PU_UK" typeName="EBOOL"/>
		<variables name="A_124SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_124SP_MK_IND" typeName="BOOL"/>
		<variables name="A_124SP_PK_IND" typeName="BOOL"/>
		<variables name="A_124VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_124VK_U" typeName="EBOOL"/>
		<variables name="A_124VK_U_OBR" typeName="BOOL"/>
		<variables name="A_124VK_U_PR" typeName="BOOL"/>
		<variables name="A_124VK_UK" typeName="EBOOL"/>
		<variables name="A_124VZ" typeName="BOOL"/>
		<variables name="A_124VZREZ_IM" typeName="BOOL"/>
		<variables name="A_124ZSTR" typeName="BOOL"/>
		<variables name="A_124ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_128_134IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_128_134RI" typeName="BOOL"/>
		<variables name="A_128_134SP_128STR_IND" typeName="BYTE"/>
		<variables name="A_128_134SP_134STR_IND" typeName="BYTE"/>
		<variables name="A_128_134SP_K" typeName="EBOOL"/>
		<variables name="A_128_134Z" typeName="BOOL"/>
		<variables name="A_128KS" typeName="BOOL"/>
		<variables name="A_128MAK_KN" typeName="BOOL"/>
		<variables name="A_128MAK_M_IM" typeName="BOOL"/>
		<variables name="A_128MAK_P_IM" typeName="BOOL"/>
		<variables name="A_128MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_128MK_K" typeName="EBOOL"/>
		<variables name="A_128MKStr" typeName="BOOL"/>
		<variables name="A_128MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_128MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_128MU_U" typeName="EBOOL"/>
		<variables name="A_128MU_U_OBR" typeName="BOOL"/>
		<variables name="A_128MU_U_PR" typeName="BOOL"/>
		<variables name="A_128MU_UK" typeName="EBOOL"/>
		<variables name="A_128NPS_U" typeName="EBOOL"/>
		<variables name="A_128P_K" typeName="EBOOL"/>
		<variables name="A_128PK_K" typeName="EBOOL"/>
		<variables name="A_128PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_128PU_U" typeName="EBOOL"/>
		<variables name="A_128PU_U_OBR" typeName="BOOL"/>
		<variables name="A_128PU_U_PR" typeName="BOOL"/>
		<variables name="A_128PU_UK" typeName="EBOOL"/>
		<variables name="A_128RI" typeName="BOOL"/>
		<variables name="A_128SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_128SP_MK_IND" typeName="BOOL"/>
		<variables name="A_128SP_PK_IND" typeName="BOOL"/>
		<variables name="A_128VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_128VK_U" typeName="EBOOL"/>
		<variables name="A_128VK_U_OBR" typeName="BOOL"/>
		<variables name="A_128VK_U_PR" typeName="BOOL"/>
		<variables name="A_128VK_UK" typeName="EBOOL"/>
		<variables name="A_128VZ" typeName="BOOL"/>
		<variables name="A_128VZREZ_IM" typeName="BOOL"/>
		<variables name="A_128Z" typeName="BOOL"/>
		<variables name="A_128ZSTR" typeName="BOOL"/>
		<variables name="A_128ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_130_164IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_130_164RI" typeName="BOOL"/>
		<variables name="A_130_164SP_130STR_IND" typeName="BYTE"/>
		<variables name="A_130_164SP_132STR_IND" typeName="BYTE"/>
		<variables name="A_130_164SP_164STR_IND" typeName="BYTE"/>
		<variables name="A_130_164SP_K" typeName="EBOOL"/>
		<variables name="A_130_164Z" typeName="BOOL"/>
		<variables name="A_130MAK_KN" typeName="BOOL"/>
		<variables name="A_130MAK_M_IM" typeName="BOOL"/>
		<variables name="A_130MAK_P_IM" typeName="BOOL"/>
		<variables name="A_130MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_130MK_K" typeName="EBOOL"/>
		<variables name="A_130MKStr" typeName="BOOL"/>
		<variables name="A_130MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_130MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_130MU_U" typeName="EBOOL"/>
		<variables name="A_130MU_U_OBR" typeName="BOOL"/>
		<variables name="A_130MU_U_PR" typeName="BOOL"/>
		<variables name="A_130MU_UK" typeName="EBOOL"/>
		<variables name="A_130NPS_U" typeName="EBOOL"/>
		<variables name="A_130PK_K" typeName="EBOOL"/>
		<variables name="A_130PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_130PU_U" typeName="EBOOL"/>
		<variables name="A_130PU_U_OBR" typeName="BOOL"/>
		<variables name="A_130PU_U_PR" typeName="BOOL"/>
		<variables name="A_130PU_UK" typeName="EBOOL"/>
		<variables name="A_130SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_130SP_MK_IND" typeName="BOOL"/>
		<variables name="A_130SP_PK_IND" typeName="BOOL"/>
		<variables name="A_130VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_130VK_U" typeName="EBOOL"/>
		<variables name="A_130VK_U_OBR" typeName="BOOL"/>
		<variables name="A_130VK_U_PR" typeName="BOOL"/>
		<variables name="A_130VK_UK" typeName="EBOOL"/>
		<variables name="A_130VZ" typeName="BOOL"/>
		<variables name="A_130VZREZ_IM" typeName="BOOL"/>
		<variables name="A_130ZSTR" typeName="BOOL"/>
		<variables name="A_130ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_132MAK_KN" typeName="BOOL"/>
		<variables name="A_132MAK_M_IM" typeName="BOOL"/>
		<variables name="A_132MAK_P_IM" typeName="BOOL"/>
		<variables name="A_132MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_132MK_K" typeName="EBOOL"/>
		<variables name="A_132MKStr" typeName="BOOL"/>
		<variables name="A_132MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_132MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_132MU_U" typeName="EBOOL"/>
		<variables name="A_132MU_U_OBR" typeName="BOOL"/>
		<variables name="A_132MU_U_PR" typeName="BOOL"/>
		<variables name="A_132MU_UK" typeName="EBOOL"/>
		<variables name="A_132NPS_U" typeName="EBOOL"/>
		<variables name="A_132PK_K" typeName="EBOOL"/>
		<variables name="A_132PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_132PU_U" typeName="EBOOL"/>
		<variables name="A_132PU_U_OBR" typeName="BOOL"/>
		<variables name="A_132PU_U_PR" typeName="BOOL"/>
		<variables name="A_132PU_UK" typeName="EBOOL"/>
		<variables name="A_132SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_132SP_MK_IND" typeName="BOOL"/>
		<variables name="A_132SP_PK_IND" typeName="BOOL"/>
		<variables name="A_132VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_132VK_U" typeName="EBOOL"/>
		<variables name="A_132VK_U_OBR" typeName="BOOL"/>
		<variables name="A_132VK_U_PR" typeName="BOOL"/>
		<variables name="A_132VK_UK" typeName="EBOOL"/>
		<variables name="A_132VZ" typeName="BOOL"/>
		<variables name="A_132VZREZ_IM" typeName="BOOL"/>
		<variables name="A_132ZSTR" typeName="BOOL"/>
		<variables name="A_132ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_134MAK_KN" typeName="BOOL"/>
		<variables name="A_134MAK_M_IM" typeName="BOOL"/>
		<variables name="A_134MAK_P_IM" typeName="BOOL"/>
		<variables name="A_134MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_134MK_K" typeName="EBOOL"/>
		<variables name="A_134MKStr" typeName="BOOL"/>
		<variables name="A_134MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_134MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_134MU_U" typeName="EBOOL"/>
		<variables name="A_134MU_U_OBR" typeName="BOOL"/>
		<variables name="A_134MU_U_PR" typeName="BOOL"/>
		<variables name="A_134MU_UK" typeName="EBOOL"/>
		<variables name="A_134NPS_U" typeName="EBOOL"/>
		<variables name="A_134PK_K" typeName="EBOOL"/>
		<variables name="A_134PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_134PU_U" typeName="EBOOL"/>
		<variables name="A_134PU_U_OBR" typeName="BOOL"/>
		<variables name="A_134PU_U_PR" typeName="BOOL"/>
		<variables name="A_134PU_UK" typeName="EBOOL"/>
		<variables name="A_134SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_134SP_MK_IND" typeName="BOOL"/>
		<variables name="A_134SP_PK_IND" typeName="BOOL"/>
		<variables name="A_134VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_134VK_U" typeName="EBOOL"/>
		<variables name="A_134VK_U_OBR" typeName="BOOL"/>
		<variables name="A_134VK_U_PR" typeName="BOOL"/>
		<variables name="A_134VK_UK" typeName="EBOOL"/>
		<variables name="A_134VZ" typeName="BOOL"/>
		<variables name="A_134VZREZ_IM" typeName="BOOL"/>
		<variables name="A_134ZSTR" typeName="BOOL"/>
		<variables name="A_134ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_136IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_136MAK_KN" typeName="BOOL"/>
		<variables name="A_136MAK_M_IM" typeName="BOOL"/>
		<variables name="A_136MAK_P_IM" typeName="BOOL"/>
		<variables name="A_136MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_136MK_K" typeName="EBOOL"/>
		<variables name="A_136MKStr" typeName="BOOL"/>
		<variables name="A_136MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_136MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_136MU_U" typeName="EBOOL"/>
		<variables name="A_136MU_U_OBR" typeName="BOOL"/>
		<variables name="A_136MU_U_PR" typeName="BOOL"/>
		<variables name="A_136MU_UK" typeName="EBOOL"/>
		<variables name="A_136NPS_U" typeName="EBOOL"/>
		<variables name="A_136PK_K" typeName="EBOOL"/>
		<variables name="A_136PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_136PU_U" typeName="EBOOL"/>
		<variables name="A_136PU_U_OBR" typeName="BOOL"/>
		<variables name="A_136PU_U_PR" typeName="BOOL"/>
		<variables name="A_136PU_UK" typeName="EBOOL"/>
		<variables name="A_136RI" typeName="BOOL"/>
		<variables name="A_136SP_IND" typeName="BYTE"/>
		<variables name="A_136SP_K" typeName="EBOOL"/>
		<variables name="A_136SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_136SP_MK_IND" typeName="BOOL"/>
		<variables name="A_136SP_PK_IND" typeName="BOOL"/>
		<variables name="A_136VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_136VK_U" typeName="EBOOL"/>
		<variables name="A_136VK_U_OBR" typeName="BOOL"/>
		<variables name="A_136VK_U_PR" typeName="BOOL"/>
		<variables name="A_136VK_UK" typeName="EBOOL"/>
		<variables name="A_136VZ" typeName="BOOL"/>
		<variables name="A_136VZREZ_IM" typeName="BOOL"/>
		<variables name="A_136Z" typeName="BOOL"/>
		<variables name="A_136ZSTR" typeName="BOOL"/>
		<variables name="A_136ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_138_140MAK_KN" typeName="BOOL"/>
		<variables name="A_138_140MAK_M_IM" typeName="BOOL"/>
		<variables name="A_138_140MAK_P_IM" typeName="BOOL"/>
		<variables name="A_138_140MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_138_140MK_K" typeName="EBOOL"/>
		<variables name="A_138_140MKStr" typeName="BOOL"/>
		<variables name="A_138_140MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_138_140MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_138_140MU_U" typeName="EBOOL"/>
		<variables name="A_138_140MU_U_OBR" typeName="BOOL"/>
		<variables name="A_138_140MU_U_PR" typeName="BOOL"/>
		<variables name="A_138_140MU_UK" typeName="EBOOL"/>
		<variables name="A_138_140NPS_U" typeName="EBOOL"/>
		<variables name="A_138_140PK_K" typeName="EBOOL"/>
		<variables name="A_138_140PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_138_140PU_U" typeName="EBOOL"/>
		<variables name="A_138_140PU_U_OBR" typeName="BOOL"/>
		<variables name="A_138_140PU_U_PR" typeName="BOOL"/>
		<variables name="A_138_140PU_UK" typeName="EBOOL"/>
		<variables name="A_138_140VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_138_140VK_U" typeName="EBOOL"/>
		<variables name="A_138_140VK_U_OBR" typeName="BOOL"/>
		<variables name="A_138_140VK_U_PR" typeName="BOOL"/>
		<variables name="A_138_140VK_UK" typeName="EBOOL"/>
		<variables name="A_138_140VZ" typeName="BOOL"/>
		<variables name="A_138_140VZREZ_IM" typeName="BOOL"/>
		<variables name="A_138_140ZSTR" typeName="BOOL"/>
		<variables name="A_138_140ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_138SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_138SP_MK_IND" typeName="BOOL"/>
		<variables name="A_138SP_PK_IND" typeName="BOOL"/>
		<variables name="A_138VZREZ_IM" typeName="BOOL"/>
		<variables name="A_140IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_140RI" typeName="BOOL"/>
		<variables name="A_140SP_IND" typeName="BYTE"/>
		<variables name="A_140SP_K" typeName="EBOOL"/>
		<variables name="A_140SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_140SP_MK_IND" typeName="BOOL"/>
		<variables name="A_140SP_PK_IND" typeName="BOOL"/>
		<variables name="A_140VZREZ_IM" typeName="BOOL"/>
		<variables name="A_140Z" typeName="BOOL"/>
		<variables name="A_142IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_142MAK_KN" typeName="BOOL"/>
		<variables name="A_142MAK_M_IM" typeName="BOOL"/>
		<variables name="A_142MAK_P_IM" typeName="BOOL"/>
		<variables name="A_142MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_142MK_K" typeName="EBOOL"/>
		<variables name="A_142MKStr" typeName="BOOL"/>
		<variables name="A_142MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_142MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_142MU_U" typeName="EBOOL"/>
		<variables name="A_142MU_U_OBR" typeName="BOOL"/>
		<variables name="A_142MU_U_PR" typeName="BOOL"/>
		<variables name="A_142MU_UK" typeName="EBOOL"/>
		<variables name="A_142NPS_U" typeName="EBOOL"/>
		<variables name="A_142PK_K" typeName="EBOOL"/>
		<variables name="A_142PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_142PU_U" typeName="EBOOL"/>
		<variables name="A_142PU_U_OBR" typeName="BOOL"/>
		<variables name="A_142PU_U_PR" typeName="BOOL"/>
		<variables name="A_142PU_UK" typeName="EBOOL"/>
		<variables name="A_142RI" typeName="BOOL"/>
		<variables name="A_142SP_IND" typeName="BYTE"/>
		<variables name="A_142SP_K" typeName="EBOOL"/>
		<variables name="A_142SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_142SP_MK_IND" typeName="BOOL"/>
		<variables name="A_142SP_PK_IND" typeName="BOOL"/>
		<variables name="A_142VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_142VK_U" typeName="EBOOL"/>
		<variables name="A_142VK_U_OBR" typeName="BOOL"/>
		<variables name="A_142VK_U_PR" typeName="BOOL"/>
		<variables name="A_142VK_UK" typeName="EBOOL"/>
		<variables name="A_142VZ" typeName="BOOL"/>
		<variables name="A_142VZREZ_IM" typeName="BOOL"/>
		<variables name="A_142Z" typeName="BOOL"/>
		<variables name="A_142ZSTR" typeName="BOOL"/>
		<variables name="A_142ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_144_146IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_144_146RI" typeName="BOOL"/>
		<variables name="A_144_146SP_144STR_IND" typeName="BYTE"/>
		<variables name="A_144_146SP_146STR_IND" typeName="BYTE"/>
		<variables name="A_144_146SP_K" typeName="EBOOL"/>
		<variables name="A_144_146Z" typeName="BOOL"/>
		<variables name="A_144MAK_KN" typeName="BOOL"/>
		<variables name="A_144MAK_M_IM" typeName="BOOL"/>
		<variables name="A_144MAK_P_IM" typeName="BOOL"/>
		<variables name="A_144MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_144MK_K" typeName="EBOOL"/>
		<variables name="A_144MKStr" typeName="BOOL"/>
		<variables name="A_144MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_144MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_144MU_U" typeName="EBOOL"/>
		<variables name="A_144MU_U_OBR" typeName="BOOL"/>
		<variables name="A_144MU_U_PR" typeName="BOOL"/>
		<variables name="A_144MU_UK" typeName="EBOOL"/>
		<variables name="A_144NPS_U" typeName="EBOOL"/>
		<variables name="A_144PK_K" typeName="EBOOL"/>
		<variables name="A_144PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_144PU_U" typeName="EBOOL"/>
		<variables name="A_144PU_U_OBR" typeName="BOOL"/>
		<variables name="A_144PU_U_PR" typeName="BOOL"/>
		<variables name="A_144PU_UK" typeName="EBOOL"/>
		<variables name="A_144SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_144SP_MK_IND" typeName="BOOL"/>
		<variables name="A_144SP_PK_IND" typeName="BOOL"/>
		<variables name="A_144VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_144VK_U" typeName="EBOOL"/>
		<variables name="A_144VK_U_OBR" typeName="BOOL"/>
		<variables name="A_144VK_U_PR" typeName="BOOL"/>
		<variables name="A_144VK_UK" typeName="EBOOL"/>
		<variables name="A_144VZ" typeName="BOOL"/>
		<variables name="A_144VZREZ_IM" typeName="BOOL"/>
		<variables name="A_144ZSTR" typeName="BOOL"/>
		<variables name="A_144ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_146MAK_KN" typeName="BOOL"/>
		<variables name="A_146MAK_M_IM" typeName="BOOL"/>
		<variables name="A_146MAK_P_IM" typeName="BOOL"/>
		<variables name="A_146MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_146MK_K" typeName="EBOOL"/>
		<variables name="A_146MKStr" typeName="BOOL"/>
		<variables name="A_146MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_146MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_146MU_U" typeName="EBOOL"/>
		<variables name="A_146MU_U_OBR" typeName="BOOL"/>
		<variables name="A_146MU_U_PR" typeName="BOOL"/>
		<variables name="A_146MU_UK" typeName="EBOOL"/>
		<variables name="A_146NPS_U" typeName="EBOOL"/>
		<variables name="A_146PK_K" typeName="EBOOL"/>
		<variables name="A_146PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_146PU_U" typeName="EBOOL"/>
		<variables name="A_146PU_U_OBR" typeName="BOOL"/>
		<variables name="A_146PU_U_PR" typeName="BOOL"/>
		<variables name="A_146PU_UK" typeName="EBOOL"/>
		<variables name="A_146SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_146SP_MK_IND" typeName="BOOL"/>
		<variables name="A_146SP_PK_IND" typeName="BOOL"/>
		<variables name="A_146VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_146VK_U" typeName="EBOOL"/>
		<variables name="A_146VK_U_OBR" typeName="BOOL"/>
		<variables name="A_146VK_U_PR" typeName="BOOL"/>
		<variables name="A_146VK_UK" typeName="EBOOL"/>
		<variables name="A_146VZ" typeName="BOOL"/>
		<variables name="A_146VZREZ_IM" typeName="BOOL"/>
		<variables name="A_146ZSTR" typeName="BOOL"/>
		<variables name="A_146ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_148IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_148MAK_KN" typeName="BOOL"/>
		<variables name="A_148MAK_M_IM" typeName="BOOL"/>
		<variables name="A_148MAK_P_IM" typeName="BOOL"/>
		<variables name="A_148MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_148MK_K" typeName="EBOOL"/>
		<variables name="A_148MKStr" typeName="BOOL"/>
		<variables name="A_148MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_148MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_148MU_U" typeName="EBOOL"/>
		<variables name="A_148MU_U_OBR" typeName="BOOL"/>
		<variables name="A_148MU_U_PR" typeName="BOOL"/>
		<variables name="A_148MU_UK" typeName="EBOOL"/>
		<variables name="A_148NPS_U" typeName="EBOOL"/>
		<variables name="A_148PK_K" typeName="EBOOL"/>
		<variables name="A_148PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_148PU_U" typeName="EBOOL"/>
		<variables name="A_148PU_U_OBR" typeName="BOOL"/>
		<variables name="A_148PU_U_PR" typeName="BOOL"/>
		<variables name="A_148PU_UK" typeName="EBOOL"/>
		<variables name="A_148RI" typeName="BOOL"/>
		<variables name="A_148SP_IND" typeName="BYTE"/>
		<variables name="A_148SP_K" typeName="EBOOL"/>
		<variables name="A_148SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_148SP_MK_IND" typeName="BOOL"/>
		<variables name="A_148SP_PK_IND" typeName="BOOL"/>
		<variables name="A_148VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_148VK_U" typeName="EBOOL"/>
		<variables name="A_148VK_U_OBR" typeName="BOOL"/>
		<variables name="A_148VK_U_PR" typeName="BOOL"/>
		<variables name="A_148VK_UK" typeName="EBOOL"/>
		<variables name="A_148VZ" typeName="BOOL"/>
		<variables name="A_148VZREZ_IM" typeName="BOOL"/>
		<variables name="A_148Z" typeName="BOOL"/>
		<variables name="A_148ZSTR" typeName="BOOL"/>
		<variables name="A_148ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_160_162IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_160_162KS" typeName="BOOL"/>
		<variables name="A_160_162P_IND" typeName="BYTE"/>
		<variables name="A_160_162P_K" typeName="EBOOL"/>
		<variables name="A_160_162RI" typeName="BOOL"/>
		<variables name="A_160_162Z" typeName="BOOL"/>
		<variables name="A_160IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_160MAK_KN" typeName="BOOL"/>
		<variables name="A_160MAK_M_IM" typeName="BOOL"/>
		<variables name="A_160MAK_P_IM" typeName="BOOL"/>
		<variables name="A_160MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_160MK_K" typeName="EBOOL"/>
		<variables name="A_160MKStr" typeName="BOOL"/>
		<variables name="A_160MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_160MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_160MU_U" typeName="EBOOL"/>
		<variables name="A_160MU_U_OBR" typeName="BOOL"/>
		<variables name="A_160MU_U_PR" typeName="BOOL"/>
		<variables name="A_160MU_UK" typeName="EBOOL"/>
		<variables name="A_160NPS_U" typeName="EBOOL"/>
		<variables name="A_160PK_K" typeName="EBOOL"/>
		<variables name="A_160PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_160PU_U" typeName="EBOOL"/>
		<variables name="A_160PU_U_OBR" typeName="BOOL"/>
		<variables name="A_160PU_U_PR" typeName="BOOL"/>
		<variables name="A_160PU_UK" typeName="EBOOL"/>
		<variables name="A_160RI" typeName="BOOL"/>
		<variables name="A_160SP_IND" typeName="BYTE"/>
		<variables name="A_160SP_K" typeName="EBOOL"/>
		<variables name="A_160SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_160SP_MK_IND" typeName="BOOL"/>
		<variables name="A_160SP_PK_IND" typeName="BOOL"/>
		<variables name="A_160VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_160VK_U" typeName="EBOOL"/>
		<variables name="A_160VK_U_OBR" typeName="BOOL"/>
		<variables name="A_160VK_U_PR" typeName="BOOL"/>
		<variables name="A_160VK_UK" typeName="EBOOL"/>
		<variables name="A_160VZ" typeName="BOOL"/>
		<variables name="A_160VZREZ_IM" typeName="BOOL"/>
		<variables name="A_160Z" typeName="BOOL"/>
		<variables name="A_160ZSTR" typeName="BOOL"/>
		<variables name="A_160ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_162_166IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_162_166KS" typeName="BOOL"/>
		<variables name="A_162_166P_IND" typeName="BYTE"/>
		<variables name="A_162_166P_K" typeName="EBOOL"/>
		<variables name="A_162_166RI" typeName="BOOL"/>
		<variables name="A_162_166Z" typeName="BOOL"/>
		<variables name="A_162IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_162MAK_KN" typeName="BOOL"/>
		<variables name="A_162MAK_M_IM" typeName="BOOL"/>
		<variables name="A_162MAK_P_IM" typeName="BOOL"/>
		<variables name="A_162MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_162MK_K" typeName="EBOOL"/>
		<variables name="A_162MKStr" typeName="BOOL"/>
		<variables name="A_162MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_162MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_162MU_U" typeName="EBOOL"/>
		<variables name="A_162MU_U_OBR" typeName="BOOL"/>
		<variables name="A_162MU_U_PR" typeName="BOOL"/>
		<variables name="A_162MU_UK" typeName="EBOOL"/>
		<variables name="A_162NPS_U" typeName="EBOOL"/>
		<variables name="A_162PK_K" typeName="EBOOL"/>
		<variables name="A_162PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_162PU_U" typeName="EBOOL"/>
		<variables name="A_162PU_U_OBR" typeName="BOOL"/>
		<variables name="A_162PU_U_PR" typeName="BOOL"/>
		<variables name="A_162PU_UK" typeName="EBOOL"/>
		<variables name="A_162RI" typeName="BOOL"/>
		<variables name="A_162SP_IND" typeName="BYTE"/>
		<variables name="A_162SP_K" typeName="EBOOL"/>
		<variables name="A_162SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_162SP_MK_IND" typeName="BOOL"/>
		<variables name="A_162SP_PK_IND" typeName="BOOL"/>
		<variables name="A_162VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_162VK_U" typeName="EBOOL"/>
		<variables name="A_162VK_U_OBR" typeName="BOOL"/>
		<variables name="A_162VK_U_PR" typeName="BOOL"/>
		<variables name="A_162VK_UK" typeName="EBOOL"/>
		<variables name="A_162VZ" typeName="BOOL"/>
		<variables name="A_162VZREZ_IM" typeName="BOOL"/>
		<variables name="A_162Z" typeName="BOOL"/>
		<variables name="A_162ZSTR" typeName="BOOL"/>
		<variables name="A_162ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_163ZSTR" typeName="BOOL"/>
		<variables name="A_164MAK_KN" typeName="BOOL"/>
		<variables name="A_164MAK_M_IM" typeName="BOOL"/>
		<variables name="A_164MAK_P_IM" typeName="BOOL"/>
		<variables name="A_164MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_164MK_K" typeName="EBOOL"/>
		<variables name="A_164MKStr" typeName="BOOL"/>
		<variables name="A_164MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_164MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_164MU_U" typeName="EBOOL"/>
		<variables name="A_164MU_U_OBR" typeName="BOOL"/>
		<variables name="A_164MU_U_PR" typeName="BOOL"/>
		<variables name="A_164MU_UK" typeName="EBOOL"/>
		<variables name="A_164NPS_U" typeName="EBOOL"/>
		<variables name="A_164PK_K" typeName="EBOOL"/>
		<variables name="A_164PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_164PU_U" typeName="EBOOL"/>
		<variables name="A_164PU_U_OBR" typeName="BOOL"/>
		<variables name="A_164PU_U_PR" typeName="BOOL"/>
		<variables name="A_164PU_UK" typeName="EBOOL"/>
		<variables name="A_164SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_164SP_MK_IND" typeName="BOOL"/>
		<variables name="A_164SP_PK_IND" typeName="BOOL"/>
		<variables name="A_164VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_164VK_U" typeName="EBOOL"/>
		<variables name="A_164VK_U_OBR" typeName="BOOL"/>
		<variables name="A_164VK_U_PR" typeName="BOOL"/>
		<variables name="A_164VK_UK" typeName="EBOOL"/>
		<variables name="A_164VZ" typeName="BOOL"/>
		<variables name="A_164VZREZ_IM" typeName="BOOL"/>
		<variables name="A_164ZSTR" typeName="BOOL"/>
		<variables name="A_164ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_166IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_166MAK_KN" typeName="BOOL"/>
		<variables name="A_166MAK_M_IM" typeName="BOOL"/>
		<variables name="A_166MAK_P_IM" typeName="BOOL"/>
		<variables name="A_166MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_166MK_K" typeName="EBOOL"/>
		<variables name="A_166MKStr" typeName="BOOL"/>
		<variables name="A_166MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_166MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_166MU_U" typeName="EBOOL"/>
		<variables name="A_166MU_U_OBR" typeName="BOOL"/>
		<variables name="A_166MU_U_PR" typeName="BOOL"/>
		<variables name="A_166MU_UK" typeName="EBOOL"/>
		<variables name="A_166NPS_U" typeName="EBOOL"/>
		<variables name="A_166PK_K" typeName="EBOOL"/>
		<variables name="A_166PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_166PU_U" typeName="EBOOL"/>
		<variables name="A_166PU_U_OBR" typeName="BOOL"/>
		<variables name="A_166PU_U_PR" typeName="BOOL"/>
		<variables name="A_166PU_UK" typeName="EBOOL"/>
		<variables name="A_166RI" typeName="BOOL"/>
		<variables name="A_166SP_IND" typeName="BYTE"/>
		<variables name="A_166SP_K" typeName="EBOOL"/>
		<variables name="A_166SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_166SP_MK_IND" typeName="BOOL"/>
		<variables name="A_166SP_PK_IND" typeName="BOOL"/>
		<variables name="A_166VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_166VK_U" typeName="EBOOL"/>
		<variables name="A_166VK_U_OBR" typeName="BOOL"/>
		<variables name="A_166VK_U_PR" typeName="BOOL"/>
		<variables name="A_166VK_UK" typeName="EBOOL"/>
		<variables name="A_166VZ" typeName="BOOL"/>
		<variables name="A_166VZREZ_IM" typeName="BOOL"/>
		<variables name="A_166Z" typeName="BOOL"/>
		<variables name="A_166ZSTR" typeName="BOOL"/>
		<variables name="A_166ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_201_203IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_201_203RI" typeName="BOOL"/>
		<variables name="A_201_203SP_201STR_IND" typeName="BYTE"/>
		<variables name="A_201_203SP_203STR_IND" typeName="BYTE"/>
		<variables name="A_201_203SP_K" typeName="EBOOL"/>
		<variables name="A_201_203Z" typeName="BOOL"/>
		<variables name="A_201MAK_KN" typeName="BOOL"/>
		<variables name="A_201MAK_M_IM" typeName="BOOL"/>
		<variables name="A_201MAK_P_IM" typeName="BOOL"/>
		<variables name="A_201MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_201MK_K" typeName="EBOOL"/>
		<variables name="A_201MKStr" typeName="BOOL"/>
		<variables name="A_201MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_201MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_201MU_U" typeName="EBOOL"/>
		<variables name="A_201MU_U_OBR" typeName="BOOL"/>
		<variables name="A_201MU_U_PR" typeName="BOOL"/>
		<variables name="A_201MU_UK" typeName="EBOOL"/>
		<variables name="A_201NPS_U" typeName="EBOOL"/>
		<variables name="A_201P_IND" typeName="BYTE"/>
		<variables name="A_201P_K" typeName="EBOOL"/>
		<variables name="A_201PCHKM" typeName="BOOL"/>
		<variables name="A_201PK_K" typeName="EBOOL"/>
		<variables name="A_201PNKM" typeName="BOOL"/>
		<variables name="A_201PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_201PU_U" typeName="EBOOL"/>
		<variables name="A_201PU_U_OBR" typeName="BOOL"/>
		<variables name="A_201PU_U_PR" typeName="BOOL"/>
		<variables name="A_201PU_UK" typeName="EBOOL"/>
		<variables name="A_201SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_201SP_MK_IND" typeName="BOOL"/>
		<variables name="A_201SP_PK_IND" typeName="BOOL"/>
		<variables name="A_201VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_201VK_U" typeName="EBOOL"/>
		<variables name="A_201VK_U_OBR" typeName="BOOL"/>
		<variables name="A_201VK_U_PR" typeName="BOOL"/>
		<variables name="A_201VK_UK" typeName="EBOOL"/>
		<variables name="A_201VZ" typeName="BOOL"/>
		<variables name="A_201VZREZ_IM" typeName="BOOL"/>
		<variables name="A_201ZSTR" typeName="BOOL"/>
		<variables name="A_201ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_202_212IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_202_212KS" typeName="BOOL"/>
		<variables name="A_202_212P_IND" typeName="BYTE"/>
		<variables name="A_202_212P_K" typeName="EBOOL"/>
		<variables name="A_202_212RI" typeName="BOOL"/>
		<variables name="A_202_212Z" typeName="BOOL"/>
		<variables name="A_202IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_202MAK_KN" typeName="BOOL"/>
		<variables name="A_202MAK_M_IM" typeName="BOOL"/>
		<variables name="A_202MAK_P_IM" typeName="BOOL"/>
		<variables name="A_202MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_202MK_K" typeName="EBOOL"/>
		<variables name="A_202MKStr" typeName="BOOL"/>
		<variables name="A_202MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_202MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_202MU_U" typeName="EBOOL"/>
		<variables name="A_202MU_U_OBR" typeName="BOOL"/>
		<variables name="A_202MU_U_PR" typeName="BOOL"/>
		<variables name="A_202MU_UK" typeName="EBOOL"/>
		<variables name="A_202NPS_U" typeName="EBOOL"/>
		<variables name="A_202P_IND" typeName="BYTE"/>
		<variables name="A_202P_K" typeName="EBOOL"/>
		<variables name="A_202PCHKM" typeName="BOOL"/>
		<variables name="A_202PK_K" typeName="EBOOL"/>
		<variables name="A_202PNKM" typeName="BOOL"/>
		<variables name="A_202PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_202PU_U" typeName="EBOOL"/>
		<variables name="A_202PU_U_OBR" typeName="BOOL"/>
		<variables name="A_202PU_U_PR" typeName="BOOL"/>
		<variables name="A_202PU_UK" typeName="EBOOL"/>
		<variables name="A_202RI" typeName="BOOL"/>
		<variables name="A_202SP_IND" typeName="BYTE"/>
		<variables name="A_202SP_K" typeName="EBOOL"/>
		<variables name="A_202SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_202SP_MK_IND" typeName="BOOL"/>
		<variables name="A_202SP_PK_IND" typeName="BOOL"/>
		<variables name="A_202VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_202VK_U" typeName="EBOOL"/>
		<variables name="A_202VK_U_OBR" typeName="BOOL"/>
		<variables name="A_202VK_U_PR" typeName="BOOL"/>
		<variables name="A_202VK_UK" typeName="EBOOL"/>
		<variables name="A_202VZ" typeName="BOOL"/>
		<variables name="A_202VZREZ_IM" typeName="BOOL"/>
		<variables name="A_202Z" typeName="BOOL"/>
		<variables name="A_202ZSTR" typeName="BOOL"/>
		<variables name="A_202ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_203MAK_KN" typeName="BOOL"/>
		<variables name="A_203MAK_M_IM" typeName="BOOL"/>
		<variables name="A_203MAK_P_IM" typeName="BOOL"/>
		<variables name="A_203MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_203MK_K" typeName="EBOOL"/>
		<variables name="A_203MKStr" typeName="BOOL"/>
		<variables name="A_203MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_203MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_203MU_U" typeName="EBOOL"/>
		<variables name="A_203MU_U_OBR" typeName="BOOL"/>
		<variables name="A_203MU_U_PR" typeName="BOOL"/>
		<variables name="A_203MU_UK" typeName="EBOOL"/>
		<variables name="A_203NPS_U" typeName="EBOOL"/>
		<variables name="A_203P_IND" typeName="BYTE"/>
		<variables name="A_203P_K" typeName="EBOOL"/>
		<variables name="A_203PCHKM" typeName="BOOL"/>
		<variables name="A_203PK_K" typeName="EBOOL"/>
		<variables name="A_203PNKM" typeName="BOOL"/>
		<variables name="A_203PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_203PU_U" typeName="EBOOL"/>
		<variables name="A_203PU_U_OBR" typeName="BOOL"/>
		<variables name="A_203PU_U_PR" typeName="BOOL"/>
		<variables name="A_203PU_UK" typeName="EBOOL"/>
		<variables name="A_203SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_203SP_MK_IND" typeName="BOOL"/>
		<variables name="A_203SP_PK_IND" typeName="BOOL"/>
		<variables name="A_203VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_203VK_U" typeName="EBOOL"/>
		<variables name="A_203VK_U_OBR" typeName="BOOL"/>
		<variables name="A_203VK_U_PR" typeName="BOOL"/>
		<variables name="A_203VK_UK" typeName="EBOOL"/>
		<variables name="A_203VZ" typeName="BOOL"/>
		<variables name="A_203VZREZ_IM" typeName="BOOL"/>
		<variables name="A_203ZSTR" typeName="BOOL"/>
		<variables name="A_203ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_204_206MAK_KN" typeName="BOOL"/>
		<variables name="A_204_206MAK_M_IM" typeName="BOOL"/>
		<variables name="A_204_206MAK_P_IM" typeName="BOOL"/>
		<variables name="A_204_206MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_204_206MK_K" typeName="BOOL"/>
		<variables name="A_204_206MKStr" typeName="BOOL"/>
		<variables name="A_204_206MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_204_206MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_204_206MU_U" typeName="EBOOL"/>
		<variables name="A_204_206MU_U_OBR" typeName="BOOL"/>
		<variables name="A_204_206MU_U_PR" typeName="BOOL"/>
		<variables name="A_204_206MU_UK" typeName="EBOOL"/>
		<variables name="A_204_206NPS_U" typeName="EBOOL"/>
		<variables name="A_204_206PK_K" typeName="EBOOL"/>
		<variables name="A_204_206PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_204_206PU_U" typeName="EBOOL"/>
		<variables name="A_204_206PU_U_OBR" typeName="BOOL"/>
		<variables name="A_204_206PU_U_PR" typeName="BOOL"/>
		<variables name="A_204_206PU_UK" typeName="EBOOL"/>
		<variables name="A_204_206VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_204_206VK_U" typeName="EBOOL"/>
		<variables name="A_204_206VK_U_OBR" typeName="BOOL"/>
		<variables name="A_204_206VK_U_PR" typeName="BOOL"/>
		<variables name="A_204_206VK_UK" typeName="EBOOL"/>
		<variables name="A_204_206VZ" typeName="BOOL"/>
		<variables name="A_204_206VZREZ_IM" typeName="BOOL"/>
		<variables name="A_204_206ZSTR" typeName="BOOL"/>
		<variables name="A_204_206ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_204_212IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_204_212RI" typeName="BOOL"/>
		<variables name="A_204_212SP_204STR_IND" typeName="BYTE"/>
		<variables name="A_204_212SP_208STR_IND" typeName="BYTE"/>
		<variables name="A_204_212SP_212STR_IND" typeName="BYTE"/>
		<variables name="A_204_212SP_K" typeName="EBOOL"/>
		<variables name="A_204_212Z" typeName="BOOL"/>
		<variables name="A_204SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_204SP_MK_IND" typeName="BOOL"/>
		<variables name="A_204SP_PK_IND" typeName="BOOL"/>
		<variables name="A_204VZREZ_IM" typeName="BOOL"/>
		<variables name="A_206IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_206RI" typeName="BOOL"/>
		<variables name="A_206SP_IND" typeName="BYTE"/>
		<variables name="A_206SP_K" typeName="EBOOL"/>
		<variables name="A_206SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_206SP_MK_IND" typeName="BOOL"/>
		<variables name="A_206SP_PK_IND" typeName="BOOL"/>
		<variables name="A_206VZREZ_IM" typeName="BOOL"/>
		<variables name="A_206Z" typeName="BOOL"/>
		<variables name="A_208MAK_KN" typeName="BOOL"/>
		<variables name="A_208MAK_M_IM" typeName="BOOL"/>
		<variables name="A_208MAK_P_IM" typeName="BOOL"/>
		<variables name="A_208MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_208MK_K" typeName="EBOOL"/>
		<variables name="A_208MKStr" typeName="BOOL"/>
		<variables name="A_208MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_208MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_208MU_U" typeName="EBOOL"/>
		<variables name="A_208MU_U_OBR" typeName="BOOL"/>
		<variables name="A_208MU_U_PR" typeName="BOOL"/>
		<variables name="A_208MU_UK" typeName="EBOOL"/>
		<variables name="A_208NPS_U" typeName="EBOOL"/>
		<variables name="A_208PK_K" typeName="EBOOL"/>
		<variables name="A_208PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_208PU_U" typeName="EBOOL"/>
		<variables name="A_208PU_U_OBR" typeName="BOOL"/>
		<variables name="A_208PU_U_PR" typeName="BOOL"/>
		<variables name="A_208PU_UK" typeName="EBOOL"/>
		<variables name="A_208SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_208SP_MK_IND" typeName="BOOL"/>
		<variables name="A_208SP_PK_IND" typeName="BOOL"/>
		<variables name="A_208VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_208VK_U" typeName="EBOOL"/>
		<variables name="A_208VK_U_OBR" typeName="BOOL"/>
		<variables name="A_208VK_U_PR" typeName="BOOL"/>
		<variables name="A_208VK_UK" typeName="EBOOL"/>
		<variables name="A_208VZ" typeName="BOOL"/>
		<variables name="A_208VZREZ_IM" typeName="BOOL"/>
		<variables name="A_208ZSTR" typeName="BOOL"/>
		<variables name="A_208ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_212MAK_KN" typeName="BOOL"/>
		<variables name="A_212MAK_M_IM" typeName="BOOL"/>
		<variables name="A_212MAK_P_IM" typeName="BOOL"/>
		<variables name="A_212MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_212MK_K" typeName="EBOOL"/>
		<variables name="A_212MKStr" typeName="BOOL"/>
		<variables name="A_212MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_212MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_212MU_U" typeName="EBOOL"/>
		<variables name="A_212MU_U_OBR" typeName="BOOL"/>
		<variables name="A_212MU_U_PR" typeName="BOOL"/>
		<variables name="A_212MU_UK" typeName="EBOOL"/>
		<variables name="A_212NPS_U" typeName="EBOOL"/>
		<variables name="A_212PK_K" typeName="EBOOL"/>
		<variables name="A_212PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_212PU_U" typeName="EBOOL"/>
		<variables name="A_212PU_U_OBR" typeName="BOOL"/>
		<variables name="A_212PU_U_PR" typeName="BOOL"/>
		<variables name="A_212PU_UK" typeName="EBOOL"/>
		<variables name="A_212SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_212SP_MK_IND" typeName="BOOL"/>
		<variables name="A_212SP_PK_IND" typeName="BOOL"/>
		<variables name="A_212VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_212VK_U" typeName="EBOOL"/>
		<variables name="A_212VK_U_OBR" typeName="BOOL"/>
		<variables name="A_212VK_U_PR" typeName="BOOL"/>
		<variables name="A_212VK_UK" typeName="EBOOL"/>
		<variables name="A_212VZ" typeName="BOOL"/>
		<variables name="A_212VZREZ_IM" typeName="BOOL"/>
		<variables name="A_212ZSTR" typeName="BOOL"/>
		<variables name="A_212ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_302IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_302MAK_KN" typeName="BOOL"/>
		<variables name="A_302MAK_M_IM" typeName="BOOL"/>
		<variables name="A_302MAK_P_IM" typeName="BOOL"/>
		<variables name="A_302MAK_UST_IM" typeName="BOOL"/>
		<variables name="A_302MK_K" typeName="EBOOL"/>
		<variables name="A_302MKStr" typeName="BOOL"/>
		<variables name="A_302MKSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_302MU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_302MU_U" typeName="EBOOL"/>
		<variables name="A_302MU_U_OBR" typeName="BOOL"/>
		<variables name="A_302MU_U_PR" typeName="BOOL"/>
		<variables name="A_302MU_UK" typeName="EBOOL"/>
		<variables name="A_302NPS_U" typeName="EBOOL"/>
		<variables name="A_302PK_K" typeName="EBOOL"/>
		<variables name="A_302PU_ARM_DSP" typeName="BOOL"/>
		<variables name="A_302PU_U" typeName="EBOOL"/>
		<variables name="A_302PU_U_OBR" typeName="BOOL"/>
		<variables name="A_302PU_U_PR" typeName="BOOL"/>
		<variables name="A_302PU_UK" typeName="EBOOL"/>
		<variables name="A_302RI" typeName="BOOL"/>
		<variables name="A_302SP_IND" typeName="BYTE"/>
		<variables name="A_302SP_K" typeName="EBOOL"/>
		<variables name="A_302SP_MAKET_IND" typeName="BOOL"/>
		<variables name="A_302SP_MK_IND" typeName="BOOL"/>
		<variables name="A_302SP_PK_IND" typeName="BOOL"/>
		<variables name="A_302VK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_302VK_U" typeName="EBOOL"/>
		<variables name="A_302VK_U_OBR" typeName="BOOL"/>
		<variables name="A_302VK_U_PR" typeName="BOOL"/>
		<variables name="A_302VK_UK" typeName="EBOOL"/>
		<variables name="A_302VZ" typeName="BOOL"/>
		<variables name="A_302VZREZ_IM" typeName="BOOL"/>
		<variables name="A_302Z" typeName="BOOL"/>
		<variables name="A_302ZSTR" typeName="BOOL"/>
		<variables name="A_302ZSTR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_11616MK_K" typeName="EBOOL"/>
		<variables name="A_A10_K" typeName="EBOOL"/>
		<variables name="A_AS_U" typeName="EBOOL"/>
		<variables name="A_AV_K" typeName="EBOOL"/>
		<variables name="A_AV_U" typeName="EBOOL"/>
		<variables name="A_AV_U_OBR" typeName="BOOL"/>
		<variables name="A_AV_U_PR" typeName="BOOL"/>
		<variables name="A_AV_UK" typeName="EBOOL"/>
		<variables name="A_BIR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_BKS" typeName="BOOL"/>
		<variables name="A_BP_IND" typeName="BYTE"/>
		<variables name="A_BP_K" typeName="EBOOL"/>
		<variables name="A_BRI" typeName="BOOL"/>
		<variables name="A_BZ" typeName="BOOL"/>
		<variables name="A_CH" typeName="BOOL"/>
		<variables name="A_CH101_ARM_DSP" typeName="BOOL"/>
		<variables name="A_CH101_BL_IM" typeName="BOOL"/>
		<variables name="A_CH101_GOT" typeName="BOOL"/>
		<variables name="A_CH101_iK" typeName="BOOL"/>
		<variables name="A_CH101_IND" typeName="BYTE"/>
		<variables name="A_CH101_KL_IM" typeName="BOOL"/>
		<variables name="A_CH101_KM" typeName="BOOL"/>
		<variables name="A_CH101_KP" typeName="BOOL"/>
		<variables name="A_CH101_KS" typeName="BOOL"/>
		<variables name="A_CH101_KS_V1" typeName="BOOL"/>
		<variables name="A_CH101_MOT" typeName="BOOL"/>
		<variables name="A_CH101_MS" typeName="BOOL"/>
		<variables name="A_CH101_MS1" typeName="BOOL"/>
		<variables name="A_CH101_MS_V1" typeName="BOOL"/>
		<variables name="A_CH101_MV1" typeName="BOOL"/>
		<variables name="A_CH101_N" typeName="BOOL"/>
		<variables name="A_CH101_NM" typeName="BOOL"/>
		<variables name="A_CH101_OT" typeName="BOOL"/>
		<variables name="A_CH101_OTM" typeName="BOOL"/>
		<variables name="A_CH101_PV1" typeName="BOOL"/>
		<variables name="A_CH101_S" typeName="BOOL"/>
		<variables name="A_CH101_S1" typeName="BOOL"/>
		<variables name="A_CH101_S_V1" typeName="BOOL"/>
		<variables name="A_CH101_UK" typeName="BOOL"/>
		<variables name="A_CH101_VCH" typeName="BOOL"/>
		<variables name="A_CH101_VCHM" typeName="BOOL"/>
		<variables name="A_CH101_ZL_IM" typeName="BOOL"/>
		<variables name="A_CH101MS_K" typeName="EBOOL"/>
		<variables name="A_CH101MS_U" typeName="EBOOL"/>
		<variables name="A_CH101MS_U_OBR" typeName="BOOL"/>
		<variables name="A_CH101MS_U_PR" typeName="BOOL"/>
		<variables name="A_CH101MS_UK" typeName="EBOOL"/>
		<variables name="A_CH101O_K" typeName="EBOOL"/>
		<variables name="A_CH101ON" typeName="BOOL"/>
		<variables name="A_CH101P_ARM_DSP" typeName="BOOL"/>
		<variables name="A_CH101S_K" typeName="BOOL"/>
		<variables name="A_CH101S_U" typeName="EBOOL"/>
		<variables name="A_CH102_ARM_DSP" typeName="BOOL"/>
		<variables name="A_CH102_BL_IM" typeName="BOOL"/>
		<variables name="A_CH102_GOT" typeName="BOOL"/>
		<variables name="A_CH102_iK" typeName="BOOL"/>
		<variables name="A_CH102_IND" typeName="BYTE"/>
		<variables name="A_CH102_KL_IM" typeName="BOOL"/>
		<variables name="A_CH102_KM" typeName="BOOL"/>
		<variables name="A_CH102_KP" typeName="BOOL"/>
		<variables name="A_CH102_KS" typeName="BOOL"/>
		<variables name="A_CH102_KS_V1" typeName="BOOL"/>
		<variables name="A_CH102_MOT" typeName="BOOL"/>
		<variables name="A_CH102_MS" typeName="BOOL"/>
		<variables name="A_CH102_MS1" typeName="BOOL"/>
		<variables name="A_CH102_MS_V1" typeName="BOOL"/>
		<variables name="A_CH102_MV1" typeName="BOOL"/>
		<variables name="A_CH102_N" typeName="BOOL"/>
		<variables name="A_CH102_NM" typeName="BOOL"/>
		<variables name="A_CH102_OT" typeName="BOOL"/>
		<variables name="A_CH102_OTM" typeName="BOOL"/>
		<variables name="A_CH102_PV1" typeName="BOOL"/>
		<variables name="A_CH102_S" typeName="BOOL"/>
		<variables name="A_CH102_S1" typeName="BOOL"/>
		<variables name="A_CH102_S_V1" typeName="BOOL"/>
		<variables name="A_CH102_UK" typeName="BOOL"/>
		<variables name="A_CH102_VCH" typeName="BOOL"/>
		<variables name="A_CH102_VCHM" typeName="BOOL"/>
		<variables name="A_CH102_ZL_IM" typeName="BOOL"/>
		<variables name="A_CH102MS_K" typeName="EBOOL"/>
		<variables name="A_CH102MS_U" typeName="EBOOL"/>
		<variables name="A_CH102MS_U_OBR" typeName="BOOL"/>
		<variables name="A_CH102MS_U_PR" typeName="BOOL"/>
		<variables name="A_CH102MS_UK" typeName="EBOOL"/>
		<variables name="A_CH102O_K" typeName="EBOOL"/>
		<variables name="A_CH102ON" typeName="BOOL"/>
		<variables name="A_CH102P_ARM_DSP" typeName="BOOL"/>
		<variables name="A_CH102S_K" typeName="BOOL"/>
		<variables name="A_CH102S_U" typeName="EBOOL"/>
		<variables name="A_CH103_ARM_DSP" typeName="BOOL"/>
		<variables name="A_CH103_BL_IM" typeName="BOOL"/>
		<variables name="A_CH103_GOT" typeName="BOOL"/>
		<variables name="A_CH103_iK" typeName="BOOL"/>
		<variables name="A_CH103_IND" typeName="BYTE"/>
		<variables name="A_CH103_KL_IM" typeName="BOOL"/>
		<variables name="A_CH103_KM" typeName="BOOL"/>
		<variables name="A_CH103_KP" typeName="BOOL"/>
		<variables name="A_CH103_KS" typeName="BOOL"/>
		<variables name="A_CH103_KS_V1" typeName="BOOL"/>
		<variables name="A_CH103_MOT" typeName="BOOL"/>
		<variables name="A_CH103_MS" typeName="BOOL"/>
		<variables name="A_CH103_MS1" typeName="BOOL"/>
		<variables name="A_CH103_MS_V1" typeName="BOOL"/>
		<variables name="A_CH103_MV1" typeName="BOOL"/>
		<variables name="A_CH103_N" typeName="BOOL"/>
		<variables name="A_CH103_NM" typeName="BOOL"/>
		<variables name="A_CH103_OT" typeName="BOOL"/>
		<variables name="A_CH103_OTM" typeName="BOOL"/>
		<variables name="A_CH103_PV1" typeName="BOOL"/>
		<variables name="A_CH103_S" typeName="BOOL"/>
		<variables name="A_CH103_S1" typeName="BOOL"/>
		<variables name="A_CH103_S_V1" typeName="BOOL"/>
		<variables name="A_CH103_UK" typeName="BOOL"/>
		<variables name="A_CH103_VCH" typeName="BOOL"/>
		<variables name="A_CH103_VCHM" typeName="BOOL"/>
		<variables name="A_CH103_ZL_IM" typeName="BOOL"/>
		<variables name="A_CH103MS_K" typeName="EBOOL"/>
		<variables name="A_CH103MS_U" typeName="EBOOL"/>
		<variables name="A_CH103MS_U_OBR" typeName="BOOL"/>
		<variables name="A_CH103MS_U_PR" typeName="BOOL"/>
		<variables name="A_CH103MS_UK" typeName="EBOOL"/>
		<variables name="A_CH103O_K" typeName="EBOOL"/>
		<variables name="A_CH103ON" typeName="BOOL"/>
		<variables name="A_CH103P_ARM_DSP" typeName="BOOL"/>
		<variables name="A_CH103S_K" typeName="BOOL"/>
		<variables name="A_CH103S_U" typeName="EBOOL"/>
		<variables name="A_CH104_ARM_DSP" typeName="BOOL"/>
		<variables name="A_CH104_BL_IM" typeName="BOOL"/>
		<variables name="A_CH104_GOT" typeName="BOOL"/>
		<variables name="A_CH104_iK" typeName="BOOL"/>
		<variables name="A_CH104_IND" typeName="BYTE"/>
		<variables name="A_CH104_KL_IM" typeName="BOOL"/>
		<variables name="A_CH104_KM" typeName="BOOL"/>
		<variables name="A_CH104_KP" typeName="BOOL"/>
		<variables name="A_CH104_KS" typeName="BOOL"/>
		<variables name="A_CH104_KS_V1" typeName="BOOL"/>
		<variables name="A_CH104_MOT" typeName="BOOL"/>
		<variables name="A_CH104_MS" typeName="BOOL"/>
		<variables name="A_CH104_MS1" typeName="BOOL"/>
		<variables name="A_CH104_MS_V1" typeName="BOOL"/>
		<variables name="A_CH104_MV1" typeName="BOOL"/>
		<variables name="A_CH104_N" typeName="BOOL"/>
		<variables name="A_CH104_NM" typeName="BOOL"/>
		<variables name="A_CH104_OT" typeName="BOOL"/>
		<variables name="A_CH104_OTM" typeName="BOOL"/>
		<variables name="A_CH104_PV1" typeName="BOOL"/>
		<variables name="A_CH104_S" typeName="BOOL"/>
		<variables name="A_CH104_S1" typeName="BOOL"/>
		<variables name="A_CH104_S_V1" typeName="BOOL"/>
		<variables name="A_CH104_UK" typeName="BOOL"/>
		<variables name="A_CH104_VCH" typeName="BOOL"/>
		<variables name="A_CH104_VCHM" typeName="BOOL"/>
		<variables name="A_CH104_ZL_IM" typeName="BOOL"/>
		<variables name="A_CH104MS_K" typeName="EBOOL"/>
		<variables name="A_CH104MS_U" typeName="EBOOL"/>
		<variables name="A_CH104MS_U_OBR" typeName="BOOL"/>
		<variables name="A_CH104MS_U_PR" typeName="BOOL"/>
		<variables name="A_CH104MS_UK" typeName="EBOOL"/>
		<variables name="A_CH104O_K" typeName="EBOOL"/>
		<variables name="A_CH104ON" typeName="BOOL"/>
		<variables name="A_CH104P_ARM_DSP" typeName="BOOL"/>
		<variables name="A_CH104S_K" typeName="BOOL"/>
		<variables name="A_CH104S_U" typeName="EBOOL"/>
		<variables name="A_CH105_ARM_DSP" typeName="BOOL"/>
		<variables name="A_CH105_BL_IM" typeName="BOOL"/>
		<variables name="A_CH105_GOT" typeName="BOOL"/>
		<variables name="A_CH105_iK" typeName="BOOL"/>
		<variables name="A_CH105_IND" typeName="BYTE"/>
		<variables name="A_CH105_KL_IM" typeName="BOOL"/>
		<variables name="A_CH105_KM" typeName="BOOL"/>
		<variables name="A_CH105_KP" typeName="BOOL"/>
		<variables name="A_CH105_KS" typeName="BOOL"/>
		<variables name="A_CH105_KS_V1" typeName="BOOL"/>
		<variables name="A_CH105_MOT" typeName="BOOL"/>
		<variables name="A_CH105_MS" typeName="BOOL"/>
		<variables name="A_CH105_MS1" typeName="BOOL"/>
		<variables name="A_CH105_MS_V1" typeName="BOOL"/>
		<variables name="A_CH105_MV1" typeName="BOOL"/>
		<variables name="A_CH105_N" typeName="BOOL"/>
		<variables name="A_CH105_NM" typeName="BOOL"/>
		<variables name="A_CH105_OT" typeName="BOOL"/>
		<variables name="A_CH105_OTM" typeName="BOOL"/>
		<variables name="A_CH105_PV1" typeName="BOOL"/>
		<variables name="A_CH105_S" typeName="BOOL"/>
		<variables name="A_CH105_S1" typeName="BOOL"/>
		<variables name="A_CH105_S_V1" typeName="BOOL"/>
		<variables name="A_CH105_UK" typeName="BOOL"/>
		<variables name="A_CH105_VCH" typeName="BOOL"/>
		<variables name="A_CH105_VCHM" typeName="BOOL"/>
		<variables name="A_CH105_ZL_IM" typeName="BOOL"/>
		<variables name="A_CH105MS_K" typeName="EBOOL"/>
		<variables name="A_CH105MS_U" typeName="EBOOL"/>
		<variables name="A_CH105MS_U_OBR" typeName="BOOL"/>
		<variables name="A_CH105MS_U_PR" typeName="BOOL"/>
		<variables name="A_CH105MS_UK" typeName="EBOOL"/>
		<variables name="A_CH105O_K" typeName="EBOOL"/>
		<variables name="A_CH105ON" typeName="BOOL"/>
		<variables name="A_CH105P_ARM_DSP" typeName="BOOL"/>
		<variables name="A_CH105S_K" typeName="BOOL"/>
		<variables name="A_CH105S_U" typeName="EBOOL"/>
		<variables name="A_CH106_ARM_DSP" typeName="BOOL"/>
		<variables name="A_CH106_BL_IM" typeName="BOOL"/>
		<variables name="A_CH106_GOT" typeName="BOOL"/>
		<variables name="A_CH106_iK" typeName="BOOL"/>
		<variables name="A_CH106_IND" typeName="BYTE"/>
		<variables name="A_CH106_KL_IM" typeName="BOOL"/>
		<variables name="A_CH106_KM" typeName="BOOL"/>
		<variables name="A_CH106_KP" typeName="BOOL"/>
		<variables name="A_CH106_KS" typeName="BOOL"/>
		<variables name="A_CH106_KS_V1" typeName="BOOL"/>
		<variables name="A_CH106_MOT" typeName="BOOL"/>
		<variables name="A_CH106_MS" typeName="BOOL"/>
		<variables name="A_CH106_MS1" typeName="BOOL"/>
		<variables name="A_CH106_MS_V1" typeName="BOOL"/>
		<variables name="A_CH106_MV1" typeName="BOOL"/>
		<variables name="A_CH106_N" typeName="BOOL"/>
		<variables name="A_CH106_NM" typeName="BOOL"/>
		<variables name="A_CH106_OT" typeName="BOOL"/>
		<variables name="A_CH106_OTM" typeName="BOOL"/>
		<variables name="A_CH106_PV1" typeName="BOOL"/>
		<variables name="A_CH106_S" typeName="BOOL"/>
		<variables name="A_CH106_S1" typeName="BOOL"/>
		<variables name="A_CH106_S_V1" typeName="BOOL"/>
		<variables name="A_CH106_UK" typeName="BOOL"/>
		<variables name="A_CH106_VCH" typeName="BOOL"/>
		<variables name="A_CH106_VCHM" typeName="BOOL"/>
		<variables name="A_CH106_ZL_IM" typeName="BOOL"/>
		<variables name="A_CH106MS_K" typeName="EBOOL"/>
		<variables name="A_CH106MS_U" typeName="EBOOL"/>
		<variables name="A_CH106MS_U_OBR" typeName="BOOL"/>
		<variables name="A_CH106MS_U_PR" typeName="BOOL"/>
		<variables name="A_CH106MS_UK" typeName="EBOOL"/>
		<variables name="A_CH106O_K" typeName="EBOOL"/>
		<variables name="A_CH106ON" typeName="BOOL"/>
		<variables name="A_CH106P_ARM_DSP" typeName="BOOL"/>
		<variables name="A_CH106S_K" typeName="BOOL"/>
		<variables name="A_CH106S_U" typeName="EBOOL"/>
		<variables name="A_CH107_ARM_DSP" typeName="BOOL"/>
		<variables name="A_CH107_BL_IM" typeName="BOOL"/>
		<variables name="A_CH107_GOT" typeName="BOOL"/>
		<variables name="A_CH107_iK" typeName="BOOL"/>
		<variables name="A_CH107_IND" typeName="BYTE"/>
		<variables name="A_CH107_KL_IM" typeName="BOOL"/>
		<variables name="A_CH107_KM" typeName="BOOL"/>
		<variables name="A_CH107_KP" typeName="BOOL"/>
		<variables name="A_CH107_KS" typeName="BOOL"/>
		<variables name="A_CH107_KS_V1" typeName="BOOL"/>
		<variables name="A_CH107_MOT" typeName="BOOL"/>
		<variables name="A_CH107_MS" typeName="BOOL"/>
		<variables name="A_CH107_MS1" typeName="BOOL"/>
		<variables name="A_CH107_MS_V1" typeName="BOOL"/>
		<variables name="A_CH107_MV1" typeName="BOOL"/>
		<variables name="A_CH107_N" typeName="BOOL"/>
		<variables name="A_CH107_NM" typeName="BOOL"/>
		<variables name="A_CH107_OT" typeName="BOOL"/>
		<variables name="A_CH107_OTM" typeName="BOOL"/>
		<variables name="A_CH107_PV1" typeName="BOOL"/>
		<variables name="A_CH107_S" typeName="BOOL"/>
		<variables name="A_CH107_S1" typeName="BOOL"/>
		<variables name="A_CH107_S_V1" typeName="BOOL"/>
		<variables name="A_CH107_UK" typeName="BOOL"/>
		<variables name="A_CH107_VCH" typeName="BOOL"/>
		<variables name="A_CH107_VCHM" typeName="BOOL"/>
		<variables name="A_CH107_ZL_IM" typeName="BOOL"/>
		<variables name="A_CH107MS_K" typeName="EBOOL"/>
		<variables name="A_CH107MS_U" typeName="EBOOL"/>
		<variables name="A_CH107MS_U_OBR" typeName="BOOL"/>
		<variables name="A_CH107MS_U_PR" typeName="BOOL"/>
		<variables name="A_CH107MS_UK" typeName="EBOOL"/>
		<variables name="A_CH107O_K" typeName="EBOOL"/>
		<variables name="A_CH107ON" typeName="BOOL"/>
		<variables name="A_CH107P_ARM_DSP" typeName="BOOL"/>
		<variables name="A_CH107S_K" typeName="BOOL"/>
		<variables name="A_CH107S_U" typeName="EBOOL"/>
		<variables name="A_CH108_ARM_DSP" typeName="BOOL"/>
		<variables name="A_CH108_BL_IM" typeName="BOOL"/>
		<variables name="A_CH108_GOT" typeName="BOOL"/>
		<variables name="A_CH108_iK" typeName="BOOL"/>
		<variables name="A_CH108_IND" typeName="BYTE"/>
		<variables name="A_CH108_KL_IM" typeName="BOOL"/>
		<variables name="A_CH108_KM" typeName="BOOL"/>
		<variables name="A_CH108_KP" typeName="BOOL"/>
		<variables name="A_CH108_KS" typeName="BOOL"/>
		<variables name="A_CH108_KS_V1" typeName="BOOL"/>
		<variables name="A_CH108_MOT" typeName="BOOL"/>
		<variables name="A_CH108_MS" typeName="BOOL"/>
		<variables name="A_CH108_MS1" typeName="BOOL"/>
		<variables name="A_CH108_MS_V1" typeName="BOOL"/>
		<variables name="A_CH108_MV1" typeName="BOOL"/>
		<variables name="A_CH108_N" typeName="BOOL"/>
		<variables name="A_CH108_NM" typeName="BOOL"/>
		<variables name="A_CH108_OT" typeName="BOOL"/>
		<variables name="A_CH108_OTM" typeName="BOOL"/>
		<variables name="A_CH108_PV1" typeName="BOOL"/>
		<variables name="A_CH108_S" typeName="BOOL"/>
		<variables name="A_CH108_S1" typeName="BOOL"/>
		<variables name="A_CH108_S_V1" typeName="BOOL"/>
		<variables name="A_CH108_UK" typeName="BOOL"/>
		<variables name="A_CH108_VCH" typeName="BOOL"/>
		<variables name="A_CH108_VCHM" typeName="BOOL"/>
		<variables name="A_CH108_ZL_IM" typeName="BOOL"/>
		<variables name="A_CH108MS_K" typeName="EBOOL"/>
		<variables name="A_CH108MS_U" typeName="EBOOL"/>
		<variables name="A_CH108MS_U_OBR" typeName="BOOL"/>
		<variables name="A_CH108MS_U_PR" typeName="BOOL"/>
		<variables name="A_CH108MS_UK" typeName="EBOOL"/>
		<variables name="A_CH108O_K" typeName="EBOOL"/>
		<variables name="A_CH108ON" typeName="BOOL"/>
		<variables name="A_CH108P_ARM_DSP" typeName="BOOL"/>
		<variables name="A_CH108S_K" typeName="BOOL"/>
		<variables name="A_CH108S_U" typeName="EBOOL"/>
		<variables name="A_CH201_ARM_DSP" typeName="BOOL"/>
		<variables name="A_CH201_BL_IM" typeName="BOOL"/>
		<variables name="A_CH201_GOT" typeName="BOOL"/>
		<variables name="A_CH201_iK" typeName="BOOL"/>
		<variables name="A_CH201_IND" typeName="BYTE"/>
		<variables name="A_CH201_KL_IM" typeName="BOOL"/>
		<variables name="A_CH201_KM" typeName="BOOL"/>
		<variables name="A_CH201_KP" typeName="BOOL"/>
		<variables name="A_CH201_KS" typeName="BOOL"/>
		<variables name="A_CH201_KS_V1" typeName="BOOL"/>
		<variables name="A_CH201_MOT" typeName="BOOL"/>
		<variables name="A_CH201_MS" typeName="BOOL"/>
		<variables name="A_CH201_MS1" typeName="BOOL"/>
		<variables name="A_CH201_MS_V1" typeName="BOOL"/>
		<variables name="A_CH201_MV1" typeName="BOOL"/>
		<variables name="A_CH201_N" typeName="BOOL"/>
		<variables name="A_CH201_NM" typeName="BOOL"/>
		<variables name="A_CH201_OT" typeName="BOOL"/>
		<variables name="A_CH201_OTM" typeName="BOOL"/>
		<variables name="A_CH201_PV1" typeName="BOOL"/>
		<variables name="A_CH201_S" typeName="BOOL"/>
		<variables name="A_CH201_S1" typeName="BOOL"/>
		<variables name="A_CH201_S_V1" typeName="BOOL"/>
		<variables name="A_CH201_UK" typeName="BOOL"/>
		<variables name="A_CH201_VCH" typeName="BOOL"/>
		<variables name="A_CH201_VCHM" typeName="BOOL"/>
		<variables name="A_CH201_ZL_IM" typeName="BOOL"/>
		<variables name="A_CH201MS_K" typeName="EBOOL"/>
		<variables name="A_CH201MS_U" typeName="EBOOL"/>
		<variables name="A_CH201MS_U_OBR" typeName="BOOL"/>
		<variables name="A_CH201MS_U_PR" typeName="BOOL"/>
		<variables name="A_CH201MS_UK" typeName="EBOOL"/>
		<variables name="A_CH201O_K" typeName="EBOOL"/>
		<variables name="A_CH201ON" typeName="BOOL"/>
		<variables name="A_CH201P_ARM_DSP" typeName="BOOL"/>
		<variables name="A_CH201S_K" typeName="EBOOL"/>
		<variables name="A_CH201S_U" typeName="EBOOL"/>
		<variables name="A_CH202_ARM_DSP" typeName="BOOL"/>
		<variables name="A_CH202_BL_IM" typeName="BOOL"/>
		<variables name="A_CH202_GOT" typeName="BOOL"/>
		<variables name="A_CH202_iK" typeName="BOOL"/>
		<variables name="A_CH202_IND" typeName="BYTE"/>
		<variables name="A_CH202_KL_IM" typeName="BOOL"/>
		<variables name="A_CH202_KM" typeName="BOOL"/>
		<variables name="A_CH202_KP" typeName="BOOL"/>
		<variables name="A_CH202_KS" typeName="BOOL"/>
		<variables name="A_CH202_KS_V1" typeName="BOOL"/>
		<variables name="A_CH202_MOT" typeName="BOOL"/>
		<variables name="A_CH202_MS" typeName="BOOL"/>
		<variables name="A_CH202_MS1" typeName="BOOL"/>
		<variables name="A_CH202_MS_V1" typeName="BOOL"/>
		<variables name="A_CH202_MV1" typeName="BOOL"/>
		<variables name="A_CH202_N" typeName="BOOL"/>
		<variables name="A_CH202_NM" typeName="BOOL"/>
		<variables name="A_CH202_OT" typeName="BOOL"/>
		<variables name="A_CH202_OTM" typeName="BOOL"/>
		<variables name="A_CH202_PV1" typeName="BOOL"/>
		<variables name="A_CH202_S" typeName="BOOL"/>
		<variables name="A_CH202_S1" typeName="BOOL"/>
		<variables name="A_CH202_S_V1" typeName="BOOL"/>
		<variables name="A_CH202_UK" typeName="BOOL"/>
		<variables name="A_CH202_VCH" typeName="BOOL"/>
		<variables name="A_CH202_VCHM" typeName="BOOL"/>
		<variables name="A_CH202_ZL_IM" typeName="BOOL"/>
		<variables name="A_CH202MS_K" typeName="EBOOL"/>
		<variables name="A_CH202MS_U" typeName="EBOOL"/>
		<variables name="A_CH202MS_U_OBR" typeName="BOOL"/>
		<variables name="A_CH202MS_U_PR" typeName="BOOL"/>
		<variables name="A_CH202MS_UK" typeName="EBOOL"/>
		<variables name="A_CH202O_K" typeName="EBOOL"/>
		<variables name="A_CH202ON" typeName="BOOL"/>
		<variables name="A_CH202P_ARM_DSP" typeName="BOOL"/>
		<variables name="A_CH202S_K" typeName="EBOOL"/>
		<variables name="A_CH202S_U" typeName="EBOOL"/>
		<variables name="A_CH203_ARM_DSP" typeName="BOOL"/>
		<variables name="A_CH203_BL_IM" typeName="BOOL"/>
		<variables name="A_CH203_GOT" typeName="BOOL"/>
		<variables name="A_CH203_iK" typeName="BOOL"/>
		<variables name="A_CH203_IND" typeName="BYTE"/>
		<variables name="A_CH203_KL_IM" typeName="BOOL"/>
		<variables name="A_CH203_KM" typeName="BOOL"/>
		<variables name="A_CH203_KP" typeName="BOOL"/>
		<variables name="A_CH203_KS" typeName="BOOL"/>
		<variables name="A_CH203_KS_V1" typeName="BOOL"/>
		<variables name="A_CH203_MOT" typeName="BOOL"/>
		<variables name="A_CH203_MS" typeName="BOOL"/>
		<variables name="A_CH203_MS1" typeName="BOOL"/>
		<variables name="A_CH203_MS_V1" typeName="BOOL"/>
		<variables name="A_CH203_MV1" typeName="BOOL"/>
		<variables name="A_CH203_N" typeName="BOOL"/>
		<variables name="A_CH203_NM" typeName="BOOL"/>
		<variables name="A_CH203_OT" typeName="BOOL"/>
		<variables name="A_CH203_OTM" typeName="BOOL"/>
		<variables name="A_CH203_PV1" typeName="BOOL"/>
		<variables name="A_CH203_S" typeName="BOOL"/>
		<variables name="A_CH203_S1" typeName="BOOL"/>
		<variables name="A_CH203_S_V1" typeName="BOOL"/>
		<variables name="A_CH203_UK" typeName="BOOL"/>
		<variables name="A_CH203_VCH" typeName="BOOL"/>
		<variables name="A_CH203_VCHM" typeName="BOOL"/>
		<variables name="A_CH203_ZL_IM" typeName="BOOL"/>
		<variables name="A_CH203MS_K" typeName="EBOOL"/>
		<variables name="A_CH203MS_U" typeName="EBOOL"/>
		<variables name="A_CH203MS_U_OBR" typeName="BOOL"/>
		<variables name="A_CH203MS_U_PR" typeName="BOOL"/>
		<variables name="A_CH203MS_UK" typeName="EBOOL"/>
		<variables name="A_CH203O_K" typeName="EBOOL"/>
		<variables name="A_CH203ON" typeName="BOOL"/>
		<variables name="A_CH203P_ARM_DSP" typeName="BOOL"/>
		<variables name="A_CH203S_K" typeName="BOOL"/>
		<variables name="A_CH203S_U" typeName="EBOOL"/>
		<variables name="A_ch_chm" typeName="BOOL"/>
		<variables name="A_CH_ik" typeName="BOOL"/>
		<variables name="A_CH_KP" typeName="BOOL"/>
		<variables name="A_CH_KS" typeName="BOOL"/>
		<variables name="A_CH_MK_UK" typeName="BOOL"/>
		<variables name="A_CH_MK_VCH" typeName="BOOL"/>
		<variables name="A_CH_MK_VCHM" typeName="BOOL"/>
		<variables name="A_CH_N" typeName="BOOL"/>
		<variables name="A_CH_OT" typeName="BOOL"/>
		<variables name="A_chdifp_schet_online" typeName="INT"/>
		<variables name="A_CHDKMG_K" typeName="EBOOL">
			<comment>???????? ???? ?????</comment>
			<attribute name="TimeStampCapability" value="0"/>
		</variables>
		<variables name="A_CHDMG_NEISPR_IM" typeName="BOOL"/>
		<variables name="A_chdps_schet_online" typeName="INT"/>
		<variables name="A_chifp_schet_online" typeName="INT"/>
		<variables name="A_CHKMG_K" typeName="EBOOL">
			<comment>???????? ???? ????</comment>
			<attribute name="TimeStampCapability" value="0"/>
		</variables>
		<variables name="A_chm" typeName="BOOL"/>
		<variables name="A_CHMG_NEISPR_IM" typeName="BOOL"/>
		<variables name="A_CHPI_K" typeName="EBOOL"/>
		<variables name="A_CHPI_U" typeName="EBOOL"/>
		<variables name="A_CHPI_U_OBR" typeName="BOOL"/>
		<variables name="A_CHPI_U_PR" typeName="BOOL"/>
		<variables name="A_CHPI_UK" typeName="EBOOL"/>
		<variables name="A_chps_schet_online" typeName="INT"/>
		<variables name="A_CHS_K" typeName="BOOL"/>
		<variables name="A_CHZ108A_1JL_IM" typeName="BOOL"/>
		<variables name="A_CHZ108A_2JL_IM" typeName="BOOL"/>
		<variables name="A_CHZ108A_BL_IM" typeName="BOOL"/>
		<variables name="A_CHZ108A_KL_IM" typeName="BOOL"/>
		<variables name="A_CHZ108A_ZL_IM" typeName="BOOL"/>
		<variables name="A_CHZ108AMS_K" typeName="EBOOL"/>
		<variables name="A_CHZ108AMS_U" typeName="EBOOL"/>
		<variables name="A_CHZ108AMS_U_OBR" typeName="BOOL"/>
		<variables name="A_CHZ108AMS_U_PR" typeName="BOOL"/>
		<variables name="A_CHZ108AMS_UK" typeName="EBOOL"/>
		<variables name="A_CHZ109A_1JL_IM" typeName="BOOL"/>
		<variables name="A_CHZ109A_2JL_IM" typeName="BOOL"/>
		<variables name="A_CHZ109A_BL_IM" typeName="BOOL"/>
		<variables name="A_CHZ109A_KL_IM" typeName="BOOL"/>
		<variables name="A_CHZ109A_ZL_IM" typeName="BOOL"/>
		<variables name="A_CHZ109AMS_K" typeName="EBOOL"/>
		<variables name="A_CHZ109AMS_U" typeName="EBOOL"/>
		<variables name="A_CHZ109AMS_U_OBR" typeName="BOOL"/>
		<variables name="A_CHZ109AMS_U_PR" typeName="BOOL"/>
		<variables name="A_CHZ109AMS_UK" typeName="EBOOL"/>
		<variables name="A_CONST_1" typeName="BOOL"/>
		<variables name="A_DIAG_RIO" typeName="WORD"/>
		<variables name="A_DN_ARM_DSP" typeName="BOOL"/>
		<variables name="A_DN_JL" typeName="BOOL"/>
		<variables name="A_DN_K" typeName="EBOOL">
			<comment>???????? ???? ??</comment>
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_DN_U" typeName="EBOOL"/>
		<variables name="A_DOG" typeName="BOOL"/>
		<variables name="A_DSN10_K" typeName="EBOOL"/>
		<variables name="A_DSN10_U" typeName="EBOOL"/>
		<variables name="A_DSN10_U_OBR" typeName="BOOL"/>
		<variables name="A_DSN10_U_PR" typeName="BOOL"/>
		<variables name="A_DSN10_UK" typeName="EBOOL"/>
		<variables name="A_DSN13_K" typeName="EBOOL"/>
		<variables name="A_DSN13_U" typeName="EBOOL"/>
		<variables name="A_DSN13_U_OBR" typeName="BOOL"/>
		<variables name="A_DSN13_U_PR" typeName="BOOL"/>
		<variables name="A_DSN13_UK" typeName="EBOOL"/>
		<variables name="A_DSP1_ETH_SUCCESS" typeName="BOOL"/>
		<variables name="A_DSP2_ETH_SUCCESS" typeName="BOOL"/>
		<variables name="A_FID_NEISPR" typeName="BOOL"/>
		<variables name="A_FID_NEISPR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_GIR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_gir_schet_online" typeName="INT" topologicalAddress="%MW1031">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_GOT" typeName="BOOL"/>
		<variables name="A_GRI" typeName="BOOL"/>
		<variables name="A_GRI1" typeName="BOOL"/>
		<variables name="A_GRK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_grk_schet_online" typeName="INT"/>
		<variables name="A_i_CH_Z" typeName="BOOL"/>
		<variables name="A_i_CHM_B" typeName="BOOL"/>
		<variables name="A_i_IR" typeName="BOOL"/>
		<variables name="A_i_IRM" typeName="BOOL"/>
		<variables name="A_i_N_Z" typeName="BOOL"/>
		<variables name="A_i_NM_B" typeName="BOOL"/>
		<variables name="A_i_O" typeName="BOOL"/>
		<variables name="A_i_OM" typeName="BOOL"/>
		<variables name="A_i_OMM" typeName="BOOL"/>
		<variables name="A_i_OP" typeName="BOOL"/>
		<variables name="A_i_OPM" typeName="BOOL"/>
		<variables name="A_i_OS" typeName="BOOL"/>
		<variables name="A_i_OSM" typeName="BOOL"/>
		<variables name="A_IG32_34P_ST94" typeName="WORD"/>
		<variables name="A_IG_1_5P_ST37" typeName="WORD"/>
		<variables name="A_IG_1AP_ST61" typeName="WORD"/>
		<variables name="A_IG_1SP_M1" typeName="WORD"/>
		<variables name="A_IG_2AP_ST32" typeName="WORD"/>
		<variables name="A_IG_3_5SP_STR3" typeName="WORD"/>
		<variables name="A_IG_4_84SP_STR84" typeName="WORD"/>
		<variables name="A_IG_4_124P_ST108" typeName="WORD"/>
		<variables name="A_IG_5_9P_ST26" typeName="WORD"/>
		<variables name="A_IG_5_201P_ST48" typeName="WORD"/>
		<variables name="A_IG_6_8P_ST70" typeName="WORD"/>
		<variables name="A_IG_6_84P_ST118" typeName="WORD"/>
		<variables name="A_IG_6P_ST73" typeName="WORD"/>
		<variables name="A_IG_6SP_M20" typeName="WORD"/>
		<variables name="A_IG_7AP_ST83" typeName="WORD"/>
		<variables name="A_IG_7P_ST85" typeName="WORD"/>
		<variables name="A_IG_8_16P_ST92" typeName="WORD"/>
		<variables name="A_IG_8CHP_ST6" typeName="WORD"/>
		<variables name="A_IG_8NP_ST7" typeName="WORD"/>
		<variables name="A_IG_8SP_STR8" typeName="WORD"/>
		<variables name="A_IG_9SP_M11" typeName="WORD"/>
		<variables name="A_IG_10_12P_ST68" typeName="WORD"/>
		<variables name="A_IG_11_19SP_ST4" typeName="WORD"/>
		<variables name="A_IG_11_19SP_STR11" typeName="WORD"/>
		<variables name="A_IG_12_14P_ST66" typeName="WORD"/>
		<variables name="A_IG_12_302_ST79" typeName="WORD"/>
		<variables name="A_IG_12SP_STR12" typeName="WORD"/>
		<variables name="A_IG_13SP_ST39" typeName="WORD"/>
		<variables name="A_IG_13SP_STR13" typeName="WORD"/>
		<variables name="A_IG_14SP_STR14" typeName="WORD"/>
		<variables name="A_IG_15_117P_ST120" typeName="WORD"/>
		<variables name="A_IG_15SP_M13" typeName="WORD"/>
		<variables name="A_IG_16SP_M44" typeName="WORD"/>
		<variables name="A_IG_18_26SP_STR18" typeName="WORD"/>
		<variables name="A_IG_20_22SP_STR20" typeName="WORD"/>
		<variables name="A_IG_22_32P_ST89" typeName="WORD"/>
		<variables name="A_IG_23_29P_ST29" typeName="WORD"/>
		<variables name="A_IG_23SP_ST28" typeName="WORD"/>
		<variables name="A_IG_24_42AP_M78" typeName="WORD"/>
		<variables name="A_IG_24_42P_ST63" typeName="WORD"/>
		<variables name="A_IG_24SP_M52" typeName="WORD"/>
		<variables name="A_IG_24SP_STR24" typeName="WORD"/>
		<variables name="A_IG_25_27SP_ST39" typeName="WORD"/>
		<variables name="A_IG_25_31P_ST58" typeName="WORD"/>
		<variables name="A_IG_26_40AP_M76" typeName="WORD"/>
		<variables name="A_IG_26_40BP_ST34" typeName="WORD"/>
		<variables name="A_IG_27_101P_ST46" typeName="WORD"/>
		<variables name="A_IG_28_302P_ST76" typeName="WORD"/>
		<variables name="A_IG_28SP_M66" typeName="WORD"/>
		<variables name="A_IG_28SP_STR28" typeName="WORD"/>
		<variables name="A_IG_29SP_M23" typeName="WORD"/>
		<variables name="A_IG_30SP_STR30" typeName="WORD"/>
		<variables name="A_IG_31SP_STR31" typeName="WORD"/>
		<variables name="A_IG_32SP_M100" typeName="WORD"/>
		<variables name="A_IG_33_43SP_M29" typeName="WORD"/>
		<variables name="A_IG_34_36SP_M60" typeName="WORD"/>
		<variables name="A_IG_35SP_STR35" typeName="WORD"/>
		<variables name="A_IG_37_39SP_M63" typeName="WORD"/>
		<variables name="A_IG_38SP_STR38" typeName="WORD"/>
		<variables name="A_IG_39_51P_ST81" typeName="WORD"/>
		<variables name="A_IG_43_32P_ST87" typeName="WORD"/>
		<variables name="A_IG_47_49SP_M39" typeName="WORD"/>
		<variables name="A_IG_47_49SP_ST14" typeName="WORD"/>
		<variables name="A_IG_51SP_STR51" typeName="WORD"/>
		<variables name="A_IG_53SP_M57" typeName="WORD"/>
		<variables name="A_IG_98_104SP_STR104" typeName="WORD"/>
		<variables name="A_IG_98_122P_ST154" typeName="WORD"/>
		<variables name="A_IG_100SP_M118" typeName="WORD"/>
		<variables name="A_IG_101_103SP_STR101" typeName="WORD"/>
		<variables name="A_IG_101_103SP_STR103" typeName="WORD"/>
		<variables name="A_IG_101P_ST97" typeName="WORD"/>
		<variables name="A_IG_101P_ST128" typeName="WORD"/>
		<variables name="A_IG_102_120SP_STR102" typeName="WORD"/>
		<variables name="A_IG_102_140P_ST114" typeName="WORD"/>
		<variables name="A_IG_102P_ST99" typeName="WORD"/>
		<variables name="A_IG_103P_ST101" typeName="WORD"/>
		<variables name="A_IG_104P_ST103" typeName="WORD"/>
		<variables name="A_IG_105AP_ST112" typeName="WORD"/>
		<variables name="A_IG_105P_ST116" typeName="WORD"/>
		<variables name="A_IG_105SP_ST10" typeName="WORD"/>
		<variables name="A_IG_106_108SP_STR106" typeName="WORD"/>
		<variables name="A_IG_106AP_ST125" typeName="WORD"/>
		<variables name="A_IG_106P_ST123" typeName="WORD"/>
		<variables name="A_IG_107_121SP_M105" typeName="WORD"/>
		<variables name="A_IG_107P_ST43" typeName="WORD"/>
		<variables name="A_IG_108AP_ST132" typeName="WORD"/>
		<variables name="A_IG_108P_ST129" typeName="WORD"/>
		<variables name="A_IG_109AP_ST137" typeName="WORD"/>
		<variables name="A_IG_109SP_ST11" typeName="WORD"/>
		<variables name="A_IG_110_112SP_STR110" typeName="WORD"/>
		<variables name="A_IG_111_113SP_ST21" typeName="WORD"/>
		<variables name="A_IG_114SP_STR114" typeName="WORD"/>
		<variables name="A_IG_115SP_ST15" typeName="WORD"/>
		<variables name="A_IG_116_124P_ST110" typeName="WORD"/>
		<variables name="A_IG_116_124P_ST111" typeName="WORD"/>
		<variables name="A_IG_116SP_ST105" typeName="WORD"/>
		<variables name="A_IG_117AP_ST122" typeName="WORD"/>
		<variables name="A_IG_117B_ST12" typeName="WORD"/>
		<variables name="A_IG_117P_ST126" typeName="WORD"/>
		<variables name="A_IG_117SP_M67" typeName="WORD"/>
		<variables name="A_IG_117SP_STR117" typeName="WORD"/>
		<variables name="A_IG_119_121P_ST40" typeName="WORD"/>
		<variables name="A_IG_119SP_STR119" typeName="WORD"/>
		<variables name="A_IG_122_160P_ST141" typeName="WORD"/>
		<variables name="A_IG_122SP_M154" typeName="WORD"/>
		<variables name="A_IG_124_138SP_STR124" typeName="WORD"/>
		<variables name="A_IG_128_134SP_STR128" typeName="WORD"/>
		<variables name="A_IG_128P_M146" typeName="WORD"/>
		<variables name="A_IG_128P_STR128" typeName="WORD"/>
		<variables name="A_IG_130_164ST_STR130" typeName="WORD"/>
		<variables name="A_IG_136SP_M140" typeName="WORD"/>
		<variables name="A_IG_140SP_ST113" typeName="WORD"/>
		<variables name="A_IG_142SP_ST16" typeName="WORD"/>
		<variables name="A_IG_144_146SP_STR144" typeName="WORD"/>
		<variables name="A_IG_148SP_M116" typeName="WORD"/>
		<variables name="A_IG_160_162P_ST135" typeName="WORD"/>
		<variables name="A_IG_160SP_M174" typeName="WORD"/>
		<variables name="A_IG_162SP_M120" typeName="WORD"/>
		<variables name="A_IG_166P_ST131" typeName="WORD"/>
		<variables name="A_IG_166SP_STR166" typeName="WORD"/>
		<variables name="A_IG_201_203SP_M203" typeName="WORD"/>
		<variables name="A_IG_201P_ST54" typeName="WORD"/>
		<variables name="A_IG_202_212_ST52" typeName="WORD"/>
		<variables name="A_IG_202P_ST50" typeName="WORD"/>
		<variables name="A_IG_202SP_M206" typeName="WORD"/>
		<variables name="A_IG_203P_ST56" typeName="WORD"/>
		<variables name="A_IG_204_212SP_STR204" typeName="WORD"/>
		<variables name="A_IG_206SP_M212" typeName="WORD"/>
		<variables name="A_IG_302SP_STR302" typeName="WORD"/>
		<variables name="A_IG_BP_ST106" typeName="WORD"/>
		<variables name="A_IG_CH101_STR103" typeName="WORD"/>
		<variables name="A_IG_CH102_STR115" typeName="WORD"/>
		<variables name="A_IG_CH103_STR115" typeName="WORD"/>
		<variables name="A_IG_CH104_STR113" typeName="WORD"/>
		<variables name="A_IG_CH105_STR111" typeName="WORD"/>
		<variables name="A_IG_CH106_STR109" typeName="WORD"/>
		<variables name="A_IG_CH107_STR109" typeName="WORD"/>
		<variables name="A_IG_CH108_STR107" typeName="WORD"/>
		<variables name="A_IG_CH201_STR203" typeName="WORD"/>
		<variables name="A_IG_CH202_STR203" typeName="WORD"/>
		<variables name="A_IG_CH203_STR201" typeName="WORD"/>
		<variables name="A_IG_M1_ST36" typeName="WORD"/>
		<variables name="A_IG_M3_STR1" typeName="WORD"/>
		<variables name="A_IG_M5_STR5" typeName="WORD"/>
		<variables name="A_IG_M7_ST25" typeName="WORD"/>
		<variables name="A_IG_M9_STR9" typeName="WORD"/>
		<variables name="A_IG_M10_6_84P" typeName="WORD"/>
		<variables name="A_IG_M10_ST119" typeName="WORD"/>
		<variables name="A_IG_M11_ST27" typeName="WORD"/>
		<variables name="A_IG_M12_4_84SP" typeName="WORD"/>
		<variables name="A_IG_M13_ST38" typeName="WORD"/>
		<variables name="A_IG_M14_ST115" typeName="WORD"/>
		<variables name="A_IG_M15_STR19" typeName="WORD"/>
		<variables name="A_IG_M16_ST109" typeName="WORD"/>
		<variables name="A_IG_M17_STR19" typeName="WORD"/>
		<variables name="A_IG_M18_STR6" typeName="WORD"/>
		<variables name="A_IG_M19_23SP" typeName="WORD"/>
		<variables name="A_IG_M20_ST71" typeName="WORD"/>
		<variables name="A_IG_M21_STR25" typeName="WORD"/>
		<variables name="A_IG_M22_8SP" typeName="WORD"/>
		<variables name="A_IG_M23_ST30" typeName="WORD"/>
		<variables name="A_IG_M25_ST59" typeName="WORD"/>
		<variables name="A_IG_M26_ST69" typeName="WORD"/>
		<variables name="A_IG_M27_31SP" typeName="WORD"/>
		<variables name="A_IG_M27_M29" typeName="BOOL"/>
		<variables name="A_IG_M28_ST93" typeName="WORD"/>
		<variables name="A_IG_M28_STR8" typeName="WORD"/>
		<variables name="A_IG_M29_M27" typeName="BOOL"/>
		<variables name="A_IG_M29_ST60" typeName="WORD"/>
		<variables name="A_IG_M30_12SP" typeName="WORD"/>
		<variables name="A_IG_M32_ST67" typeName="WORD"/>
		<variables name="A_IG_M33_STR41" typeName="WORD"/>
		<variables name="A_IG_M34_STR16" typeName="WORD"/>
		<variables name="A_IG_M35_ST32" typeName="WORD"/>
		<variables name="A_IG_M35_STR37" typeName="WORD"/>
		<variables name="A_IG_M36_14SP" typeName="WORD"/>
		<variables name="A_IG_M37_STR43" typeName="WORD"/>
		<variables name="A_IG_M38_ST143" typeName="WORD"/>
		<variables name="A_IG_M38_STR102" typeName="WORD"/>
		<variables name="A_IG_M39_ST14" typeName="WORD"/>
		<variables name="A_IG_M40_18_26SP" typeName="WORD"/>
		<variables name="A_IG_M41_STR39" typeName="WORD"/>
		<variables name="A_IG_M42_20_22SP" typeName="WORD"/>
		<variables name="A_IG_M43_STR39" typeName="WORD"/>
		<variables name="A_IG_M44_ST91" typeName="WORD"/>
		<variables name="A_IG_M45_STR47" typeName="WORD"/>
		<variables name="A_IG_M46_STR36" typeName="WORD"/>
		<variables name="A_IG_M47_STR49" typeName="WORD"/>
		<variables name="A_IG_M48_STR36" typeName="WORD"/>
		<variables name="A_IG_M49_STR49" typeName="WORD"/>
		<variables name="A_IG_M50_ST35" typeName="WORD"/>
		<variables name="A_IG_M52_ST64" typeName="WORD"/>
		<variables name="A_IG_M53_ST82" typeName="WORD"/>
		<variables name="A_IG_M54_ST90" typeName="WORD"/>
		<variables name="A_IG_M55_51SP" typeName="WORD"/>
		<variables name="A_IG_M56_STR32" typeName="WORD"/>
		<variables name="A_IG_M57_ST84" typeName="WORD"/>
		<variables name="A_IG_M58_STR32" typeName="WORD"/>
		<variables name="A_IG_M59_STR53" typeName="WORD"/>
		<variables name="A_IG_M60_ST95" typeName="WORD"/>
		<variables name="A_IG_M61_STR53" typeName="WORD"/>
		<variables name="A_IG_M62_STR28" typeName="WORD"/>
		<variables name="A_IG_M63_ST31" typeName="WORD"/>
		<variables name="A_IG_M64_38SP" typeName="WORD"/>
		<variables name="A_IG_M65_STR15" typeName="WORD"/>
		<variables name="A_IG_M66_ST75" typeName="WORD"/>
		<variables name="A_IG_M67_ST121" typeName="WORD"/>
		<variables name="A_IG_M68_26_40AP" typeName="WORD"/>
		<variables name="A_IG_M69_STR27" typeName="WORD"/>
		<variables name="A_IG_M70_24_42AP" typeName="WORD"/>
		<variables name="A_IG_M72_ST86" typeName="WORD"/>
		<variables name="A_IG_M74_ST74" typeName="WORD"/>
		<variables name="A_IG_M76_ST32" typeName="WORD"/>
		<variables name="A_IG_M78_ST62" typeName="WORD"/>
		<variables name="A_IG_M80_STR100" typeName="WORD"/>
		<variables name="A_IG_M82_STR148" typeName="WORD"/>
		<variables name="A_IG_M97_117B" typeName="WORD"/>
		<variables name="A_IG_M100_ST88" typeName="WORD"/>
		<variables name="A_IG_M101_STR107" typeName="WORD"/>
		<variables name="A_IG_M102_110_112SP" typeName="WORD"/>
		<variables name="A_IG_M103_119SP" typeName="WORD"/>
		<variables name="A_IG_M104_STR140" typeName="WORD"/>
		<variables name="A_IG_M105_ST41" typeName="WORD"/>
		<variables name="A_IG_M106_ST98" typeName="WORD"/>
		<variables name="A_IG_M107_STR117" typeName="WORD"/>
		<variables name="A_IG_M108_ST100" typeName="WORD"/>
		<variables name="A_IG_M109_ST47" typeName="WORD"/>
		<variables name="A_IG_M110_ST102" typeName="WORD"/>
		<variables name="A_IG_M110_STR114" typeName="WORD"/>
		<variables name="A_IG_M112_ST104" typeName="WORD"/>
		<variables name="A_IG_M114_ST117" typeName="WORD"/>
		<variables name="A_IG_M116_ST124" typeName="WORD"/>
		<variables name="A_IG_M118_ST44" typeName="WORD"/>
		<variables name="A_IG_M120_ST130" typeName="WORD"/>
		<variables name="A_IG_M122_STR120" typeName="WORD"/>
		<variables name="A_IG_M124_124_138SP" typeName="WORD"/>
		<variables name="A_IG_M126_ST111" typeName="WORD"/>
		<variables name="A_IG_M128_ST107" typeName="WORD"/>
		<variables name="A_IG_M130_STR116" typeName="WORD"/>
		<variables name="A_IG_M132_STR116" typeName="WORD"/>
		<variables name="A_IG_M134_STR136" typeName="WORD"/>
		<variables name="A_IG_M136_ST138" typeName="WORD"/>
		<variables name="A_IG_M142_ST153" typeName="WORD"/>
		<variables name="A_IG_M144_ST133" typeName="WORD"/>
		<variables name="A_IG_M146_ST127" typeName="WORD"/>
		<variables name="A_IG_M148_106AP" typeName="WORD"/>
		<variables name="A_IG_M148_STR134" typeName="WORD"/>
		<variables name="A_IG_M150_ST155" typeName="WORD"/>
		<variables name="A_IG_M152_STR160" typeName="WORD"/>
		<variables name="A_IG_M154_ST142" typeName="WORD"/>
		<variables name="A_IG_M158_STR122" typeName="WORD"/>
		<variables name="A_IG_M160_STR142" typeName="WORD"/>
		<variables name="A_IG_M162_166SP" typeName="WORD"/>
		<variables name="A_IG_M164_ST42" typeName="WORD"/>
		<variables name="A_IG_M166_ST134" typeName="WORD"/>
		<variables name="A_IG_M168_STR162" typeName="WORD"/>
		<variables name="A_IG_M172_STR160" typeName="WORD"/>
		<variables name="A_IG_M174_ST136" typeName="WORD"/>
		<variables name="A_IG_M176_STR162" typeName="WORD"/>
		<variables name="A_IG_M201_STR5" typeName="WORD"/>
		<variables name="A_IG_M202_STR202" typeName="WORD"/>
		<variables name="A_IG_M203_ST49" typeName="WORD"/>
		<variables name="A_IG_M204_STR202" typeName="WORD"/>
		<variables name="A_IG_M206_ST53" typeName="WORD"/>
		<variables name="A_IG_M208_STR206" typeName="WORD"/>
		<variables name="A_IG_M210_ST52" typeName="WORD"/>
		<variables name="A_IG_M210_STR212" typeName="WORD"/>
		<variables name="A_IG_M212_ST55" typeName="WORD"/>
		<variables name="A_IG_M214_ST51" typeName="WORD"/>
		<variables name="A_IG_M216_ST57" typeName="WORD"/>
		<variables name="A_IG_M218_STR212" typeName="WORD"/>
		<variables name="A_IG_M302P_ST78" typeName="WORD"/>
		<variables name="A_IG_M304_ST77" typeName="WORD"/>
		<variables name="A_IG_M306_ST80" typeName="WORD"/>
		<variables name="A_IG_M308_STR12" typeName="WORD"/>
		<variables name="A_IG_M977_117B" typeName="WORD"/>
		<variables name="A_IG_MK_8NP" typeName="WORD"/>
		<variables name="A_IG_MK_NP" typeName="WORD"/>
		<variables name="A_IG_NP_MK" typeName="WORD"/>
		<variables name="A_IG_ST1_STR1" typeName="WORD"/>
		<variables name="A_IG_ST2_STR11" typeName="WORD"/>
		<variables name="A_IG_ST2_STR13" typeName="WORD"/>
		<variables name="A_IG_ST3_STR15" typeName="WORD"/>
		<variables name="A_IG_ST4_STR9" typeName="WORD"/>
		<variables name="A_IG_ST5_STR29" typeName="WORD"/>
		<variables name="A_IG_ST6_M302P" typeName="WORD"/>
		<variables name="A_IG_ST7_8CHP" typeName="WORD"/>
		<variables name="A_IG_ST9_STR27" typeName="WORD"/>
		<variables name="A_IG_ST10_STR103" typeName="WORD"/>
		<variables name="A_IG_ST11_STR105" typeName="WORD"/>
		<variables name="A_IG_ST12_117AP" typeName="WORD"/>
		<variables name="A_IG_ST13_STR24" typeName="WORD"/>
		<variables name="A_IG_ST14_STR43" typeName="WORD"/>
		<variables name="A_IG_ST15_STR113" typeName="WORD"/>
		<variables name="A_IG_ST16_142SP" typeName="WORD"/>
		<variables name="A_IG_ST16_144_146SP" typeName="WORD"/>
		<variables name="A_IG_ST17_STR142" typeName="WORD"/>
		<variables name="A_IG_ST18_114SP" typeName="WORD"/>
		<variables name="A_IG_ST19_128_134SP" typeName="WORD"/>
		<variables name="A_IG_ST20_STR120" typeName="WORD"/>
		<variables name="A_IG_ST21_STR105" typeName="WORD"/>
		<variables name="A_IG_ST22_STR146" typeName="WORD"/>
		<variables name="A_IG_ST22_STR148" typeName="WORD"/>
		<variables name="A_IG_ST23_106_108SP" typeName="WORD"/>
		<variables name="A_IG_ST24_STR122" typeName="WORD"/>
		<variables name="A_IG_ST25_2GP" typeName="WORD"/>
		<variables name="A_IG_ST26_M5" typeName="WORD"/>
		<variables name="A_IG_ST27_5_9P" typeName="WORD"/>
		<variables name="A_IG_ST28_STR17" typeName="WORD"/>
		<variables name="A_IG_ST29_M19" typeName="WORD"/>
		<variables name="A_IG_ST30_23_29P" typeName="WORD"/>
		<variables name="A_IG_ST31_35SP" typeName="WORD"/>
		<variables name="A_IG_ST32_M35" typeName="WORD"/>
		<variables name="A_IG_ST33_2AP" typeName="WORD"/>
		<variables name="A_IG_ST34_M68" typeName="WORD"/>
		<variables name="A_IG_ST35_26_40BP" typeName="WORD"/>
		<variables name="A_IG_ST36_1GP" typeName="WORD"/>
		<variables name="A_IG_ST37_M3" typeName="WORD"/>
		<variables name="A_IG_ST38_1_5P" typeName="WORD"/>
		<variables name="A_IG_ST39_13SP" typeName="WORD"/>
		<variables name="A_IG_ST40_M103" typeName="WORD"/>
		<variables name="A_IG_ST41_119_121P" typeName="WORD"/>
		<variables name="A_IG_ST42_STR121" typeName="WORD"/>
		<variables name="A_IG_ST43_CH107" typeName="WORD"/>
		<variables name="A_IG_ST44_107P" typeName="WORD"/>
		<variables name="A_IG_ST45_STR100" typeName="WORD"/>
		<variables name="A_IG_ST46_M69" typeName="WORD"/>
		<variables name="A_IG_ST47_27_101P" typeName="WORD"/>
		<variables name="A_IG_ST48_M201" typeName="WORD"/>
		<variables name="A_IG_ST49_5_201P" typeName="WORD"/>
		<variables name="A_IG_ST50_CH202" typeName="WORD"/>
		<variables name="A_IG_ST51_202P" typeName="WORD"/>
		<variables name="A_IG_ST52_M210" typeName="WORD"/>
		<variables name="A_IG_ST53_202_212P" typeName="WORD"/>
		<variables name="A_IG_ST54_CH201" typeName="WORD"/>
		<variables name="A_IG_ST55_201P" typeName="WORD"/>
		<variables name="A_IG_ST56_CH203" typeName="WORD"/>
		<variables name="A_IG_ST57_203P" typeName="WORD"/>
		<variables name="A_IG_ST58_M21" typeName="WORD"/>
		<variables name="A_IG_ST59_25_31P" typeName="WORD"/>
		<variables name="A_IG_ST60_M27" typeName="WORD"/>
		<variables name="A_IG_ST61_M33" typeName="WORD"/>
		<variables name="A_IG_ST62_1AP" typeName="WORD"/>
		<variables name="A_IG_ST63_M70" typeName="WORD"/>
		<variables name="A_IG_ST64_24_42P" typeName="WORD"/>
		<variables name="A_IG_ST65_M40" typeName="WORD"/>
		<variables name="A_IG_ST66_M36" typeName="WORD"/>
		<variables name="A_IG_ST67_12_14P" typeName="WORD"/>
		<variables name="A_IG_ST68_M30" typeName="WORD"/>
		<variables name="A_IG_ST69_10_12P" typeName="WORD"/>
		<variables name="A_IG_ST70_M22" typeName="WORD"/>
		<variables name="A_IG_ST71_6_8P" typeName="WORD"/>
		<variables name="A_IG_ST73_M43" typeName="WORD"/>
		<variables name="A_IG_ST74_6P" typeName="WORD"/>
		<variables name="A_IG_ST75_M66" typeName="WORD"/>
		<variables name="A_IG_ST76_M62" typeName="WORD"/>
		<variables name="A_IG_ST77_28_302P" typeName="WORD"/>
		<variables name="A_IG_ST78_M302" typeName="WORD"/>
		<variables name="A_IG_ST79_308P" typeName="WORD"/>
		<variables name="A_IG_ST80_12_302P" typeName="WORD"/>
		<variables name="A_IG_ST81_M41" typeName="WORD"/>
		<variables name="A_IG_ST82_39_51P" typeName="WORD"/>
		<variables name="A_IG_ST83_M55" typeName="WORD"/>
		<variables name="A_IG_ST84_7AP" typeName="WORD"/>
		<variables name="A_IG_ST85_M61" typeName="WORD"/>
		<variables name="A_IG_ST86_7P" typeName="WORD"/>
		<variables name="A_IG_ST87_M37" typeName="WORD"/>
		<variables name="A_IG_ST88_43_32P" typeName="WORD"/>
		<variables name="A_IG_ST89_M56" typeName="WORD"/>
		<variables name="A_IG_ST90_22_32P" typeName="WORD"/>
		<variables name="A_IG_ST91_M42" typeName="WORD"/>
		<variables name="A_IG_ST92_M34" typeName="WORD"/>
		<variables name="A_IG_ST93_8_16P" typeName="WORD"/>
		<variables name="A_IG_ST94_M58" typeName="WORD"/>
		<variables name="A_IG_ST95_32_34P" typeName="WORD"/>
		<variables name="A_IG_ST96_STR34" typeName="WORD"/>
		<variables name="A_IG_ST97_CH101" typeName="WORD"/>
		<variables name="A_IG_ST98_101P" typeName="WORD"/>
		<variables name="A_IG_ST99_CH102" typeName="WORD"/>
		<variables name="A_IG_ST100_102P" typeName="WORD"/>
		<variables name="A_IG_ST101_CH103" typeName="WORD"/>
		<variables name="A_IG_ST102_103P" typeName="WORD"/>
		<variables name="A_IG_ST103_CH104" typeName="WORD"/>
		<variables name="A_IG_ST104_104P" typeName="WORD"/>
		<variables name="A_IG_ST105_M102" typeName="WORD"/>
		<variables name="A_IG_ST106_M132" typeName="WORD"/>
		<variables name="A_IG_ST107_BP" typeName="WORD"/>
		<variables name="A_IG_ST107_M128" typeName="WORD"/>
		<variables name="A_IG_ST108_M124" typeName="WORD"/>
		<variables name="A_IG_ST109_4_124P" typeName="WORD"/>
		<variables name="A_IG_ST110_M130" typeName="WORD"/>
		<variables name="A_IG_ST111_116_124P" typeName="WORD"/>
		<variables name="A_IG_ST112_M160" typeName="WORD"/>
		<variables name="A_IG_ST113_130_164SP" typeName="WORD"/>
		<variables name="A_IG_ST114_M104" typeName="WORD"/>
		<variables name="A_IG_ST115_102_140P" typeName="WORD"/>
		<variables name="A_IG_ST116_CH105" typeName="WORD"/>
		<variables name="A_IG_ST117_105P" typeName="WORD"/>
		<variables name="A_IG_ST118_M12" typeName="WORD"/>
		<variables name="A_IG_ST119_6_84P" typeName="WORD"/>
		<variables name="A_IG_ST120_M65" typeName="WORD"/>
		<variables name="A_IG_ST121_15_117P" typeName="WORD"/>
		<variables name="A_IG_ST122_M107" typeName="WORD"/>
		<variables name="A_IG_ST123_CH106" typeName="WORD"/>
		<variables name="A_IG_ST124_106P" typeName="WORD"/>
		<variables name="A_IG_ST125_M82" typeName="WORD"/>
		<variables name="A_IG_ST126_M80" typeName="WORD"/>
		<variables name="A_IG_ST127_117P" typeName="WORD"/>
		<variables name="A_IG_ST128_M101" typeName="WORD"/>
		<variables name="A_IG_ST129_CH108" typeName="WORD"/>
		<variables name="A_IG_ST130_108P" typeName="WORD"/>
		<variables name="A_IG_ST131_M168" typeName="WORD"/>
		<variables name="A_IG_ST132_M132" typeName="WORD"/>
		<variables name="A_IG_ST133_108AP" typeName="WORD"/>
		<variables name="A_IG_ST134_162_166P" typeName="WORD"/>
		<variables name="A_IG_ST135_M176" typeName="WORD"/>
		<variables name="A_IG_ST136_160_162P" typeName="WORD"/>
		<variables name="A_IG_ST137_M172" typeName="WORD"/>
		<variables name="A_IG_ST138_109AP" typeName="WORD"/>
		<variables name="A_IG_ST139_M134" typeName="WORD"/>
		<variables name="A_IG_ST140_STR136" typeName="WORD"/>
		<variables name="A_IG_ST141_M152" typeName="WORD"/>
		<variables name="A_IG_ST142_122_160P" typeName="WORD"/>
		<variables name="A_IG_ST143_98_104SP" typeName="WORD"/>
		<variables name="A_IG_ST144_STR28" typeName="WORD"/>
		<variables name="A_IG_ST145_STR29" typeName="WORD"/>
		<variables name="A_IG_ST146_STR33" typeName="WORD"/>
		<variables name="A_IG_ST147_STR117" typeName="WORD"/>
		<variables name="A_IG_ST148_STR140" typeName="WORD"/>
		<variables name="A_IG_ST149_STR206" typeName="WORD"/>
		<variables name="A_IG_ST150_STR24" typeName="WORD"/>
		<variables name="A_IG_ST151_STR16" typeName="WORD"/>
		<variables name="A_IG_ST153_98_122P" typeName="WORD"/>
		<variables name="A_IG_ST154_M158" typeName="WORD"/>
		<variables name="A_IG_ST155_105AP" typeName="WORD"/>
		<variables name="A_IG_STR1_1SP" typeName="WORD"/>
		<variables name="A_IG_STR3_M7" typeName="WORD"/>
		<variables name="A_IG_STR3_ST1" typeName="WORD"/>
		<variables name="A_IG_STR4_M16" typeName="WORD"/>
		<variables name="A_IG_STR4_ST20" typeName="WORD"/>
		<variables name="A_IG_STR4_STR84" typeName="WORD"/>
		<variables name="A_IG_STR5_3_5SP" typeName="WORD"/>
		<variables name="A_IG_STR5_M201" typeName="WORD"/>
		<variables name="A_IG_STR6_6SP" typeName="WORD"/>
		<variables name="A_IG_STR6_M10" typeName="WORD"/>
		<variables name="A_IG_STR8_M26" typeName="WORD"/>
		<variables name="A_IG_STR8_M28" typeName="WORD"/>
		<variables name="A_IG_STR9_9SP" typeName="WORD"/>
		<variables name="A_IG_STR11_11_19SP" typeName="WORD"/>
		<variables name="A_IG_STR12_12SP" typeName="WORD"/>
		<variables name="A_IG_STR12_M32" typeName="WORD"/>
		<variables name="A_IG_STR13_ST2" typeName="WORD"/>
		<variables name="A_IG_STR13_ST3" typeName="WORD"/>
		<variables name="A_IG_STR14_ST65" typeName="WORD"/>
		<variables name="A_IG_STR14_ST151" typeName="WORD"/>
		<variables name="A_IG_STR15_15SP" typeName="WORD"/>
		<variables name="A_IG_STR16_16SP" typeName="WORD"/>
		<variables name="A_IG_STR17_STR11" typeName="WORD"/>
		<variables name="A_IG_STR18_ST13" typeName="WORD"/>
		<variables name="A_IG_STR18_STR26" typeName="WORD"/>
		<variables name="A_IG_STR19_STR17" typeName="WORD"/>
		<variables name="A_IG_STR20_20_22SP" typeName="WORD"/>
		<variables name="A_IG_STR20_30SP" typeName="WORD"/>
		<variables name="A_IG_STR20_STR22" typeName="WORD"/>
		<variables name="A_IG_STR22_M54" typeName="WORD"/>
		<variables name="A_IG_STR22_ST150" typeName="WORD"/>
		<variables name="A_IG_STR24_24SP" typeName="WORD"/>
		<variables name="A_IG_STR24_ST150" typeName="WORD"/>
		<variables name="A_IG_STR25_25_27SP" typeName="WORD"/>
		<variables name="A_IG_STR26_M50" typeName="WORD"/>
		<variables name="A_IG_STR26_ST144" typeName="WORD"/>
		<variables name="A_IG_STR26_STR18" typeName="WORD"/>
		<variables name="A_IG_STR27_STR25" typeName="WORD"/>
		<variables name="A_IG_STR28_28SP" typeName="WORD"/>
		<variables name="A_IG_STR28_ST144" typeName="WORD"/>
		<variables name="A_IG_STR29_29SP" typeName="WORD"/>
		<variables name="A_IG_STR30_ST96" typeName="WORD"/>
		<variables name="A_IG_STR31_M25" typeName="WORD"/>
		<variables name="A_IG_STR31_ST145" typeName="WORD"/>
		<variables name="A_IG_STR32_32SP" typeName="WORD"/>
		<variables name="A_IG_STR33_33_43SP" typeName="WORD"/>
		<variables name="A_IG_STR33_STR41" typeName="WORD"/>
		<variables name="A_IG_STR34_34_36SP" typeName="WORD"/>
		<variables name="A_IG_STR34_STR36" typeName="WORD"/>
		<variables name="A_IG_STR35_ST5" typeName="WORD"/>
		<variables name="A_IG_STR35_ST146" typeName="WORD"/>
		<variables name="A_IG_STR36_STR34" typeName="WORD"/>
		<variables name="A_IG_STR37_37_39SP" typeName="WORD"/>
		<variables name="A_IG_STR37_STR39" typeName="WORD"/>
		<variables name="A_IG_STR38_M72" typeName="WORD"/>
		<variables name="A_IG_STR38_M74" typeName="WORD"/>
		<variables name="A_IG_STR39_STR37" typeName="WORD"/>
		<variables name="A_IG_STR41_STR33" typeName="WORD"/>
		<variables name="A_IG_STR43_STR41" typeName="WORD"/>
		<variables name="A_IG_STR47_47_49SP" typeName="WORD"/>
		<variables name="A_IG_STR49_STR47" typeName="WORD"/>
		<variables name="A_IG_STR51_M51" typeName="WORD"/>
		<variables name="A_IG_STR51_M53" typeName="WORD"/>
		<variables name="A_IG_STR53_53SP" typeName="WORD"/>
		<variables name="A_IG_STR84_M84" typeName="WORD"/>
		<variables name="A_IG_STR84_STR4" typeName="WORD"/>
		<variables name="A_IG_STR98_M142" typeName="WORD"/>
		<variables name="A_IG_STR98_M156" typeName="WORD"/>
		<variables name="A_IG_STR100_100SP" typeName="WORD"/>
		<variables name="A_IG_STR100_M80" typeName="WORD"/>
		<variables name="A_IG_STR100_ST45" typeName="WORD"/>
		<variables name="A_IG_STR101_M109" typeName="WORD"/>
		<variables name="A_IG_STR101_ST42" typeName="WORD"/>
		<variables name="A_IG_STR102_M14" typeName="WORD"/>
		<variables name="A_IG_STR102_M38" typeName="WORD"/>
		<variables name="A_IG_STR103_101_103SP" typeName="WORD"/>
		<variables name="A_IG_STR103_CH101" typeName="WORD"/>
		<variables name="A_IG_STR104_ST23" typeName="WORD"/>
		<variables name="A_IG_STR104_STR98" typeName="WORD"/>
		<variables name="A_IG_STR105_105SP" typeName="WORD"/>
		<variables name="A_IG_STR105_ST21" typeName="WORD"/>
		<variables name="A_IG_STR106_ST139" typeName="WORD"/>
		<variables name="A_IG_STR106_STR108" typeName="WORD"/>
		<variables name="A_IG_STR107_CH108" typeName="WORD"/>
		<variables name="A_IG_STR107_STR121" typeName="WORD"/>
		<variables name="A_IG_STR108_M138" typeName="WORD"/>
		<variables name="A_IG_STR108_ST24" typeName="WORD"/>
		<variables name="A_IG_STR109_109SP" typeName="WORD"/>
		<variables name="A_IG_STR110_110_112SP" typeName="WORD"/>
		<variables name="A_IG_STR110_ST18" typeName="WORD"/>
		<variables name="A_IG_STR110_STR112" typeName="WORD"/>
		<variables name="A_IG_STR111_111_113SP" typeName="WORD"/>
		<variables name="A_IG_STR112_M106" typeName="WORD"/>
		<variables name="A_IG_STR112_M108" typeName="WORD"/>
		<variables name="A_IG_STR112_STR110" typeName="WORD"/>
		<variables name="A_IG_STR113_STR111" typeName="WORD"/>
		<variables name="A_IG_STR114_M110" typeName="WORD"/>
		<variables name="A_IG_STR114_ST17" typeName="WORD"/>
		<variables name="A_IG_STR115_115SP" typeName="WORD"/>
		<variables name="A_IG_STR116_116SP" typeName="WORD"/>
		<variables name="A_IG_STR117_117SP" typeName="WORD"/>
		<variables name="A_IG_STR119_ST9" typeName="WORD"/>
		<variables name="A_IG_STR119_ST147" typeName="WORD"/>
		<variables name="A_IG_STR120_102_120SP" typeName="WORD"/>
		<variables name="A_IG_STR121_107_121SP" typeName="WORD"/>
		<variables name="A_IG_STR122_122SP" typeName="WORD"/>
		<variables name="A_IG_STR124_M128" typeName="WORD"/>
		<variables name="A_IG_STR124_STR138" typeName="WORD"/>
		<variables name="A_IG_STR128_128P" typeName="WORD"/>
		<variables name="A_IG_STR128_STR134" typeName="WORD"/>
		<variables name="A_IG_STR130_130_164ST" typeName="WORD"/>
		<variables name="A_IG_STR130_ST19" typeName="WORD"/>
		<variables name="A_IG_STR130_STR132" typeName="WORD"/>
		<variables name="A_IG_STR132_M144" typeName="WORD"/>
		<variables name="A_IG_STR132_STR130" typeName="WORD"/>
		<variables name="A_IG_STR132_STR164" typeName="WORD"/>
		<variables name="A_IG_STR134_M148" typeName="WORD"/>
		<variables name="A_IG_STR134_M150" typeName="WORD"/>
		<variables name="A_IG_STR136_136SP" typeName="WORD"/>
		<variables name="A_IG_STR138_M128" typeName="WORD"/>
		<variables name="A_IG_STR138_ST148" typeName="WORD"/>
		<variables name="A_IG_STR140_140SP" typeName="WORD"/>
		<variables name="A_IG_STR142_142SP" typeName="WORD"/>
		<variables name="A_IG_STR142_M160" typeName="WORD"/>
		<variables name="A_IG_STR144_M112" typeName="WORD"/>
		<variables name="A_IG_STR144_STR146" typeName="WORD"/>
		<variables name="A_IG_STR146_M114" typeName="WORD"/>
		<variables name="A_IG_STR146_ST22" typeName="WORD"/>
		<variables name="A_IG_STR148_148SP" typeName="WORD"/>
		<variables name="A_IG_STR160_160SP" typeName="WORD"/>
		<variables name="A_IG_STR162_162SP" typeName="WORD"/>
		<variables name="A_IG_STR162_M168" typeName="WORD"/>
		<variables name="A_IG_STR162_M176" typeName="WORD"/>
		<variables name="A_IG_STR164_M136" typeName="WORD"/>
		<variables name="A_IG_STR164_ST140" typeName="WORD"/>
		<variables name="A_IG_STR166_M164" typeName="WORD"/>
		<variables name="A_IG_STR166_M166" typeName="WORD"/>
		<variables name="A_IG_STR201_201_203SP" typeName="WORD"/>
		<variables name="A_IG_STR202_202SP" typeName="WORD"/>
		<variables name="A_IG_STR203_STR201" typeName="WORD"/>
		<variables name="A_IG_STR204_ST149" typeName="WORD"/>
		<variables name="A_IG_STR204_STR208" typeName="WORD"/>
		<variables name="A_IG_STR206_206SP" typeName="WORD"/>
		<variables name="A_IG_STR208_M214" typeName="WORD"/>
		<variables name="A_IG_STR208_M216" typeName="WORD"/>
		<variables name="A_IG_STR212_204_212SP" typeName="WORD"/>
		<variables name="A_IG_STR302_M304" typeName="WORD"/>
		<variables name="A_IG_STR302_M306" typeName="WORD"/>
		<variables name="A_IG_TVD_M208" typeName="WORD"/>
		<variables name="A_IGM302_302SP" typeName="WORD"/>
		<variables name="A_IR_KL" typeName="BOOL"/>
		<variables name="A_IR_MIG_KL" typeName="BOOL"/>
		<variables name="A_IR_SrL" typeName="BOOL"/>
		<variables name="A_IV" typeName="BOOL"/>
		<variables name="A_IZ" typeName="BOOL"/>
		<variables name="A_IZ_KL" typeName="BOOL"/>
		<variables name="A_IZ_SrL" typeName="BOOL"/>
		<variables name="A_K_1_3MKSTR" typeName="BOOL"/>
		<variables name="A_K_1_3MU" typeName="BOOL"/>
		<variables name="A_K_1_3PU" typeName="BOOL"/>
		<variables name="A_K_1_3VK" typeName="BOOL"/>
		<variables name="A_K_1_3ZSTR" typeName="BOOL"/>
		<variables name="A_K_1_15IR" typeName="BOOL"/>
		<variables name="A_K_1AIR" typeName="BOOL"/>
		<variables name="A_K_1GIR" typeName="BOOL"/>
		<variables name="A_K_1IR" typeName="BOOL"/>
		<variables name="A_K_2AIR" typeName="BOOL"/>
		<variables name="A_K_2GIR" typeName="BOOL"/>
		<variables name="A_K_3_5IR" typeName="BOOL"/>
		<variables name="A_K_4_84IR" typeName="BOOL"/>
		<variables name="A_K_4_124IR" typeName="BOOL"/>
		<variables name="A_K_4MKSTR" typeName="BOOL"/>
		<variables name="A_K_4MU" typeName="BOOL"/>
		<variables name="A_K_4PU" typeName="BOOL"/>
		<variables name="A_K_4VK" typeName="BOOL"/>
		<variables name="A_K_4ZSTR" typeName="BOOL"/>
		<variables name="A_K_5_9IR" typeName="BOOL"/>
		<variables name="A_K_5_201IR" typeName="BOOL"/>
		<variables name="A_K_5MKSTR" typeName="BOOL"/>
		<variables name="A_K_5MU" typeName="BOOL"/>
		<variables name="A_K_5PU" typeName="BOOL"/>
		<variables name="A_K_5VK" typeName="BOOL"/>
		<variables name="A_K_5ZSTR" typeName="BOOL"/>
		<variables name="A_K_6_8IR" typeName="BOOL"/>
		<variables name="A_K_6_84IR" typeName="BOOL"/>
		<variables name="A_K_6IR" typeName="BOOL"/>
		<variables name="A_K_6MKSTR" typeName="BOOL"/>
		<variables name="A_K_6MU" typeName="BOOL"/>
		<variables name="A_K_6PIR" typeName="BOOL"/>
		<variables name="A_K_6PU" typeName="BOOL"/>
		<variables name="A_K_6VK" typeName="BOOL"/>
		<variables name="A_K_6ZSTR" typeName="BOOL"/>
		<variables name="A_K_7AIR" typeName="BOOL"/>
		<variables name="A_K_7IR" typeName="BOOL"/>
		<variables name="A_K_8_16IR" typeName="BOOL"/>
		<variables name="A_K_8CHIR" typeName="BOOL"/>
		<variables name="A_K_8IR" typeName="BOOL"/>
		<variables name="A_K_8MKSTR" typeName="BOOL"/>
		<variables name="A_K_8MU" typeName="BOOL"/>
		<variables name="A_K_8NIR" typeName="BOOL"/>
		<variables name="A_K_8PU" typeName="BOOL"/>
		<variables name="A_K_8VK" typeName="BOOL"/>
		<variables name="A_K_8ZSTR" typeName="BOOL"/>
		<variables name="A_K_9IR" typeName="BOOL"/>
		<variables name="A_K_9MKSTR" typeName="BOOL"/>
		<variables name="A_K_9MU" typeName="BOOL"/>
		<variables name="A_K_9PU" typeName="BOOL"/>
		<variables name="A_K_9VK" typeName="BOOL"/>
		<variables name="A_K_9ZSTR" typeName="BOOL"/>
		<variables name="A_K_10_12IR" typeName="BOOL"/>
		<variables name="A_K_11_13MKSTR" typeName="BOOL"/>
		<variables name="A_K_11_13MU" typeName="BOOL"/>
		<variables name="A_K_11_13PU" typeName="BOOL"/>
		<variables name="A_K_11_13VK" typeName="BOOL"/>
		<variables name="A_K_11_13ZSTR" typeName="BOOL"/>
		<variables name="A_K_11_19IR" typeName="BOOL"/>
		<variables name="A_K_12_14IR" typeName="BOOL"/>
		<variables name="A_K_12_302IR" typeName="BOOL"/>
		<variables name="A_K_12IR" typeName="BOOL"/>
		<variables name="A_K_12MKSTR" typeName="BOOL"/>
		<variables name="A_K_12MU" typeName="BOOL"/>
		<variables name="A_K_12PU" typeName="BOOL"/>
		<variables name="A_K_12VK" typeName="BOOL"/>
		<variables name="A_K_12ZSTR" typeName="BOOL"/>
		<variables name="A_K_13IR" typeName="BOOL"/>
		<variables name="A_K_14_16MKSTR" typeName="BOOL"/>
		<variables name="A_K_14_16MU" typeName="BOOL"/>
		<variables name="A_K_14_16PU" typeName="BOOL"/>
		<variables name="A_K_14_16VK" typeName="BOOL"/>
		<variables name="A_K_14_16ZSTR" typeName="BOOL"/>
		<variables name="A_K_14IR" typeName="BOOL"/>
		<variables name="A_K_15_117IR" typeName="BOOL"/>
		<variables name="A_K_15IR" typeName="BOOL"/>
		<variables name="A_K_15MKSTR" typeName="BOOL"/>
		<variables name="A_K_15MU" typeName="BOOL"/>
		<variables name="A_K_15PU" typeName="BOOL"/>
		<variables name="A_K_15VK" typeName="BOOL"/>
		<variables name="A_K_15ZSTR" typeName="BOOL"/>
		<variables name="A_K_16IR" typeName="BOOL"/>
		<variables name="A_K_17MKSTR" typeName="BOOL"/>
		<variables name="A_K_17MU" typeName="BOOL"/>
		<variables name="A_K_17PU" typeName="BOOL"/>
		<variables name="A_K_17VK" typeName="BOOL"/>
		<variables name="A_K_17ZSTR" typeName="BOOL"/>
		<variables name="A_K_18_26IR" typeName="BOOL"/>
		<variables name="A_K_18MKSTR" typeName="BOOL"/>
		<variables name="A_K_18MU" typeName="BOOL"/>
		<variables name="A_K_18PU" typeName="BOOL"/>
		<variables name="A_K_18VK" typeName="BOOL"/>
		<variables name="A_K_18ZSTR" typeName="BOOL"/>
		<variables name="A_K_19MKSTR" typeName="BOOL"/>
		<variables name="A_K_19MU" typeName="BOOL"/>
		<variables name="A_K_19PU" typeName="BOOL"/>
		<variables name="A_K_19VK" typeName="BOOL"/>
		<variables name="A_K_19ZSTR" typeName="BOOL"/>
		<variables name="A_K_20_22IR" typeName="BOOL"/>
		<variables name="A_K_20MKSTR" typeName="BOOL"/>
		<variables name="A_K_20MU" typeName="BOOL"/>
		<variables name="A_K_20PU" typeName="BOOL"/>
		<variables name="A_K_20VK" typeName="BOOL"/>
		<variables name="A_K_20ZSTR" typeName="BOOL"/>
		<variables name="A_K_22_24MKSTR" typeName="BOOL"/>
		<variables name="A_K_22_24MU" typeName="BOOL"/>
		<variables name="A_K_22_24PU" typeName="BOOL"/>
		<variables name="A_K_22_24VK" typeName="BOOL"/>
		<variables name="A_K_22_24ZSTR" typeName="BOOL"/>
		<variables name="A_K_22_32IR" typeName="BOOL"/>
		<variables name="A_K_23_29IR" typeName="BOOL"/>
		<variables name="A_K_23IR" typeName="BOOL"/>
		<variables name="A_K_24_42AIR" typeName="BOOL"/>
		<variables name="A_K_24_42IR" typeName="BOOL"/>
		<variables name="A_K_24IR" typeName="BOOL"/>
		<variables name="A_K_25_27IR" typeName="BOOL"/>
		<variables name="A_K_25_31IR" typeName="BOOL"/>
		<variables name="A_K_25MKSTR" typeName="BOOL"/>
		<variables name="A_K_25MU" typeName="BOOL"/>
		<variables name="A_K_25PU" typeName="BOOL"/>
		<variables name="A_K_25VK" typeName="BOOL"/>
		<variables name="A_K_25ZSTR" typeName="BOOL"/>
		<variables name="A_K_26_28MKSTR" typeName="BOOL"/>
		<variables name="A_K_26_28MU" typeName="BOOL"/>
		<variables name="A_K_26_28PU" typeName="BOOL"/>
		<variables name="A_K_26_28VK" typeName="BOOL"/>
		<variables name="A_K_26_28ZSTR" typeName="BOOL"/>
		<variables name="A_K_26_40AIR" typeName="BOOL"/>
		<variables name="A_K_26_40BIR" typeName="BOOL"/>
		<variables name="A_K_27_101IR" typeName="BOOL"/>
		<variables name="A_K_27MKSTR" typeName="BOOL"/>
		<variables name="A_K_27MU" typeName="BOOL"/>
		<variables name="A_K_27PU" typeName="BOOL"/>
		<variables name="A_K_27VK" typeName="BOOL"/>
		<variables name="A_K_27ZSTR" typeName="BOOL"/>
		<variables name="A_K_28_302IR" typeName="BOOL"/>
		<variables name="A_K_28IR" typeName="BOOL"/>
		<variables name="A_K_29_31MKSTR" typeName="BOOL"/>
		<variables name="A_K_29_31MU" typeName="BOOL"/>
		<variables name="A_K_29_31PU" typeName="BOOL"/>
		<variables name="A_K_29_31VK" typeName="BOOL"/>
		<variables name="A_K_29_31ZSTR" typeName="BOOL"/>
		<variables name="A_K_29IR" typeName="BOOL"/>
		<variables name="A_K_30_34MKSTR" typeName="BOOL"/>
		<variables name="A_K_30_34MU" typeName="BOOL"/>
		<variables name="A_K_30_34PU" typeName="BOOL"/>
		<variables name="A_K_30_34VK" typeName="BOOL"/>
		<variables name="A_K_30_34ZSTR" typeName="BOOL"/>
		<variables name="A_K_30IR" typeName="BOOL"/>
		<variables name="A_K_31IR" typeName="BOOL"/>
		<variables name="A_K_32_34IR" typeName="BOOL"/>
		<variables name="A_K_32IR" typeName="BOOL"/>
		<variables name="A_K_32MKSTR" typeName="BOOL"/>
		<variables name="A_K_32MU" typeName="BOOL"/>
		<variables name="A_K_32PU" typeName="BOOL"/>
		<variables name="A_K_32VK" typeName="BOOL"/>
		<variables name="A_K_32ZSTR" typeName="BOOL"/>
		<variables name="A_K_33_35MKSTR" typeName="BOOL"/>
		<variables name="A_K_33_35MU" typeName="BOOL"/>
		<variables name="A_K_33_35PU" typeName="BOOL"/>
		<variables name="A_K_33_35VK" typeName="BOOL"/>
		<variables name="A_K_33_35ZSTR" typeName="BOOL"/>
		<variables name="A_K_33_43IR" typeName="BOOL"/>
		<variables name="A_K_34_36IR" typeName="BOOL"/>
		<variables name="A_K_35IR" typeName="BOOL"/>
		<variables name="A_K_36MKSTR" typeName="BOOL"/>
		<variables name="A_K_36MU" typeName="BOOL"/>
		<variables name="A_K_36PU" typeName="BOOL"/>
		<variables name="A_K_36VK" typeName="BOOL"/>
		<variables name="A_K_36ZSTR" typeName="BOOL"/>
		<variables name="A_K_37_39IR" typeName="BOOL"/>
		<variables name="A_K_37MKSTR" typeName="BOOL"/>
		<variables name="A_K_37MU" typeName="BOOL"/>
		<variables name="A_K_37PU" typeName="BOOL"/>
		<variables name="A_K_37VK" typeName="BOOL"/>
		<variables name="A_K_37ZSTR" typeName="BOOL"/>
		<variables name="A_K_38IR" typeName="BOOL"/>
		<variables name="A_K_38MKSTR" typeName="BOOL"/>
		<variables name="A_K_38MU" typeName="BOOL"/>
		<variables name="A_K_38PU" typeName="BOOL"/>
		<variables name="A_K_38VK" typeName="BOOL"/>
		<variables name="A_K_38ZSTR" typeName="BOOL"/>
		<variables name="A_K_39_51IR" typeName="BOOL"/>
		<variables name="A_K_39MKSTR" typeName="BOOL"/>
		<variables name="A_K_39MU" typeName="BOOL"/>
		<variables name="A_K_39PU" typeName="BOOL"/>
		<variables name="A_K_39VK" typeName="BOOL"/>
		<variables name="A_K_39ZSTR" typeName="BOOL"/>
		<variables name="A_K_41MKSTR" typeName="BOOL"/>
		<variables name="A_K_41MU" typeName="BOOL"/>
		<variables name="A_K_41PU" typeName="BOOL"/>
		<variables name="A_K_41VK" typeName="BOOL"/>
		<variables name="A_K_41ZSTR" typeName="BOOL"/>
		<variables name="A_K_43_32IR" typeName="BOOL"/>
		<variables name="A_K_43MKSTR" typeName="BOOL"/>
		<variables name="A_K_43MU" typeName="BOOL"/>
		<variables name="A_K_43PU" typeName="BOOL"/>
		<variables name="A_K_43VK" typeName="BOOL"/>
		<variables name="A_K_43ZSTR" typeName="BOOL"/>
		<variables name="A_K_47_49IR" typeName="BOOL"/>
		<variables name="A_K_47MKSTR" typeName="BOOL"/>
		<variables name="A_K_47MU" typeName="BOOL"/>
		<variables name="A_K_47PU" typeName="BOOL"/>
		<variables name="A_K_47VK" typeName="BOOL"/>
		<variables name="A_K_47ZSTR" typeName="BOOL"/>
		<variables name="A_K_49MKSTR" typeName="BOOL"/>
		<variables name="A_K_49MU" typeName="BOOL"/>
		<variables name="A_K_49PU" typeName="BOOL"/>
		<variables name="A_K_49VK" typeName="BOOL"/>
		<variables name="A_K_49ZSTR" typeName="BOOL"/>
		<variables name="A_K_51IR" typeName="BOOL"/>
		<variables name="A_K_51MKSTR" typeName="BOOL"/>
		<variables name="A_K_51MU" typeName="BOOL"/>
		<variables name="A_K_51PU" typeName="BOOL"/>
		<variables name="A_K_51VK" typeName="BOOL"/>
		<variables name="A_K_51ZSTR" typeName="BOOL"/>
		<variables name="A_K_53IR" typeName="BOOL"/>
		<variables name="A_K_53MKSTR" typeName="BOOL"/>
		<variables name="A_K_53MU" typeName="BOOL"/>
		<variables name="A_K_53PU" typeName="BOOL"/>
		<variables name="A_K_53VK" typeName="BOOL"/>
		<variables name="A_K_53ZSTR" typeName="BOOL"/>
		<variables name="A_K_84MKSTR" typeName="BOOL"/>
		<variables name="A_K_84MU" typeName="BOOL"/>
		<variables name="A_K_84PU" typeName="BOOL"/>
		<variables name="A_K_84VK" typeName="BOOL"/>
		<variables name="A_K_84ZSTR" typeName="BOOL"/>
		<variables name="A_K_98_104IR" typeName="BOOL"/>
		<variables name="A_K_98_122IR" typeName="BOOL"/>
		<variables name="A_K_98MKSTR" typeName="BOOL"/>
		<variables name="A_K_98MU" typeName="BOOL"/>
		<variables name="A_K_98PU" typeName="BOOL"/>
		<variables name="A_K_98VK" typeName="BOOL"/>
		<variables name="A_K_98ZSTR" typeName="BOOL"/>
		<variables name="A_K_100IR" typeName="BOOL"/>
		<variables name="A_K_100MKSTR" typeName="BOOL"/>
		<variables name="A_K_100MU" typeName="BOOL"/>
		<variables name="A_K_100PU" typeName="BOOL"/>
		<variables name="A_K_100VK" typeName="BOOL"/>
		<variables name="A_K_100ZSTR" typeName="BOOL"/>
		<variables name="A_K_101_103IR" typeName="BOOL"/>
		<variables name="A_K_101MKSTR" typeName="BOOL"/>
		<variables name="A_K_101MU" typeName="BOOL"/>
		<variables name="A_K_101PU" typeName="BOOL"/>
		<variables name="A_K_101VK" typeName="BOOL"/>
		<variables name="A_K_101ZSTR" typeName="BOOL"/>
		<variables name="A_K_102_120IR" typeName="BOOL"/>
		<variables name="A_K_102_140IR" typeName="BOOL"/>
		<variables name="A_K_102MKSTR" typeName="BOOL"/>
		<variables name="A_K_102MU" typeName="BOOL"/>
		<variables name="A_K_102PU" typeName="BOOL"/>
		<variables name="A_K_102VK" typeName="BOOL"/>
		<variables name="A_K_102ZSTR" typeName="BOOL"/>
		<variables name="A_K_103MKSTR" typeName="BOOL"/>
		<variables name="A_K_103MU" typeName="BOOL"/>
		<variables name="A_K_103PU" typeName="BOOL"/>
		<variables name="A_K_103VK" typeName="BOOL"/>
		<variables name="A_K_103ZSTR" typeName="BOOL"/>
		<variables name="A_K_104MKSTR" typeName="BOOL"/>
		<variables name="A_K_104MU" typeName="BOOL"/>
		<variables name="A_K_104PU" typeName="BOOL"/>
		<variables name="A_K_104VK" typeName="BOOL"/>
		<variables name="A_K_104ZSTR" typeName="BOOL"/>
		<variables name="A_K_105AIR" typeName="BOOL"/>
		<variables name="A_K_105IR" typeName="BOOL"/>
		<variables name="A_K_105MKSTR" typeName="BOOL"/>
		<variables name="A_K_105MU" typeName="BOOL"/>
		<variables name="A_K_105PU" typeName="BOOL"/>
		<variables name="A_K_105VK" typeName="BOOL"/>
		<variables name="A_K_105ZSTR" typeName="BOOL"/>
		<variables name="A_K_106_108IR" typeName="BOOL"/>
		<variables name="A_K_106AIR" typeName="BOOL"/>
		<variables name="A_K_106MKSTR" typeName="BOOL"/>
		<variables name="A_K_106MU" typeName="BOOL"/>
		<variables name="A_K_106PU" typeName="BOOL"/>
		<variables name="A_K_106VK" typeName="BOOL"/>
		<variables name="A_K_106ZSTR" typeName="BOOL"/>
		<variables name="A_K_107_121IR" typeName="BOOL"/>
		<variables name="A_K_107_12121VK" typeName="BOOL"/>
		<variables name="A_K_107MKSTR" typeName="BOOL"/>
		<variables name="A_K_107MU" typeName="BOOL"/>
		<variables name="A_K_107PU" typeName="BOOL"/>
		<variables name="A_K_107VK" typeName="BOOL"/>
		<variables name="A_K_107ZSTR" typeName="BOOL"/>
		<variables name="A_K_108AIR" typeName="BOOL"/>
		<variables name="A_K_108MKSTR" typeName="BOOL"/>
		<variables name="A_K_108MU" typeName="BOOL"/>
		<variables name="A_K_108PU" typeName="BOOL"/>
		<variables name="A_K_108VK" typeName="BOOL"/>
		<variables name="A_K_108ZSTR" typeName="BOOL"/>
		<variables name="A_K_109AIR" typeName="BOOL"/>
		<variables name="A_K_109IR" typeName="BOOL"/>
		<variables name="A_K_109MKSTR" typeName="BOOL"/>
		<variables name="A_K_109MU" typeName="BOOL"/>
		<variables name="A_K_109PU" typeName="BOOL"/>
		<variables name="A_K_109VK" typeName="BOOL"/>
		<variables name="A_K_109ZSTR" typeName="BOOL"/>
		<variables name="A_K_110_112IR" typeName="BOOL"/>
		<variables name="A_K_110MKSTR" typeName="BOOL"/>
		<variables name="A_K_110MU" typeName="BOOL"/>
		<variables name="A_K_110PU" typeName="BOOL"/>
		<variables name="A_K_110VK" typeName="BOOL"/>
		<variables name="A_K_110ZSTR" typeName="BOOL"/>
		<variables name="A_K_111_113IR" typeName="BOOL"/>
		<variables name="A_K_111MKSTR" typeName="BOOL"/>
		<variables name="A_K_111MU" typeName="BOOL"/>
		<variables name="A_K_111PU" typeName="BOOL"/>
		<variables name="A_K_111VK" typeName="BOOL"/>
		<variables name="A_K_111ZSTR" typeName="BOOL"/>
		<variables name="A_K_112MKSTR" typeName="BOOL"/>
		<variables name="A_K_112MU" typeName="BOOL"/>
		<variables name="A_K_112PU" typeName="BOOL"/>
		<variables name="A_K_112VK" typeName="BOOL"/>
		<variables name="A_K_112ZSTR" typeName="BOOL"/>
		<variables name="A_K_113MKSTR" typeName="BOOL"/>
		<variables name="A_K_113MU" typeName="BOOL"/>
		<variables name="A_K_113PU" typeName="BOOL"/>
		<variables name="A_K_113VK" typeName="BOOL"/>
		<variables name="A_K_113ZSTR" typeName="BOOL"/>
		<variables name="A_K_114IR" typeName="BOOL"/>
		<variables name="A_K_114MKSTR" typeName="BOOL"/>
		<variables name="A_K_114MU" typeName="BOOL"/>
		<variables name="A_K_114PU" typeName="BOOL"/>
		<variables name="A_K_114VK" typeName="BOOL"/>
		<variables name="A_K_114ZSTR" typeName="BOOL"/>
		<variables name="A_K_115IR" typeName="BOOL"/>
		<variables name="A_K_115MKSTR" typeName="BOOL"/>
		<variables name="A_K_115MU" typeName="BOOL"/>
		<variables name="A_K_115PU" typeName="BOOL"/>
		<variables name="A_K_115VK" typeName="BOOL"/>
		<variables name="A_K_115ZSTR" typeName="BOOL"/>
		<variables name="A_K_116_124IR" typeName="BOOL"/>
		<variables name="A_K_116IR" typeName="BOOL"/>
		<variables name="A_K_116MKSTR" typeName="BOOL"/>
		<variables name="A_K_116MU" typeName="BOOL"/>
		<variables name="A_K_116PU" typeName="BOOL"/>
		<variables name="A_K_116VK" typeName="BOOL"/>
		<variables name="A_K_116ZSTR" typeName="BOOL"/>
		<variables name="A_K_117_119MKSTR" typeName="BOOL"/>
		<variables name="A_K_117_119MU" typeName="BOOL"/>
		<variables name="A_K_117_119PU" typeName="BOOL"/>
		<variables name="A_K_117_119VK" typeName="BOOL"/>
		<variables name="A_K_117_119ZSTR" typeName="BOOL"/>
		<variables name="A_K_117AIR" typeName="BOOL"/>
		<variables name="A_K_117BIR" typeName="BOOL"/>
		<variables name="A_K_117IR" typeName="BOOL"/>
		<variables name="A_K_117PIR" typeName="BOOL"/>
		<variables name="A_K_119_121IR" typeName="BOOL"/>
		<variables name="A_K_119IR" typeName="BOOL"/>
		<variables name="A_K_120MKSTR" typeName="BOOL"/>
		<variables name="A_K_120MU" typeName="BOOL"/>
		<variables name="A_K_120PU" typeName="BOOL"/>
		<variables name="A_K_120VK" typeName="BOOL"/>
		<variables name="A_K_120ZSTR" typeName="BOOL"/>
		<variables name="A_K_121MKSTR" typeName="BOOL"/>
		<variables name="A_K_121MU" typeName="BOOL"/>
		<variables name="A_K_121PU" typeName="BOOL"/>
		<variables name="A_K_121VK" typeName="BOOL"/>
		<variables name="A_K_121ZSTR" typeName="BOOL"/>
		<variables name="A_K_122_160IR" typeName="BOOL"/>
		<variables name="A_K_122IR" typeName="BOOL"/>
		<variables name="A_K_122MKSTR" typeName="BOOL"/>
		<variables name="A_K_122MU" typeName="BOOL"/>
		<variables name="A_K_122PU" typeName="BOOL"/>
		<variables name="A_K_122VK" typeName="BOOL"/>
		<variables name="A_K_122ZSTR" typeName="BOOL"/>
		<variables name="A_K_124_138IR" typeName="BOOL"/>
		<variables name="A_K_124MKSTR" typeName="BOOL"/>
		<variables name="A_K_124MU" typeName="BOOL"/>
		<variables name="A_K_124PU" typeName="BOOL"/>
		<variables name="A_K_124VK" typeName="BOOL"/>
		<variables name="A_K_124ZSTR" typeName="BOOL"/>
		<variables name="A_K_128_134IR" typeName="BOOL"/>
		<variables name="A_K_128IR" typeName="BOOL"/>
		<variables name="A_K_128MKSTR" typeName="BOOL"/>
		<variables name="A_K_128MU" typeName="BOOL"/>
		<variables name="A_K_128PU" typeName="BOOL"/>
		<variables name="A_K_128VK" typeName="BOOL"/>
		<variables name="A_K_128ZSTR" typeName="BOOL"/>
		<variables name="A_K_130_164IR" typeName="BOOL"/>
		<variables name="A_K_130MKSTR" typeName="BOOL"/>
		<variables name="A_K_130MU" typeName="BOOL"/>
		<variables name="A_K_130PU" typeName="BOOL"/>
		<variables name="A_K_130VK" typeName="BOOL"/>
		<variables name="A_K_130ZSTR" typeName="BOOL"/>
		<variables name="A_K_132MKSTR" typeName="BOOL"/>
		<variables name="A_K_132MU" typeName="BOOL"/>
		<variables name="A_K_132PU" typeName="BOOL"/>
		<variables name="A_K_132VK" typeName="BOOL"/>
		<variables name="A_K_132ZSTR" typeName="BOOL"/>
		<variables name="A_K_134MKSTR" typeName="BOOL"/>
		<variables name="A_K_134MU" typeName="BOOL"/>
		<variables name="A_K_134PU" typeName="BOOL"/>
		<variables name="A_K_134VK" typeName="BOOL"/>
		<variables name="A_K_134ZSTR" typeName="BOOL"/>
		<variables name="A_K_136IR" typeName="BOOL"/>
		<variables name="A_K_136MKSTR" typeName="BOOL"/>
		<variables name="A_K_136MU" typeName="BOOL"/>
		<variables name="A_K_136PU" typeName="BOOL"/>
		<variables name="A_K_136VK" typeName="BOOL"/>
		<variables name="A_K_136ZSTR" typeName="BOOL"/>
		<variables name="A_K_138_140MKSTR" typeName="BOOL"/>
		<variables name="A_K_138_140MU" typeName="BOOL"/>
		<variables name="A_K_138_140PU" typeName="BOOL"/>
		<variables name="A_K_138_140VK" typeName="BOOL"/>
		<variables name="A_K_138_140ZSTR" typeName="BOOL"/>
		<variables name="A_K_140IR" typeName="BOOL"/>
		<variables name="A_K_142IR" typeName="BOOL"/>
		<variables name="A_K_142MKSTR" typeName="BOOL"/>
		<variables name="A_K_142MU" typeName="BOOL"/>
		<variables name="A_K_142PU" typeName="BOOL"/>
		<variables name="A_K_142VK" typeName="BOOL"/>
		<variables name="A_K_142ZSTR" typeName="BOOL"/>
		<variables name="A_K_144_146IR" typeName="BOOL"/>
		<variables name="A_K_144MKSTR" typeName="BOOL"/>
		<variables name="A_K_144MU" typeName="BOOL"/>
		<variables name="A_K_144PU" typeName="BOOL"/>
		<variables name="A_K_144VK" typeName="BOOL"/>
		<variables name="A_K_144ZSTR" typeName="BOOL"/>
		<variables name="A_K_146MKSTR" typeName="BOOL"/>
		<variables name="A_K_146MU" typeName="BOOL"/>
		<variables name="A_K_146PU" typeName="BOOL"/>
		<variables name="A_K_146VK" typeName="BOOL"/>
		<variables name="A_K_146ZSTR" typeName="BOOL"/>
		<variables name="A_K_148IR" typeName="BOOL"/>
		<variables name="A_K_148MKSTR" typeName="BOOL"/>
		<variables name="A_K_148MU" typeName="BOOL"/>
		<variables name="A_K_148PU" typeName="BOOL"/>
		<variables name="A_K_148VK" typeName="BOOL"/>
		<variables name="A_K_148ZSTR" typeName="BOOL"/>
		<variables name="A_K_160_162IR" typeName="BOOL"/>
		<variables name="A_K_160IR" typeName="BOOL"/>
		<variables name="A_K_160MKSTR" typeName="BOOL"/>
		<variables name="A_K_160MU" typeName="BOOL"/>
		<variables name="A_K_160PU" typeName="BOOL"/>
		<variables name="A_K_160VK" typeName="BOOL"/>
		<variables name="A_K_160ZSTR" typeName="BOOL"/>
		<variables name="A_K_162_166IR" typeName="BOOL"/>
		<variables name="A_K_162IR" typeName="BOOL"/>
		<variables name="A_K_162MKSTR" typeName="BOOL"/>
		<variables name="A_K_162MU" typeName="BOOL"/>
		<variables name="A_K_162PU" typeName="BOOL"/>
		<variables name="A_K_162VK" typeName="BOOL"/>
		<variables name="A_K_162ZSTR" typeName="BOOL"/>
		<variables name="A_K_163ZSTR" typeName="BOOL"/>
		<variables name="A_K_164MKSTR" typeName="BOOL"/>
		<variables name="A_K_164MU" typeName="BOOL"/>
		<variables name="A_K_164PU" typeName="BOOL"/>
		<variables name="A_K_164VK" typeName="BOOL"/>
		<variables name="A_K_164ZSTR" typeName="BOOL"/>
		<variables name="A_K_166IR" typeName="BOOL"/>
		<variables name="A_K_166MKSTR" typeName="BOOL"/>
		<variables name="A_K_166MU" typeName="BOOL"/>
		<variables name="A_K_166PU" typeName="BOOL"/>
		<variables name="A_K_166VK" typeName="BOOL"/>
		<variables name="A_K_166ZSTR" typeName="BOOL"/>
		<variables name="A_K_201_203IR" typeName="BOOL"/>
		<variables name="A_K_201MKSTR" typeName="BOOL"/>
		<variables name="A_K_201MU" typeName="BOOL"/>
		<variables name="A_K_201PU" typeName="BOOL"/>
		<variables name="A_K_201VK" typeName="BOOL"/>
		<variables name="A_K_201ZSTR" typeName="BOOL"/>
		<variables name="A_K_202_212IR" typeName="BOOL"/>
		<variables name="A_K_202IR" typeName="BOOL"/>
		<variables name="A_K_202MKSTR" typeName="BOOL"/>
		<variables name="A_K_202MU" typeName="BOOL"/>
		<variables name="A_K_202PU" typeName="BOOL"/>
		<variables name="A_K_202VK" typeName="BOOL"/>
		<variables name="A_K_202ZSTR" typeName="BOOL"/>
		<variables name="A_K_203MKSTR" typeName="BOOL"/>
		<variables name="A_K_203MU" typeName="BOOL"/>
		<variables name="A_K_203PU" typeName="BOOL"/>
		<variables name="A_K_203VK" typeName="BOOL"/>
		<variables name="A_K_203ZSTR" typeName="BOOL"/>
		<variables name="A_K_204_206MKSTR" typeName="BOOL"/>
		<variables name="A_K_204_206MU" typeName="BOOL"/>
		<variables name="A_K_204_206PU" typeName="BOOL"/>
		<variables name="A_K_204_206VK" typeName="BOOL"/>
		<variables name="A_K_204_206ZSTR" typeName="BOOL"/>
		<variables name="A_K_204_212IR" typeName="BOOL"/>
		<variables name="A_K_206IR" typeName="BOOL"/>
		<variables name="A_K_208MKSTR" typeName="BOOL"/>
		<variables name="A_K_208MU" typeName="BOOL"/>
		<variables name="A_K_208PU" typeName="BOOL"/>
		<variables name="A_K_208VK" typeName="BOOL"/>
		<variables name="A_K_208ZSTR" typeName="BOOL"/>
		<variables name="A_K_212MKSTR" typeName="BOOL"/>
		<variables name="A_K_212MU" typeName="BOOL"/>
		<variables name="A_K_212PU" typeName="BOOL"/>
		<variables name="A_K_212VK" typeName="BOOL"/>
		<variables name="A_K_212ZSTR" typeName="BOOL"/>
		<variables name="A_K_302IR" typeName="BOOL"/>
		<variables name="A_K_302MKSTR" typeName="BOOL"/>
		<variables name="A_K_302MU" typeName="BOOL"/>
		<variables name="A_K_302PU" typeName="BOOL"/>
		<variables name="A_K_302VK" typeName="BOOL"/>
		<variables name="A_K_302ZSTR" typeName="BOOL"/>
		<variables name="A_K_10909VK" typeName="BOOL"/>
		<variables name="A_K_BIR" typeName="BOOL"/>
		<variables name="A_K_CH101" typeName="BOOL"/>
		<variables name="A_K_CH102" typeName="BOOL"/>
		<variables name="A_K_CH103" typeName="BOOL"/>
		<variables name="A_K_CH104" typeName="BOOL"/>
		<variables name="A_K_CH105" typeName="BOOL"/>
		<variables name="A_K_CH106" typeName="BOOL"/>
		<variables name="A_K_CH107" typeName="BOOL"/>
		<variables name="A_K_CH108" typeName="BOOL"/>
		<variables name="A_K_CH201" typeName="BOOL"/>
		<variables name="A_K_CH202" typeName="BOOL"/>
		<variables name="A_K_CH203" typeName="BOOL"/>
		<variables name="A_K_CHP" typeName="BOOL"/>
		<variables name="A_K_DN" typeName="BOOL"/>
		<variables name="A_K_FID_NEISPR" typeName="BOOL"/>
		<variables name="A_K_GIR" typeName="BOOL"/>
		<variables name="A_K_GRK" typeName="BOOL"/>
		<variables name="A_K_M1" typeName="BOOL"/>
		<variables name="A_K_M3" typeName="BOOL"/>
		<variables name="A_K_M5" typeName="BOOL"/>
		<variables name="A_K_M7" typeName="BOOL"/>
		<variables name="A_K_M9" typeName="BOOL"/>
		<variables name="A_K_M10" typeName="BOOL"/>
		<variables name="A_K_M11" typeName="BOOL"/>
		<variables name="A_K_M12" typeName="BOOL"/>
		<variables name="A_K_M13" typeName="BOOL"/>
		<variables name="A_K_M14" typeName="BOOL"/>
		<variables name="A_K_M15" typeName="BOOL"/>
		<variables name="A_K_M16" typeName="BOOL"/>
		<variables name="A_K_M17" typeName="BOOL"/>
		<variables name="A_K_M18" typeName="BOOL"/>
		<variables name="A_K_M19" typeName="BOOL"/>
		<variables name="A_K_M20" typeName="BOOL"/>
		<variables name="A_K_M21" typeName="BOOL"/>
		<variables name="A_K_M22" typeName="BOOL"/>
		<variables name="A_K_M23" typeName="BOOL"/>
		<variables name="A_K_M25" typeName="BOOL"/>
		<variables name="A_K_M26" typeName="BOOL"/>
		<variables name="A_K_M27" typeName="BOOL"/>
		<variables name="A_K_M28" typeName="BOOL"/>
		<variables name="A_K_M29" typeName="BOOL"/>
		<variables name="A_K_M30" typeName="BOOL"/>
		<variables name="A_K_M32" typeName="BOOL"/>
		<variables name="A_K_M33" typeName="BOOL"/>
		<variables name="A_K_M34" typeName="BOOL"/>
		<variables name="A_K_M35" typeName="BOOL"/>
		<variables name="A_K_M36" typeName="BOOL"/>
		<variables name="A_K_M37" typeName="BOOL"/>
		<variables name="A_K_M38" typeName="BOOL"/>
		<variables name="A_K_M39" typeName="BOOL"/>
		<variables name="A_K_M40" typeName="BOOL"/>
		<variables name="A_K_M41" typeName="BOOL"/>
		<variables name="A_K_M42" typeName="BOOL"/>
		<variables name="A_K_M43" typeName="BOOL"/>
		<variables name="A_K_M44" typeName="BOOL"/>
		<variables name="A_K_M45" typeName="BOOL"/>
		<variables name="A_K_M46" typeName="BOOL"/>
		<variables name="A_K_M47" typeName="BOOL"/>
		<variables name="A_K_M48" typeName="BOOL"/>
		<variables name="A_K_M49" typeName="BOOL"/>
		<variables name="A_K_M50" typeName="BOOL"/>
		<variables name="A_K_M51" typeName="BOOL"/>
		<variables name="A_K_M52" typeName="BOOL"/>
		<variables name="A_K_M53" typeName="BOOL"/>
		<variables name="A_K_M54" typeName="BOOL"/>
		<variables name="A_K_M55" typeName="BOOL"/>
		<variables name="A_K_M56" typeName="BOOL"/>
		<variables name="A_K_M57" typeName="BOOL"/>
		<variables name="A_K_M58" typeName="BOOL"/>
		<variables name="A_K_M59" typeName="BOOL"/>
		<variables name="A_K_M60" typeName="BOOL"/>
		<variables name="A_K_M61" typeName="BOOL"/>
		<variables name="A_K_M62" typeName="BOOL"/>
		<variables name="A_K_M63" typeName="BOOL"/>
		<variables name="A_K_M64" typeName="BOOL"/>
		<variables name="A_K_M65" typeName="BOOL"/>
		<variables name="A_K_M66" typeName="BOOL"/>
		<variables name="A_K_M67" typeName="BOOL"/>
		<variables name="A_K_M68" typeName="BOOL"/>
		<variables name="A_K_M69" typeName="BOOL"/>
		<variables name="A_K_M70" typeName="BOOL"/>
		<variables name="A_K_M72" typeName="BOOL"/>
		<variables name="A_K_M74" typeName="BOOL"/>
		<variables name="A_K_M76" typeName="BOOL"/>
		<variables name="A_K_M78" typeName="BOOL"/>
		<variables name="A_K_M80" typeName="BOOL"/>
		<variables name="A_K_M82" typeName="BOOL"/>
		<variables name="A_K_M84" typeName="BOOL"/>
		<variables name="A_K_M97" typeName="BOOL"/>
		<variables name="A_K_M100" typeName="BOOL"/>
		<variables name="A_K_M101" typeName="BOOL"/>
		<variables name="A_K_M101IR" typeName="BOOL"/>
		<variables name="A_K_M102" typeName="BOOL"/>
		<variables name="A_K_M103" typeName="BOOL"/>
		<variables name="A_K_M104" typeName="BOOL"/>
		<variables name="A_K_M105" typeName="BOOL"/>
		<variables name="A_K_M106" typeName="BOOL"/>
		<variables name="A_K_M107" typeName="BOOL"/>
		<variables name="A_K_M108" typeName="BOOL"/>
		<variables name="A_K_M109" typeName="BOOL"/>
		<variables name="A_K_M110" typeName="BOOL"/>
		<variables name="A_K_M112" typeName="BOOL"/>
		<variables name="A_K_M114" typeName="BOOL"/>
		<variables name="A_K_M116" typeName="BOOL"/>
		<variables name="A_K_M118" typeName="BOOL"/>
		<variables name="A_K_M120" typeName="BOOL"/>
		<variables name="A_K_M122" typeName="BOOL"/>
		<variables name="A_K_M124" typeName="BOOL"/>
		<variables name="A_K_M126" typeName="BOOL"/>
		<variables name="A_K_M128" typeName="BOOL"/>
		<variables name="A_K_M130" typeName="BOOL"/>
		<variables name="A_K_M132" typeName="BOOL"/>
		<variables name="A_K_M134" typeName="BOOL"/>
		<variables name="A_K_M136" typeName="BOOL"/>
		<variables name="A_K_M138" typeName="BOOL"/>
		<variables name="A_K_M140" typeName="BOOL"/>
		<variables name="A_K_M142" typeName="BOOL"/>
		<variables name="A_K_M144" typeName="BOOL"/>
		<variables name="A_K_M146" typeName="BOOL"/>
		<variables name="A_K_M148" typeName="BOOL"/>
		<variables name="A_K_M150" typeName="BOOL"/>
		<variables name="A_K_M152" typeName="BOOL"/>
		<variables name="A_K_M154" typeName="BOOL"/>
		<variables name="A_K_M156" typeName="BOOL"/>
		<variables name="A_K_M158" typeName="BOOL"/>
		<variables name="A_K_M160" typeName="BOOL"/>
		<variables name="A_K_M162" typeName="BOOL"/>
		<variables name="A_K_M164" typeName="BOOL"/>
		<variables name="A_K_M166" typeName="BOOL"/>
		<variables name="A_K_M168" typeName="BOOL"/>
		<variables name="A_K_M172" typeName="BOOL"/>
		<variables name="A_K_M174" typeName="BOOL"/>
		<variables name="A_K_M176" typeName="BOOL"/>
		<variables name="A_K_M201" typeName="BOOL"/>
		<variables name="A_K_M202" typeName="BOOL"/>
		<variables name="A_K_M203" typeName="BOOL"/>
		<variables name="A_K_M204" typeName="BOOL"/>
		<variables name="A_K_M206" typeName="BOOL"/>
		<variables name="A_K_M208" typeName="BOOL"/>
		<variables name="A_K_M210" typeName="BOOL"/>
		<variables name="A_K_M212" typeName="BOOL"/>
		<variables name="A_K_M214" typeName="BOOL"/>
		<variables name="A_K_M216" typeName="BOOL"/>
		<variables name="A_K_M218" typeName="BOOL"/>
		<variables name="A_K_M302" typeName="BOOL"/>
		<variables name="A_K_M302IR" typeName="BOOL"/>
		<variables name="A_K_M304" typeName="BOOL"/>
		<variables name="A_K_M306" typeName="BOOL"/>
		<variables name="A_K_M308" typeName="BOOL"/>
		<variables name="A_K_MAK_STR" typeName="BOOL"/>
		<variables name="A_K_MAKET_M" typeName="BOOL"/>
		<variables name="A_K_MAKET_P" typeName="BOOL"/>
		<variables name="A_K_MAKET_SR" typeName="BOOL"/>
		<variables name="A_K_MK" typeName="BOOL"/>
		<variables name="A_K_MN" typeName="BOOL"/>
		<variables name="A_K_MZ" typeName="BOOL"/>
		<variables name="A_K_NEISPR_SIST" typeName="BOOL"/>
		<variables name="A_K_NIR" typeName="BOOL"/>
		<variables name="A_K_OG" typeName="BOOL"/>
		<variables name="A_K_ON" typeName="BOOL"/>
		<variables name="A_K_PCHDIP_SLZ" typeName="BOOL"/>
		<variables name="A_K_PCHIP_SLZ" typeName="BOOL"/>
		<variables name="A_K_PNDIP_SLZ" typeName="BOOL"/>
		<variables name="A_K_PNIP_SLZ" typeName="BOOL"/>
		<variables name="A_K_RESET_DIAG" typeName="BOOL"/>
		<variables name="A_K_TVDIR" typeName="BOOL"/>
		<variables name="A_K_VZ_STR" typeName="BOOL"/>
		<variables name="A_KK_K" typeName="EBOOL">
			<comment>???????? ???? ??</comment>
			<attribute name="TimeStampCapability" value="0"/>
		</variables>
		<variables name="A_KK_U" typeName="BOOL"/>
		<variables name="A_KM" typeName="BOOL"/>
		<variables name="A_KMM_K" typeName="EBOOL">
			<comment>???????? ???? ???</comment>
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_KMP_K" typeName="EBOOL">
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_KONTR_ARM_DIAG" typeName="BOOL"/>
		<variables name="A_KONTR_NET_DIAG" typeName="BOOL"/>
		<variables name="A_KONTR_PLC_DIAG" typeName="BOOL"/>
		<variables name="A_KONTR_SIST_KL" typeName="BOOL"/>
		<variables name="A_KONTR_SIST_MIG_KL" typeName="BOOL"/>
		<variables name="A_KONTR_SIST_SrL" typeName="BOOL"/>
		<variables name="A_KP10P_K" typeName="EBOOL"/>
		<variables name="A_KP13_K" typeName="EBOOL"/>
		<variables name="A_KP13P_K" typeName="EBOOL"/>
		<variables name="A_KPA_IM" typeName="BOOL"/>
		<variables name="A_KPA_K" typeName="EBOOL">
			<comment>???????? ???? ???</comment>
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_KPN" typeName="BOOL"/>
		<variables name="A_KZP13_K" typeName="EBOOL"/>
		<variables name="A_L1_K" typeName="EBOOL"/>
		<variables name="A_L2_K" typeName="EBOOL"/>
		<variables name="A_L3_K" typeName="EBOOL"/>
		<variables name="A_L4_K" typeName="EBOOL"/>
		<variables name="A_L5_K" typeName="EBOOL"/>
		<variables name="A_L6_K" typeName="EBOOL"/>
		<variables name="A_L7_K" typeName="EBOOL"/>
		<variables name="A_L8_K" typeName="EBOOL"/>
		<variables name="A_L10_K" typeName="EBOOL"/>
		<variables name="A_M" typeName="BOOL"/>
		<variables name="A_M1_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M1_BL_IM" typeName="BOOL"/>
		<variables name="A_M1_GOT" typeName="BOOL"/>
		<variables name="A_M1_iK" typeName="BOOL"/>
		<variables name="A_M1_IND" typeName="BYTE"/>
		<variables name="A_M1_KM" typeName="BOOL"/>
		<variables name="A_M1_KS" typeName="BOOL"/>
		<variables name="A_M1_MOT" typeName="BOOL"/>
		<variables name="A_M1_MV1" typeName="BOOL"/>
		<variables name="A_M1_N" typeName="BOOL"/>
		<variables name="A_M1_OT" typeName="BOOL"/>
		<variables name="A_M1_OT1" typeName="BOOL"/>
		<variables name="A_M1_S" typeName="BOOL"/>
		<variables name="A_M1_S1" typeName="BOOL"/>
		<variables name="A_M1_SnL_IM" typeName="BOOL"/>
		<variables name="A_M1_UK" typeName="BOOL"/>
		<variables name="A_M1_VNM" typeName="BOOL"/>
		<variables name="A_M1_VO" typeName="BOOL"/>
		<variables name="A_M1KM" typeName="BOOL"/>
		<variables name="A_M1MS_K" typeName="EBOOL"/>
		<variables name="A_M1MS_U" typeName="EBOOL"/>
		<variables name="A_M1MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M1MS_U_PR" typeName="BOOL"/>
		<variables name="A_M1MS_UK" typeName="EBOOL"/>
		<variables name="A_M1N" typeName="BOOL"/>
		<variables name="A_M1O_K" typeName="EBOOL"/>
		<variables name="A_M3_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M3_BL_IM" typeName="BOOL"/>
		<variables name="A_M3_GOT" typeName="BOOL"/>
		<variables name="A_M3_iK" typeName="BOOL"/>
		<variables name="A_M3_IND" typeName="BYTE"/>
		<variables name="A_M3_KM" typeName="BOOL"/>
		<variables name="A_M3_KS" typeName="BOOL"/>
		<variables name="A_M3_MOT" typeName="BOOL"/>
		<variables name="A_M3_MS" typeName="BOOL"/>
		<variables name="A_M3_MS1" typeName="BOOL"/>
		<variables name="A_M3_MV1" typeName="BOOL"/>
		<variables name="A_M3_N" typeName="BOOL"/>
		<variables name="A_M3_NM" typeName="BOOL"/>
		<variables name="A_M3_OT" typeName="BOOL"/>
		<variables name="A_M3_OT1" typeName="BOOL"/>
		<variables name="A_M3_OTM" typeName="BOOL"/>
		<variables name="A_M3_S" typeName="BOOL"/>
		<variables name="A_M3_S1" typeName="BOOL"/>
		<variables name="A_M3_SnL_IM" typeName="BOOL"/>
		<variables name="A_M3_UK" typeName="BOOL"/>
		<variables name="A_M3_VCHM" typeName="BOOL"/>
		<variables name="A_M3_VN" typeName="BOOL"/>
		<variables name="A_M3_VO" typeName="BOOL"/>
		<variables name="A_M3KM" typeName="BOOL"/>
		<variables name="A_M3MS_K" typeName="EBOOL"/>
		<variables name="A_M3MS_U" typeName="EBOOL"/>
		<variables name="A_M3MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M3MS_U_PR" typeName="BOOL"/>
		<variables name="A_M3MS_UK" typeName="EBOOL"/>
		<variables name="A_M3N" typeName="BOOL"/>
		<variables name="A_M3O_K" typeName="EBOOL"/>
		<variables name="A_M5_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M5_BL_IM" typeName="BOOL"/>
		<variables name="A_M5_GOT" typeName="BOOL"/>
		<variables name="A_M5_iK" typeName="BOOL"/>
		<variables name="A_M5_IND" typeName="BYTE"/>
		<variables name="A_M5_KM" typeName="BOOL"/>
		<variables name="A_M5_KS" typeName="BOOL"/>
		<variables name="A_M5_MOT" typeName="BOOL"/>
		<variables name="A_M5_MV1" typeName="BOOL"/>
		<variables name="A_M5_N" typeName="BOOL"/>
		<variables name="A_M5_OT" typeName="BOOL"/>
		<variables name="A_M5_OT1" typeName="BOOL"/>
		<variables name="A_M5_S" typeName="BOOL"/>
		<variables name="A_M5_S1" typeName="BOOL"/>
		<variables name="A_M5_SnL_IM" typeName="BOOL"/>
		<variables name="A_M5_UK" typeName="BOOL"/>
		<variables name="A_M5_VCHM" typeName="BOOL"/>
		<variables name="A_M5_VO" typeName="BOOL"/>
		<variables name="A_M5KM" typeName="BOOL"/>
		<variables name="A_M5MS_K" typeName="EBOOL"/>
		<variables name="A_M5MS_U" typeName="EBOOL"/>
		<variables name="A_M5MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M5MS_U_PR" typeName="BOOL"/>
		<variables name="A_M5MS_UK" typeName="EBOOL"/>
		<variables name="A_M5N" typeName="BOOL"/>
		<variables name="A_M5O_K" typeName="EBOOL"/>
		<variables name="A_M7_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M7_BL_IM" typeName="BOOL"/>
		<variables name="A_M7_GOT" typeName="BOOL"/>
		<variables name="A_M7_iK" typeName="BOOL"/>
		<variables name="A_M7_IND" typeName="BYTE"/>
		<variables name="A_M7_KM" typeName="BOOL"/>
		<variables name="A_M7_KS" typeName="BOOL"/>
		<variables name="A_M7_MOT" typeName="BOOL"/>
		<variables name="A_M7_MV1" typeName="BOOL"/>
		<variables name="A_M7_N" typeName="BOOL"/>
		<variables name="A_M7_OT" typeName="BOOL"/>
		<variables name="A_M7_OT1" typeName="BOOL"/>
		<variables name="A_M7_S" typeName="BOOL"/>
		<variables name="A_M7_S1" typeName="BOOL"/>
		<variables name="A_M7_SnL_IM" typeName="BOOL"/>
		<variables name="A_M7_UK" typeName="BOOL"/>
		<variables name="A_M7_VNM" typeName="BOOL"/>
		<variables name="A_M7_VO" typeName="BOOL"/>
		<variables name="A_M7iK" typeName="BOOL"/>
		<variables name="A_M7KM" typeName="BOOL"/>
		<variables name="A_M7MS_K" typeName="EBOOL"/>
		<variables name="A_M7MS_U" typeName="EBOOL"/>
		<variables name="A_M7MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M7MS_U_PR" typeName="BOOL"/>
		<variables name="A_M7MS_UK" typeName="EBOOL"/>
		<variables name="A_M7N" typeName="BOOL"/>
		<variables name="A_M7O_K" typeName="EBOOL"/>
		<variables name="A_M7UK" typeName="BOOL"/>
		<variables name="A_M9_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M9_BL_IM" typeName="BOOL"/>
		<variables name="A_M9_GOT" typeName="BOOL"/>
		<variables name="A_M9_iK" typeName="BOOL"/>
		<variables name="A_M9_IND" typeName="BYTE"/>
		<variables name="A_M9_KM" typeName="BOOL"/>
		<variables name="A_M9_KS" typeName="BOOL"/>
		<variables name="A_M9_MOT" typeName="BOOL"/>
		<variables name="A_M9_MV1" typeName="BOOL"/>
		<variables name="A_M9_N" typeName="BOOL"/>
		<variables name="A_M9_OT" typeName="BOOL"/>
		<variables name="A_M9_OT1" typeName="BOOL"/>
		<variables name="A_M9_S" typeName="BOOL"/>
		<variables name="A_M9_S1" typeName="BOOL"/>
		<variables name="A_M9_SnL_IM" typeName="BOOL"/>
		<variables name="A_M9_UK" typeName="BOOL"/>
		<variables name="A_M9_VCHM" typeName="BOOL"/>
		<variables name="A_M9_VO" typeName="BOOL"/>
		<variables name="A_M9KM" typeName="BOOL"/>
		<variables name="A_M9MS_K" typeName="EBOOL"/>
		<variables name="A_M9MS_U" typeName="EBOOL"/>
		<variables name="A_M9MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M9MS_U_PR" typeName="BOOL"/>
		<variables name="A_M9MS_UK" typeName="EBOOL"/>
		<variables name="A_M9N" typeName="BOOL"/>
		<variables name="A_M9O_K" typeName="EBOOL"/>
		<variables name="A_M9S1" typeName="BOOL"/>
		<variables name="A_M9UK" typeName="BOOL"/>
		<variables name="A_M10_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M10_BL_IM" typeName="BOOL"/>
		<variables name="A_M10_GOT" typeName="BOOL"/>
		<variables name="A_M10_iK" typeName="BOOL"/>
		<variables name="A_M10_IND" typeName="BYTE"/>
		<variables name="A_M10_KL_IM" typeName="BOOL"/>
		<variables name="A_M10_KM" typeName="BOOL"/>
		<variables name="A_M10_KS" typeName="BOOL"/>
		<variables name="A_M10_MOT" typeName="BOOL"/>
		<variables name="A_M10_MV1" typeName="BOOL"/>
		<variables name="A_M10_N" typeName="BOOL"/>
		<variables name="A_M10_OT" typeName="BOOL"/>
		<variables name="A_M10_OT1" typeName="BOOL"/>
		<variables name="A_M10_S" typeName="BOOL"/>
		<variables name="A_M10_S1" typeName="BOOL"/>
		<variables name="A_M10_SnL_IM" typeName="BOOL"/>
		<variables name="A_M10_UK" typeName="BOOL"/>
		<variables name="A_M10_VCHM" typeName="BOOL"/>
		<variables name="A_M10_VO" typeName="BOOL"/>
		<variables name="A_M10KM" typeName="BOOL"/>
		<variables name="A_M10MS_K" typeName="EBOOL"/>
		<variables name="A_M10MS_U" typeName="EBOOL"/>
		<variables name="A_M10MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M10MS_U_PR" typeName="BOOL"/>
		<variables name="A_M10MS_UK" typeName="EBOOL"/>
		<variables name="A_M10N" typeName="BOOL"/>
		<variables name="A_M10O_K" typeName="EBOOL"/>
		<variables name="A_M11_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M11_BL_IM" typeName="BOOL"/>
		<variables name="A_M11_GOT" typeName="BOOL"/>
		<variables name="A_M11_iK" typeName="BOOL"/>
		<variables name="A_M11_IND" typeName="BYTE"/>
		<variables name="A_M11_KM" typeName="BOOL"/>
		<variables name="A_M11_KS" typeName="BOOL"/>
		<variables name="A_M11_MOT" typeName="BOOL"/>
		<variables name="A_M11_MV1" typeName="BOOL"/>
		<variables name="A_M11_N" typeName="BOOL"/>
		<variables name="A_M11_OT" typeName="BOOL"/>
		<variables name="A_M11_OT1" typeName="BOOL"/>
		<variables name="A_M11_S" typeName="BOOL"/>
		<variables name="A_M11_S1" typeName="BOOL"/>
		<variables name="A_M11_SnL_IM" typeName="BOOL"/>
		<variables name="A_M11_UK" typeName="BOOL"/>
		<variables name="A_M11_VNM" typeName="BOOL"/>
		<variables name="A_M11_VO" typeName="BOOL"/>
		<variables name="A_M11KM" typeName="BOOL"/>
		<variables name="A_M11MS_K" typeName="EBOOL"/>
		<variables name="A_M11MS_U" typeName="EBOOL"/>
		<variables name="A_M11MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M11MS_U_PR" typeName="BOOL"/>
		<variables name="A_M11MS_UK" typeName="EBOOL"/>
		<variables name="A_M11N" typeName="BOOL"/>
		<variables name="A_M11O_K" typeName="EBOOL"/>
		<variables name="A_M12_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M12_BL_IM" typeName="BOOL"/>
		<variables name="A_M12_GOT" typeName="BOOL"/>
		<variables name="A_M12_iK" typeName="BOOL"/>
		<variables name="A_M12_IND" typeName="BYTE"/>
		<variables name="A_M12_KL_IM" typeName="BOOL"/>
		<variables name="A_M12_KM" typeName="BOOL"/>
		<variables name="A_M12_KS" typeName="BOOL"/>
		<variables name="A_M12_MOT" typeName="BOOL"/>
		<variables name="A_M12_MV1" typeName="BOOL"/>
		<variables name="A_M12_N" typeName="BOOL"/>
		<variables name="A_M12_OT" typeName="BOOL"/>
		<variables name="A_M12_OT1" typeName="BOOL"/>
		<variables name="A_M12_S" typeName="BOOL"/>
		<variables name="A_M12_S1" typeName="BOOL"/>
		<variables name="A_M12_SnL_IM" typeName="BOOL"/>
		<variables name="A_M12_UK" typeName="BOOL"/>
		<variables name="A_M12_VCHM" typeName="BOOL"/>
		<variables name="A_M12_VNM" typeName="BOOL"/>
		<variables name="A_M12_VO" typeName="BOOL"/>
		<variables name="A_M12KM" typeName="BOOL"/>
		<variables name="A_M12MS_K" typeName="EBOOL"/>
		<variables name="A_M12MS_U" typeName="EBOOL"/>
		<variables name="A_M12MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M12MS_U_PR" typeName="BOOL"/>
		<variables name="A_M12MS_UK" typeName="EBOOL"/>
		<variables name="A_M12N" typeName="BOOL"/>
		<variables name="A_M12O_K" typeName="EBOOL"/>
		<variables name="A_M13_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M13_BL_IM" typeName="BOOL"/>
		<variables name="A_M13_GOT" typeName="BOOL"/>
		<variables name="A_M13_iK" typeName="BOOL"/>
		<variables name="A_M13_IND" typeName="BYTE"/>
		<variables name="A_M13_KM" typeName="BOOL"/>
		<variables name="A_M13_KS" typeName="BOOL"/>
		<variables name="A_M13_MOT" typeName="BOOL"/>
		<variables name="A_M13_MV1" typeName="BOOL"/>
		<variables name="A_M13_N" typeName="BOOL"/>
		<variables name="A_M13_OT" typeName="BOOL"/>
		<variables name="A_M13_OT1" typeName="BOOL"/>
		<variables name="A_M13_S" typeName="BOOL"/>
		<variables name="A_M13_S1" typeName="BOOL"/>
		<variables name="A_M13_SnL_IM" typeName="BOOL"/>
		<variables name="A_M13_UK" typeName="BOOL"/>
		<variables name="A_M13_VNM" typeName="BOOL"/>
		<variables name="A_M13_VO" typeName="BOOL"/>
		<variables name="A_M13KM" typeName="BOOL"/>
		<variables name="A_M13MS_K" typeName="EBOOL"/>
		<variables name="A_M13MS_U" typeName="EBOOL"/>
		<variables name="A_M13MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M13MS_U_PR" typeName="BOOL"/>
		<variables name="A_M13MS_UK" typeName="EBOOL"/>
		<variables name="A_M13N" typeName="BOOL"/>
		<variables name="A_M13O_K" typeName="EBOOL"/>
		<variables name="A_M14_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M14_BL_IM" typeName="BOOL"/>
		<variables name="A_M14_GOT" typeName="BOOL"/>
		<variables name="A_M14_iK" typeName="BOOL"/>
		<variables name="A_M14_IND" typeName="BYTE"/>
		<variables name="A_M14_KL_IM" typeName="BOOL"/>
		<variables name="A_M14_KM" typeName="BOOL"/>
		<variables name="A_M14_KS" typeName="BOOL"/>
		<variables name="A_M14_MOT" typeName="BOOL"/>
		<variables name="A_M14_MV1" typeName="BOOL"/>
		<variables name="A_M14_N" typeName="BOOL"/>
		<variables name="A_M14_OT" typeName="BOOL"/>
		<variables name="A_M14_OT1" typeName="BOOL"/>
		<variables name="A_M14_S" typeName="BOOL"/>
		<variables name="A_M14_S1" typeName="BOOL"/>
		<variables name="A_M14_SnL_IM" typeName="BOOL"/>
		<variables name="A_M14_UK" typeName="BOOL"/>
		<variables name="A_M14_VNM" typeName="BOOL"/>
		<variables name="A_M14_VO" typeName="BOOL"/>
		<variables name="A_M14KM" typeName="BOOL"/>
		<variables name="A_M14MS_K" typeName="EBOOL"/>
		<variables name="A_M14MS_U" typeName="EBOOL"/>
		<variables name="A_M14MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M14MS_U_PR" typeName="BOOL"/>
		<variables name="A_M14MS_UK" typeName="EBOOL"/>
		<variables name="A_M14N" typeName="BOOL"/>
		<variables name="A_M14O_K" typeName="EBOOL"/>
		<variables name="A_M15_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M15_BL_IM" typeName="BOOL"/>
		<variables name="A_M15_GOT" typeName="BOOL"/>
		<variables name="A_M15_iK" typeName="BOOL"/>
		<variables name="A_M15_IND" typeName="BYTE"/>
		<variables name="A_M15_KL_IM" typeName="BOOL"/>
		<variables name="A_M15_KM" typeName="BOOL"/>
		<variables name="A_M15_KS" typeName="BOOL"/>
		<variables name="A_M15_MOT" typeName="BOOL"/>
		<variables name="A_M15_MV1" typeName="BOOL"/>
		<variables name="A_M15_N" typeName="BOOL"/>
		<variables name="A_M15_OT" typeName="BOOL"/>
		<variables name="A_M15_OT1" typeName="BOOL"/>
		<variables name="A_M15_S" typeName="BOOL"/>
		<variables name="A_M15_S1" typeName="BOOL"/>
		<variables name="A_M15_SnL_IM" typeName="BOOL"/>
		<variables name="A_M15_UK" typeName="BOOL"/>
		<variables name="A_M15_VCHM" typeName="BOOL"/>
		<variables name="A_M15_VO" typeName="BOOL"/>
		<variables name="A_M15iK" typeName="BOOL"/>
		<variables name="A_M15KM" typeName="BOOL"/>
		<variables name="A_M15MS_K" typeName="EBOOL"/>
		<variables name="A_M15MS_U" typeName="EBOOL"/>
		<variables name="A_M15MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M15MS_U_PR" typeName="BOOL"/>
		<variables name="A_M15MS_UK" typeName="EBOOL"/>
		<variables name="A_M15N" typeName="BOOL"/>
		<variables name="A_M15O_K" typeName="EBOOL"/>
		<variables name="A_M15OT" typeName="BOOL"/>
		<variables name="A_M15OT1" typeName="BOOL"/>
		<variables name="A_M15S1" typeName="BOOL"/>
		<variables name="A_M15UK" typeName="BOOL"/>
		<variables name="A_M16_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M16_BL_IM" typeName="BOOL"/>
		<variables name="A_M16_GOT" typeName="BOOL"/>
		<variables name="A_M16_iK" typeName="BOOL"/>
		<variables name="A_M16_IND" typeName="BYTE"/>
		<variables name="A_M16_KL_IM" typeName="BOOL"/>
		<variables name="A_M16_KM" typeName="BOOL"/>
		<variables name="A_M16_KS" typeName="BOOL"/>
		<variables name="A_M16_MOT" typeName="BOOL"/>
		<variables name="A_M16_MV1" typeName="BOOL"/>
		<variables name="A_M16_N" typeName="BOOL"/>
		<variables name="A_M16_OT" typeName="BOOL"/>
		<variables name="A_M16_OT1" typeName="BOOL"/>
		<variables name="A_M16_S" typeName="BOOL"/>
		<variables name="A_M16_S1" typeName="BOOL"/>
		<variables name="A_M16_SnL_IM" typeName="BOOL"/>
		<variables name="A_M16_UK" typeName="BOOL"/>
		<variables name="A_M16_VNM" typeName="BOOL"/>
		<variables name="A_M16_VO" typeName="BOOL"/>
		<variables name="A_M16KM" typeName="BOOL"/>
		<variables name="A_M16MS_K" typeName="EBOOL"/>
		<variables name="A_M16MS_U" typeName="EBOOL"/>
		<variables name="A_M16MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M16MS_U_PR" typeName="BOOL"/>
		<variables name="A_M16MS_UK" typeName="EBOOL"/>
		<variables name="A_M16N" typeName="BOOL"/>
		<variables name="A_M16O_K" typeName="EBOOL"/>
		<variables name="A_M17_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M17_BL_IM" typeName="BOOL"/>
		<variables name="A_M17_GOT" typeName="BOOL"/>
		<variables name="A_M17_iK" typeName="BOOL"/>
		<variables name="A_M17_IND" typeName="BYTE"/>
		<variables name="A_M17_KM" typeName="BOOL"/>
		<variables name="A_M17_KS" typeName="BOOL"/>
		<variables name="A_M17_MOT" typeName="BOOL"/>
		<variables name="A_M17_MV1" typeName="BOOL"/>
		<variables name="A_M17_N" typeName="BOOL"/>
		<variables name="A_M17_OT" typeName="BOOL"/>
		<variables name="A_M17_OT1" typeName="BOOL"/>
		<variables name="A_M17_S" typeName="BOOL"/>
		<variables name="A_M17_S1" typeName="BOOL"/>
		<variables name="A_M17_SnL_IM" typeName="BOOL"/>
		<variables name="A_M17_UK" typeName="BOOL"/>
		<variables name="A_M17_VCHM" typeName="BOOL"/>
		<variables name="A_M17_VO" typeName="BOOL"/>
		<variables name="A_M17KM" typeName="BOOL"/>
		<variables name="A_M17MS_K" typeName="EBOOL"/>
		<variables name="A_M17MS_U" typeName="EBOOL"/>
		<variables name="A_M17MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M17MS_U_PR" typeName="BOOL"/>
		<variables name="A_M17MS_UK" typeName="EBOOL"/>
		<variables name="A_M17N" typeName="BOOL"/>
		<variables name="A_M17O_K" typeName="EBOOL"/>
		<variables name="A_M18_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M18_BL_IM" typeName="BOOL"/>
		<variables name="A_M18_GOT" typeName="BOOL"/>
		<variables name="A_M18_iK" typeName="BOOL"/>
		<variables name="A_M18_IND" typeName="BYTE"/>
		<variables name="A_M18_KL_IM" typeName="BOOL"/>
		<variables name="A_M18_KM" typeName="BOOL"/>
		<variables name="A_M18_KS" typeName="BOOL"/>
		<variables name="A_M18_MOT" typeName="BOOL"/>
		<variables name="A_M18_MV1" typeName="BOOL"/>
		<variables name="A_M18_N" typeName="BOOL"/>
		<variables name="A_M18_OT" typeName="BOOL"/>
		<variables name="A_M18_OT1" typeName="BOOL"/>
		<variables name="A_M18_S" typeName="BOOL"/>
		<variables name="A_M18_S1" typeName="BOOL"/>
		<variables name="A_M18_SnL_IM" typeName="BOOL"/>
		<variables name="A_M18_UK" typeName="BOOL"/>
		<variables name="A_M18_VCHM" typeName="BOOL"/>
		<variables name="A_M18_VO" typeName="BOOL"/>
		<variables name="A_M18KM" typeName="BOOL"/>
		<variables name="A_M18MS_K" typeName="EBOOL"/>
		<variables name="A_M18MS_U" typeName="EBOOL"/>
		<variables name="A_M18MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M18MS_U_PR" typeName="BOOL"/>
		<variables name="A_M18MS_UK" typeName="EBOOL"/>
		<variables name="A_M18N" typeName="BOOL"/>
		<variables name="A_M18O_K" typeName="EBOOL"/>
		<variables name="A_M19_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M19_BL_IM" typeName="BOOL"/>
		<variables name="A_M19_GOT" typeName="BOOL"/>
		<variables name="A_M19_iK" typeName="BOOL"/>
		<variables name="A_M19_IND" typeName="BYTE"/>
		<variables name="A_M19_KM" typeName="BOOL"/>
		<variables name="A_M19_KS" typeName="BOOL"/>
		<variables name="A_M19_MOT" typeName="BOOL"/>
		<variables name="A_M19_MV1" typeName="BOOL"/>
		<variables name="A_M19_N" typeName="BOOL"/>
		<variables name="A_M19_OT" typeName="BOOL"/>
		<variables name="A_M19_OT1" typeName="BOOL"/>
		<variables name="A_M19_S" typeName="BOOL"/>
		<variables name="A_M19_S1" typeName="BOOL"/>
		<variables name="A_M19_SnL_IM" typeName="BOOL"/>
		<variables name="A_M19_UK" typeName="BOOL"/>
		<variables name="A_M19_VCHM" typeName="BOOL"/>
		<variables name="A_M19_VO" typeName="BOOL"/>
		<variables name="A_M19KM" typeName="BOOL"/>
		<variables name="A_M19MS_K" typeName="EBOOL"/>
		<variables name="A_M19MS_U" typeName="EBOOL"/>
		<variables name="A_M19MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M19MS_U_PR" typeName="BOOL"/>
		<variables name="A_M19MS_UK" typeName="EBOOL"/>
		<variables name="A_M19N" typeName="BOOL"/>
		<variables name="A_M19O_K" typeName="EBOOL"/>
		<variables name="A_M20_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M20_BL_IM" typeName="BOOL"/>
		<variables name="A_M20_GOT" typeName="BOOL"/>
		<variables name="A_M20_iK" typeName="BOOL"/>
		<variables name="A_M20_IND" typeName="BYTE"/>
		<variables name="A_M20_KL_IM" typeName="BOOL"/>
		<variables name="A_M20_KM" typeName="BOOL"/>
		<variables name="A_M20_KS" typeName="BOOL"/>
		<variables name="A_M20_MOT" typeName="BOOL"/>
		<variables name="A_M20_MV1" typeName="BOOL"/>
		<variables name="A_M20_N" typeName="BOOL"/>
		<variables name="A_M20_OT" typeName="BOOL"/>
		<variables name="A_M20_OT1" typeName="BOOL"/>
		<variables name="A_M20_S" typeName="BOOL"/>
		<variables name="A_M20_S1" typeName="BOOL"/>
		<variables name="A_M20_SnL_IM" typeName="BOOL"/>
		<variables name="A_M20_UK" typeName="BOOL"/>
		<variables name="A_M20_VNM" typeName="BOOL"/>
		<variables name="A_M20_VO" typeName="BOOL"/>
		<variables name="A_M20KM" typeName="BOOL"/>
		<variables name="A_M20MS_K" typeName="EBOOL"/>
		<variables name="A_M20MS_U" typeName="EBOOL"/>
		<variables name="A_M20MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M20MS_U_PR" typeName="BOOL"/>
		<variables name="A_M20MS_UK" typeName="EBOOL"/>
		<variables name="A_M20N" typeName="BOOL"/>
		<variables name="A_M20O_K" typeName="EBOOL"/>
		<variables name="A_M21_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M21_BL_IM" typeName="BOOL"/>
		<variables name="A_M21_GOT" typeName="BOOL"/>
		<variables name="A_M21_iK" typeName="BOOL"/>
		<variables name="A_M21_IND" typeName="BYTE"/>
		<variables name="A_M21_KM" typeName="BOOL"/>
		<variables name="A_M21_KS" typeName="BOOL"/>
		<variables name="A_M21_MOT" typeName="BOOL"/>
		<variables name="A_M21_MV1" typeName="BOOL"/>
		<variables name="A_M21_N" typeName="BOOL"/>
		<variables name="A_M21_OT" typeName="BOOL"/>
		<variables name="A_M21_OT1" typeName="BOOL"/>
		<variables name="A_M21_S" typeName="BOOL"/>
		<variables name="A_M21_S1" typeName="BOOL"/>
		<variables name="A_M21_SnL_IM" typeName="BOOL"/>
		<variables name="A_M21_UK" typeName="BOOL"/>
		<variables name="A_M21_VCHM" typeName="BOOL"/>
		<variables name="A_M21_VO" typeName="BOOL"/>
		<variables name="A_M21KM" typeName="BOOL"/>
		<variables name="A_M21MS_K" typeName="EBOOL"/>
		<variables name="A_M21MS_U" typeName="EBOOL"/>
		<variables name="A_M21MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M21MS_U_PR" typeName="BOOL"/>
		<variables name="A_M21MS_UK" typeName="EBOOL"/>
		<variables name="A_M21N" typeName="BOOL"/>
		<variables name="A_M21O_K" typeName="EBOOL"/>
		<variables name="A_M22_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M22_BL_IM" typeName="BOOL"/>
		<variables name="A_M22_GOT" typeName="BOOL"/>
		<variables name="A_M22_iK" typeName="BOOL"/>
		<variables name="A_M22_IND" typeName="BYTE"/>
		<variables name="A_M22_KL_IM" typeName="BOOL"/>
		<variables name="A_M22_KM" typeName="BOOL"/>
		<variables name="A_M22_KS" typeName="BOOL"/>
		<variables name="A_M22_MOT" typeName="BOOL"/>
		<variables name="A_M22_MV1" typeName="BOOL"/>
		<variables name="A_M22_N" typeName="BOOL"/>
		<variables name="A_M22_OT" typeName="BOOL"/>
		<variables name="A_M22_OT1" typeName="BOOL"/>
		<variables name="A_M22_S" typeName="BOOL"/>
		<variables name="A_M22_S1" typeName="BOOL"/>
		<variables name="A_M22_SnL_IM" typeName="BOOL"/>
		<variables name="A_M22_UK" typeName="BOOL"/>
		<variables name="A_M22_VCHM" typeName="BOOL"/>
		<variables name="A_M22_VO" typeName="BOOL"/>
		<variables name="A_M22KM" typeName="BOOL"/>
		<variables name="A_M22MS_K" typeName="EBOOL"/>
		<variables name="A_M22MS_U" typeName="EBOOL"/>
		<variables name="A_M22MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M22MS_U_PR" typeName="BOOL"/>
		<variables name="A_M22MS_UK" typeName="EBOOL"/>
		<variables name="A_M22N" typeName="BOOL"/>
		<variables name="A_M22O_K" typeName="EBOOL"/>
		<variables name="A_M23_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M23_BL_IM" typeName="BOOL"/>
		<variables name="A_M23_GOT" typeName="BOOL"/>
		<variables name="A_M23_iK" typeName="BOOL"/>
		<variables name="A_M23_IND" typeName="BYTE"/>
		<variables name="A_M23_KM" typeName="BOOL"/>
		<variables name="A_M23_KS" typeName="BOOL"/>
		<variables name="A_M23_MOT" typeName="BOOL"/>
		<variables name="A_M23_MV1" typeName="BOOL"/>
		<variables name="A_M23_N" typeName="BOOL"/>
		<variables name="A_M23_OT" typeName="BOOL"/>
		<variables name="A_M23_OT1" typeName="BOOL"/>
		<variables name="A_M23_S" typeName="BOOL"/>
		<variables name="A_M23_S1" typeName="BOOL"/>
		<variables name="A_M23_SnL_IM" typeName="BOOL"/>
		<variables name="A_M23_UK" typeName="BOOL"/>
		<variables name="A_M23_VNM" typeName="BOOL"/>
		<variables name="A_M23_VO" typeName="BOOL"/>
		<variables name="A_M23KM" typeName="BOOL"/>
		<variables name="A_M23MS_K" typeName="EBOOL"/>
		<variables name="A_M23MS_U" typeName="EBOOL"/>
		<variables name="A_M23MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M23MS_U_PR" typeName="BOOL"/>
		<variables name="A_M23MS_UK" typeName="EBOOL"/>
		<variables name="A_M23N" typeName="BOOL"/>
		<variables name="A_M23O_K" typeName="EBOOL"/>
		<variables name="A_M25_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M25_BL_IM" typeName="BOOL"/>
		<variables name="A_M25_GOT" typeName="BOOL"/>
		<variables name="A_M25_iK" typeName="BOOL"/>
		<variables name="A_M25_IND" typeName="BYTE"/>
		<variables name="A_M25_KM" typeName="BOOL"/>
		<variables name="A_M25_KS" typeName="BOOL"/>
		<variables name="A_M25_MOT" typeName="BOOL"/>
		<variables name="A_M25_MV1" typeName="BOOL"/>
		<variables name="A_M25_N" typeName="BOOL"/>
		<variables name="A_M25_OT" typeName="BOOL"/>
		<variables name="A_M25_OT1" typeName="BOOL"/>
		<variables name="A_M25_S" typeName="BOOL"/>
		<variables name="A_M25_S1" typeName="BOOL"/>
		<variables name="A_M25_SnL_IM" typeName="BOOL"/>
		<variables name="A_M25_UK" typeName="BOOL"/>
		<variables name="A_M25_VNM" typeName="BOOL"/>
		<variables name="A_M25_VO" typeName="BOOL"/>
		<variables name="A_M25KM" typeName="BOOL"/>
		<variables name="A_M25MS_K" typeName="EBOOL"/>
		<variables name="A_M25MS_U" typeName="EBOOL"/>
		<variables name="A_M25MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M25MS_U_PR" typeName="BOOL"/>
		<variables name="A_M25MS_UK" typeName="EBOOL"/>
		<variables name="A_M25N" typeName="BOOL"/>
		<variables name="A_M25O_K" typeName="EBOOL"/>
		<variables name="A_M26_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M26_BL_IM" typeName="BOOL"/>
		<variables name="A_M26_GOT" typeName="BOOL"/>
		<variables name="A_M26_iK" typeName="BOOL"/>
		<variables name="A_M26_IND" typeName="BYTE"/>
		<variables name="A_M26_KL_IM" typeName="BOOL"/>
		<variables name="A_M26_KM" typeName="BOOL"/>
		<variables name="A_M26_KS" typeName="BOOL"/>
		<variables name="A_M26_MOT" typeName="BOOL"/>
		<variables name="A_M26_MV1" typeName="BOOL"/>
		<variables name="A_M26_N" typeName="BOOL"/>
		<variables name="A_M26_OT" typeName="BOOL"/>
		<variables name="A_M26_OT1" typeName="BOOL"/>
		<variables name="A_M26_S" typeName="BOOL"/>
		<variables name="A_M26_S1" typeName="BOOL"/>
		<variables name="A_M26_SnL_IM" typeName="BOOL"/>
		<variables name="A_M26_UK" typeName="BOOL"/>
		<variables name="A_M26_VNM" typeName="BOOL"/>
		<variables name="A_M26_VO" typeName="BOOL"/>
		<variables name="A_M26KM" typeName="BOOL"/>
		<variables name="A_M26MS_K" typeName="EBOOL"/>
		<variables name="A_M26MS_U" typeName="EBOOL"/>
		<variables name="A_M26MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M26MS_U_PR" typeName="BOOL"/>
		<variables name="A_M26MS_UK" typeName="EBOOL"/>
		<variables name="A_M26N" typeName="BOOL"/>
		<variables name="A_M26O_K" typeName="EBOOL"/>
		<variables name="A_M27_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M27_BL_IM" typeName="BOOL"/>
		<variables name="A_M27_GOT" typeName="BOOL"/>
		<variables name="A_M27_iK" typeName="BOOL"/>
		<variables name="A_M27_IND" typeName="BYTE"/>
		<variables name="A_M27_KL_IM" typeName="BOOL"/>
		<variables name="A_M27_KM" typeName="BOOL"/>
		<variables name="A_M27_KS" typeName="BOOL"/>
		<variables name="A_M27_MOT" typeName="BOOL"/>
		<variables name="A_M27_MV1" typeName="BOOL"/>
		<variables name="A_M27_N" typeName="BOOL"/>
		<variables name="A_M27_OT" typeName="BOOL"/>
		<variables name="A_M27_OT1" typeName="BOOL"/>
		<variables name="A_M27_S" typeName="BOOL"/>
		<variables name="A_M27_S1" typeName="BOOL"/>
		<variables name="A_M27_SnL_IM" typeName="BOOL"/>
		<variables name="A_M27_UK" typeName="BOOL"/>
		<variables name="A_M27_VCHM" typeName="BOOL"/>
		<variables name="A_M27_VO" typeName="BOOL"/>
		<variables name="A_M27KM" typeName="BOOL"/>
		<variables name="A_M27MS_K" typeName="EBOOL"/>
		<variables name="A_M27MS_U" typeName="EBOOL"/>
		<variables name="A_M27MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M27MS_U_PR" typeName="BOOL"/>
		<variables name="A_M27MS_UK" typeName="EBOOL"/>
		<variables name="A_M27N" typeName="BOOL"/>
		<variables name="A_M27O_K" typeName="EBOOL"/>
		<variables name="A_M28_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M28_BL_IM" typeName="BOOL"/>
		<variables name="A_M28_GOT" typeName="BOOL"/>
		<variables name="A_M28_iK" typeName="BOOL"/>
		<variables name="A_M28_IND" typeName="BYTE"/>
		<variables name="A_M28_KL_IM" typeName="BOOL"/>
		<variables name="A_M28_KM" typeName="BOOL"/>
		<variables name="A_M28_KS" typeName="BOOL"/>
		<variables name="A_M28_MOT" typeName="BOOL"/>
		<variables name="A_M28_MV1" typeName="BOOL"/>
		<variables name="A_M28_N" typeName="BOOL"/>
		<variables name="A_M28_OT" typeName="BOOL"/>
		<variables name="A_M28_OT1" typeName="BOOL"/>
		<variables name="A_M28_S" typeName="BOOL"/>
		<variables name="A_M28_S1" typeName="BOOL"/>
		<variables name="A_M28_SnL_IM" typeName="BOOL"/>
		<variables name="A_M28_UK" typeName="BOOL"/>
		<variables name="A_M28_VNM" typeName="BOOL"/>
		<variables name="A_M28_VO" typeName="BOOL"/>
		<variables name="A_M28KM" typeName="BOOL"/>
		<variables name="A_M28MS_K" typeName="EBOOL"/>
		<variables name="A_M28MS_U" typeName="EBOOL"/>
		<variables name="A_M28MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M28MS_U_PR" typeName="BOOL"/>
		<variables name="A_M28MS_UK" typeName="EBOOL"/>
		<variables name="A_M28N" typeName="BOOL"/>
		<variables name="A_M28O_K" typeName="EBOOL"/>
		<variables name="A_M29_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M29_BL_IM" typeName="BOOL"/>
		<variables name="A_M29_GOT" typeName="BOOL"/>
		<variables name="A_M29_iK" typeName="BOOL"/>
		<variables name="A_M29_IND" typeName="BYTE"/>
		<variables name="A_M29_KL_IM" typeName="BOOL"/>
		<variables name="A_M29_KM" typeName="BOOL"/>
		<variables name="A_M29_KS" typeName="BOOL"/>
		<variables name="A_M29_MOT" typeName="BOOL"/>
		<variables name="A_M29_MV1" typeName="BOOL"/>
		<variables name="A_M29_N" typeName="BOOL"/>
		<variables name="A_M29_OT" typeName="BOOL"/>
		<variables name="A_M29_OT1" typeName="BOOL"/>
		<variables name="A_M29_S" typeName="BOOL"/>
		<variables name="A_M29_S1" typeName="BOOL"/>
		<variables name="A_M29_SnL_IM" typeName="BOOL"/>
		<variables name="A_M29_UK" typeName="BOOL"/>
		<variables name="A_M29_VNM" typeName="BOOL"/>
		<variables name="A_M29_VO" typeName="BOOL"/>
		<variables name="A_M29KM" typeName="BOOL"/>
		<variables name="A_M29MS_K" typeName="EBOOL"/>
		<variables name="A_M29MS_U" typeName="EBOOL"/>
		<variables name="A_M29MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M29MS_U_PR" typeName="BOOL"/>
		<variables name="A_M29MS_UK" typeName="EBOOL"/>
		<variables name="A_M29N" typeName="BOOL"/>
		<variables name="A_M29O_K" typeName="EBOOL"/>
		<variables name="A_M30_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M30_BL_IM" typeName="BOOL"/>
		<variables name="A_M30_GOT" typeName="BOOL"/>
		<variables name="A_M30_iK" typeName="BOOL"/>
		<variables name="A_M30_IND" typeName="BYTE"/>
		<variables name="A_M30_KL_IM" typeName="BOOL"/>
		<variables name="A_M30_KM" typeName="BOOL"/>
		<variables name="A_M30_KS" typeName="BOOL"/>
		<variables name="A_M30_MOT" typeName="BOOL"/>
		<variables name="A_M30_MV1" typeName="BOOL"/>
		<variables name="A_M30_N" typeName="BOOL"/>
		<variables name="A_M30_OT" typeName="BOOL"/>
		<variables name="A_M30_OT1" typeName="BOOL"/>
		<variables name="A_M30_S" typeName="BOOL"/>
		<variables name="A_M30_S1" typeName="BOOL"/>
		<variables name="A_M30_SnL_IM" typeName="BOOL"/>
		<variables name="A_M30_UK" typeName="BOOL"/>
		<variables name="A_M30_VCHM" typeName="BOOL"/>
		<variables name="A_M30_VO" typeName="BOOL"/>
		<variables name="A_M30KM" typeName="BOOL"/>
		<variables name="A_M30MS_K" typeName="EBOOL"/>
		<variables name="A_M30MS_U" typeName="EBOOL"/>
		<variables name="A_M30MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M30MS_U_PR" typeName="BOOL"/>
		<variables name="A_M30MS_UK" typeName="EBOOL"/>
		<variables name="A_M30N" typeName="BOOL"/>
		<variables name="A_M30O_K" typeName="BOOL"/>
		<variables name="A_M32_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M32_BL_IM" typeName="BOOL"/>
		<variables name="A_M32_GOT" typeName="BOOL"/>
		<variables name="A_M32_iK" typeName="BOOL"/>
		<variables name="A_M32_IND" typeName="BYTE"/>
		<variables name="A_M32_KL_IM" typeName="BOOL"/>
		<variables name="A_M32_KM" typeName="BOOL"/>
		<variables name="A_M32_KS" typeName="BOOL"/>
		<variables name="A_M32_MOT" typeName="BOOL"/>
		<variables name="A_M32_MV1" typeName="BOOL"/>
		<variables name="A_M32_N" typeName="BOOL"/>
		<variables name="A_M32_OT" typeName="BOOL"/>
		<variables name="A_M32_OT1" typeName="BOOL"/>
		<variables name="A_M32_S" typeName="BOOL"/>
		<variables name="A_M32_S1" typeName="BOOL"/>
		<variables name="A_M32_SnL_IM" typeName="BOOL"/>
		<variables name="A_M32_UK" typeName="BOOL"/>
		<variables name="A_M32_VNM" typeName="BOOL"/>
		<variables name="A_M32_VO" typeName="BOOL"/>
		<variables name="A_M32KM" typeName="BOOL"/>
		<variables name="A_M32MS_K" typeName="EBOOL"/>
		<variables name="A_M32MS_U" typeName="EBOOL"/>
		<variables name="A_M32MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M32MS_U_PR" typeName="BOOL"/>
		<variables name="A_M32MS_UK" typeName="EBOOL"/>
		<variables name="A_M32N" typeName="BOOL"/>
		<variables name="A_M32O_K" typeName="EBOOL"/>
		<variables name="A_M33_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M33_BL_IM" typeName="BOOL"/>
		<variables name="A_M33_GOT" typeName="BOOL"/>
		<variables name="A_M33_iK" typeName="BOOL"/>
		<variables name="A_M33_IND" typeName="BYTE"/>
		<variables name="A_M33_KL_IM" typeName="BOOL"/>
		<variables name="A_M33_KM" typeName="BOOL"/>
		<variables name="A_M33_KS" typeName="BOOL"/>
		<variables name="A_M33_MOT" typeName="BOOL"/>
		<variables name="A_M33_MV1" typeName="BOOL"/>
		<variables name="A_M33_N" typeName="BOOL"/>
		<variables name="A_M33_OT" typeName="BOOL"/>
		<variables name="A_M33_OT1" typeName="BOOL"/>
		<variables name="A_M33_S" typeName="BOOL"/>
		<variables name="A_M33_S1" typeName="BOOL"/>
		<variables name="A_M33_SnL_IM" typeName="BOOL"/>
		<variables name="A_M33_UK" typeName="BOOL"/>
		<variables name="A_M33_VCHM" typeName="BOOL"/>
		<variables name="A_M33_VO" typeName="BOOL"/>
		<variables name="A_M33KM" typeName="BOOL"/>
		<variables name="A_M33MS_K" typeName="EBOOL"/>
		<variables name="A_M33MS_U" typeName="EBOOL"/>
		<variables name="A_M33MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M33MS_U_PR" typeName="BOOL"/>
		<variables name="A_M33MS_UK" typeName="EBOOL"/>
		<variables name="A_M33N" typeName="BOOL"/>
		<variables name="A_M33O_K" typeName="EBOOL"/>
		<variables name="A_M34_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M34_BL_IM" typeName="BOOL"/>
		<variables name="A_M34_GOT" typeName="BOOL"/>
		<variables name="A_M34_iK" typeName="BOOL"/>
		<variables name="A_M34_IND" typeName="BYTE"/>
		<variables name="A_M34_KL_IM" typeName="BOOL"/>
		<variables name="A_M34_KM" typeName="BOOL"/>
		<variables name="A_M34_KS" typeName="BOOL"/>
		<variables name="A_M34_MOT" typeName="BOOL"/>
		<variables name="A_M34_MV1" typeName="BOOL"/>
		<variables name="A_M34_N" typeName="BOOL"/>
		<variables name="A_M34_OT" typeName="BOOL"/>
		<variables name="A_M34_OT1" typeName="BOOL"/>
		<variables name="A_M34_S" typeName="BOOL"/>
		<variables name="A_M34_S1" typeName="BOOL"/>
		<variables name="A_M34_SnL_IM" typeName="BOOL"/>
		<variables name="A_M34_UK" typeName="BOOL"/>
		<variables name="A_M34_VCHM" typeName="BOOL"/>
		<variables name="A_M34_VO" typeName="BOOL"/>
		<variables name="A_M34KM" typeName="BOOL"/>
		<variables name="A_M34MS_K" typeName="EBOOL"/>
		<variables name="A_M34MS_U" typeName="EBOOL"/>
		<variables name="A_M34MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M34MS_U_PR" typeName="BOOL"/>
		<variables name="A_M34MS_UK" typeName="EBOOL"/>
		<variables name="A_M34N" typeName="BOOL"/>
		<variables name="A_M34O_K" typeName="EBOOL"/>
		<variables name="A_M35_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M35_BL_IM" typeName="BOOL"/>
		<variables name="A_M35_GOT" typeName="BOOL"/>
		<variables name="A_M35_iK" typeName="BOOL"/>
		<variables name="A_M35_IND" typeName="BYTE"/>
		<variables name="A_M35_KM" typeName="BOOL"/>
		<variables name="A_M35_KS" typeName="BOOL"/>
		<variables name="A_M35_MOT" typeName="BOOL"/>
		<variables name="A_M35_MV1" typeName="BOOL"/>
		<variables name="A_M35_N" typeName="BOOL"/>
		<variables name="A_M35_OT" typeName="BOOL"/>
		<variables name="A_M35_OT1" typeName="BOOL"/>
		<variables name="A_M35_S" typeName="BOOL"/>
		<variables name="A_M35_S1" typeName="BOOL"/>
		<variables name="A_M35_SnL_IM" typeName="BOOL"/>
		<variables name="A_M35_UK" typeName="BOOL"/>
		<variables name="A_M35_VCHM" typeName="BOOL"/>
		<variables name="A_M35_VO" typeName="BOOL"/>
		<variables name="A_M35KM" typeName="BOOL"/>
		<variables name="A_M35MS_K" typeName="EBOOL"/>
		<variables name="A_M35MS_U" typeName="EBOOL"/>
		<variables name="A_M35MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M35MS_U_PR" typeName="BOOL"/>
		<variables name="A_M35MS_UK" typeName="EBOOL"/>
		<variables name="A_M35N" typeName="BOOL"/>
		<variables name="A_M35O_K" typeName="EBOOL"/>
		<variables name="A_M36_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M36_BL_IM" typeName="BOOL"/>
		<variables name="A_M36_GOT" typeName="BOOL"/>
		<variables name="A_M36_iK" typeName="BOOL"/>
		<variables name="A_M36_IND" typeName="BYTE"/>
		<variables name="A_M36_KL_IM" typeName="BOOL"/>
		<variables name="A_M36_KM" typeName="BOOL"/>
		<variables name="A_M36_KS" typeName="BOOL"/>
		<variables name="A_M36_MOT" typeName="BOOL"/>
		<variables name="A_M36_MV1" typeName="BOOL"/>
		<variables name="A_M36_N" typeName="BOOL"/>
		<variables name="A_M36_OT" typeName="BOOL"/>
		<variables name="A_M36_OT1" typeName="BOOL"/>
		<variables name="A_M36_S" typeName="BOOL"/>
		<variables name="A_M36_S1" typeName="BOOL"/>
		<variables name="A_M36_SnL_IM" typeName="BOOL"/>
		<variables name="A_M36_UK" typeName="BOOL"/>
		<variables name="A_M36_VCHM" typeName="BOOL"/>
		<variables name="A_M36_VO" typeName="BOOL"/>
		<variables name="A_M36KM" typeName="BOOL"/>
		<variables name="A_M36MS_K" typeName="EBOOL"/>
		<variables name="A_M36MS_U" typeName="EBOOL"/>
		<variables name="A_M36MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M36MS_U_PR" typeName="BOOL"/>
		<variables name="A_M36MS_UK" typeName="EBOOL"/>
		<variables name="A_M36N" typeName="BOOL"/>
		<variables name="A_M36O_K" typeName="EBOOL"/>
		<variables name="A_M37_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M37_BL_IM" typeName="BOOL"/>
		<variables name="A_M37_GOT" typeName="BOOL"/>
		<variables name="A_M37_iK" typeName="BOOL"/>
		<variables name="A_M37_IND" typeName="BYTE"/>
		<variables name="A_M37_KM" typeName="BOOL"/>
		<variables name="A_M37_KS" typeName="BOOL"/>
		<variables name="A_M37_MOT" typeName="BOOL"/>
		<variables name="A_M37_MV1" typeName="BOOL"/>
		<variables name="A_M37_N" typeName="BOOL"/>
		<variables name="A_M37_OT" typeName="BOOL"/>
		<variables name="A_M37_OT1" typeName="BOOL"/>
		<variables name="A_M37_S" typeName="BOOL"/>
		<variables name="A_M37_S1" typeName="BOOL"/>
		<variables name="A_M37_SnL_IM" typeName="BOOL"/>
		<variables name="A_M37_UK" typeName="BOOL"/>
		<variables name="A_M37_VCHM" typeName="BOOL"/>
		<variables name="A_M37_VO" typeName="BOOL"/>
		<variables name="A_M37KM" typeName="BOOL"/>
		<variables name="A_M37MS_K" typeName="EBOOL"/>
		<variables name="A_M37MS_U" typeName="EBOOL"/>
		<variables name="A_M37MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M37MS_U_PR" typeName="BOOL"/>
		<variables name="A_M37MS_UK" typeName="EBOOL"/>
		<variables name="A_M37N" typeName="BOOL"/>
		<variables name="A_M37O_K" typeName="EBOOL"/>
		<variables name="A_M38_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M38_BL_IM" typeName="BOOL"/>
		<variables name="A_M38_GOT" typeName="BOOL"/>
		<variables name="A_M38_iK" typeName="BOOL"/>
		<variables name="A_M38_IND" typeName="BYTE"/>
		<variables name="A_M38_KL_IM" typeName="BOOL"/>
		<variables name="A_M38_KM" typeName="BOOL"/>
		<variables name="A_M38_KS" typeName="BOOL"/>
		<variables name="A_M38_MOT" typeName="BOOL"/>
		<variables name="A_M38_MV1" typeName="BOOL"/>
		<variables name="A_M38_N" typeName="BOOL"/>
		<variables name="A_M38_OT" typeName="BOOL"/>
		<variables name="A_M38_OT1" typeName="BOOL"/>
		<variables name="A_M38_S" typeName="BOOL"/>
		<variables name="A_M38_S1" typeName="BOOL"/>
		<variables name="A_M38_SnL_IM" typeName="BOOL"/>
		<variables name="A_M38_UK" typeName="BOOL"/>
		<variables name="A_M38_VCHM" typeName="BOOL"/>
		<variables name="A_M38_VNM" typeName="BOOL"/>
		<variables name="A_M38_VO" typeName="BOOL"/>
		<variables name="A_M38MS_K" typeName="EBOOL"/>
		<variables name="A_M38MS_U" typeName="EBOOL"/>
		<variables name="A_M38MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M38MS_U_PR" typeName="BOOL"/>
		<variables name="A_M38MS_UK" typeName="EBOOL"/>
		<variables name="A_M38N" typeName="BOOL"/>
		<variables name="A_M38O_K" typeName="EBOOL"/>
		<variables name="A_M39_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M39_BL_IM" typeName="BOOL"/>
		<variables name="A_M39_GOT" typeName="BOOL"/>
		<variables name="A_M39_iK" typeName="BOOL"/>
		<variables name="A_M39_IND" typeName="BYTE"/>
		<variables name="A_M39_KM" typeName="BOOL"/>
		<variables name="A_M39_KS" typeName="BOOL"/>
		<variables name="A_M39_MOT" typeName="BOOL"/>
		<variables name="A_M39_MV1" typeName="BOOL"/>
		<variables name="A_M39_N" typeName="BOOL"/>
		<variables name="A_M39_OT" typeName="BOOL"/>
		<variables name="A_M39_OT1" typeName="BOOL"/>
		<variables name="A_M39_S" typeName="BOOL"/>
		<variables name="A_M39_S1" typeName="BOOL"/>
		<variables name="A_M39_SnL_IM" typeName="BOOL"/>
		<variables name="A_M39_VCHM" typeName="BOOL"/>
		<variables name="A_M39_VNM" typeName="BOOL"/>
		<variables name="A_M39_VO" typeName="BOOL"/>
		<variables name="A_M39MS_K" typeName="BOOL"/>
		<variables name="A_M39MS_U" typeName="EBOOL">
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_M39MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M39MS_U_PR" typeName="BOOL"/>
		<variables name="A_M39MS_UK" typeName="EBOOL"/>
		<variables name="A_M39N" typeName="BOOL"/>
		<variables name="A_M39O_K" typeName="EBOOL"/>
		<variables name="A_M40_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M40_BL_IM" typeName="BOOL"/>
		<variables name="A_M40_GOT" typeName="BOOL"/>
		<variables name="A_M40_iK" typeName="BOOL"/>
		<variables name="A_M40_IND" typeName="BYTE"/>
		<variables name="A_M40_KL_IM" typeName="BOOL"/>
		<variables name="A_M40_KM" typeName="BOOL"/>
		<variables name="A_M40_KS" typeName="BOOL"/>
		<variables name="A_M40_MOT" typeName="BOOL"/>
		<variables name="A_M40_MV1" typeName="BOOL"/>
		<variables name="A_M40_N" typeName="BOOL"/>
		<variables name="A_M40_OT" typeName="BOOL"/>
		<variables name="A_M40_OT1" typeName="BOOL"/>
		<variables name="A_M40_S" typeName="BOOL"/>
		<variables name="A_M40_S1" typeName="BOOL"/>
		<variables name="A_M40_SnL_IM" typeName="BOOL"/>
		<variables name="A_M40_UK" typeName="BOOL"/>
		<variables name="A_M40_VCHM" typeName="BOOL"/>
		<variables name="A_M40_VNM" typeName="BOOL"/>
		<variables name="A_M40_VO" typeName="BOOL"/>
		<variables name="A_M40MS_K" typeName="EBOOL"/>
		<variables name="A_M40MS_U" typeName="EBOOL"/>
		<variables name="A_M40MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M40MS_U_PR" typeName="BOOL"/>
		<variables name="A_M40MS_UK" typeName="EBOOL"/>
		<variables name="A_M40N" typeName="BOOL"/>
		<variables name="A_M40O_K" typeName="EBOOL"/>
		<variables name="A_M41_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M41_BL_IM" typeName="BOOL"/>
		<variables name="A_M41_GOT" typeName="BOOL"/>
		<variables name="A_M41_iK" typeName="BOOL"/>
		<variables name="A_M41_IND" typeName="BYTE"/>
		<variables name="A_M41_KM" typeName="BOOL"/>
		<variables name="A_M41_KS" typeName="BOOL"/>
		<variables name="A_M41_MOT" typeName="BOOL"/>
		<variables name="A_M41_MV1" typeName="BOOL"/>
		<variables name="A_M41_N" typeName="BOOL"/>
		<variables name="A_M41_OT" typeName="BOOL"/>
		<variables name="A_M41_OT1" typeName="BOOL"/>
		<variables name="A_M41_S" typeName="BOOL"/>
		<variables name="A_M41_S1" typeName="BOOL"/>
		<variables name="A_M41_SnL_IM" typeName="BOOL"/>
		<variables name="A_M41_UK" typeName="BOOL"/>
		<variables name="A_M41_VCHM" typeName="BOOL"/>
		<variables name="A_M41_VO" typeName="BOOL"/>
		<variables name="A_M41KM" typeName="BOOL"/>
		<variables name="A_M41MS_K" typeName="EBOOL"/>
		<variables name="A_M41MS_U" typeName="EBOOL"/>
		<variables name="A_M41MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M41MS_U_PR" typeName="BOOL"/>
		<variables name="A_M41MS_UK" typeName="EBOOL"/>
		<variables name="A_M41N" typeName="BOOL"/>
		<variables name="A_M41O_K" typeName="EBOOL"/>
		<variables name="A_M42_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M42_BL_IM" typeName="BOOL"/>
		<variables name="A_M42_GOT" typeName="BOOL"/>
		<variables name="A_M42_iK" typeName="BOOL"/>
		<variables name="A_M42_IND" typeName="BYTE"/>
		<variables name="A_M42_KL_IM" typeName="BOOL"/>
		<variables name="A_M42_KM" typeName="BOOL"/>
		<variables name="A_M42_KS" typeName="BOOL"/>
		<variables name="A_M42_MOT" typeName="BOOL"/>
		<variables name="A_M42_MV1" typeName="BOOL"/>
		<variables name="A_M42_N" typeName="BOOL"/>
		<variables name="A_M42_OT" typeName="BOOL"/>
		<variables name="A_M42_OT1" typeName="BOOL"/>
		<variables name="A_M42_S" typeName="BOOL"/>
		<variables name="A_M42_S1" typeName="BOOL"/>
		<variables name="A_M42_SnL_IM" typeName="BOOL"/>
		<variables name="A_M42_UK" typeName="BOOL"/>
		<variables name="A_M42_VCHM" typeName="BOOL"/>
		<variables name="A_M42_VO" typeName="BOOL"/>
		<variables name="A_M42MS_K" typeName="EBOOL"/>
		<variables name="A_M42MS_U" typeName="EBOOL"/>
		<variables name="A_M42MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M42MS_U_PR" typeName="BOOL"/>
		<variables name="A_M42MS_UK" typeName="EBOOL"/>
		<variables name="A_M42N" typeName="BOOL"/>
		<variables name="A_M42O_K" typeName="EBOOL"/>
		<variables name="A_M43_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M43_BL_IM" typeName="BOOL"/>
		<variables name="A_M43_GOT" typeName="BOOL"/>
		<variables name="A_M43_iK" typeName="BOOL"/>
		<variables name="A_M43_IND" typeName="BYTE"/>
		<variables name="A_M43_KL_IM" typeName="BOOL"/>
		<variables name="A_M43_KM" typeName="BOOL"/>
		<variables name="A_M43_KS" typeName="BOOL"/>
		<variables name="A_M43_MOT" typeName="BOOL"/>
		<variables name="A_M43_MV1" typeName="BOOL"/>
		<variables name="A_M43_N" typeName="BOOL"/>
		<variables name="A_M43_OT" typeName="BOOL"/>
		<variables name="A_M43_OT1" typeName="BOOL"/>
		<variables name="A_M43_S" typeName="BOOL"/>
		<variables name="A_M43_S1" typeName="BOOL"/>
		<variables name="A_M43_SnL_IM" typeName="BOOL"/>
		<variables name="A_M43_UK" typeName="BOOL"/>
		<variables name="A_M43_VCHM" typeName="BOOL"/>
		<variables name="A_M43_VO" typeName="BOOL"/>
		<variables name="A_M43KM" typeName="BOOL"/>
		<variables name="A_M43MS_K" typeName="EBOOL"/>
		<variables name="A_M43MS_U" typeName="EBOOL"/>
		<variables name="A_M43MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M43MS_U_PR" typeName="BOOL"/>
		<variables name="A_M43MS_UK" typeName="EBOOL"/>
		<variables name="A_M43N" typeName="BOOL"/>
		<variables name="A_M43O_K" typeName="EBOOL"/>
		<variables name="A_M44_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M44_BL_IM" typeName="BOOL"/>
		<variables name="A_M44_GOT" typeName="BOOL"/>
		<variables name="A_M44_iK" typeName="BOOL"/>
		<variables name="A_M44_IND" typeName="BYTE"/>
		<variables name="A_M44_KL_IM" typeName="BOOL"/>
		<variables name="A_M44_KM" typeName="BOOL"/>
		<variables name="A_M44_KS" typeName="BOOL"/>
		<variables name="A_M44_MOT" typeName="BOOL"/>
		<variables name="A_M44_MV1" typeName="BOOL"/>
		<variables name="A_M44_N" typeName="BOOL"/>
		<variables name="A_M44_OT" typeName="BOOL"/>
		<variables name="A_M44_OT1" typeName="BOOL"/>
		<variables name="A_M44_S" typeName="BOOL"/>
		<variables name="A_M44_S1" typeName="BOOL"/>
		<variables name="A_M44_SnL_IM" typeName="BOOL"/>
		<variables name="A_M44_UK" typeName="BOOL"/>
		<variables name="A_M44_VNM" typeName="BOOL"/>
		<variables name="A_M44_VO" typeName="BOOL"/>
		<variables name="A_M44MS_K" typeName="EBOOL"/>
		<variables name="A_M44MS_U" typeName="EBOOL"/>
		<variables name="A_M44MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M44MS_U_PR" typeName="BOOL"/>
		<variables name="A_M44MS_UK" typeName="EBOOL"/>
		<variables name="A_M44N" typeName="BOOL"/>
		<variables name="A_M44O_K" typeName="EBOOL"/>
		<variables name="A_M45_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M45_BL_IM" typeName="BOOL"/>
		<variables name="A_M45_GOT" typeName="BOOL"/>
		<variables name="A_M45_iK" typeName="BOOL"/>
		<variables name="A_M45_IND" typeName="BYTE"/>
		<variables name="A_M45_KL_IM" typeName="BOOL"/>
		<variables name="A_M45_KM" typeName="BOOL"/>
		<variables name="A_M45_KS" typeName="BOOL"/>
		<variables name="A_M45_MOT" typeName="BOOL"/>
		<variables name="A_M45_MV1" typeName="BOOL"/>
		<variables name="A_M45_N" typeName="BOOL"/>
		<variables name="A_M45_OT" typeName="BOOL"/>
		<variables name="A_M45_OT1" typeName="BOOL"/>
		<variables name="A_M45_S" typeName="BOOL"/>
		<variables name="A_M45_S1" typeName="BOOL"/>
		<variables name="A_M45_SnL_IM" typeName="BOOL"/>
		<variables name="A_M45_UK" typeName="BOOL"/>
		<variables name="A_M45_VCHM" typeName="BOOL"/>
		<variables name="A_M45_VO" typeName="BOOL"/>
		<variables name="A_M45KM" typeName="BOOL"/>
		<variables name="A_M45MS_K" typeName="EBOOL"/>
		<variables name="A_M45MS_U" typeName="EBOOL"/>
		<variables name="A_M45MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M45MS_U_PR" typeName="BOOL"/>
		<variables name="A_M45MS_UK" typeName="EBOOL"/>
		<variables name="A_M45N" typeName="BOOL"/>
		<variables name="A_M45O_K" typeName="EBOOL"/>
		<variables name="A_M46_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M46_BL_IM" typeName="BOOL"/>
		<variables name="A_M46_GOT" typeName="BOOL"/>
		<variables name="A_M46_iK" typeName="BOOL"/>
		<variables name="A_M46_IND" typeName="BYTE"/>
		<variables name="A_M46_KL_IM" typeName="BOOL"/>
		<variables name="A_M46_KM" typeName="BOOL"/>
		<variables name="A_M46_KS" typeName="BOOL"/>
		<variables name="A_M46_MOT" typeName="BOOL"/>
		<variables name="A_M46_MV1" typeName="BOOL"/>
		<variables name="A_M46_N" typeName="BOOL"/>
		<variables name="A_M46_OT" typeName="BOOL"/>
		<variables name="A_M46_OT1" typeName="BOOL"/>
		<variables name="A_M46_S" typeName="BOOL"/>
		<variables name="A_M46_S1" typeName="BOOL"/>
		<variables name="A_M46_SnL_IM" typeName="BOOL"/>
		<variables name="A_M46_UK" typeName="BOOL"/>
		<variables name="A_M46_VCHM" typeName="BOOL"/>
		<variables name="A_M46_VO" typeName="BOOL"/>
		<variables name="A_M46KM" typeName="BOOL"/>
		<variables name="A_M46MS_K" typeName="EBOOL"/>
		<variables name="A_M46MS_U" typeName="EBOOL"/>
		<variables name="A_M46MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M46MS_U_PR" typeName="BOOL"/>
		<variables name="A_M46MS_UK" typeName="EBOOL"/>
		<variables name="A_M46N" typeName="BOOL"/>
		<variables name="A_M46O_K" typeName="EBOOL"/>
		<variables name="A_M47_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M47_BL_IM" typeName="BOOL"/>
		<variables name="A_M47_GOT" typeName="BOOL"/>
		<variables name="A_M47_iK" typeName="BOOL"/>
		<variables name="A_M47_IND" typeName="BYTE"/>
		<variables name="A_M47_KL_IM" typeName="BOOL"/>
		<variables name="A_M47_KM" typeName="BOOL"/>
		<variables name="A_M47_KS" typeName="BOOL"/>
		<variables name="A_M47_MOT" typeName="BOOL"/>
		<variables name="A_M47_MV1" typeName="BOOL"/>
		<variables name="A_M47_N" typeName="BOOL"/>
		<variables name="A_M47_OT" typeName="BOOL"/>
		<variables name="A_M47_OT1" typeName="BOOL"/>
		<variables name="A_M47_S" typeName="BOOL"/>
		<variables name="A_M47_S1" typeName="BOOL"/>
		<variables name="A_M47_SnL_IM" typeName="BOOL"/>
		<variables name="A_M47_UK" typeName="BOOL"/>
		<variables name="A_M47_VCHM" typeName="BOOL"/>
		<variables name="A_M47_VO" typeName="BOOL"/>
		<variables name="A_M47KM" typeName="BOOL"/>
		<variables name="A_M47MS_K" typeName="EBOOL"/>
		<variables name="A_M47MS_U" typeName="EBOOL"/>
		<variables name="A_M47MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M47MS_U_PR" typeName="BOOL"/>
		<variables name="A_M47MS_UK" typeName="EBOOL"/>
		<variables name="A_M47N" typeName="BOOL"/>
		<variables name="A_M47O_K" typeName="EBOOL"/>
		<variables name="A_M48_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M48_BL_IM" typeName="BOOL"/>
		<variables name="A_M48_GOT" typeName="BOOL"/>
		<variables name="A_M48_iK" typeName="BOOL"/>
		<variables name="A_M48_IND" typeName="BYTE"/>
		<variables name="A_M48_KL_IM" typeName="BOOL"/>
		<variables name="A_M48_KM" typeName="BOOL"/>
		<variables name="A_M48_KS" typeName="BOOL"/>
		<variables name="A_M48_MOT" typeName="BOOL"/>
		<variables name="A_M48_MV1" typeName="BOOL"/>
		<variables name="A_M48_N" typeName="BOOL"/>
		<variables name="A_M48_OT" typeName="BOOL"/>
		<variables name="A_M48_OT1" typeName="BOOL"/>
		<variables name="A_M48_S" typeName="BOOL"/>
		<variables name="A_M48_S1" typeName="BOOL"/>
		<variables name="A_M48_SnL_IM" typeName="BOOL"/>
		<variables name="A_M48_UK" typeName="BOOL"/>
		<variables name="A_M48_VCHM" typeName="BOOL"/>
		<variables name="A_M48_VO" typeName="BOOL"/>
		<variables name="A_M48KM" typeName="BOOL"/>
		<variables name="A_M48MS_K" typeName="EBOOL"/>
		<variables name="A_M48MS_U" typeName="EBOOL"/>
		<variables name="A_M48MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M48MS_U_PR" typeName="BOOL"/>
		<variables name="A_M48MS_UK" typeName="EBOOL"/>
		<variables name="A_M48N" typeName="BOOL"/>
		<variables name="A_M48O_K" typeName="EBOOL"/>
		<variables name="A_M49_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M49_BL_IM" typeName="BOOL"/>
		<variables name="A_M49_GOT" typeName="BOOL"/>
		<variables name="A_M49_iK" typeName="BOOL"/>
		<variables name="A_M49_IND" typeName="BYTE"/>
		<variables name="A_M49_KL_IM" typeName="BOOL"/>
		<variables name="A_M49_KM" typeName="BOOL"/>
		<variables name="A_M49_KS" typeName="BOOL"/>
		<variables name="A_M49_MOT" typeName="BOOL"/>
		<variables name="A_M49_MV1" typeName="BOOL"/>
		<variables name="A_M49_N" typeName="BOOL"/>
		<variables name="A_M49_OT" typeName="BOOL"/>
		<variables name="A_M49_OT1" typeName="BOOL"/>
		<variables name="A_M49_S" typeName="BOOL"/>
		<variables name="A_M49_S1" typeName="BOOL"/>
		<variables name="A_M49_SnL_IM" typeName="BOOL"/>
		<variables name="A_M49_UK" typeName="BOOL"/>
		<variables name="A_M49_VCHM" typeName="BOOL"/>
		<variables name="A_M49_VO" typeName="BOOL"/>
		<variables name="A_M49KM" typeName="BOOL"/>
		<variables name="A_M49MS_K" typeName="EBOOL"/>
		<variables name="A_M49MS_U" typeName="EBOOL"/>
		<variables name="A_M49MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M49MS_U_PR" typeName="BOOL"/>
		<variables name="A_M49MS_UK" typeName="EBOOL"/>
		<variables name="A_M49N" typeName="BOOL"/>
		<variables name="A_M49O_K" typeName="EBOOL"/>
		<variables name="A_M50_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M50_BL_IM" typeName="BOOL"/>
		<variables name="A_M50_GOT" typeName="BOOL"/>
		<variables name="A_M50_iK" typeName="BOOL"/>
		<variables name="A_M50_IND" typeName="BYTE"/>
		<variables name="A_M50_KL_IM" typeName="BOOL"/>
		<variables name="A_M50_KM" typeName="BOOL"/>
		<variables name="A_M50_KS" typeName="BOOL"/>
		<variables name="A_M50_MOT" typeName="BOOL"/>
		<variables name="A_M50_MV1" typeName="BOOL"/>
		<variables name="A_M50_N" typeName="BOOL"/>
		<variables name="A_M50_OT" typeName="BOOL"/>
		<variables name="A_M50_OT1" typeName="BOOL"/>
		<variables name="A_M50_S" typeName="BOOL"/>
		<variables name="A_M50_S1" typeName="BOOL"/>
		<variables name="A_M50_SnL_IM" typeName="BOOL"/>
		<variables name="A_M50_UK" typeName="BOOL"/>
		<variables name="A_M50_VNM" typeName="BOOL"/>
		<variables name="A_M50_VO" typeName="BOOL"/>
		<variables name="A_M50KM" typeName="BOOL"/>
		<variables name="A_M50MS_K" typeName="EBOOL"/>
		<variables name="A_M50MS_U" typeName="EBOOL"/>
		<variables name="A_M50MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M50MS_U_PR" typeName="BOOL"/>
		<variables name="A_M50MS_UK" typeName="EBOOL"/>
		<variables name="A_M50N" typeName="BOOL"/>
		<variables name="A_M50O_K" typeName="EBOOL"/>
		<variables name="A_M51_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M51_BL_IM" typeName="BOOL"/>
		<variables name="A_M51_GOT" typeName="BOOL"/>
		<variables name="A_M51_iK" typeName="BOOL"/>
		<variables name="A_M51_IND" typeName="BYTE"/>
		<variables name="A_M51_KM" typeName="BOOL"/>
		<variables name="A_M51_KS" typeName="BOOL"/>
		<variables name="A_M51_MOT" typeName="BOOL"/>
		<variables name="A_M51_MV1" typeName="BOOL"/>
		<variables name="A_M51_N" typeName="BOOL"/>
		<variables name="A_M51_OT" typeName="BOOL"/>
		<variables name="A_M51_OT1" typeName="BOOL"/>
		<variables name="A_M51_S" typeName="BOOL"/>
		<variables name="A_M51_S1" typeName="BOOL"/>
		<variables name="A_M51_SnL_IM" typeName="BOOL"/>
		<variables name="A_M51_UK" typeName="BOOL"/>
		<variables name="A_M51_VNM" typeName="BOOL"/>
		<variables name="A_M51_VO" typeName="BOOL"/>
		<variables name="A_M51KM" typeName="BOOL"/>
		<variables name="A_M51MS_K" typeName="EBOOL"/>
		<variables name="A_M51MS_U" typeName="EBOOL"/>
		<variables name="A_M51MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M51MS_U_PR" typeName="BOOL"/>
		<variables name="A_M51MS_UK" typeName="EBOOL"/>
		<variables name="A_M51N" typeName="BOOL"/>
		<variables name="A_M51O_K" typeName="EBOOL"/>
		<variables name="A_M52_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M52_BL_IM" typeName="BOOL"/>
		<variables name="A_M52_GOT" typeName="BOOL"/>
		<variables name="A_M52_iK" typeName="BOOL"/>
		<variables name="A_M52_IND" typeName="BYTE"/>
		<variables name="A_M52_KL_IM" typeName="BOOL"/>
		<variables name="A_M52_KM" typeName="BOOL"/>
		<variables name="A_M52_KS" typeName="BOOL"/>
		<variables name="A_M52_MOT" typeName="BOOL"/>
		<variables name="A_M52_MV1" typeName="BOOL"/>
		<variables name="A_M52_N" typeName="BOOL"/>
		<variables name="A_M52_OT" typeName="BOOL"/>
		<variables name="A_M52_OT1" typeName="BOOL"/>
		<variables name="A_M52_S" typeName="BOOL"/>
		<variables name="A_M52_S1" typeName="BOOL"/>
		<variables name="A_M52_SnL_IM" typeName="BOOL"/>
		<variables name="A_M52_UK" typeName="BOOL"/>
		<variables name="A_M52_VNM" typeName="BOOL"/>
		<variables name="A_M52_VO" typeName="BOOL"/>
		<variables name="A_M52KM" typeName="BOOL"/>
		<variables name="A_M52MS_K" typeName="EBOOL"/>
		<variables name="A_M52MS_U" typeName="EBOOL"/>
		<variables name="A_M52MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M52MS_U_PR" typeName="BOOL"/>
		<variables name="A_M52MS_UK" typeName="EBOOL"/>
		<variables name="A_M52N" typeName="BOOL"/>
		<variables name="A_M52O_K" typeName="EBOOL"/>
		<variables name="A_M53_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M53_BL_IM" typeName="BOOL"/>
		<variables name="A_M53_GOT" typeName="BOOL"/>
		<variables name="A_M53_iK" typeName="BOOL"/>
		<variables name="A_M53_IND" typeName="BYTE"/>
		<variables name="A_M53_KM" typeName="BOOL"/>
		<variables name="A_M53_KS" typeName="BOOL"/>
		<variables name="A_M53_MOT" typeName="BOOL"/>
		<variables name="A_M53_MV1" typeName="BOOL"/>
		<variables name="A_M53_N" typeName="BOOL"/>
		<variables name="A_M53_OT" typeName="BOOL"/>
		<variables name="A_M53_OT1" typeName="BOOL"/>
		<variables name="A_M53_S" typeName="BOOL"/>
		<variables name="A_M53_S1" typeName="BOOL"/>
		<variables name="A_M53_SnL_IM" typeName="BOOL"/>
		<variables name="A_M53_UK" typeName="BOOL"/>
		<variables name="A_M53_VNM" typeName="BOOL"/>
		<variables name="A_M53_VO" typeName="BOOL"/>
		<variables name="A_M53KM" typeName="BOOL"/>
		<variables name="A_M53MS_K" typeName="EBOOL"/>
		<variables name="A_M53MS_U" typeName="EBOOL"/>
		<variables name="A_M53MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M53MS_U_PR" typeName="BOOL"/>
		<variables name="A_M53MS_UK" typeName="EBOOL"/>
		<variables name="A_M53N" typeName="BOOL"/>
		<variables name="A_M53O_K" typeName="EBOOL"/>
		<variables name="A_M54_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M54_BL_IM" typeName="BOOL"/>
		<variables name="A_M54_GOT" typeName="BOOL"/>
		<variables name="A_M54_iK" typeName="BOOL"/>
		<variables name="A_M54_IND" typeName="BYTE"/>
		<variables name="A_M54_KL_IM" typeName="BOOL"/>
		<variables name="A_M54_KM" typeName="BOOL"/>
		<variables name="A_M54_KS" typeName="BOOL"/>
		<variables name="A_M54_MOT" typeName="BOOL"/>
		<variables name="A_M54_MV1" typeName="BOOL"/>
		<variables name="A_M54_N" typeName="BOOL"/>
		<variables name="A_M54_OT" typeName="BOOL"/>
		<variables name="A_M54_OT1" typeName="BOOL"/>
		<variables name="A_M54_S" typeName="BOOL"/>
		<variables name="A_M54_S1" typeName="BOOL"/>
		<variables name="A_M54_SnL_IM" typeName="BOOL"/>
		<variables name="A_M54_UK" typeName="BOOL"/>
		<variables name="A_M54_VNM" typeName="BOOL"/>
		<variables name="A_M54_VO" typeName="BOOL"/>
		<variables name="A_M54KM" typeName="BOOL"/>
		<variables name="A_M54MS_K" typeName="EBOOL"/>
		<variables name="A_M54MS_U" typeName="EBOOL"/>
		<variables name="A_M54MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M54MS_U_PR" typeName="BOOL"/>
		<variables name="A_M54MS_UK" typeName="EBOOL"/>
		<variables name="A_M54N" typeName="BOOL"/>
		<variables name="A_M54O_K" typeName="EBOOL"/>
		<variables name="A_M55_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M55_BL_IM" typeName="BOOL"/>
		<variables name="A_M55_GOT" typeName="BOOL"/>
		<variables name="A_M55_iK" typeName="BOOL"/>
		<variables name="A_M55_IND" typeName="BYTE"/>
		<variables name="A_M55_KM" typeName="BOOL"/>
		<variables name="A_M55_KS" typeName="BOOL"/>
		<variables name="A_M55_MOT" typeName="BOOL"/>
		<variables name="A_M55_MV1" typeName="BOOL"/>
		<variables name="A_M55_N" typeName="BOOL"/>
		<variables name="A_M55_OT" typeName="BOOL"/>
		<variables name="A_M55_OT1" typeName="BOOL"/>
		<variables name="A_M55_S" typeName="BOOL"/>
		<variables name="A_M55_S1" typeName="BOOL"/>
		<variables name="A_M55_SnL_IM" typeName="BOOL"/>
		<variables name="A_M55_UK" typeName="BOOL"/>
		<variables name="A_M55_VCHM" typeName="BOOL"/>
		<variables name="A_M55_VO" typeName="BOOL"/>
		<variables name="A_M55KM" typeName="BOOL"/>
		<variables name="A_M55MS_K" typeName="EBOOL"/>
		<variables name="A_M55MS_U" typeName="EBOOL"/>
		<variables name="A_M55MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M55MS_U_PR" typeName="BOOL"/>
		<variables name="A_M55MS_UK" typeName="EBOOL"/>
		<variables name="A_M55N" typeName="BOOL"/>
		<variables name="A_M55O_K" typeName="EBOOL"/>
		<variables name="A_M56_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M56_BL_IM" typeName="BOOL"/>
		<variables name="A_M56_GOT" typeName="BOOL"/>
		<variables name="A_M56_iK" typeName="BOOL"/>
		<variables name="A_M56_IND" typeName="BYTE"/>
		<variables name="A_M56_KL_IM" typeName="BOOL"/>
		<variables name="A_M56_KM" typeName="BOOL"/>
		<variables name="A_M56_KS" typeName="BOOL"/>
		<variables name="A_M56_MOT" typeName="BOOL"/>
		<variables name="A_M56_MV1" typeName="BOOL"/>
		<variables name="A_M56_N" typeName="BOOL"/>
		<variables name="A_M56_OT" typeName="BOOL"/>
		<variables name="A_M56_OT1" typeName="BOOL"/>
		<variables name="A_M56_S" typeName="BOOL"/>
		<variables name="A_M56_S1" typeName="BOOL"/>
		<variables name="A_M56_SnL_IM" typeName="BOOL"/>
		<variables name="A_M56_UK" typeName="BOOL"/>
		<variables name="A_M56_VCHM" typeName="BOOL"/>
		<variables name="A_M56_VO" typeName="BOOL"/>
		<variables name="A_M56KM" typeName="BOOL"/>
		<variables name="A_M56MS_K" typeName="EBOOL"/>
		<variables name="A_M56MS_U" typeName="EBOOL"/>
		<variables name="A_M56MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M56MS_U_PR" typeName="BOOL"/>
		<variables name="A_M56MS_UK" typeName="EBOOL"/>
		<variables name="A_M56N" typeName="BOOL"/>
		<variables name="A_M56O_K" typeName="EBOOL"/>
		<variables name="A_M57_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M57_BL_IM" typeName="BOOL"/>
		<variables name="A_M57_GOT" typeName="BOOL"/>
		<variables name="A_M57_iK" typeName="BOOL"/>
		<variables name="A_M57_IND" typeName="BYTE"/>
		<variables name="A_M57_KM" typeName="BOOL"/>
		<variables name="A_M57_KS" typeName="BOOL"/>
		<variables name="A_M57_MOT" typeName="BOOL"/>
		<variables name="A_M57_MV1" typeName="BOOL"/>
		<variables name="A_M57_N" typeName="BOOL"/>
		<variables name="A_M57_OT" typeName="BOOL"/>
		<variables name="A_M57_OT1" typeName="BOOL"/>
		<variables name="A_M57_S" typeName="BOOL"/>
		<variables name="A_M57_S1" typeName="BOOL"/>
		<variables name="A_M57_SnL_IM" typeName="BOOL"/>
		<variables name="A_M57_UK" typeName="BOOL"/>
		<variables name="A_M57_VNM" typeName="BOOL"/>
		<variables name="A_M57_VO" typeName="BOOL"/>
		<variables name="A_M57KM" typeName="BOOL"/>
		<variables name="A_M57MS_K" typeName="EBOOL"/>
		<variables name="A_M57MS_U" typeName="EBOOL"/>
		<variables name="A_M57MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M57MS_U_PR" typeName="BOOL"/>
		<variables name="A_M57MS_UK" typeName="EBOOL"/>
		<variables name="A_M57N" typeName="BOOL"/>
		<variables name="A_M57O_K" typeName="EBOOL"/>
		<variables name="A_M58_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M58_BL_IM" typeName="BOOL"/>
		<variables name="A_M58_GOT" typeName="BOOL"/>
		<variables name="A_M58_iK" typeName="BOOL"/>
		<variables name="A_M58_IND" typeName="BYTE"/>
		<variables name="A_M58_KL_IM" typeName="BOOL"/>
		<variables name="A_M58_KM" typeName="BOOL"/>
		<variables name="A_M58_KS" typeName="BOOL"/>
		<variables name="A_M58_MOT" typeName="BOOL"/>
		<variables name="A_M58_MV1" typeName="BOOL"/>
		<variables name="A_M58_N" typeName="BOOL"/>
		<variables name="A_M58_OT" typeName="BOOL"/>
		<variables name="A_M58_OT1" typeName="BOOL"/>
		<variables name="A_M58_S" typeName="BOOL"/>
		<variables name="A_M58_S1" typeName="BOOL"/>
		<variables name="A_M58_SnL_IM" typeName="BOOL"/>
		<variables name="A_M58_UK" typeName="BOOL"/>
		<variables name="A_M58_VCHM" typeName="BOOL"/>
		<variables name="A_M58_VO" typeName="BOOL"/>
		<variables name="A_M58KM" typeName="BOOL"/>
		<variables name="A_M58MS_K" typeName="EBOOL"/>
		<variables name="A_M58MS_U" typeName="EBOOL"/>
		<variables name="A_M58MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M58MS_U_PR" typeName="BOOL"/>
		<variables name="A_M58MS_UK" typeName="EBOOL"/>
		<variables name="A_M58N" typeName="BOOL"/>
		<variables name="A_M58O_K" typeName="EBOOL"/>
		<variables name="A_M59_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M59_BL_IM" typeName="BOOL"/>
		<variables name="A_M59_GOT" typeName="BOOL"/>
		<variables name="A_M59_iK" typeName="BOOL"/>
		<variables name="A_M59_IND" typeName="BYTE"/>
		<variables name="A_M59_KM" typeName="BOOL"/>
		<variables name="A_M59_KS" typeName="BOOL"/>
		<variables name="A_M59_MOT" typeName="BOOL"/>
		<variables name="A_M59_MV1" typeName="BOOL"/>
		<variables name="A_M59_N" typeName="BOOL"/>
		<variables name="A_M59_OT" typeName="BOOL"/>
		<variables name="A_M59_OT1" typeName="BOOL"/>
		<variables name="A_M59_S" typeName="BOOL"/>
		<variables name="A_M59_S1" typeName="BOOL"/>
		<variables name="A_M59_SnL_IM" typeName="BOOL"/>
		<variables name="A_M59_UK" typeName="BOOL"/>
		<variables name="A_M59_VCHM" typeName="BOOL"/>
		<variables name="A_M59_VO" typeName="BOOL"/>
		<variables name="A_M59KM" typeName="BOOL"/>
		<variables name="A_M59MS_K" typeName="EBOOL"/>
		<variables name="A_M59MS_U" typeName="EBOOL"/>
		<variables name="A_M59MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M59MS_U_PR" typeName="BOOL"/>
		<variables name="A_M59MS_UK" typeName="EBOOL"/>
		<variables name="A_M59N" typeName="BOOL"/>
		<variables name="A_M59O_K" typeName="EBOOL"/>
		<variables name="A_M60_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M60_BL_IM" typeName="BOOL"/>
		<variables name="A_M60_GOT" typeName="BOOL"/>
		<variables name="A_M60_iK" typeName="BOOL"/>
		<variables name="A_M60_IND" typeName="BYTE"/>
		<variables name="A_M60_KL_IM" typeName="BOOL"/>
		<variables name="A_M60_KM" typeName="BOOL"/>
		<variables name="A_M60_KS" typeName="BOOL"/>
		<variables name="A_M60_MOT" typeName="BOOL"/>
		<variables name="A_M60_MV1" typeName="BOOL"/>
		<variables name="A_M60_N" typeName="BOOL"/>
		<variables name="A_M60_OT" typeName="BOOL"/>
		<variables name="A_M60_OT1" typeName="BOOL"/>
		<variables name="A_M60_S" typeName="BOOL"/>
		<variables name="A_M60_S1" typeName="BOOL"/>
		<variables name="A_M60_SnL_IM" typeName="BOOL"/>
		<variables name="A_M60_UK" typeName="BOOL"/>
		<variables name="A_M60_VNM" typeName="BOOL"/>
		<variables name="A_M60_VO" typeName="BOOL"/>
		<variables name="A_M60KM" typeName="BOOL"/>
		<variables name="A_M60MS_K" typeName="EBOOL"/>
		<variables name="A_M60MS_U" typeName="EBOOL"/>
		<variables name="A_M60MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M60MS_U_PR" typeName="BOOL"/>
		<variables name="A_M60MS_UK" typeName="EBOOL"/>
		<variables name="A_M60N" typeName="BOOL"/>
		<variables name="A_M60O_K" typeName="EBOOL"/>
		<variables name="A_M61_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M61_BL_IM" typeName="BOOL"/>
		<variables name="A_M61_GOT" typeName="BOOL"/>
		<variables name="A_M61_iK" typeName="BOOL"/>
		<variables name="A_M61_IND" typeName="BYTE"/>
		<variables name="A_M61_KM" typeName="BOOL"/>
		<variables name="A_M61_KS" typeName="BOOL"/>
		<variables name="A_M61_MOT" typeName="BOOL"/>
		<variables name="A_M61_MV1" typeName="BOOL"/>
		<variables name="A_M61_N" typeName="BOOL"/>
		<variables name="A_M61_OT" typeName="BOOL"/>
		<variables name="A_M61_OT1" typeName="BOOL"/>
		<variables name="A_M61_S" typeName="BOOL"/>
		<variables name="A_M61_S1" typeName="BOOL"/>
		<variables name="A_M61_SnL_IM" typeName="BOOL"/>
		<variables name="A_M61_UK" typeName="BOOL"/>
		<variables name="A_M61_VCHM" typeName="BOOL"/>
		<variables name="A_M61_VO" typeName="BOOL"/>
		<variables name="A_M61KM" typeName="BOOL"/>
		<variables name="A_M61MS_K" typeName="EBOOL"/>
		<variables name="A_M61MS_U" typeName="EBOOL"/>
		<variables name="A_M61MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M61MS_U_PR" typeName="BOOL"/>
		<variables name="A_M61MS_UK" typeName="EBOOL"/>
		<variables name="A_M61N" typeName="BOOL"/>
		<variables name="A_M61O_K" typeName="EBOOL"/>
		<variables name="A_M62_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M62_BL_IM" typeName="BOOL"/>
		<variables name="A_M62_GOT" typeName="BOOL"/>
		<variables name="A_M62_iK" typeName="BOOL"/>
		<variables name="A_M62_IND" typeName="BYTE"/>
		<variables name="A_M62_KL_IM" typeName="BOOL"/>
		<variables name="A_M62_KM" typeName="BOOL"/>
		<variables name="A_M62_KS" typeName="BOOL"/>
		<variables name="A_M62_MOT" typeName="BOOL"/>
		<variables name="A_M62_MV1" typeName="BOOL"/>
		<variables name="A_M62_N" typeName="BOOL"/>
		<variables name="A_M62_OT" typeName="BOOL"/>
		<variables name="A_M62_OT1" typeName="BOOL"/>
		<variables name="A_M62_S" typeName="BOOL"/>
		<variables name="A_M62_S1" typeName="BOOL"/>
		<variables name="A_M62_SnL_IM" typeName="BOOL"/>
		<variables name="A_M62_UK" typeName="BOOL"/>
		<variables name="A_M62_VCHM" typeName="BOOL"/>
		<variables name="A_M62_VO" typeName="BOOL"/>
		<variables name="A_M62KM" typeName="BOOL"/>
		<variables name="A_M62MS_K" typeName="EBOOL"/>
		<variables name="A_M62MS_U" typeName="EBOOL"/>
		<variables name="A_M62MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M62MS_U_PR" typeName="BOOL"/>
		<variables name="A_M62MS_UK" typeName="EBOOL"/>
		<variables name="A_M62N" typeName="BOOL"/>
		<variables name="A_M62O_K" typeName="EBOOL"/>
		<variables name="A_M63_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M63_BL_IM" typeName="BOOL"/>
		<variables name="A_M63_GOT" typeName="BOOL"/>
		<variables name="A_M63_iK" typeName="BOOL"/>
		<variables name="A_M63_IND" typeName="BYTE"/>
		<variables name="A_M63_KM" typeName="BOOL"/>
		<variables name="A_M63_KS" typeName="BOOL"/>
		<variables name="A_M63_MOT" typeName="BOOL"/>
		<variables name="A_M63_MV1" typeName="BOOL"/>
		<variables name="A_M63_N" typeName="BOOL"/>
		<variables name="A_M63_OT" typeName="BOOL"/>
		<variables name="A_M63_OT1" typeName="BOOL"/>
		<variables name="A_M63_S" typeName="BOOL"/>
		<variables name="A_M63_S1" typeName="BOOL"/>
		<variables name="A_M63_SnL_IM" typeName="BOOL"/>
		<variables name="A_M63_UK" typeName="BOOL"/>
		<variables name="A_M63_VCHM" typeName="BOOL"/>
		<variables name="A_M63_VNM" typeName="BOOL"/>
		<variables name="A_M63_VO" typeName="BOOL"/>
		<variables name="A_M63MS_K" typeName="EBOOL"/>
		<variables name="A_M63MS_U" typeName="EBOOL"/>
		<variables name="A_M63MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M63MS_U_PR" typeName="BOOL"/>
		<variables name="A_M63MS_UK" typeName="EBOOL"/>
		<variables name="A_M63N" typeName="BOOL"/>
		<variables name="A_M63O_K" typeName="EBOOL"/>
		<variables name="A_M64_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M64_BL_IM" typeName="BOOL"/>
		<variables name="A_M64_GOT" typeName="BOOL"/>
		<variables name="A_M64_iK" typeName="BOOL"/>
		<variables name="A_M64_IND" typeName="BYTE"/>
		<variables name="A_M64_KL_IM" typeName="BOOL"/>
		<variables name="A_M64_KM" typeName="BOOL"/>
		<variables name="A_M64_KS" typeName="BOOL"/>
		<variables name="A_M64_MOT" typeName="BOOL"/>
		<variables name="A_M64_MV1" typeName="BOOL"/>
		<variables name="A_M64_N" typeName="BOOL"/>
		<variables name="A_M64_OT" typeName="BOOL"/>
		<variables name="A_M64_OT1" typeName="BOOL"/>
		<variables name="A_M64_S" typeName="BOOL"/>
		<variables name="A_M64_S1" typeName="BOOL"/>
		<variables name="A_M64_SnL_IM" typeName="BOOL"/>
		<variables name="A_M64_UK" typeName="BOOL"/>
		<variables name="A_M64_VCHM" typeName="BOOL"/>
		<variables name="A_M64_VO" typeName="BOOL"/>
		<variables name="A_M64MS_K" typeName="EBOOL"/>
		<variables name="A_M64MS_U" typeName="EBOOL"/>
		<variables name="A_M64MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M64MS_U_PR" typeName="BOOL"/>
		<variables name="A_M64MS_UK" typeName="EBOOL"/>
		<variables name="A_M64N" typeName="BOOL"/>
		<variables name="A_M64O_K" typeName="EBOOL"/>
		<variables name="A_M65_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M65_BL_IM" typeName="BOOL"/>
		<variables name="A_M65_GOT" typeName="BOOL"/>
		<variables name="A_M65_iK" typeName="BOOL"/>
		<variables name="A_M65_IND" typeName="BYTE"/>
		<variables name="A_M65_KM" typeName="BOOL"/>
		<variables name="A_M65_KS" typeName="BOOL"/>
		<variables name="A_M65_MOT" typeName="BOOL"/>
		<variables name="A_M65_MV1" typeName="BOOL"/>
		<variables name="A_M65_N" typeName="BOOL"/>
		<variables name="A_M65_OT" typeName="BOOL"/>
		<variables name="A_M65_OT1" typeName="BOOL"/>
		<variables name="A_M65_S" typeName="BOOL"/>
		<variables name="A_M65_S1" typeName="BOOL"/>
		<variables name="A_M65_SnL_IM" typeName="BOOL"/>
		<variables name="A_M65_UK" typeName="BOOL"/>
		<variables name="A_M65_VCHM" typeName="BOOL"/>
		<variables name="A_M65_VO" typeName="BOOL"/>
		<variables name="A_M65KM" typeName="BOOL"/>
		<variables name="A_M65MS_K" typeName="EBOOL"/>
		<variables name="A_M65MS_U" typeName="EBOOL"/>
		<variables name="A_M65MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M65MS_U_PR" typeName="BOOL"/>
		<variables name="A_M65MS_UK" typeName="EBOOL"/>
		<variables name="A_M65N" typeName="BOOL"/>
		<variables name="A_M65O_K" typeName="EBOOL"/>
		<variables name="A_M66_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M66_BL_IM" typeName="BOOL"/>
		<variables name="A_M66_GOT" typeName="BOOL"/>
		<variables name="A_M66_iK" typeName="BOOL"/>
		<variables name="A_M66_IND" typeName="BYTE"/>
		<variables name="A_M66_KL_IM" typeName="BOOL"/>
		<variables name="A_M66_KM" typeName="BOOL"/>
		<variables name="A_M66_KS" typeName="BOOL"/>
		<variables name="A_M66_MOT" typeName="BOOL"/>
		<variables name="A_M66_MV1" typeName="BOOL"/>
		<variables name="A_M66_N" typeName="BOOL"/>
		<variables name="A_M66_OT" typeName="BOOL"/>
		<variables name="A_M66_OT1" typeName="BOOL"/>
		<variables name="A_M66_S" typeName="BOOL"/>
		<variables name="A_M66_S1" typeName="BOOL"/>
		<variables name="A_M66_SnL_IM" typeName="BOOL"/>
		<variables name="A_M66_UK" typeName="BOOL"/>
		<variables name="A_M66_VNM" typeName="BOOL"/>
		<variables name="A_M66_VO" typeName="BOOL"/>
		<variables name="A_M66MS_K" typeName="EBOOL"/>
		<variables name="A_M66MS_U" typeName="EBOOL"/>
		<variables name="A_M66MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M66MS_U_PR" typeName="BOOL"/>
		<variables name="A_M66MS_UK" typeName="EBOOL"/>
		<variables name="A_M66N" typeName="BOOL"/>
		<variables name="A_M66O_K" typeName="EBOOL"/>
		<variables name="A_M67_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M67_BL_IM" typeName="BOOL"/>
		<variables name="A_M67_GOT" typeName="BOOL"/>
		<variables name="A_M67_iK" typeName="BOOL"/>
		<variables name="A_M67_IND" typeName="BYTE"/>
		<variables name="A_M67_KM" typeName="BOOL"/>
		<variables name="A_M67_KS" typeName="BOOL"/>
		<variables name="A_M67_MOT" typeName="BOOL"/>
		<variables name="A_M67_MS_K" typeName="EBOOL"/>
		<variables name="A_M67_MV1" typeName="BOOL"/>
		<variables name="A_M67_N" typeName="BOOL"/>
		<variables name="A_M67_OT" typeName="BOOL"/>
		<variables name="A_M67_OT1" typeName="BOOL"/>
		<variables name="A_M67_S" typeName="BOOL"/>
		<variables name="A_M67_S1" typeName="BOOL"/>
		<variables name="A_M67_SnL_IM" typeName="BOOL"/>
		<variables name="A_M67_UK" typeName="BOOL"/>
		<variables name="A_M67_VNM" typeName="BOOL"/>
		<variables name="A_M67_VO" typeName="BOOL"/>
		<variables name="A_M67KM" typeName="BOOL"/>
		<variables name="A_M67MS_K" typeName="EBOOL"/>
		<variables name="A_M67MS_U" typeName="EBOOL"/>
		<variables name="A_M67MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M67MS_U_PR" typeName="BOOL"/>
		<variables name="A_M67MS_UK" typeName="EBOOL"/>
		<variables name="A_M67N" typeName="BOOL"/>
		<variables name="A_M67O_K" typeName="EBOOL"/>
		<variables name="A_M68_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M68_BL_IM" typeName="BOOL"/>
		<variables name="A_M68_GOT" typeName="BOOL"/>
		<variables name="A_M68_iK" typeName="BOOL"/>
		<variables name="A_M68_IND" typeName="BYTE"/>
		<variables name="A_M68_KL_IM" typeName="BOOL"/>
		<variables name="A_M68_KM" typeName="BOOL"/>
		<variables name="A_M68_KS" typeName="BOOL"/>
		<variables name="A_M68_MOT" typeName="BOOL"/>
		<variables name="A_M68_MV1" typeName="BOOL"/>
		<variables name="A_M68_N" typeName="BOOL"/>
		<variables name="A_M68_OT" typeName="BOOL"/>
		<variables name="A_M68_OT1" typeName="BOOL"/>
		<variables name="A_M68_S" typeName="BOOL"/>
		<variables name="A_M68_S1" typeName="BOOL"/>
		<variables name="A_M68_SnL_IM" typeName="BOOL"/>
		<variables name="A_M68_UK" typeName="BOOL"/>
		<variables name="A_M68_VCHM" typeName="BOOL"/>
		<variables name="A_M68_VO" typeName="BOOL"/>
		<variables name="A_M68KM" typeName="BOOL"/>
		<variables name="A_M68MS_K" typeName="EBOOL"/>
		<variables name="A_M68MS_U" typeName="EBOOL"/>
		<variables name="A_M68MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M68MS_U_PR" typeName="BOOL"/>
		<variables name="A_M68MS_UK" typeName="EBOOL"/>
		<variables name="A_M68N" typeName="BOOL"/>
		<variables name="A_M68O_K" typeName="EBOOL"/>
		<variables name="A_M69_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M69_BL_IM" typeName="BOOL"/>
		<variables name="A_M69_GOT" typeName="BOOL"/>
		<variables name="A_M69_iK" typeName="BOOL"/>
		<variables name="A_M69_IND" typeName="BYTE"/>
		<variables name="A_M69_KM" typeName="BOOL"/>
		<variables name="A_M69_KS" typeName="BOOL"/>
		<variables name="A_M69_MOT" typeName="BOOL"/>
		<variables name="A_M69_MV1" typeName="BOOL"/>
		<variables name="A_M69_N" typeName="BOOL"/>
		<variables name="A_M69_OT" typeName="BOOL"/>
		<variables name="A_M69_OT1" typeName="BOOL"/>
		<variables name="A_M69_S" typeName="BOOL"/>
		<variables name="A_M69_S1" typeName="BOOL"/>
		<variables name="A_M69_SnL_IM" typeName="BOOL"/>
		<variables name="A_M69_UK" typeName="BOOL"/>
		<variables name="A_M69_VCHM" typeName="BOOL"/>
		<variables name="A_M69_VO" typeName="BOOL"/>
		<variables name="A_M69KM" typeName="BOOL"/>
		<variables name="A_M69MS_K" typeName="EBOOL"/>
		<variables name="A_M69MS_U" typeName="EBOOL"/>
		<variables name="A_M69MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M69MS_U_PR" typeName="BOOL"/>
		<variables name="A_M69MS_UK" typeName="EBOOL"/>
		<variables name="A_M69N" typeName="BOOL"/>
		<variables name="A_M69O_K" typeName="EBOOL"/>
		<variables name="A_M70_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M70_BL_IM" typeName="BOOL"/>
		<variables name="A_M70_GOT" typeName="BOOL"/>
		<variables name="A_M70_iK" typeName="BOOL"/>
		<variables name="A_M70_IND" typeName="BYTE"/>
		<variables name="A_M70_KL_IM" typeName="BOOL"/>
		<variables name="A_M70_KM" typeName="BOOL"/>
		<variables name="A_M70_KS" typeName="BOOL"/>
		<variables name="A_M70_MOT" typeName="BOOL"/>
		<variables name="A_M70_MV1" typeName="BOOL"/>
		<variables name="A_M70_N" typeName="BOOL"/>
		<variables name="A_M70_OT" typeName="BOOL"/>
		<variables name="A_M70_OT1" typeName="BOOL"/>
		<variables name="A_M70_S" typeName="BOOL"/>
		<variables name="A_M70_S1" typeName="BOOL"/>
		<variables name="A_M70_SnL_IM" typeName="BOOL"/>
		<variables name="A_M70_UK" typeName="BOOL"/>
		<variables name="A_M70_VCHM" typeName="BOOL"/>
		<variables name="A_M70_VO" typeName="BOOL"/>
		<variables name="A_M70KM" typeName="BOOL"/>
		<variables name="A_M70MS_K" typeName="EBOOL"/>
		<variables name="A_M70MS_U" typeName="EBOOL"/>
		<variables name="A_M70MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M70MS_U_PR" typeName="BOOL"/>
		<variables name="A_M70MS_UK" typeName="EBOOL"/>
		<variables name="A_M70N" typeName="BOOL"/>
		<variables name="A_M70O_K" typeName="EBOOL"/>
		<variables name="A_M72_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M72_BL_IM" typeName="BOOL"/>
		<variables name="A_M72_GOT" typeName="BOOL"/>
		<variables name="A_M72_iK" typeName="BOOL"/>
		<variables name="A_M72_IND" typeName="BYTE"/>
		<variables name="A_M72_KL_IM" typeName="BOOL"/>
		<variables name="A_M72_KM" typeName="BOOL"/>
		<variables name="A_M72_KS" typeName="BOOL"/>
		<variables name="A_M72_MOT" typeName="BOOL"/>
		<variables name="A_M72_MV1" typeName="BOOL"/>
		<variables name="A_M72_N" typeName="BOOL"/>
		<variables name="A_M72_OT" typeName="BOOL"/>
		<variables name="A_M72_OT1" typeName="BOOL"/>
		<variables name="A_M72_S" typeName="BOOL"/>
		<variables name="A_M72_S1" typeName="BOOL"/>
		<variables name="A_M72_SnL_IM" typeName="BOOL"/>
		<variables name="A_M72_UK" typeName="BOOL"/>
		<variables name="A_M72_VNM" typeName="BOOL"/>
		<variables name="A_M72_VO" typeName="BOOL"/>
		<variables name="A_M72KM" typeName="BOOL"/>
		<variables name="A_M72MS_K" typeName="EBOOL"/>
		<variables name="A_M72MS_U" typeName="EBOOL"/>
		<variables name="A_M72MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M72MS_U_PR" typeName="BOOL"/>
		<variables name="A_M72MS_UK" typeName="EBOOL"/>
		<variables name="A_M72N" typeName="BOOL"/>
		<variables name="A_M72O_K" typeName="EBOOL"/>
		<variables name="A_M74_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M74_BL_IM" typeName="BOOL"/>
		<variables name="A_M74_GOT" typeName="BOOL"/>
		<variables name="A_M74_iK" typeName="BOOL"/>
		<variables name="A_M74_IND" typeName="BYTE"/>
		<variables name="A_M74_KL_IM" typeName="BOOL"/>
		<variables name="A_M74_KM" typeName="BOOL"/>
		<variables name="A_M74_KS" typeName="BOOL"/>
		<variables name="A_M74_MOT" typeName="BOOL"/>
		<variables name="A_M74_MV1" typeName="BOOL"/>
		<variables name="A_M74_N" typeName="BOOL"/>
		<variables name="A_M74_OT" typeName="BOOL"/>
		<variables name="A_M74_OT1" typeName="BOOL"/>
		<variables name="A_M74_S" typeName="BOOL"/>
		<variables name="A_M74_S1" typeName="BOOL"/>
		<variables name="A_M74_SnL_IM" typeName="BOOL"/>
		<variables name="A_M74_UK" typeName="BOOL"/>
		<variables name="A_M74_VNM" typeName="BOOL"/>
		<variables name="A_M74_VO" typeName="BOOL"/>
		<variables name="A_M74KM" typeName="BOOL"/>
		<variables name="A_M74MS_K" typeName="EBOOL"/>
		<variables name="A_M74MS_U" typeName="EBOOL"/>
		<variables name="A_M74MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M74MS_U_PR" typeName="BOOL"/>
		<variables name="A_M74MS_UK" typeName="EBOOL"/>
		<variables name="A_M74N" typeName="BOOL"/>
		<variables name="A_M74O_K" typeName="EBOOL"/>
		<variables name="A_M76_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M76_BL_IM" typeName="BOOL"/>
		<variables name="A_M76_GOT" typeName="BOOL"/>
		<variables name="A_M76_iK" typeName="BOOL"/>
		<variables name="A_M76_IND" typeName="BYTE"/>
		<variables name="A_M76_KL_IM" typeName="BOOL"/>
		<variables name="A_M76_KM" typeName="BOOL"/>
		<variables name="A_M76_KS" typeName="BOOL"/>
		<variables name="A_M76_MOT" typeName="BOOL"/>
		<variables name="A_M76_MV1" typeName="BOOL"/>
		<variables name="A_M76_N" typeName="BOOL"/>
		<variables name="A_M76_OT" typeName="BOOL"/>
		<variables name="A_M76_OT1" typeName="BOOL"/>
		<variables name="A_M76_S" typeName="BOOL"/>
		<variables name="A_M76_S1" typeName="BOOL"/>
		<variables name="A_M76_SnL_IM" typeName="BOOL"/>
		<variables name="A_M76_UK" typeName="BOOL"/>
		<variables name="A_M76_VNM" typeName="BOOL"/>
		<variables name="A_M76_VO" typeName="BOOL"/>
		<variables name="A_M76KM" typeName="BOOL"/>
		<variables name="A_M76MS_K" typeName="EBOOL"/>
		<variables name="A_M76MS_U" typeName="EBOOL"/>
		<variables name="A_M76MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M76MS_U_PR" typeName="BOOL"/>
		<variables name="A_M76MS_UK" typeName="EBOOL"/>
		<variables name="A_M76N" typeName="BOOL"/>
		<variables name="A_M76O_K" typeName="EBOOL"/>
		<variables name="A_M78_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M78_BL_IM" typeName="BOOL"/>
		<variables name="A_M78_GOT" typeName="BOOL"/>
		<variables name="A_M78_iK" typeName="BOOL"/>
		<variables name="A_M78_IND" typeName="BYTE"/>
		<variables name="A_M78_KL_IM" typeName="BOOL"/>
		<variables name="A_M78_KM" typeName="BOOL"/>
		<variables name="A_M78_KS" typeName="BOOL"/>
		<variables name="A_M78_MOT" typeName="BOOL"/>
		<variables name="A_M78_MV1" typeName="BOOL"/>
		<variables name="A_M78_N" typeName="BOOL"/>
		<variables name="A_M78_OT" typeName="BOOL"/>
		<variables name="A_M78_OT1" typeName="BOOL"/>
		<variables name="A_M78_S" typeName="BOOL"/>
		<variables name="A_M78_S1" typeName="BOOL"/>
		<variables name="A_M78_SnL_IM" typeName="BOOL"/>
		<variables name="A_M78_UK" typeName="BOOL"/>
		<variables name="A_M78_VNM" typeName="BOOL"/>
		<variables name="A_M78_VO" typeName="BOOL"/>
		<variables name="A_M78KM" typeName="BOOL"/>
		<variables name="A_M78MS_K" typeName="EBOOL"/>
		<variables name="A_M78MS_U" typeName="EBOOL"/>
		<variables name="A_M78MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M78MS_U_PR" typeName="BOOL"/>
		<variables name="A_M78MS_UK" typeName="EBOOL"/>
		<variables name="A_M78N" typeName="BOOL"/>
		<variables name="A_M78O_K" typeName="EBOOL"/>
		<variables name="A_M80_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M80_BL_IM" typeName="BOOL"/>
		<variables name="A_M80_GOT" typeName="BOOL"/>
		<variables name="A_M80_iK" typeName="BOOL"/>
		<variables name="A_M80_IND" typeName="BYTE"/>
		<variables name="A_M80_KL_IM" typeName="BOOL"/>
		<variables name="A_M80_KM" typeName="BOOL"/>
		<variables name="A_M80_KS" typeName="BOOL"/>
		<variables name="A_M80_MOT" typeName="BOOL"/>
		<variables name="A_M80_MV1" typeName="BOOL"/>
		<variables name="A_M80_N" typeName="BOOL"/>
		<variables name="A_M80_OT" typeName="BOOL"/>
		<variables name="A_M80_OT1" typeName="BOOL"/>
		<variables name="A_M80_S" typeName="BOOL"/>
		<variables name="A_M80_S1" typeName="BOOL"/>
		<variables name="A_M80_SnL_IM" typeName="BOOL"/>
		<variables name="A_M80_UK" typeName="BOOL"/>
		<variables name="A_M80_VCHM" typeName="BOOL"/>
		<variables name="A_M80_VO" typeName="BOOL"/>
		<variables name="A_M80KM" typeName="BOOL"/>
		<variables name="A_M80MS_K" typeName="EBOOL"/>
		<variables name="A_M80MS_U" typeName="EBOOL"/>
		<variables name="A_M80MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M80MS_U_PR" typeName="BOOL"/>
		<variables name="A_M80MS_UK" typeName="EBOOL"/>
		<variables name="A_M80N" typeName="BOOL"/>
		<variables name="A_M80O_K" typeName="EBOOL"/>
		<variables name="A_M82_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M82_BL_IM" typeName="BOOL"/>
		<variables name="A_M82_GOT" typeName="BOOL"/>
		<variables name="A_M82_iK" typeName="BOOL"/>
		<variables name="A_M82_IND" typeName="BYTE"/>
		<variables name="A_M82_KL_IM" typeName="BOOL"/>
		<variables name="A_M82_KM" typeName="BOOL"/>
		<variables name="A_M82_KS" typeName="BOOL"/>
		<variables name="A_M82_MOT" typeName="BOOL"/>
		<variables name="A_M82_MV1" typeName="BOOL"/>
		<variables name="A_M82_N" typeName="BOOL"/>
		<variables name="A_M82_OT" typeName="BOOL"/>
		<variables name="A_M82_OT1" typeName="BOOL"/>
		<variables name="A_M82_S" typeName="BOOL"/>
		<variables name="A_M82_S1" typeName="BOOL"/>
		<variables name="A_M82_SnL_IM" typeName="BOOL"/>
		<variables name="A_M82_UK" typeName="BOOL"/>
		<variables name="A_M82_VCHM" typeName="BOOL"/>
		<variables name="A_M82_VO" typeName="BOOL"/>
		<variables name="A_M82KM" typeName="BOOL"/>
		<variables name="A_M82MS_K" typeName="EBOOL"/>
		<variables name="A_M82MS_U" typeName="EBOOL"/>
		<variables name="A_M82MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M82MS_U_PR" typeName="BOOL"/>
		<variables name="A_M82MS_UK" typeName="EBOOL"/>
		<variables name="A_M82N" typeName="BOOL"/>
		<variables name="A_M82O_K" typeName="EBOOL"/>
		<variables name="A_M84_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M84_BL_IM" typeName="BOOL"/>
		<variables name="A_M84_GOT" typeName="BOOL"/>
		<variables name="A_M84_iK" typeName="BOOL"/>
		<variables name="A_M84_IND" typeName="BYTE"/>
		<variables name="A_M84_KL_IM" typeName="BOOL"/>
		<variables name="A_M84_KM" typeName="BOOL"/>
		<variables name="A_M84_KS" typeName="BOOL"/>
		<variables name="A_M84_MOT" typeName="BOOL"/>
		<variables name="A_M84_MV1" typeName="BOOL"/>
		<variables name="A_M84_N" typeName="BOOL"/>
		<variables name="A_M84_OT" typeName="BOOL"/>
		<variables name="A_M84_OT1" typeName="BOOL"/>
		<variables name="A_M84_S" typeName="BOOL"/>
		<variables name="A_M84_S1" typeName="BOOL"/>
		<variables name="A_M84_SnL_IM" typeName="BOOL"/>
		<variables name="A_M84_UK" typeName="BOOL"/>
		<variables name="A_M84_VNM" typeName="BOOL"/>
		<variables name="A_M84_VO" typeName="BOOL"/>
		<variables name="A_M84KM" typeName="BOOL"/>
		<variables name="A_M84MS_K" typeName="EBOOL"/>
		<variables name="A_M84MS_U" typeName="EBOOL"/>
		<variables name="A_M84MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M84MS_U_PR" typeName="BOOL"/>
		<variables name="A_M84MS_UK" typeName="EBOOL"/>
		<variables name="A_M84N" typeName="BOOL"/>
		<variables name="A_M84O_K" typeName="EBOOL"/>
		<variables name="A_M97_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M97_BL_IM" typeName="BOOL"/>
		<variables name="A_M97_GOT" typeName="BOOL"/>
		<variables name="A_M97_iK" typeName="BOOL"/>
		<variables name="A_M97_IND" typeName="BYTE"/>
		<variables name="A_M97_KM" typeName="BOOL"/>
		<variables name="A_M97_KS" typeName="BOOL"/>
		<variables name="A_M97_MOT" typeName="BOOL"/>
		<variables name="A_M97_MV1" typeName="BOOL"/>
		<variables name="A_M97_N" typeName="BOOL"/>
		<variables name="A_M97_OT" typeName="BOOL"/>
		<variables name="A_M97_OT1" typeName="BOOL"/>
		<variables name="A_M97_S" typeName="BOOL"/>
		<variables name="A_M97_S1" typeName="BOOL"/>
		<variables name="A_M97_SnL_IM" typeName="BOOL"/>
		<variables name="A_M97_UK" typeName="BOOL"/>
		<variables name="A_M97_VCHM" typeName="BOOL"/>
		<variables name="A_M97_VO" typeName="BOOL"/>
		<variables name="A_M97KM" typeName="BOOL"/>
		<variables name="A_M97MS_K" typeName="EBOOL"/>
		<variables name="A_M97MS_U" typeName="EBOOL"/>
		<variables name="A_M97MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M97MS_U_PR" typeName="BOOL"/>
		<variables name="A_M97MS_UK" typeName="EBOOL"/>
		<variables name="A_M97N" typeName="BOOL"/>
		<variables name="A_M97O_K" typeName="EBOOL"/>
		<variables name="A_M98_BL_IM" typeName="BOOL"/>
		<variables name="A_M98_KL_IM" typeName="BOOL"/>
		<variables name="A_M98_SnL_IM" typeName="BOOL"/>
		<variables name="A_M98MS_K" typeName="BOOL"/>
		<variables name="A_M98MS_U" typeName="EBOOL"/>
		<variables name="A_M98O_K" typeName="EBOOL"/>
		<variables name="A_M100_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M100_BL_IM" typeName="BOOL"/>
		<variables name="A_M100_GOT" typeName="BOOL"/>
		<variables name="A_M100_iK" typeName="BOOL"/>
		<variables name="A_M100_IND" typeName="BYTE"/>
		<variables name="A_M100_KL_IM" typeName="BOOL"/>
		<variables name="A_M100_KM" typeName="BOOL"/>
		<variables name="A_M100_KS" typeName="BOOL"/>
		<variables name="A_M100_MOT" typeName="BOOL"/>
		<variables name="A_M100_MV1" typeName="BOOL"/>
		<variables name="A_M100_N" typeName="BOOL"/>
		<variables name="A_M100_OT" typeName="BOOL"/>
		<variables name="A_M100_OT1" typeName="BOOL"/>
		<variables name="A_M100_S" typeName="BOOL"/>
		<variables name="A_M100_S1" typeName="BOOL"/>
		<variables name="A_M100_SnL_IM" typeName="BOOL"/>
		<variables name="A_M100_UK" typeName="BOOL"/>
		<variables name="A_M100_VNM" typeName="BOOL"/>
		<variables name="A_M100_VO" typeName="BOOL"/>
		<variables name="A_M100KM" typeName="BOOL"/>
		<variables name="A_M100MS_K" typeName="EBOOL"/>
		<variables name="A_M100MS_U" typeName="EBOOL"/>
		<variables name="A_M100MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M100MS_U_PR" typeName="BOOL"/>
		<variables name="A_M100MS_UK" typeName="EBOOL"/>
		<variables name="A_M100N" typeName="BOOL"/>
		<variables name="A_M100O_K" typeName="EBOOL"/>
		<variables name="A_M101_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M101_BL_IM" typeName="BOOL"/>
		<variables name="A_M101_GOT" typeName="BOOL"/>
		<variables name="A_M101_iK" typeName="BOOL"/>
		<variables name="A_M101_IND" typeName="BYTE"/>
		<variables name="A_M101_KM" typeName="BOOL"/>
		<variables name="A_M101_KS" typeName="BOOL"/>
		<variables name="A_M101_MOT" typeName="BOOL"/>
		<variables name="A_M101_MV1" typeName="BOOL"/>
		<variables name="A_M101_N" typeName="BOOL"/>
		<variables name="A_M101_OT" typeName="BOOL"/>
		<variables name="A_M101_OT1" typeName="BOOL"/>
		<variables name="A_M101_S" typeName="BOOL"/>
		<variables name="A_M101_S1" typeName="BOOL"/>
		<variables name="A_M101_SnL_IM" typeName="BOOL"/>
		<variables name="A_M101_UK" typeName="BOOL"/>
		<variables name="A_M101_VCHM" typeName="BOOL"/>
		<variables name="A_M101_VO" typeName="BOOL"/>
		<variables name="A_M101IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M101KM" typeName="BOOL"/>
		<variables name="A_M101KS" typeName="BOOL"/>
		<variables name="A_M101MS_K" typeName="EBOOL"/>
		<variables name="A_M101MS_U" typeName="EBOOL"/>
		<variables name="A_M101MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M101MS_U_PR" typeName="BOOL"/>
		<variables name="A_M101MS_UK" typeName="EBOOL"/>
		<variables name="A_M101N" typeName="BOOL"/>
		<variables name="A_M101O_K" typeName="EBOOL"/>
		<variables name="A_M101P_IND" typeName="BYTE"/>
		<variables name="A_M101P_K" typeName="EBOOL"/>
		<variables name="A_M101RI" typeName="BOOL"/>
		<variables name="A_M101Z" typeName="BOOL"/>
		<variables name="A_M102_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M102_BL_IM" typeName="BOOL"/>
		<variables name="A_M102_GOT" typeName="BOOL"/>
		<variables name="A_M102_iK" typeName="BOOL"/>
		<variables name="A_M102_IND" typeName="BYTE"/>
		<variables name="A_M102_KL_IM" typeName="BOOL"/>
		<variables name="A_M102_KM" typeName="BOOL"/>
		<variables name="A_M102_KS" typeName="BOOL"/>
		<variables name="A_M102_MOT" typeName="BOOL"/>
		<variables name="A_M102_MV1" typeName="BOOL"/>
		<variables name="A_M102_N" typeName="BOOL"/>
		<variables name="A_M102_OT" typeName="BOOL"/>
		<variables name="A_M102_OT1" typeName="BOOL"/>
		<variables name="A_M102_S" typeName="BOOL"/>
		<variables name="A_M102_S1" typeName="BOOL"/>
		<variables name="A_M102_SnL_IM" typeName="BOOL"/>
		<variables name="A_M102_VCHM" typeName="BOOL"/>
		<variables name="A_M102_VNM" typeName="BOOL"/>
		<variables name="A_M102_VO" typeName="BOOL"/>
		<variables name="A_M102MS_K" typeName="EBOOL"/>
		<variables name="A_M102MS_U" typeName="EBOOL"/>
		<variables name="A_M102MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M102MS_U_PR" typeName="BOOL"/>
		<variables name="A_M102MS_UK" typeName="EBOOL"/>
		<variables name="A_M102N" typeName="BOOL"/>
		<variables name="A_M102O_K" typeName="EBOOL"/>
		<variables name="A_M103_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M103_BL_IM" typeName="BOOL"/>
		<variables name="A_M103_GOT" typeName="BOOL"/>
		<variables name="A_M103_iK" typeName="BOOL"/>
		<variables name="A_M103_IND" typeName="BYTE"/>
		<variables name="A_M103_KM" typeName="BOOL"/>
		<variables name="A_M103_KS" typeName="BOOL"/>
		<variables name="A_M103_MOT" typeName="BOOL"/>
		<variables name="A_M103_MV1" typeName="BOOL"/>
		<variables name="A_M103_N" typeName="BOOL"/>
		<variables name="A_M103_OT" typeName="BOOL"/>
		<variables name="A_M103_OT1" typeName="BOOL"/>
		<variables name="A_M103_S" typeName="BOOL"/>
		<variables name="A_M103_S1" typeName="BOOL"/>
		<variables name="A_M103_SnL_IM" typeName="BOOL"/>
		<variables name="A_M103_UK" typeName="BOOL"/>
		<variables name="A_M103_VCHM" typeName="BOOL"/>
		<variables name="A_M103_VO" typeName="BOOL"/>
		<variables name="A_M103KM" typeName="BOOL"/>
		<variables name="A_M103MS_K" typeName="EBOOL"/>
		<variables name="A_M103MS_U" typeName="EBOOL"/>
		<variables name="A_M103MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M103MS_U_PR" typeName="BOOL"/>
		<variables name="A_M103MS_UK" typeName="EBOOL"/>
		<variables name="A_M103N" typeName="BOOL"/>
		<variables name="A_M103O_K" typeName="EBOOL"/>
		<variables name="A_M104_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M104_BL_IM" typeName="BOOL"/>
		<variables name="A_M104_GOT" typeName="BOOL"/>
		<variables name="A_M104_iK" typeName="BOOL"/>
		<variables name="A_M104_IND" typeName="BYTE"/>
		<variables name="A_M104_KL_IM" typeName="BOOL"/>
		<variables name="A_M104_KM" typeName="BOOL"/>
		<variables name="A_M104_KS" typeName="BOOL"/>
		<variables name="A_M104_MOT" typeName="BOOL"/>
		<variables name="A_M104_MV1" typeName="BOOL"/>
		<variables name="A_M104_N" typeName="BOOL"/>
		<variables name="A_M104_OT" typeName="BOOL"/>
		<variables name="A_M104_OT1" typeName="BOOL"/>
		<variables name="A_M104_S" typeName="BOOL"/>
		<variables name="A_M104_S1" typeName="BOOL"/>
		<variables name="A_M104_SnL_IM" typeName="BOOL"/>
		<variables name="A_M104_UK" typeName="BOOL"/>
		<variables name="A_M104_VCHM" typeName="BOOL"/>
		<variables name="A_M104_VO" typeName="BOOL"/>
		<variables name="A_M104KM" typeName="BOOL"/>
		<variables name="A_M104MS_K" typeName="EBOOL"/>
		<variables name="A_M104MS_U" typeName="EBOOL"/>
		<variables name="A_M104MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M104MS_U_PR" typeName="BOOL"/>
		<variables name="A_M104MS_UK" typeName="EBOOL"/>
		<variables name="A_M104N" typeName="BOOL"/>
		<variables name="A_M104O_K" typeName="EBOOL"/>
		<variables name="A_M105_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M105_BL_IM" typeName="BOOL"/>
		<variables name="A_M105_GOT" typeName="BOOL"/>
		<variables name="A_M105_iK" typeName="BOOL"/>
		<variables name="A_M105_IND" typeName="BYTE"/>
		<variables name="A_M105_KM" typeName="BOOL"/>
		<variables name="A_M105_KS" typeName="BOOL"/>
		<variables name="A_M105_MOT" typeName="BOOL"/>
		<variables name="A_M105_MV1" typeName="BOOL"/>
		<variables name="A_M105_N" typeName="BOOL"/>
		<variables name="A_M105_OT" typeName="BOOL"/>
		<variables name="A_M105_OT1" typeName="BOOL"/>
		<variables name="A_M105_S" typeName="BOOL"/>
		<variables name="A_M105_S1" typeName="BOOL"/>
		<variables name="A_M105_SnL_IM" typeName="BOOL"/>
		<variables name="A_M105_UK" typeName="BOOL"/>
		<variables name="A_M105_VNM" typeName="BOOL"/>
		<variables name="A_M105_VO" typeName="BOOL"/>
		<variables name="A_M105KM" typeName="BOOL"/>
		<variables name="A_M105MS_K" typeName="EBOOL"/>
		<variables name="A_M105MS_U" typeName="EBOOL"/>
		<variables name="A_M105MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M105MS_U_PR" typeName="BOOL"/>
		<variables name="A_M105MS_UK" typeName="EBOOL"/>
		<variables name="A_M105N" typeName="BOOL"/>
		<variables name="A_M105O_K" typeName="EBOOL"/>
		<variables name="A_M106_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M106_BL_IM" typeName="BOOL"/>
		<variables name="A_M106_GOT" typeName="BOOL"/>
		<variables name="A_M106_iK" typeName="BOOL"/>
		<variables name="A_M106_IND" typeName="BYTE"/>
		<variables name="A_M106_KL_IM" typeName="BOOL"/>
		<variables name="A_M106_KM" typeName="BOOL"/>
		<variables name="A_M106_KS" typeName="BOOL"/>
		<variables name="A_M106_MOT" typeName="BOOL"/>
		<variables name="A_M106_MV1" typeName="BOOL"/>
		<variables name="A_M106_N" typeName="BOOL"/>
		<variables name="A_M106_OT" typeName="BOOL"/>
		<variables name="A_M106_OT1" typeName="BOOL"/>
		<variables name="A_M106_S" typeName="BOOL"/>
		<variables name="A_M106_S1" typeName="BOOL"/>
		<variables name="A_M106_SnL_IM" typeName="BOOL"/>
		<variables name="A_M106_UK" typeName="BOOL"/>
		<variables name="A_M106_VNM" typeName="BOOL"/>
		<variables name="A_M106_VO" typeName="BOOL"/>
		<variables name="A_M106MS_K" typeName="EBOOL"/>
		<variables name="A_M106MS_U" typeName="EBOOL"/>
		<variables name="A_M106MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M106MS_U_PR" typeName="BOOL"/>
		<variables name="A_M106MS_UK" typeName="EBOOL"/>
		<variables name="A_M106N" typeName="BOOL"/>
		<variables name="A_M106O_K" typeName="EBOOL"/>
		<variables name="A_M107_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M107_BL_IM" typeName="BOOL"/>
		<variables name="A_M107_GOT" typeName="BOOL"/>
		<variables name="A_M107_iK" typeName="BOOL"/>
		<variables name="A_M107_IND" typeName="BYTE"/>
		<variables name="A_M107_KM" typeName="BOOL"/>
		<variables name="A_M107_KS" typeName="BOOL"/>
		<variables name="A_M107_MOT" typeName="BOOL"/>
		<variables name="A_M107_MV1" typeName="BOOL"/>
		<variables name="A_M107_N" typeName="BOOL"/>
		<variables name="A_M107_OT" typeName="BOOL"/>
		<variables name="A_M107_OT1" typeName="BOOL"/>
		<variables name="A_M107_S" typeName="BOOL"/>
		<variables name="A_M107_S1" typeName="BOOL"/>
		<variables name="A_M107_SnL_IM" typeName="BOOL"/>
		<variables name="A_M107_UK" typeName="BOOL"/>
		<variables name="A_M107_VCHM" typeName="BOOL"/>
		<variables name="A_M107_VO" typeName="BOOL"/>
		<variables name="A_M107KM" typeName="BOOL"/>
		<variables name="A_M107MS_K" typeName="EBOOL"/>
		<variables name="A_M107MS_U" typeName="EBOOL"/>
		<variables name="A_M107MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M107MS_U_PR" typeName="BOOL"/>
		<variables name="A_M107MS_UK" typeName="EBOOL"/>
		<variables name="A_M107N" typeName="BOOL"/>
		<variables name="A_M107O_K" typeName="EBOOL"/>
		<variables name="A_M108_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M108_BL_IM" typeName="BOOL"/>
		<variables name="A_M108_GOT" typeName="BOOL"/>
		<variables name="A_M108_iK" typeName="BOOL"/>
		<variables name="A_M108_IND" typeName="BYTE"/>
		<variables name="A_M108_KL_IM" typeName="BOOL"/>
		<variables name="A_M108_KM" typeName="BOOL"/>
		<variables name="A_M108_KS" typeName="BOOL"/>
		<variables name="A_M108_MOT" typeName="BOOL"/>
		<variables name="A_M108_MV1" typeName="BOOL"/>
		<variables name="A_M108_N" typeName="BOOL"/>
		<variables name="A_M108_OT" typeName="BOOL"/>
		<variables name="A_M108_OT1" typeName="BOOL"/>
		<variables name="A_M108_S" typeName="BOOL"/>
		<variables name="A_M108_S1" typeName="BOOL"/>
		<variables name="A_M108_SnL_IM" typeName="BOOL"/>
		<variables name="A_M108_UK" typeName="BOOL"/>
		<variables name="A_M108_VNM" typeName="BOOL"/>
		<variables name="A_M108_VO" typeName="BOOL"/>
		<variables name="A_M108MS_K" typeName="EBOOL"/>
		<variables name="A_M108MS_U" typeName="EBOOL"/>
		<variables name="A_M108MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M108MS_U_PR" typeName="BOOL"/>
		<variables name="A_M108MS_UK" typeName="EBOOL"/>
		<variables name="A_M108N" typeName="BOOL"/>
		<variables name="A_M108O_K" typeName="EBOOL"/>
		<variables name="A_M109_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M109_BL_IM" typeName="BOOL"/>
		<variables name="A_M109_GOT" typeName="BOOL"/>
		<variables name="A_M109_iK" typeName="BOOL"/>
		<variables name="A_M109_IND" typeName="BYTE"/>
		<variables name="A_M109_KM" typeName="BOOL"/>
		<variables name="A_M109_KS" typeName="BOOL"/>
		<variables name="A_M109_MOT" typeName="BOOL"/>
		<variables name="A_M109_MV1" typeName="BOOL"/>
		<variables name="A_M109_N" typeName="BOOL"/>
		<variables name="A_M109_OT" typeName="BOOL"/>
		<variables name="A_M109_OT1" typeName="BOOL"/>
		<variables name="A_M109_S" typeName="BOOL"/>
		<variables name="A_M109_S1" typeName="BOOL"/>
		<variables name="A_M109_SnL_IM" typeName="BOOL"/>
		<variables name="A_M109_UK" typeName="BOOL"/>
		<variables name="A_M109_VNM" typeName="BOOL"/>
		<variables name="A_M109_VO" typeName="BOOL"/>
		<variables name="A_M109KM" typeName="BOOL"/>
		<variables name="A_M109MS_K" typeName="EBOOL"/>
		<variables name="A_M109MS_U" typeName="EBOOL"/>
		<variables name="A_M109MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M109MS_U_PR" typeName="BOOL"/>
		<variables name="A_M109MS_UK" typeName="EBOOL"/>
		<variables name="A_M109N" typeName="BOOL"/>
		<variables name="A_M109O_K" typeName="EBOOL"/>
		<variables name="A_M110_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M110_BL_IM" typeName="BOOL"/>
		<variables name="A_M110_GOT" typeName="BOOL"/>
		<variables name="A_M110_iK" typeName="BOOL"/>
		<variables name="A_M110_IND" typeName="BYTE"/>
		<variables name="A_M110_KL_IM" typeName="BOOL"/>
		<variables name="A_M110_KM" typeName="BOOL"/>
		<variables name="A_M110_KS" typeName="BOOL"/>
		<variables name="A_M110_MOT" typeName="BOOL"/>
		<variables name="A_M110_MV1" typeName="BOOL"/>
		<variables name="A_M110_N" typeName="BOOL"/>
		<variables name="A_M110_OT" typeName="BOOL"/>
		<variables name="A_M110_OT1" typeName="BOOL"/>
		<variables name="A_M110_S" typeName="BOOL"/>
		<variables name="A_M110_S1" typeName="BOOL"/>
		<variables name="A_M110_SnL_IM" typeName="BOOL"/>
		<variables name="A_M110_UK" typeName="BOOL"/>
		<variables name="A_M110_VNM" typeName="BOOL"/>
		<variables name="A_M110_VO" typeName="BOOL"/>
		<variables name="A_M110MS_K" typeName="EBOOL"/>
		<variables name="A_M110MS_U" typeName="EBOOL"/>
		<variables name="A_M110MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M110MS_U_PR" typeName="BOOL"/>
		<variables name="A_M110MS_UK" typeName="EBOOL"/>
		<variables name="A_M110N" typeName="BOOL"/>
		<variables name="A_M110O_K" typeName="EBOOL"/>
		<variables name="A_M112_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M112_BL_IM" typeName="BOOL"/>
		<variables name="A_M112_GOT" typeName="BOOL"/>
		<variables name="A_M112_iK" typeName="BOOL"/>
		<variables name="A_M112_IND" typeName="BYTE"/>
		<variables name="A_M112_KL_IM" typeName="BOOL"/>
		<variables name="A_M112_KM" typeName="BOOL"/>
		<variables name="A_M112_KS" typeName="BOOL"/>
		<variables name="A_M112_MOT" typeName="BOOL"/>
		<variables name="A_M112_MV1" typeName="BOOL"/>
		<variables name="A_M112_N" typeName="BOOL"/>
		<variables name="A_M112_OT" typeName="BOOL"/>
		<variables name="A_M112_OT1" typeName="BOOL"/>
		<variables name="A_M112_S" typeName="BOOL"/>
		<variables name="A_M112_S1" typeName="BOOL"/>
		<variables name="A_M112_SnL_IM" typeName="BOOL"/>
		<variables name="A_M112_UK" typeName="BOOL"/>
		<variables name="A_M112_VNM" typeName="BOOL"/>
		<variables name="A_M112_VO" typeName="BOOL"/>
		<variables name="A_M112MS_K" typeName="EBOOL"/>
		<variables name="A_M112MS_U" typeName="EBOOL"/>
		<variables name="A_M112MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M112MS_U_PR" typeName="BOOL"/>
		<variables name="A_M112MS_UK" typeName="EBOOL"/>
		<variables name="A_M112N" typeName="BOOL"/>
		<variables name="A_M112O_K" typeName="EBOOL"/>
		<variables name="A_M114_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M114_BL_IM" typeName="BOOL"/>
		<variables name="A_M114_GOT" typeName="BOOL"/>
		<variables name="A_M114_iK" typeName="BOOL"/>
		<variables name="A_M114_IND" typeName="BYTE"/>
		<variables name="A_M114_KL_IM" typeName="BOOL"/>
		<variables name="A_M114_KM" typeName="BOOL"/>
		<variables name="A_M114_KS" typeName="BOOL"/>
		<variables name="A_M114_MOT" typeName="BOOL"/>
		<variables name="A_M114_MV1" typeName="BOOL"/>
		<variables name="A_M114_N" typeName="BOOL"/>
		<variables name="A_M114_OT" typeName="BOOL"/>
		<variables name="A_M114_OT1" typeName="BOOL"/>
		<variables name="A_M114_S" typeName="BOOL"/>
		<variables name="A_M114_S1" typeName="BOOL"/>
		<variables name="A_M114_SnL_IM" typeName="BOOL"/>
		<variables name="A_M114_UK" typeName="BOOL"/>
		<variables name="A_M114_VNM" typeName="BOOL"/>
		<variables name="A_M114_VO" typeName="BOOL"/>
		<variables name="A_M114MS_K" typeName="EBOOL"/>
		<variables name="A_M114MS_U" typeName="EBOOL"/>
		<variables name="A_M114MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M114MS_U_PR" typeName="BOOL"/>
		<variables name="A_M114MS_UK" typeName="EBOOL"/>
		<variables name="A_M114N" typeName="BOOL"/>
		<variables name="A_M114O_K" typeName="EBOOL"/>
		<variables name="A_M116_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M116_BL_IM" typeName="BOOL"/>
		<variables name="A_M116_GOT" typeName="BOOL"/>
		<variables name="A_M116_iK" typeName="BOOL"/>
		<variables name="A_M116_IND" typeName="BYTE"/>
		<variables name="A_M116_KL_IM" typeName="BOOL"/>
		<variables name="A_M116_KM" typeName="BOOL"/>
		<variables name="A_M116_KS" typeName="BOOL"/>
		<variables name="A_M116_MOT" typeName="BOOL"/>
		<variables name="A_M116_MV1" typeName="BOOL"/>
		<variables name="A_M116_N" typeName="BOOL"/>
		<variables name="A_M116_OT" typeName="BOOL"/>
		<variables name="A_M116_OT1" typeName="BOOL"/>
		<variables name="A_M116_S" typeName="BOOL"/>
		<variables name="A_M116_S1" typeName="BOOL"/>
		<variables name="A_M116_SnL_IM" typeName="BOOL"/>
		<variables name="A_M116_UK" typeName="BOOL"/>
		<variables name="A_M116_VNM" typeName="BOOL"/>
		<variables name="A_M116_VO" typeName="BOOL"/>
		<variables name="A_M116MS_K" typeName="EBOOL"/>
		<variables name="A_M116MS_U" typeName="EBOOL"/>
		<variables name="A_M116MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M116MS_U_PR" typeName="BOOL"/>
		<variables name="A_M116MS_UK" typeName="EBOOL"/>
		<variables name="A_M116N" typeName="BOOL"/>
		<variables name="A_M116O_K" typeName="EBOOL"/>
		<variables name="A_M118_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M118_BL_IM" typeName="BOOL"/>
		<variables name="A_M118_GOT" typeName="BOOL"/>
		<variables name="A_M118_iK" typeName="BOOL"/>
		<variables name="A_M118_IND" typeName="BYTE"/>
		<variables name="A_M118_KL_IM" typeName="BOOL"/>
		<variables name="A_M118_KM" typeName="BOOL"/>
		<variables name="A_M118_KS" typeName="BOOL"/>
		<variables name="A_M118_MOT" typeName="BOOL"/>
		<variables name="A_M118_MV1" typeName="BOOL"/>
		<variables name="A_M118_N" typeName="BOOL"/>
		<variables name="A_M118_OT" typeName="BOOL"/>
		<variables name="A_M118_OT1" typeName="BOOL"/>
		<variables name="A_M118_S" typeName="BOOL"/>
		<variables name="A_M118_S1" typeName="BOOL"/>
		<variables name="A_M118_SnL_IM" typeName="BOOL"/>
		<variables name="A_M118_UK" typeName="BOOL"/>
		<variables name="A_M118_VNM" typeName="BOOL"/>
		<variables name="A_M118_VO" typeName="BOOL"/>
		<variables name="A_M118MS_K" typeName="EBOOL"/>
		<variables name="A_M118MS_U" typeName="EBOOL"/>
		<variables name="A_M118MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M118MS_U_PR" typeName="BOOL"/>
		<variables name="A_M118MS_UK" typeName="EBOOL"/>
		<variables name="A_M118N" typeName="BOOL"/>
		<variables name="A_M118O_K" typeName="EBOOL"/>
		<variables name="A_M120_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M120_BL_IM" typeName="BOOL"/>
		<variables name="A_M120_GOT" typeName="BOOL"/>
		<variables name="A_M120_iK" typeName="BOOL"/>
		<variables name="A_M120_IND" typeName="BYTE"/>
		<variables name="A_M120_KL_IM" typeName="BOOL"/>
		<variables name="A_M120_KM" typeName="BOOL"/>
		<variables name="A_M120_KS" typeName="BOOL"/>
		<variables name="A_M120_MOT" typeName="BOOL"/>
		<variables name="A_M120_MV1" typeName="BOOL"/>
		<variables name="A_M120_N" typeName="BOOL"/>
		<variables name="A_M120_OT" typeName="BOOL"/>
		<variables name="A_M120_OT1" typeName="BOOL"/>
		<variables name="A_M120_S" typeName="BOOL"/>
		<variables name="A_M120_S1" typeName="BOOL"/>
		<variables name="A_M120_SnL_IM" typeName="BOOL"/>
		<variables name="A_M120_UK" typeName="BOOL"/>
		<variables name="A_M120_VO" typeName="BOOL"/>
		<variables name="A_M120MS_K" typeName="BOOL"/>
		<variables name="A_M120MS_U" typeName="EBOOL"/>
		<variables name="A_M120MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M120MS_U_PR" typeName="BOOL"/>
		<variables name="A_M120MS_UK" typeName="EBOOL"/>
		<variables name="A_M120N" typeName="BOOL"/>
		<variables name="A_M120O_K" typeName="EBOOL"/>
		<variables name="A_M122_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M122_BL_IM" typeName="BOOL"/>
		<variables name="A_M122_GOT" typeName="BOOL"/>
		<variables name="A_M122_iK" typeName="BOOL"/>
		<variables name="A_M122_IND" typeName="BYTE"/>
		<variables name="A_M122_KL_IM" typeName="BOOL"/>
		<variables name="A_M122_KM" typeName="BOOL"/>
		<variables name="A_M122_KS" typeName="BOOL"/>
		<variables name="A_M122_MOT" typeName="BOOL"/>
		<variables name="A_M122_MV1" typeName="BOOL"/>
		<variables name="A_M122_N" typeName="BOOL"/>
		<variables name="A_M122_OT" typeName="BOOL"/>
		<variables name="A_M122_OT1" typeName="BOOL"/>
		<variables name="A_M122_S" typeName="BOOL"/>
		<variables name="A_M122_S1" typeName="BOOL"/>
		<variables name="A_M122_SnL_IM" typeName="BOOL"/>
		<variables name="A_M122_UK" typeName="BOOL"/>
		<variables name="A_M122_VCHM" typeName="BOOL"/>
		<variables name="A_M122_VO" typeName="BOOL"/>
		<variables name="A_M122KM" typeName="BOOL"/>
		<variables name="A_M122MS_K" typeName="EBOOL"/>
		<variables name="A_M122MS_U" typeName="EBOOL"/>
		<variables name="A_M122MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M122MS_U_PR" typeName="BOOL"/>
		<variables name="A_M122MS_UK" typeName="EBOOL"/>
		<variables name="A_M122N" typeName="BOOL"/>
		<variables name="A_M122O_K" typeName="EBOOL"/>
		<variables name="A_M124_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M124_BL_IM" typeName="BOOL"/>
		<variables name="A_M124_GOT" typeName="BOOL"/>
		<variables name="A_M124_iK" typeName="BOOL"/>
		<variables name="A_M124_IND" typeName="BYTE"/>
		<variables name="A_M124_KL_IM" typeName="BOOL"/>
		<variables name="A_M124_KM" typeName="BOOL"/>
		<variables name="A_M124_KS" typeName="BOOL"/>
		<variables name="A_M124_MOT" typeName="BOOL"/>
		<variables name="A_M124_MV1" typeName="BOOL"/>
		<variables name="A_M124_N" typeName="BOOL"/>
		<variables name="A_M124_OT" typeName="BOOL"/>
		<variables name="A_M124_OT1" typeName="BOOL"/>
		<variables name="A_M124_S" typeName="BOOL"/>
		<variables name="A_M124_S1" typeName="BOOL"/>
		<variables name="A_M124_SnL_IM" typeName="BOOL"/>
		<variables name="A_M124_UK" typeName="BOOL"/>
		<variables name="A_M124_VCHM" typeName="BOOL"/>
		<variables name="A_M124_VO" typeName="BOOL"/>
		<variables name="A_M124KM" typeName="BOOL"/>
		<variables name="A_M124MS_K" typeName="EBOOL"/>
		<variables name="A_M124MS_U" typeName="EBOOL"/>
		<variables name="A_M124MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M124MS_U_PR" typeName="BOOL"/>
		<variables name="A_M124MS_UK" typeName="EBOOL"/>
		<variables name="A_M124N" typeName="BOOL"/>
		<variables name="A_M124O_K" typeName="EBOOL"/>
		<variables name="A_M126_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M126_BL_IM" typeName="BOOL"/>
		<variables name="A_M126_GOT" typeName="BOOL"/>
		<variables name="A_M126_iK" typeName="BOOL"/>
		<variables name="A_M126_IND" typeName="BYTE"/>
		<variables name="A_M126_KL_IM" typeName="BOOL"/>
		<variables name="A_M126_KM" typeName="BOOL"/>
		<variables name="A_M126_KS" typeName="BOOL"/>
		<variables name="A_M126_MOT" typeName="BOOL"/>
		<variables name="A_M126_MV1" typeName="BOOL"/>
		<variables name="A_M126_N" typeName="BOOL"/>
		<variables name="A_M126_OT" typeName="BOOL"/>
		<variables name="A_M126_OT1" typeName="BOOL"/>
		<variables name="A_M126_S" typeName="BOOL"/>
		<variables name="A_M126_S1" typeName="BOOL"/>
		<variables name="A_M126_SnL_IM" typeName="BOOL"/>
		<variables name="A_M126_UK" typeName="BOOL"/>
		<variables name="A_M126_VNM" typeName="BOOL"/>
		<variables name="A_M126_VO" typeName="BOOL"/>
		<variables name="A_M126KM" typeName="BOOL"/>
		<variables name="A_M126MS_K" typeName="EBOOL"/>
		<variables name="A_M126MS_U" typeName="EBOOL"/>
		<variables name="A_M126MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M126MS_U_PR" typeName="BOOL"/>
		<variables name="A_M126MS_UK" typeName="EBOOL"/>
		<variables name="A_M126N" typeName="BOOL"/>
		<variables name="A_M126O_K" typeName="EBOOL"/>
		<variables name="A_M128_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M128_BL_IM" typeName="BOOL"/>
		<variables name="A_M128_GOT" typeName="BOOL"/>
		<variables name="A_M128_iK" typeName="BOOL"/>
		<variables name="A_M128_IND" typeName="BYTE"/>
		<variables name="A_M128_KL_IM" typeName="BOOL"/>
		<variables name="A_M128_KM" typeName="BOOL"/>
		<variables name="A_M128_KS" typeName="BOOL"/>
		<variables name="A_M128_MOT" typeName="BOOL"/>
		<variables name="A_M128_MV1" typeName="BOOL"/>
		<variables name="A_M128_N" typeName="BOOL"/>
		<variables name="A_M128_OT" typeName="BOOL"/>
		<variables name="A_M128_OT1" typeName="BOOL"/>
		<variables name="A_M128_S" typeName="BOOL"/>
		<variables name="A_M128_S1" typeName="BOOL"/>
		<variables name="A_M128_SnL_IM" typeName="BOOL"/>
		<variables name="A_M128_UK" typeName="BOOL"/>
		<variables name="A_M128_VNM" typeName="BOOL"/>
		<variables name="A_M128_VO" typeName="BOOL"/>
		<variables name="A_M128KM" typeName="BOOL"/>
		<variables name="A_M128MS_K" typeName="EBOOL"/>
		<variables name="A_M128MS_U" typeName="EBOOL"/>
		<variables name="A_M128MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M128MS_U_PR" typeName="BOOL"/>
		<variables name="A_M128MS_UK" typeName="EBOOL"/>
		<variables name="A_M128N" typeName="BOOL"/>
		<variables name="A_M128O_K" typeName="EBOOL"/>
		<variables name="A_M130_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M130_BL_IM" typeName="BOOL"/>
		<variables name="A_M130_GOT" typeName="BOOL"/>
		<variables name="A_M130_iK" typeName="BOOL"/>
		<variables name="A_M130_IND" typeName="BYTE"/>
		<variables name="A_M130_KL_IM" typeName="BOOL"/>
		<variables name="A_M130_KM" typeName="BOOL"/>
		<variables name="A_M130_KS" typeName="BOOL"/>
		<variables name="A_M130_MOT" typeName="BOOL"/>
		<variables name="A_M130_MV1" typeName="BOOL"/>
		<variables name="A_M130_N" typeName="BOOL"/>
		<variables name="A_M130_OT" typeName="BOOL"/>
		<variables name="A_M130_OT1" typeName="BOOL"/>
		<variables name="A_M130_S" typeName="BOOL"/>
		<variables name="A_M130_S1" typeName="BOOL"/>
		<variables name="A_M130_SnL_IM" typeName="BOOL"/>
		<variables name="A_M130_UK" typeName="BOOL"/>
		<variables name="A_M130_VCHM" typeName="BOOL"/>
		<variables name="A_M130_VO" typeName="BOOL"/>
		<variables name="A_M130KM" typeName="BOOL"/>
		<variables name="A_M130MS_K" typeName="EBOOL"/>
		<variables name="A_M130MS_U" typeName="EBOOL"/>
		<variables name="A_M130MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M130MS_U_PR" typeName="BOOL"/>
		<variables name="A_M130MS_UK" typeName="EBOOL"/>
		<variables name="A_M130N" typeName="BOOL"/>
		<variables name="A_M130O_K" typeName="EBOOL"/>
		<variables name="A_M132_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M132_BL_IM" typeName="BOOL"/>
		<variables name="A_M132_GOT" typeName="BOOL"/>
		<variables name="A_M132_iK" typeName="BOOL"/>
		<variables name="A_M132_IND" typeName="BYTE"/>
		<variables name="A_M132_KL_IM" typeName="BOOL"/>
		<variables name="A_M132_KM" typeName="BOOL"/>
		<variables name="A_M132_KS" typeName="BOOL"/>
		<variables name="A_M132_MOT" typeName="BOOL"/>
		<variables name="A_M132_MV1" typeName="BOOL"/>
		<variables name="A_M132_N" typeName="BOOL"/>
		<variables name="A_M132_OT" typeName="BOOL"/>
		<variables name="A_M132_OT1" typeName="BOOL"/>
		<variables name="A_M132_S" typeName="BOOL"/>
		<variables name="A_M132_S1" typeName="BOOL"/>
		<variables name="A_M132_SnL_IM" typeName="BOOL"/>
		<variables name="A_M132_UK" typeName="BOOL"/>
		<variables name="A_M132_VCHM" typeName="BOOL"/>
		<variables name="A_M132_VO" typeName="BOOL"/>
		<variables name="A_M132KM" typeName="BOOL"/>
		<variables name="A_M132MS_K" typeName="EBOOL"/>
		<variables name="A_M132MS_U" typeName="EBOOL"/>
		<variables name="A_M132MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M132MS_U_PR" typeName="BOOL"/>
		<variables name="A_M132MS_UK" typeName="EBOOL"/>
		<variables name="A_M132N" typeName="BOOL"/>
		<variables name="A_M132O_K" typeName="EBOOL"/>
		<variables name="A_M134_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M134_BL_IM" typeName="BOOL"/>
		<variables name="A_M134_GOT" typeName="BOOL"/>
		<variables name="A_M134_iK" typeName="BOOL"/>
		<variables name="A_M134_IND" typeName="BYTE"/>
		<variables name="A_M134_KL_IM" typeName="BOOL"/>
		<variables name="A_M134_KM" typeName="BOOL"/>
		<variables name="A_M134_KS" typeName="BOOL"/>
		<variables name="A_M134_MOT" typeName="BOOL"/>
		<variables name="A_M134_MV1" typeName="BOOL"/>
		<variables name="A_M134_N" typeName="BOOL"/>
		<variables name="A_M134_OT" typeName="BOOL"/>
		<variables name="A_M134_OT1" typeName="BOOL"/>
		<variables name="A_M134_S" typeName="BOOL"/>
		<variables name="A_M134_S1" typeName="BOOL"/>
		<variables name="A_M134_SnL_IM" typeName="BOOL"/>
		<variables name="A_M134_VCHM" typeName="BOOL"/>
		<variables name="A_M134_VNM" typeName="BOOL"/>
		<variables name="A_M134_VO" typeName="BOOL"/>
		<variables name="A_M134MS_K" typeName="BOOL"/>
		<variables name="A_M134MS_U" typeName="EBOOL"/>
		<variables name="A_M134MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M134MS_U_PR" typeName="BOOL"/>
		<variables name="A_M134MS_UK" typeName="EBOOL"/>
		<variables name="A_M134N" typeName="BOOL"/>
		<variables name="A_M134O_K" typeName="EBOOL"/>
		<variables name="A_M136_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M136_BL_IM" typeName="BOOL"/>
		<variables name="A_M136_GOT" typeName="BOOL"/>
		<variables name="A_M136_iK" typeName="BOOL"/>
		<variables name="A_M136_IND" typeName="BYTE"/>
		<variables name="A_M136_KL_IM" typeName="BOOL"/>
		<variables name="A_M136_KM" typeName="BOOL"/>
		<variables name="A_M136_KS" typeName="BOOL"/>
		<variables name="A_M136_MOT" typeName="BOOL"/>
		<variables name="A_M136_MV1" typeName="BOOL"/>
		<variables name="A_M136_N" typeName="BOOL"/>
		<variables name="A_M136_OT" typeName="BOOL"/>
		<variables name="A_M136_OT1" typeName="BOOL"/>
		<variables name="A_M136_S" typeName="BOOL"/>
		<variables name="A_M136_S1" typeName="BOOL"/>
		<variables name="A_M136_SnL_IM" typeName="BOOL"/>
		<variables name="A_M136_UK" typeName="BOOL"/>
		<variables name="A_M136_VNM" typeName="BOOL"/>
		<variables name="A_M136_VO" typeName="BOOL"/>
		<variables name="A_M136KM" typeName="BOOL"/>
		<variables name="A_M136MS_K" typeName="EBOOL"/>
		<variables name="A_M136MS_U" typeName="EBOOL"/>
		<variables name="A_M136MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M136MS_U_PR" typeName="BOOL"/>
		<variables name="A_M136MS_UK" typeName="EBOOL"/>
		<variables name="A_M136N" typeName="BOOL"/>
		<variables name="A_M136O_K" typeName="EBOOL"/>
		<variables name="A_M138_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M138_BL_IM" typeName="BOOL"/>
		<variables name="A_M138_GOT" typeName="BOOL"/>
		<variables name="A_M138_iK" typeName="BOOL"/>
		<variables name="A_M138_IND" typeName="BYTE"/>
		<variables name="A_M138_KL_IM" typeName="BOOL"/>
		<variables name="A_M138_KM" typeName="BOOL"/>
		<variables name="A_M138_KS" typeName="BOOL"/>
		<variables name="A_M138_MOT" typeName="BOOL"/>
		<variables name="A_M138_MV1" typeName="BOOL"/>
		<variables name="A_M138_N" typeName="BOOL"/>
		<variables name="A_M138_OT" typeName="BOOL"/>
		<variables name="A_M138_OT1" typeName="BOOL"/>
		<variables name="A_M138_S" typeName="BOOL"/>
		<variables name="A_M138_S1" typeName="BOOL"/>
		<variables name="A_M138_SnL_IM" typeName="BOOL"/>
		<variables name="A_M138_UK" typeName="BOOL"/>
		<variables name="A_M138_VNM" typeName="BOOL"/>
		<variables name="A_M138_VO" typeName="BOOL"/>
		<variables name="A_M138KM" typeName="BOOL"/>
		<variables name="A_M138MS_K" typeName="EBOOL"/>
		<variables name="A_M138MS_U" typeName="EBOOL"/>
		<variables name="A_M138MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M138MS_U_PR" typeName="BOOL"/>
		<variables name="A_M138MS_UK" typeName="EBOOL"/>
		<variables name="A_M138N" typeName="BOOL"/>
		<variables name="A_M138O_K" typeName="EBOOL"/>
		<variables name="A_M140_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M140_BL_IM" typeName="BOOL"/>
		<variables name="A_M140_GOT" typeName="BOOL"/>
		<variables name="A_M140_iK" typeName="BOOL"/>
		<variables name="A_M140_IND" typeName="BYTE"/>
		<variables name="A_M140_KL_IM" typeName="BOOL"/>
		<variables name="A_M140_KM" typeName="BOOL"/>
		<variables name="A_M140_KS" typeName="BOOL"/>
		<variables name="A_M140_MOT" typeName="BOOL"/>
		<variables name="A_M140_MV1" typeName="BOOL"/>
		<variables name="A_M140_N" typeName="BOOL"/>
		<variables name="A_M140_OT" typeName="BOOL"/>
		<variables name="A_M140_OT1" typeName="BOOL"/>
		<variables name="A_M140_S" typeName="BOOL"/>
		<variables name="A_M140_S1" typeName="BOOL"/>
		<variables name="A_M140_SnL_IM" typeName="BOOL"/>
		<variables name="A_M140_UK" typeName="BOOL"/>
		<variables name="A_M140_VNM" typeName="BOOL"/>
		<variables name="A_M140_VO" typeName="BOOL"/>
		<variables name="A_M140KM" typeName="BOOL"/>
		<variables name="A_M140MS_K" typeName="EBOOL"/>
		<variables name="A_M140MS_U" typeName="EBOOL"/>
		<variables name="A_M140MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M140MS_U_PR" typeName="BOOL"/>
		<variables name="A_M140MS_UK" typeName="EBOOL"/>
		<variables name="A_M140N" typeName="BOOL"/>
		<variables name="A_M140O_K" typeName="EBOOL"/>
		<variables name="A_M142_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M142_BL_IM" typeName="BOOL"/>
		<variables name="A_M142_iK" typeName="BOOL"/>
		<variables name="A_M142_IND" typeName="BYTE"/>
		<variables name="A_M142_KL_IM" typeName="BOOL"/>
		<variables name="A_M142_KM" typeName="BOOL"/>
		<variables name="A_M142_KS" typeName="BOOL"/>
		<variables name="A_M142_N" typeName="BOOL"/>
		<variables name="A_M142_OT" typeName="BOOL"/>
		<variables name="A_M142_OT1" typeName="BOOL"/>
		<variables name="A_M142_S" typeName="BOOL"/>
		<variables name="A_M142_S1" typeName="BOOL"/>
		<variables name="A_M142_SnL_IM" typeName="BOOL"/>
		<variables name="A_M142_UK" typeName="BOOL"/>
		<variables name="A_M142_VNM" typeName="BOOL"/>
		<variables name="A_M142_VO" typeName="BOOL"/>
		<variables name="A_M142KM" typeName="BOOL"/>
		<variables name="A_M142MS_K" typeName="EBOOL"/>
		<variables name="A_M142MS_U" typeName="EBOOL"/>
		<variables name="A_M142MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M142MS_U_PR" typeName="BOOL"/>
		<variables name="A_M142MS_UK" typeName="EBOOL"/>
		<variables name="A_M142O_K" typeName="EBOOL"/>
		<variables name="A_M144_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M144_BL_IM" typeName="BOOL"/>
		<variables name="A_M144_GOT" typeName="BOOL"/>
		<variables name="A_M144_iK" typeName="BOOL"/>
		<variables name="A_M144_IND" typeName="BYTE"/>
		<variables name="A_M144_KL_IM" typeName="BOOL"/>
		<variables name="A_M144_KM" typeName="BOOL"/>
		<variables name="A_M144_KS" typeName="BOOL"/>
		<variables name="A_M144_MOT" typeName="BOOL"/>
		<variables name="A_M144_MV1" typeName="BOOL"/>
		<variables name="A_M144_N" typeName="BOOL"/>
		<variables name="A_M144_OT" typeName="BOOL"/>
		<variables name="A_M144_OT1" typeName="BOOL"/>
		<variables name="A_M144_S" typeName="BOOL"/>
		<variables name="A_M144_S1" typeName="BOOL"/>
		<variables name="A_M144_SnL_IM" typeName="BOOL"/>
		<variables name="A_M144_UK" typeName="BOOL"/>
		<variables name="A_M144_VNM" typeName="BOOL"/>
		<variables name="A_M144_VO" typeName="BOOL"/>
		<variables name="A_M144KM" typeName="BOOL"/>
		<variables name="A_M144MS_K" typeName="EBOOL"/>
		<variables name="A_M144MS_U" typeName="EBOOL"/>
		<variables name="A_M144MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M144MS_U_PR" typeName="BOOL"/>
		<variables name="A_M144MS_UK" typeName="EBOOL"/>
		<variables name="A_M144N" typeName="BOOL"/>
		<variables name="A_M144O_K" typeName="EBOOL"/>
		<variables name="A_M146_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M146_BL_IM" typeName="BOOL"/>
		<variables name="A_M146_GOT" typeName="BOOL"/>
		<variables name="A_M146_iK" typeName="BOOL"/>
		<variables name="A_M146_IND" typeName="BYTE"/>
		<variables name="A_M146_KL_IM" typeName="BOOL"/>
		<variables name="A_M146_KM" typeName="BOOL"/>
		<variables name="A_M146_KS" typeName="BOOL"/>
		<variables name="A_M146_MOT" typeName="BOOL"/>
		<variables name="A_M146_MV1" typeName="BOOL"/>
		<variables name="A_M146_N" typeName="BOOL"/>
		<variables name="A_M146_OT" typeName="BOOL"/>
		<variables name="A_M146_OT1" typeName="BOOL"/>
		<variables name="A_M146_S" typeName="BOOL"/>
		<variables name="A_M146_S1" typeName="BOOL"/>
		<variables name="A_M146_SnL_IM" typeName="BOOL"/>
		<variables name="A_M146_UK" typeName="BOOL"/>
		<variables name="A_M146_VNM" typeName="BOOL"/>
		<variables name="A_M146_VO" typeName="BOOL"/>
		<variables name="A_M146KM" typeName="BOOL"/>
		<variables name="A_M146MS_K" typeName="EBOOL"/>
		<variables name="A_M146MS_U" typeName="EBOOL"/>
		<variables name="A_M146MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M146MS_U_PR" typeName="BOOL"/>
		<variables name="A_M146MS_UK" typeName="EBOOL"/>
		<variables name="A_M146N" typeName="BOOL"/>
		<variables name="A_M146O_K" typeName="EBOOL"/>
		<variables name="A_M148_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M148_BL_IM" typeName="BOOL"/>
		<variables name="A_M148_GOT" typeName="BOOL"/>
		<variables name="A_M148_iK" typeName="BOOL"/>
		<variables name="A_M148_IND" typeName="BYTE"/>
		<variables name="A_M148_KL_IM" typeName="BOOL"/>
		<variables name="A_M148_KM" typeName="BOOL"/>
		<variables name="A_M148_KS" typeName="BOOL"/>
		<variables name="A_M148_MOT" typeName="BOOL"/>
		<variables name="A_M148_MV1" typeName="BOOL"/>
		<variables name="A_M148_N" typeName="BOOL"/>
		<variables name="A_M148_OT" typeName="BOOL"/>
		<variables name="A_M148_OT1" typeName="BOOL"/>
		<variables name="A_M148_S" typeName="BOOL"/>
		<variables name="A_M148_S1" typeName="BOOL"/>
		<variables name="A_M148_SnL_IM" typeName="BOOL"/>
		<variables name="A_M148_UK" typeName="BOOL"/>
		<variables name="A_M148_VNM" typeName="BOOL"/>
		<variables name="A_M148_VO" typeName="BOOL"/>
		<variables name="A_M148KM" typeName="BOOL"/>
		<variables name="A_M148MS_K" typeName="EBOOL"/>
		<variables name="A_M148MS_U" typeName="EBOOL"/>
		<variables name="A_M148MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M148MS_U_PR" typeName="BOOL"/>
		<variables name="A_M148MS_UK" typeName="EBOOL"/>
		<variables name="A_M148N" typeName="BOOL"/>
		<variables name="A_M148O_K" typeName="EBOOL"/>
		<variables name="A_M150_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M150_BL_IM" typeName="BOOL"/>
		<variables name="A_M150_GOT" typeName="BOOL"/>
		<variables name="A_M150_iK" typeName="BOOL"/>
		<variables name="A_M150_IND" typeName="BYTE"/>
		<variables name="A_M150_KL_IM" typeName="BOOL"/>
		<variables name="A_M150_KM" typeName="BOOL"/>
		<variables name="A_M150_KS" typeName="BOOL"/>
		<variables name="A_M150_MOT" typeName="BOOL"/>
		<variables name="A_M150_MV1" typeName="BOOL"/>
		<variables name="A_M150_N" typeName="BOOL"/>
		<variables name="A_M150_OT" typeName="BOOL"/>
		<variables name="A_M150_OT1" typeName="BOOL"/>
		<variables name="A_M150_S" typeName="BOOL"/>
		<variables name="A_M150_S1" typeName="BOOL"/>
		<variables name="A_M150_SnL_IM" typeName="BOOL"/>
		<variables name="A_M150_UK" typeName="BOOL"/>
		<variables name="A_M150_VNM" typeName="BOOL"/>
		<variables name="A_M150_VO" typeName="BOOL"/>
		<variables name="A_M150KM" typeName="BOOL"/>
		<variables name="A_M150MS_K" typeName="EBOOL"/>
		<variables name="A_M150MS_U" typeName="EBOOL"/>
		<variables name="A_M150MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M150MS_U_PR" typeName="BOOL"/>
		<variables name="A_M150MS_UK" typeName="EBOOL"/>
		<variables name="A_M150N" typeName="BOOL"/>
		<variables name="A_M150O_K" typeName="EBOOL"/>
		<variables name="A_M152_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M152_BL_IM" typeName="BOOL"/>
		<variables name="A_M152_GOT" typeName="BOOL"/>
		<variables name="A_M152_iK" typeName="BOOL"/>
		<variables name="A_M152_IND" typeName="BYTE"/>
		<variables name="A_M152_KL_IM" typeName="BOOL"/>
		<variables name="A_M152_KM" typeName="BOOL"/>
		<variables name="A_M152_KS" typeName="BOOL"/>
		<variables name="A_M152_MOT" typeName="BOOL"/>
		<variables name="A_M152_MV1" typeName="BOOL"/>
		<variables name="A_M152_N" typeName="BOOL"/>
		<variables name="A_M152_OT" typeName="BOOL"/>
		<variables name="A_M152_OT1" typeName="BOOL"/>
		<variables name="A_M152_S" typeName="BOOL"/>
		<variables name="A_M152_S1" typeName="BOOL"/>
		<variables name="A_M152_SnL_IM" typeName="BOOL"/>
		<variables name="A_M152_UK" typeName="BOOL"/>
		<variables name="A_M152_VCHM" typeName="BOOL"/>
		<variables name="A_M152_VO" typeName="BOOL"/>
		<variables name="A_M152KM" typeName="BOOL"/>
		<variables name="A_M152MS_K" typeName="EBOOL"/>
		<variables name="A_M152MS_U" typeName="EBOOL"/>
		<variables name="A_M152MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M152MS_U_PR" typeName="BOOL"/>
		<variables name="A_M152MS_UK" typeName="EBOOL"/>
		<variables name="A_M152N" typeName="BOOL"/>
		<variables name="A_M152O_K" typeName="EBOOL"/>
		<variables name="A_M154_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M154_BL_IM" typeName="BOOL"/>
		<variables name="A_M154_GOT" typeName="BOOL"/>
		<variables name="A_M154_iK" typeName="BOOL"/>
		<variables name="A_M154_IND" typeName="BYTE"/>
		<variables name="A_M154_KL_IM" typeName="BOOL"/>
		<variables name="A_M154_KM" typeName="BOOL"/>
		<variables name="A_M154_KS" typeName="BOOL"/>
		<variables name="A_M154_MOT" typeName="BOOL"/>
		<variables name="A_M154_MV1" typeName="BOOL"/>
		<variables name="A_M154_N" typeName="BOOL"/>
		<variables name="A_M154_OT" typeName="BOOL"/>
		<variables name="A_M154_OT1" typeName="BOOL"/>
		<variables name="A_M154_S" typeName="BOOL"/>
		<variables name="A_M154_S1" typeName="BOOL"/>
		<variables name="A_M154_SnL_IM" typeName="BOOL"/>
		<variables name="A_M154_UK" typeName="BOOL"/>
		<variables name="A_M154_VNM" typeName="BOOL"/>
		<variables name="A_M154_VO" typeName="BOOL"/>
		<variables name="A_M154KM" typeName="BOOL"/>
		<variables name="A_M154MS_K" typeName="EBOOL"/>
		<variables name="A_M154MS_U" typeName="EBOOL"/>
		<variables name="A_M154MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M154MS_U_PR" typeName="BOOL"/>
		<variables name="A_M154MS_UK" typeName="EBOOL"/>
		<variables name="A_M154N" typeName="BOOL"/>
		<variables name="A_M154O_K" typeName="EBOOL"/>
		<variables name="A_M156_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M156_BL_IM" typeName="BOOL"/>
		<variables name="A_M156_GOT" typeName="BOOL"/>
		<variables name="A_M156_iK" typeName="BOOL"/>
		<variables name="A_M156_IND" typeName="BYTE"/>
		<variables name="A_M156_KL_IM" typeName="BOOL"/>
		<variables name="A_M156_KM" typeName="BOOL"/>
		<variables name="A_M156_KS" typeName="BOOL"/>
		<variables name="A_M156_MOT" typeName="BOOL"/>
		<variables name="A_M156_MV1" typeName="BOOL"/>
		<variables name="A_M156_N" typeName="BOOL"/>
		<variables name="A_M156_OT" typeName="BOOL"/>
		<variables name="A_M156_OT1" typeName="BOOL"/>
		<variables name="A_M156_S" typeName="BOOL"/>
		<variables name="A_M156_S1" typeName="BOOL"/>
		<variables name="A_M156_SnL_IM" typeName="BOOL"/>
		<variables name="A_M156_UK" typeName="BOOL"/>
		<variables name="A_M156_VNM" typeName="BOOL"/>
		<variables name="A_M156_VO" typeName="BOOL"/>
		<variables name="A_M156KM" typeName="BOOL"/>
		<variables name="A_M156MS_K" typeName="EBOOL"/>
		<variables name="A_M156MS_U" typeName="EBOOL"/>
		<variables name="A_M156MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M156MS_U_PR" typeName="BOOL"/>
		<variables name="A_M156MS_UK" typeName="EBOOL"/>
		<variables name="A_M156N" typeName="BOOL"/>
		<variables name="A_M156O_K" typeName="EBOOL"/>
		<variables name="A_M158_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M158_BL_IM" typeName="BOOL"/>
		<variables name="A_M158_GOT" typeName="BOOL"/>
		<variables name="A_M158_iK" typeName="BOOL"/>
		<variables name="A_M158_IND" typeName="BYTE"/>
		<variables name="A_M158_KL_IM" typeName="BOOL"/>
		<variables name="A_M158_KM" typeName="BOOL"/>
		<variables name="A_M158_KS" typeName="BOOL"/>
		<variables name="A_M158_MOT" typeName="BOOL"/>
		<variables name="A_M158_MV1" typeName="BOOL"/>
		<variables name="A_M158_N" typeName="BOOL"/>
		<variables name="A_M158_OT" typeName="BOOL"/>
		<variables name="A_M158_OT1" typeName="BOOL"/>
		<variables name="A_M158_S" typeName="BOOL"/>
		<variables name="A_M158_S1" typeName="BOOL"/>
		<variables name="A_M158_SnL_IM" typeName="BOOL"/>
		<variables name="A_M158_UK" typeName="BOOL"/>
		<variables name="A_M158_VCHM" typeName="BOOL"/>
		<variables name="A_M158_VO" typeName="BOOL"/>
		<variables name="A_M158KM" typeName="BOOL"/>
		<variables name="A_M158MS_K" typeName="EBOOL"/>
		<variables name="A_M158MS_U" typeName="EBOOL"/>
		<variables name="A_M158MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M158MS_U_PR" typeName="BOOL"/>
		<variables name="A_M158MS_UK" typeName="EBOOL"/>
		<variables name="A_M158N" typeName="BOOL"/>
		<variables name="A_M158O_K" typeName="EBOOL"/>
		<variables name="A_M160_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M160_BL_IM" typeName="BOOL"/>
		<variables name="A_M160_GOT" typeName="BOOL"/>
		<variables name="A_M160_iK" typeName="BOOL"/>
		<variables name="A_M160_IND" typeName="BYTE"/>
		<variables name="A_M160_KL_IM" typeName="BOOL"/>
		<variables name="A_M160_KM" typeName="BOOL"/>
		<variables name="A_M160_KS" typeName="BOOL"/>
		<variables name="A_M160_MOT" typeName="BOOL"/>
		<variables name="A_M160_MV1" typeName="BOOL"/>
		<variables name="A_M160_N" typeName="BOOL"/>
		<variables name="A_M160_OT" typeName="BOOL"/>
		<variables name="A_M160_OT1" typeName="BOOL"/>
		<variables name="A_M160_S" typeName="BOOL"/>
		<variables name="A_M160_S1" typeName="BOOL"/>
		<variables name="A_M160_SnL_IM" typeName="BOOL"/>
		<variables name="A_M160_UK" typeName="BOOL"/>
		<variables name="A_M160_VCHM" typeName="BOOL"/>
		<variables name="A_M160_VO" typeName="BOOL"/>
		<variables name="A_M160KM" typeName="BOOL"/>
		<variables name="A_M160MS_K" typeName="EBOOL"/>
		<variables name="A_M160MS_U" typeName="EBOOL"/>
		<variables name="A_M160MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M160MS_U_PR" typeName="BOOL"/>
		<variables name="A_M160MS_UK" typeName="EBOOL"/>
		<variables name="A_M160N" typeName="BOOL"/>
		<variables name="A_M160O_K" typeName="EBOOL"/>
		<variables name="A_M162_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M162_BL_IM" typeName="BOOL"/>
		<variables name="A_M162_GOT" typeName="BOOL"/>
		<variables name="A_M162_iK" typeName="BOOL"/>
		<variables name="A_M162_IND" typeName="BYTE"/>
		<variables name="A_M162_KL_IM" typeName="BOOL"/>
		<variables name="A_M162_KM" typeName="BOOL"/>
		<variables name="A_M162_KS" typeName="BOOL"/>
		<variables name="A_M162_MOT" typeName="BOOL"/>
		<variables name="A_M162_MV1" typeName="BOOL"/>
		<variables name="A_M162_N" typeName="BOOL"/>
		<variables name="A_M162_OT" typeName="BOOL"/>
		<variables name="A_M162_OT1" typeName="BOOL"/>
		<variables name="A_M162_S" typeName="BOOL"/>
		<variables name="A_M162_S1" typeName="BOOL"/>
		<variables name="A_M162_SnL_IM" typeName="BOOL"/>
		<variables name="A_M162_UK" typeName="BOOL"/>
		<variables name="A_M162_VCHM" typeName="BOOL"/>
		<variables name="A_M162_VO" typeName="BOOL"/>
		<variables name="A_M162KM" typeName="BOOL"/>
		<variables name="A_M162MS_K" typeName="EBOOL"/>
		<variables name="A_M162MS_U" typeName="EBOOL"/>
		<variables name="A_M162MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M162MS_U_PR" typeName="BOOL"/>
		<variables name="A_M162MS_UK" typeName="EBOOL"/>
		<variables name="A_M162N" typeName="BOOL"/>
		<variables name="A_M162O_K" typeName="EBOOL"/>
		<variables name="A_M164_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M164_BL_IM" typeName="BOOL"/>
		<variables name="A_M164_GOT" typeName="BOOL"/>
		<variables name="A_M164_iK" typeName="BOOL"/>
		<variables name="A_M164_IND" typeName="BYTE"/>
		<variables name="A_M164_KL_IM" typeName="BOOL"/>
		<variables name="A_M164_KM" typeName="BOOL"/>
		<variables name="A_M164_KS" typeName="BOOL"/>
		<variables name="A_M164_MOT" typeName="BOOL"/>
		<variables name="A_M164_MV1" typeName="BOOL"/>
		<variables name="A_M164_N" typeName="BOOL"/>
		<variables name="A_M164_OT" typeName="BOOL"/>
		<variables name="A_M164_OT1" typeName="BOOL"/>
		<variables name="A_M164_S" typeName="BOOL"/>
		<variables name="A_M164_S1" typeName="BOOL"/>
		<variables name="A_M164_SnL_IM" typeName="BOOL"/>
		<variables name="A_M164_VCHM" typeName="BOOL"/>
		<variables name="A_M164_VNM" typeName="BOOL"/>
		<variables name="A_M164_VO" typeName="BOOL"/>
		<variables name="A_M164MS_K" typeName="EBOOL"/>
		<variables name="A_M164MS_U" typeName="EBOOL"/>
		<variables name="A_M164MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M164MS_U_PR" typeName="BOOL"/>
		<variables name="A_M164MS_UK" typeName="EBOOL"/>
		<variables name="A_M164N" typeName="BOOL"/>
		<variables name="A_M164O_K" typeName="EBOOL"/>
		<variables name="A_M166_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M166_BL_IM" typeName="BOOL"/>
		<variables name="A_M166_GOT" typeName="BOOL"/>
		<variables name="A_M166_iK" typeName="BOOL"/>
		<variables name="A_M166_IND" typeName="BYTE"/>
		<variables name="A_M166_KL_IM" typeName="BOOL"/>
		<variables name="A_M166_KM" typeName="BOOL"/>
		<variables name="A_M166_KS" typeName="BOOL"/>
		<variables name="A_M166_MOT" typeName="BOOL"/>
		<variables name="A_M166_MV1" typeName="BOOL"/>
		<variables name="A_M166_N" typeName="BOOL"/>
		<variables name="A_M166_OT" typeName="BOOL"/>
		<variables name="A_M166_OT1" typeName="BOOL"/>
		<variables name="A_M166_S" typeName="BOOL"/>
		<variables name="A_M166_S1" typeName="BOOL"/>
		<variables name="A_M166_SnL_IM" typeName="BOOL"/>
		<variables name="A_M166_UK" typeName="BOOL"/>
		<variables name="A_M166_VNM" typeName="BOOL"/>
		<variables name="A_M166_VO" typeName="BOOL"/>
		<variables name="A_M166KM" typeName="BOOL"/>
		<variables name="A_M166MS_K" typeName="EBOOL"/>
		<variables name="A_M166MS_U" typeName="EBOOL"/>
		<variables name="A_M166MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M166MS_U_PR" typeName="BOOL"/>
		<variables name="A_M166MS_UK" typeName="EBOOL"/>
		<variables name="A_M166N" typeName="BOOL"/>
		<variables name="A_M166O_K" typeName="EBOOL"/>
		<variables name="A_M168_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M168_BL_IM" typeName="BOOL"/>
		<variables name="A_M168_GOT" typeName="BOOL"/>
		<variables name="A_M168_iK" typeName="BOOL"/>
		<variables name="A_M168_IND" typeName="BYTE"/>
		<variables name="A_M168_KL_IM" typeName="BOOL"/>
		<variables name="A_M168_KM" typeName="BOOL"/>
		<variables name="A_M168_KS" typeName="BOOL"/>
		<variables name="A_M168_MOT" typeName="BOOL"/>
		<variables name="A_M168_MV1" typeName="BOOL"/>
		<variables name="A_M168_N" typeName="BOOL"/>
		<variables name="A_M168_OT" typeName="BOOL"/>
		<variables name="A_M168_OT1" typeName="BOOL"/>
		<variables name="A_M168_S" typeName="BOOL"/>
		<variables name="A_M168_S1" typeName="BOOL"/>
		<variables name="A_M168_SnL_IM" typeName="BOOL"/>
		<variables name="A_M168_UK" typeName="BOOL"/>
		<variables name="A_M168_VCHM" typeName="BOOL"/>
		<variables name="A_M168_VO" typeName="BOOL"/>
		<variables name="A_M168KM" typeName="BOOL"/>
		<variables name="A_M168MS_K" typeName="EBOOL"/>
		<variables name="A_M168MS_U" typeName="EBOOL"/>
		<variables name="A_M168MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M168MS_U_PR" typeName="BOOL"/>
		<variables name="A_M168MS_UK" typeName="EBOOL"/>
		<variables name="A_M168N" typeName="BOOL"/>
		<variables name="A_M168O_K" typeName="EBOOL"/>
		<variables name="A_M172_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M172_BL_IM" typeName="BOOL"/>
		<variables name="A_M172_GOT" typeName="BOOL"/>
		<variables name="A_M172_iK" typeName="BOOL"/>
		<variables name="A_M172_IND" typeName="BYTE"/>
		<variables name="A_M172_KL_IM" typeName="BOOL"/>
		<variables name="A_M172_KM" typeName="BOOL"/>
		<variables name="A_M172_KS" typeName="BOOL"/>
		<variables name="A_M172_MOT" typeName="BOOL"/>
		<variables name="A_M172_MV1" typeName="BOOL"/>
		<variables name="A_M172_N" typeName="BOOL"/>
		<variables name="A_M172_OT" typeName="BOOL"/>
		<variables name="A_M172_OT1" typeName="BOOL"/>
		<variables name="A_M172_S" typeName="BOOL"/>
		<variables name="A_M172_S1" typeName="BOOL"/>
		<variables name="A_M172_SnL_IM" typeName="BOOL"/>
		<variables name="A_M172_UK" typeName="BOOL"/>
		<variables name="A_M172_VCHM" typeName="BOOL"/>
		<variables name="A_M172_VO" typeName="BOOL"/>
		<variables name="A_M172KM" typeName="BOOL"/>
		<variables name="A_M172MS_K" typeName="BOOL"/>
		<variables name="A_M172MS_U" typeName="EBOOL"/>
		<variables name="A_M172MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M172MS_U_PR" typeName="BOOL"/>
		<variables name="A_M172MS_UK" typeName="EBOOL"/>
		<variables name="A_M172N" typeName="BOOL"/>
		<variables name="A_M172O_K" typeName="EBOOL"/>
		<variables name="A_M173_iK" typeName="BOOL"/>
		<variables name="A_M173_KM" typeName="BOOL"/>
		<variables name="A_M173_KS" typeName="BOOL"/>
		<variables name="A_M173_OT" typeName="BOOL"/>
		<variables name="A_M173_OT1" typeName="BOOL"/>
		<variables name="A_M173_S1" typeName="BOOL"/>
		<variables name="A_M173_UK" typeName="BOOL"/>
		<variables name="A_M173_VO" typeName="BOOL"/>
		<variables name="A_M173MS_K" typeName="EBOOL"/>
		<variables name="A_M174_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M174_BL_IM" typeName="BOOL"/>
		<variables name="A_M174_GOT" typeName="BOOL"/>
		<variables name="A_M174_iK" typeName="BOOL"/>
		<variables name="A_M174_IND" typeName="BYTE"/>
		<variables name="A_M174_KL_IM" typeName="BOOL"/>
		<variables name="A_M174_KM" typeName="BOOL"/>
		<variables name="A_M174_KS" typeName="BOOL"/>
		<variables name="A_M174_MOT" typeName="BOOL"/>
		<variables name="A_M174_MV1" typeName="BOOL"/>
		<variables name="A_M174_N" typeName="BOOL"/>
		<variables name="A_M174_OT" typeName="BOOL"/>
		<variables name="A_M174_OT1" typeName="BOOL"/>
		<variables name="A_M174_S" typeName="BOOL"/>
		<variables name="A_M174_S1" typeName="BOOL"/>
		<variables name="A_M174_SnL_IM" typeName="BOOL"/>
		<variables name="A_M174_UK" typeName="BOOL"/>
		<variables name="A_M174_VNM" typeName="BOOL"/>
		<variables name="A_M174_VO" typeName="BOOL"/>
		<variables name="A_M174KM" typeName="BOOL"/>
		<variables name="A_M174MS_K" typeName="EBOOL"/>
		<variables name="A_M174MS_U" typeName="EBOOL"/>
		<variables name="A_M174MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M174MS_U_PR" typeName="BOOL"/>
		<variables name="A_M174MS_UK" typeName="EBOOL"/>
		<variables name="A_M174N" typeName="BOOL"/>
		<variables name="A_M174O_K" typeName="EBOOL"/>
		<variables name="A_M176_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M176_BL_IM" typeName="BOOL"/>
		<variables name="A_M176_GOT" typeName="BOOL"/>
		<variables name="A_M176_iK" typeName="BOOL"/>
		<variables name="A_M176_IND" typeName="BYTE"/>
		<variables name="A_M176_KL_IM" typeName="BOOL"/>
		<variables name="A_M176_KM" typeName="BOOL"/>
		<variables name="A_M176_KS" typeName="BOOL"/>
		<variables name="A_M176_MOT" typeName="BOOL"/>
		<variables name="A_M176_MV1" typeName="BOOL"/>
		<variables name="A_M176_N" typeName="BOOL"/>
		<variables name="A_M176_OT" typeName="BOOL"/>
		<variables name="A_M176_OT1" typeName="BOOL"/>
		<variables name="A_M176_S" typeName="BOOL"/>
		<variables name="A_M176_S1" typeName="BOOL"/>
		<variables name="A_M176_SnL_IM" typeName="BOOL"/>
		<variables name="A_M176_UK" typeName="BOOL"/>
		<variables name="A_M176_VCHM" typeName="BOOL"/>
		<variables name="A_M176_VO" typeName="BOOL"/>
		<variables name="A_M176KM" typeName="BOOL"/>
		<variables name="A_M176MS_K" typeName="EBOOL"/>
		<variables name="A_M176MS_U" typeName="EBOOL"/>
		<variables name="A_M176MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M176MS_U_PR" typeName="BOOL"/>
		<variables name="A_M176MS_UK" typeName="EBOOL"/>
		<variables name="A_M176N" typeName="BOOL"/>
		<variables name="A_M176O_K" typeName="EBOOL"/>
		<variables name="A_M201_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M201_BL_IM" typeName="BOOL"/>
		<variables name="A_M201_GOT" typeName="BOOL"/>
		<variables name="A_M201_iK" typeName="BOOL"/>
		<variables name="A_M201_IND" typeName="BYTE"/>
		<variables name="A_M201_KM" typeName="BOOL"/>
		<variables name="A_M201_KS" typeName="BOOL"/>
		<variables name="A_M201_MOT" typeName="BOOL"/>
		<variables name="A_M201_MV1" typeName="BOOL"/>
		<variables name="A_M201_N" typeName="BOOL"/>
		<variables name="A_M201_OT" typeName="BOOL"/>
		<variables name="A_M201_OT1" typeName="BOOL"/>
		<variables name="A_M201_S" typeName="BOOL"/>
		<variables name="A_M201_S1" typeName="BOOL"/>
		<variables name="A_M201_SnL_IM" typeName="BOOL"/>
		<variables name="A_M201_UK" typeName="BOOL"/>
		<variables name="A_M201_VCHM" typeName="BOOL"/>
		<variables name="A_M201_VO" typeName="BOOL"/>
		<variables name="A_M201KM" typeName="BOOL"/>
		<variables name="A_M201MS_K" typeName="EBOOL"/>
		<variables name="A_M201MS_U" typeName="EBOOL"/>
		<variables name="A_M201MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M201MS_U_PR" typeName="BOOL"/>
		<variables name="A_M201MS_UK" typeName="EBOOL"/>
		<variables name="A_M201N" typeName="BOOL"/>
		<variables name="A_M201O_K" typeName="EBOOL"/>
		<variables name="A_M202_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M202_BL_IM" typeName="BOOL"/>
		<variables name="A_M202_GOT" typeName="BOOL"/>
		<variables name="A_M202_iK" typeName="BOOL"/>
		<variables name="A_M202_IND" typeName="BYTE"/>
		<variables name="A_M202_KL_IM" typeName="BOOL"/>
		<variables name="A_M202_KM" typeName="BOOL"/>
		<variables name="A_M202_KS" typeName="BOOL"/>
		<variables name="A_M202_MOT" typeName="BOOL"/>
		<variables name="A_M202_MV1" typeName="BOOL"/>
		<variables name="A_M202_N" typeName="BOOL"/>
		<variables name="A_M202_OT" typeName="BOOL"/>
		<variables name="A_M202_OT1" typeName="BOOL"/>
		<variables name="A_M202_S" typeName="BOOL"/>
		<variables name="A_M202_S1" typeName="BOOL"/>
		<variables name="A_M202_SnL_IM" typeName="BOOL"/>
		<variables name="A_M202_UK" typeName="BOOL"/>
		<variables name="A_M202_VCHM" typeName="BOOL"/>
		<variables name="A_M202_VO" typeName="BOOL"/>
		<variables name="A_M202KM" typeName="BOOL"/>
		<variables name="A_M202MS_K" typeName="EBOOL"/>
		<variables name="A_M202MS_U" typeName="EBOOL"/>
		<variables name="A_M202MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M202MS_U_PR" typeName="BOOL"/>
		<variables name="A_M202MS_UK" typeName="EBOOL"/>
		<variables name="A_M202N" typeName="BOOL"/>
		<variables name="A_M202O_K" typeName="EBOOL"/>
		<variables name="A_M203_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M203_BL_IM" typeName="BOOL"/>
		<variables name="A_M203_GOT" typeName="BOOL"/>
		<variables name="A_M203_iK" typeName="BOOL"/>
		<variables name="A_M203_IND" typeName="BYTE"/>
		<variables name="A_M203_KM" typeName="BOOL"/>
		<variables name="A_M203_KS" typeName="BOOL"/>
		<variables name="A_M203_MOT" typeName="BOOL"/>
		<variables name="A_M203_MV1" typeName="BOOL"/>
		<variables name="A_M203_N" typeName="BOOL"/>
		<variables name="A_M203_OT" typeName="BOOL"/>
		<variables name="A_M203_OT1" typeName="BOOL"/>
		<variables name="A_M203_S" typeName="BOOL"/>
		<variables name="A_M203_S1" typeName="BOOL"/>
		<variables name="A_M203_SnL_IM" typeName="BOOL"/>
		<variables name="A_M203_UK" typeName="BOOL"/>
		<variables name="A_M203_VNM" typeName="BOOL"/>
		<variables name="A_M203_VO" typeName="BOOL"/>
		<variables name="A_M203KM" typeName="BOOL"/>
		<variables name="A_M203MS_K" typeName="EBOOL"/>
		<variables name="A_M203MS_U" typeName="EBOOL"/>
		<variables name="A_M203MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M203MS_U_PR" typeName="BOOL"/>
		<variables name="A_M203MS_UK" typeName="EBOOL"/>
		<variables name="A_M203N" typeName="BOOL"/>
		<variables name="A_M203O_K" typeName="EBOOL"/>
		<variables name="A_M204_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M204_BL_IM" typeName="BOOL"/>
		<variables name="A_M204_GOT" typeName="BOOL"/>
		<variables name="A_M204_iK" typeName="BOOL"/>
		<variables name="A_M204_IND" typeName="BYTE"/>
		<variables name="A_M204_KL_IM" typeName="BOOL"/>
		<variables name="A_M204_KM" typeName="BOOL"/>
		<variables name="A_M204_KS" typeName="BOOL"/>
		<variables name="A_M204_MOT" typeName="BOOL"/>
		<variables name="A_M204_MV1" typeName="BOOL"/>
		<variables name="A_M204_N" typeName="BOOL"/>
		<variables name="A_M204_OT" typeName="BOOL"/>
		<variables name="A_M204_OT1" typeName="BOOL"/>
		<variables name="A_M204_S" typeName="BOOL"/>
		<variables name="A_M204_S1" typeName="BOOL"/>
		<variables name="A_M204_SnL_IM" typeName="BOOL"/>
		<variables name="A_M204_UK" typeName="BOOL"/>
		<variables name="A_M204_VCHM" typeName="BOOL"/>
		<variables name="A_M204_VO" typeName="BOOL"/>
		<variables name="A_M204KM" typeName="BOOL"/>
		<variables name="A_M204MS_K" typeName="EBOOL"/>
		<variables name="A_M204MS_U" typeName="EBOOL"/>
		<variables name="A_M204MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M204MS_U_PR" typeName="BOOL"/>
		<variables name="A_M204MS_UK" typeName="EBOOL"/>
		<variables name="A_M204N" typeName="BOOL"/>
		<variables name="A_M204O_K" typeName="EBOOL"/>
		<variables name="A_M206_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M206_BL_IM" typeName="BOOL"/>
		<variables name="A_M206_GOT" typeName="BOOL"/>
		<variables name="A_M206_iK" typeName="BOOL"/>
		<variables name="A_M206_IND" typeName="BYTE"/>
		<variables name="A_M206_KL_IM" typeName="BOOL"/>
		<variables name="A_M206_KM" typeName="BOOL"/>
		<variables name="A_M206_KS" typeName="BOOL"/>
		<variables name="A_M206_MOT" typeName="BOOL"/>
		<variables name="A_M206_MV1" typeName="BOOL"/>
		<variables name="A_M206_N" typeName="BOOL"/>
		<variables name="A_M206_OT" typeName="BOOL"/>
		<variables name="A_M206_OT1" typeName="BOOL"/>
		<variables name="A_M206_S" typeName="BOOL"/>
		<variables name="A_M206_S1" typeName="BOOL"/>
		<variables name="A_M206_SnL_IM" typeName="BOOL"/>
		<variables name="A_M206_UK" typeName="BOOL"/>
		<variables name="A_M206_VNM" typeName="BOOL"/>
		<variables name="A_M206_VO" typeName="BOOL"/>
		<variables name="A_M206KM" typeName="BOOL"/>
		<variables name="A_M206MS_K" typeName="EBOOL"/>
		<variables name="A_M206MS_U" typeName="EBOOL"/>
		<variables name="A_M206MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M206MS_U_PR" typeName="BOOL"/>
		<variables name="A_M206MS_UK" typeName="EBOOL"/>
		<variables name="A_M206N" typeName="BOOL"/>
		<variables name="A_M206O_K" typeName="EBOOL"/>
		<variables name="A_M208_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M208_BL_IM" typeName="BOOL"/>
		<variables name="A_M208_GOT" typeName="BOOL"/>
		<variables name="A_M208_iK" typeName="BOOL"/>
		<variables name="A_M208_IND" typeName="BYTE"/>
		<variables name="A_M208_KL_IM" typeName="BOOL"/>
		<variables name="A_M208_KM" typeName="BOOL"/>
		<variables name="A_M208_KS" typeName="BOOL"/>
		<variables name="A_M208_MOT" typeName="BOOL"/>
		<variables name="A_M208_MV1" typeName="BOOL"/>
		<variables name="A_M208_N" typeName="BOOL"/>
		<variables name="A_M208_OT" typeName="BOOL"/>
		<variables name="A_M208_OT1" typeName="BOOL"/>
		<variables name="A_M208_S" typeName="BOOL"/>
		<variables name="A_M208_S1" typeName="BOOL"/>
		<variables name="A_M208_SnL_IM" typeName="BOOL"/>
		<variables name="A_M208_UK" typeName="BOOL"/>
		<variables name="A_M208_VCHM" typeName="BOOL"/>
		<variables name="A_M208_VO" typeName="BOOL"/>
		<variables name="A_M208KM" typeName="BOOL"/>
		<variables name="A_M208MS_K" typeName="EBOOL"/>
		<variables name="A_M208MS_U" typeName="EBOOL"/>
		<variables name="A_M208MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M208MS_U_PR" typeName="BOOL"/>
		<variables name="A_M208MS_UK" typeName="EBOOL"/>
		<variables name="A_M208N" typeName="BOOL"/>
		<variables name="A_M208O_K" typeName="EBOOL"/>
		<variables name="A_M210_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M210_BL_IM" typeName="BOOL"/>
		<variables name="A_M210_GOT" typeName="BOOL"/>
		<variables name="A_M210_iK" typeName="BOOL"/>
		<variables name="A_M210_IND" typeName="BYTE"/>
		<variables name="A_M210_KL_IM" typeName="BOOL"/>
		<variables name="A_M210_KM" typeName="BOOL"/>
		<variables name="A_M210_KS" typeName="BOOL"/>
		<variables name="A_M210_MOT" typeName="BOOL"/>
		<variables name="A_M210_MV1" typeName="BOOL"/>
		<variables name="A_M210_N" typeName="BOOL"/>
		<variables name="A_M210_OT" typeName="BOOL"/>
		<variables name="A_M210_OT1" typeName="BOOL"/>
		<variables name="A_M210_S" typeName="BOOL"/>
		<variables name="A_M210_S1" typeName="BOOL"/>
		<variables name="A_M210_SnL_IM" typeName="BOOL"/>
		<variables name="A_M210_UK" typeName="BOOL"/>
		<variables name="A_M210_VCHM" typeName="BOOL"/>
		<variables name="A_M210_VO" typeName="BOOL"/>
		<variables name="A_M210KM" typeName="BOOL"/>
		<variables name="A_M210MS_K" typeName="EBOOL"/>
		<variables name="A_M210MS_U" typeName="EBOOL"/>
		<variables name="A_M210MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M210MS_U_PR" typeName="BOOL"/>
		<variables name="A_M210MS_UK" typeName="EBOOL"/>
		<variables name="A_M210N" typeName="BOOL"/>
		<variables name="A_M210O_K" typeName="EBOOL"/>
		<variables name="A_M212_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M212_BL_IM" typeName="BOOL"/>
		<variables name="A_M212_GOT" typeName="BOOL"/>
		<variables name="A_M212_iK" typeName="BOOL"/>
		<variables name="A_M212_IND" typeName="BYTE"/>
		<variables name="A_M212_KL_IM" typeName="BOOL"/>
		<variables name="A_M212_KM" typeName="BOOL"/>
		<variables name="A_M212_KS" typeName="BOOL"/>
		<variables name="A_M212_MOT" typeName="BOOL"/>
		<variables name="A_M212_MV1" typeName="BOOL"/>
		<variables name="A_M212_N" typeName="BOOL"/>
		<variables name="A_M212_OT" typeName="BOOL"/>
		<variables name="A_M212_OT1" typeName="BOOL"/>
		<variables name="A_M212_S" typeName="BOOL"/>
		<variables name="A_M212_S1" typeName="BOOL"/>
		<variables name="A_M212_SnL_IM" typeName="BOOL"/>
		<variables name="A_M212_UK" typeName="BOOL"/>
		<variables name="A_M212_VNM" typeName="BOOL"/>
		<variables name="A_M212_VO" typeName="BOOL"/>
		<variables name="A_M212MS_K" typeName="EBOOL"/>
		<variables name="A_M212MS_U" typeName="EBOOL"/>
		<variables name="A_M212MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M212MS_U_PR" typeName="BOOL"/>
		<variables name="A_M212MS_UK" typeName="EBOOL"/>
		<variables name="A_M212N" typeName="BOOL"/>
		<variables name="A_M212O_K" typeName="EBOOL"/>
		<variables name="A_M214_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M214_BL_IM" typeName="BOOL"/>
		<variables name="A_M214_GOT" typeName="BOOL"/>
		<variables name="A_M214_iK" typeName="BOOL"/>
		<variables name="A_M214_IND" typeName="BYTE"/>
		<variables name="A_M214_KL_IM" typeName="BOOL"/>
		<variables name="A_M214_KM" typeName="BOOL"/>
		<variables name="A_M214_KS" typeName="BOOL"/>
		<variables name="A_M214_MOT" typeName="BOOL"/>
		<variables name="A_M214_MV1" typeName="BOOL"/>
		<variables name="A_M214_N" typeName="BOOL"/>
		<variables name="A_M214_OT" typeName="BOOL"/>
		<variables name="A_M214_OT1" typeName="BOOL"/>
		<variables name="A_M214_S" typeName="BOOL"/>
		<variables name="A_M214_S1" typeName="BOOL"/>
		<variables name="A_M214_SnL_IM" typeName="BOOL"/>
		<variables name="A_M214_UK" typeName="BOOL"/>
		<variables name="A_M214_VNM" typeName="BOOL"/>
		<variables name="A_M214_VO" typeName="BOOL"/>
		<variables name="A_M214MS_K" typeName="EBOOL"/>
		<variables name="A_M214MS_U" typeName="EBOOL"/>
		<variables name="A_M214MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M214MS_U_PR" typeName="BOOL"/>
		<variables name="A_M214MS_UK" typeName="EBOOL"/>
		<variables name="A_M214N" typeName="BOOL"/>
		<variables name="A_M214O_K" typeName="EBOOL"/>
		<variables name="A_M216_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M216_BL_IM" typeName="BOOL"/>
		<variables name="A_M216_GOT" typeName="BOOL"/>
		<variables name="A_M216_iK" typeName="BOOL"/>
		<variables name="A_M216_IND" typeName="BYTE"/>
		<variables name="A_M216_KL_IM" typeName="BOOL"/>
		<variables name="A_M216_KM" typeName="BOOL"/>
		<variables name="A_M216_KS" typeName="BOOL"/>
		<variables name="A_M216_MOT" typeName="BOOL"/>
		<variables name="A_M216_MV1" typeName="BOOL"/>
		<variables name="A_M216_N" typeName="BOOL"/>
		<variables name="A_M216_OT" typeName="BOOL"/>
		<variables name="A_M216_OT1" typeName="BOOL"/>
		<variables name="A_M216_S" typeName="BOOL"/>
		<variables name="A_M216_S1" typeName="BOOL"/>
		<variables name="A_M216_SnL_IM" typeName="BOOL"/>
		<variables name="A_M216_UK" typeName="BOOL"/>
		<variables name="A_M216_VNM" typeName="BOOL"/>
		<variables name="A_M216_VO" typeName="BOOL"/>
		<variables name="A_M216MS_K" typeName="BOOL"/>
		<variables name="A_M216MS_U" typeName="EBOOL"/>
		<variables name="A_M216MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M216MS_U_PR" typeName="BOOL"/>
		<variables name="A_M216MS_UK" typeName="EBOOL"/>
		<variables name="A_M216N" typeName="BOOL"/>
		<variables name="A_M216O_K" typeName="EBOOL"/>
		<variables name="A_M218_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M218_BL_IM" typeName="BOOL"/>
		<variables name="A_M218_GOT" typeName="BOOL"/>
		<variables name="A_M218_iK" typeName="BOOL"/>
		<variables name="A_M218_IND" typeName="BYTE"/>
		<variables name="A_M218_KL_IM" typeName="BOOL"/>
		<variables name="A_M218_KM" typeName="BOOL"/>
		<variables name="A_M218_KS" typeName="BOOL"/>
		<variables name="A_M218_MOT" typeName="BOOL"/>
		<variables name="A_M218_MV1" typeName="BOOL"/>
		<variables name="A_M218_N" typeName="BOOL"/>
		<variables name="A_M218_OT" typeName="BOOL"/>
		<variables name="A_M218_OT1" typeName="BOOL"/>
		<variables name="A_M218_S" typeName="BOOL"/>
		<variables name="A_M218_S1" typeName="BOOL"/>
		<variables name="A_M218_SnL_IM" typeName="BOOL"/>
		<variables name="A_M218_UK" typeName="BOOL"/>
		<variables name="A_M218_VCHM" typeName="BOOL"/>
		<variables name="A_M218_VO" typeName="BOOL"/>
		<variables name="A_M218KM" typeName="BOOL"/>
		<variables name="A_M218MS_K" typeName="EBOOL"/>
		<variables name="A_M218MS_U" typeName="EBOOL"/>
		<variables name="A_M218MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M218MS_U_PR" typeName="BOOL"/>
		<variables name="A_M218MS_UK" typeName="EBOOL"/>
		<variables name="A_M218N" typeName="BOOL"/>
		<variables name="A_M218O_K" typeName="EBOOL"/>
		<variables name="A_M302_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M302_BL_IM" typeName="BOOL"/>
		<variables name="A_M302_GOT" typeName="BOOL"/>
		<variables name="A_M302_iK" typeName="BOOL"/>
		<variables name="A_M302_IND" typeName="BYTE"/>
		<variables name="A_M302_KL_IM" typeName="BOOL"/>
		<variables name="A_M302_KM" typeName="BOOL"/>
		<variables name="A_M302_KS" typeName="BOOL"/>
		<variables name="A_M302_MOT" typeName="BOOL"/>
		<variables name="A_M302_MV1" typeName="BOOL"/>
		<variables name="A_M302_N" typeName="BOOL"/>
		<variables name="A_M302_OT" typeName="BOOL"/>
		<variables name="A_M302_OT1" typeName="BOOL"/>
		<variables name="A_M302_S" typeName="BOOL"/>
		<variables name="A_M302_S1" typeName="BOOL"/>
		<variables name="A_M302_SnL_IM" typeName="BOOL"/>
		<variables name="A_M302_UK" typeName="BOOL"/>
		<variables name="A_M302_VCHM" typeName="BOOL"/>
		<variables name="A_M302_VO" typeName="BOOL"/>
		<variables name="A_M302IR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M302KM" typeName="BOOL"/>
		<variables name="A_M302KS" typeName="BOOL"/>
		<variables name="A_M302MS_K" typeName="EBOOL"/>
		<variables name="A_M302MS_U" typeName="EBOOL"/>
		<variables name="A_M302MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M302MS_U_PR" typeName="BOOL"/>
		<variables name="A_M302MS_UK" typeName="EBOOL"/>
		<variables name="A_M302N" typeName="BOOL"/>
		<variables name="A_M302O_K" typeName="EBOOL"/>
		<variables name="A_M302P1_K" typeName="EBOOL"/>
		<variables name="A_M302P1_U" typeName="EBOOL"/>
		<variables name="A_M302P1_U_OBR" typeName="BOOL"/>
		<variables name="A_M302P1_U_PR" typeName="BOOL"/>
		<variables name="A_M302P1_UK" typeName="EBOOL"/>
		<variables name="A_M302P_IND" typeName="BYTE"/>
		<variables name="A_M302P_K" typeName="EBOOL"/>
		<variables name="A_M302RI" typeName="BOOL"/>
		<variables name="A_M302Z" typeName="BOOL"/>
		<variables name="A_M304_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M304_BL_IM" typeName="BOOL"/>
		<variables name="A_M304_GOT" typeName="BOOL"/>
		<variables name="A_M304_iK" typeName="BOOL"/>
		<variables name="A_M304_IND" typeName="BYTE"/>
		<variables name="A_M304_KL_IM" typeName="BOOL"/>
		<variables name="A_M304_KM" typeName="BOOL"/>
		<variables name="A_M304_KS" typeName="BOOL"/>
		<variables name="A_M304_MOT" typeName="BOOL"/>
		<variables name="A_M304_MV1" typeName="BOOL"/>
		<variables name="A_M304_N" typeName="BOOL"/>
		<variables name="A_M304_OT" typeName="BOOL"/>
		<variables name="A_M304_OT1" typeName="BOOL"/>
		<variables name="A_M304_S" typeName="BOOL"/>
		<variables name="A_M304_S1" typeName="BOOL"/>
		<variables name="A_M304_SnL_IM" typeName="BOOL"/>
		<variables name="A_M304_UK" typeName="BOOL"/>
		<variables name="A_M304_VNM" typeName="BOOL"/>
		<variables name="A_M304_VO" typeName="BOOL"/>
		<variables name="A_M304KM" typeName="BOOL"/>
		<variables name="A_M304MS_K" typeName="EBOOL"/>
		<variables name="A_M304MS_U" typeName="EBOOL"/>
		<variables name="A_M304MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M304MS_U_PR" typeName="BOOL"/>
		<variables name="A_M304MS_UK" typeName="EBOOL"/>
		<variables name="A_M304N" typeName="BOOL"/>
		<variables name="A_M304O_K" typeName="EBOOL"/>
		<variables name="A_M306_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M306_BL_IM" typeName="BOOL"/>
		<variables name="A_M306_GOT" typeName="BOOL"/>
		<variables name="A_M306_iK" typeName="BOOL"/>
		<variables name="A_M306_IND" typeName="BYTE"/>
		<variables name="A_M306_KL_IM" typeName="BOOL"/>
		<variables name="A_M306_KM" typeName="BOOL"/>
		<variables name="A_M306_KS" typeName="BOOL"/>
		<variables name="A_M306_MOT" typeName="BOOL"/>
		<variables name="A_M306_MV1" typeName="BOOL"/>
		<variables name="A_M306_N" typeName="BOOL"/>
		<variables name="A_M306_OT" typeName="BOOL"/>
		<variables name="A_M306_OT1" typeName="BOOL"/>
		<variables name="A_M306_S" typeName="BOOL"/>
		<variables name="A_M306_S1" typeName="BOOL"/>
		<variables name="A_M306_SnL_IM" typeName="BOOL"/>
		<variables name="A_M306_UK" typeName="BOOL"/>
		<variables name="A_M306_VNM" typeName="BOOL"/>
		<variables name="A_M306_VO" typeName="BOOL"/>
		<variables name="A_M306KM" typeName="BOOL"/>
		<variables name="A_M306MS_K" typeName="EBOOL"/>
		<variables name="A_M306MS_U" typeName="EBOOL"/>
		<variables name="A_M306MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M306MS_U_PR" typeName="BOOL"/>
		<variables name="A_M306MS_UK" typeName="EBOOL"/>
		<variables name="A_M306N" typeName="BOOL"/>
		<variables name="A_M306O_K" typeName="EBOOL"/>
		<variables name="A_M308_ARM_DSP" typeName="BOOL"/>
		<variables name="A_M308_BL_IM" typeName="BOOL"/>
		<variables name="A_M308_GOT" typeName="BOOL"/>
		<variables name="A_M308_iK" typeName="BOOL"/>
		<variables name="A_M308_IND" typeName="BYTE"/>
		<variables name="A_M308_KL_IM" typeName="BOOL"/>
		<variables name="A_M308_KM" typeName="BOOL"/>
		<variables name="A_M308_KS" typeName="BOOL"/>
		<variables name="A_M308_MOT" typeName="BOOL"/>
		<variables name="A_M308_MV1" typeName="BOOL"/>
		<variables name="A_M308_N" typeName="BOOL"/>
		<variables name="A_M308_OT" typeName="BOOL"/>
		<variables name="A_M308_OT1" typeName="BOOL"/>
		<variables name="A_M308_S" typeName="BOOL"/>
		<variables name="A_M308_S1" typeName="BOOL"/>
		<variables name="A_M308_SnL_IM" typeName="BOOL"/>
		<variables name="A_M308_UK" typeName="BOOL"/>
		<variables name="A_M308_VCHM" typeName="BOOL"/>
		<variables name="A_M308_VO" typeName="BOOL"/>
		<variables name="A_M308KM" typeName="BOOL"/>
		<variables name="A_M308MS_K" typeName="EBOOL"/>
		<variables name="A_M308MS_U" typeName="EBOOL"/>
		<variables name="A_M308MS_U_OBR" typeName="BOOL"/>
		<variables name="A_M308MS_U_PR" typeName="BOOL"/>
		<variables name="A_M308MS_UK" typeName="EBOOL"/>
		<variables name="A_M308N" typeName="BOOL"/>
		<variables name="A_M308O_K" typeName="EBOOL"/>
		<variables name="A_MAK_KN" typeName="BOOL"/>
		<variables name="A_MAK_M_IM" typeName="BOOL"/>
		<variables name="A_MAK_P_IM" typeName="BOOL"/>
		<variables name="A_MAK_STR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_MAK_STR_KL" typeName="BOOL"/>
		<variables name="A_MAK_STR_MIG_KL" typeName="BOOL"/>
		<variables name="A_MAK_STR_SrL" typeName="BOOL"/>
		<variables name="A_MAKET_M" typeName="BOOL"/>
		<variables name="A_MAKET_M_ARM_DSP" typeName="BOOL"/>
		<variables name="A_MAKET_M_JL" typeName="BOOL"/>
		<variables name="A_MAKET_M_SrL" typeName="BOOL"/>
		<variables name="A_MAKET_P" typeName="BOOL"/>
		<variables name="A_MAKET_P_ARM_DSP" typeName="BOOL"/>
		<variables name="A_MAKET_P_SrL" typeName="BOOL"/>
		<variables name="A_MAKET_P_ZL" typeName="BOOL"/>
		<variables name="A_MAKET_SR" typeName="BOOL"/>
		<variables name="A_MAKET_SR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_MAKET_SR_KL" typeName="BOOL"/>
		<variables name="A_MAKET_SR_SrL" typeName="BOOL"/>
		<variables name="A_MAN" typeName="BOOL"/>
		<variables name="A_MFI_K" typeName="EBOOL"/>
		<variables name="A_MFS_K" typeName="EBOOL"/>
		<variables name="A_MGOT" typeName="BOOL"/>
		<variables name="A_MI" typeName="BOOL"/>
		<variables name="A_MIV" typeName="BOOL"/>
		<variables name="A_MK_ARM_DSP" typeName="BOOL"/>
		<variables name="A_MK_BL_IM" typeName="BOOL"/>
		<variables name="A_MK_DK_NC_K" typeName="EBOOL"/>
		<variables name="A_MK_DK_NO_K" typeName="EBOOL"/>
		<variables name="A_MK_GOT" typeName="BOOL"/>
		<variables name="A_MK_ik" typeName="BOOL"/>
		<variables name="A_MK_IND" typeName="BYTE"/>
		<variables name="A_MK_KM" typeName="BOOL"/>
		<variables name="A_MK_KS" typeName="BOOL"/>
		<variables name="A_MK_MOT" typeName="BOOL"/>
		<variables name="A_MK_MS" typeName="BOOL"/>
		<variables name="A_MK_MS1" typeName="BOOL"/>
		<variables name="A_MK_MV1" typeName="BOOL"/>
		<variables name="A_MK_N" typeName="BOOL"/>
		<variables name="A_MK_OT" typeName="BOOL"/>
		<variables name="A_MK_OT1" typeName="BOOL"/>
		<variables name="A_MK_S" typeName="BOOL"/>
		<variables name="A_MK_S1" typeName="BOOL"/>
		<variables name="A_MK_S_U" typeName="EBOOL"/>
		<variables name="A_MK_S_U_OBR" typeName="BOOL"/>
		<variables name="A_MK_S_U_PR" typeName="BOOL"/>
		<variables name="A_MK_S_UK" typeName="EBOOL"/>
		<variables name="A_MK_SnL_IM" typeName="BOOL"/>
		<variables name="A_MKI_K" typeName="EBOOL"/>
		<variables name="A_MKI_U" typeName="EBOOL"/>
		<variables name="A_MKI_U_OBR" typeName="BOOL"/>
		<variables name="A_MKI_U_PR" typeName="BOOL"/>
		<variables name="A_MKI_UK" typeName="EBOOL"/>
		<variables name="A_MKKM" typeName="BOOL"/>
		<variables name="A_MKM" typeName="BOOL"/>
		<variables name="A_MKM_K" typeName="EBOOL"/>
		<variables name="A_MKMS_K" typeName="EBOOL"/>
		<variables name="A_MKMS_U" typeName="EBOOL"/>
		<variables name="A_MKMS_U_OBR" typeName="BOOL"/>
		<variables name="A_MKMS_U_PR" typeName="BOOL"/>
		<variables name="A_MKMS_UK" typeName="EBOOL"/>
		<variables name="A_MKN" typeName="BOOL"/>
		<variables name="A_MKO_K" typeName="EBOOL"/>
		<variables name="A_MKS_K" typeName="EBOOL"/>
		<variables name="A_MKS_U" typeName="EBOOL"/>
		<variables name="A_MKS_U_OBR" typeName="BOOL"/>
		<variables name="A_MKS_U_PR" typeName="BOOL"/>
		<variables name="A_MKS_UK" typeName="EBOOL"/>
		<variables name="A_MMK_K" typeName="EBOOL">
			<comment>???????? ???? ???</comment>
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_MMK_U" typeName="EBOOL">
			<comment>?????????? ???? ???</comment>
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_MMNV" typeName="BOOL"/>
		<variables name="A_MMV" typeName="BOOL"/>
		<variables name="A_MN_ARM_DSP" typeName="BOOL"/>
		<variables name="A_MN_BL_IM" typeName="BOOL"/>
		<variables name="A_MN_iK" typeName="BOOL"/>
		<variables name="A_MN_IND" typeName="BYTE"/>
		<variables name="A_MN_KM" typeName="BOOL"/>
		<variables name="A_MN_KS" typeName="BOOL"/>
		<variables name="A_MN_MS" typeName="BOOL"/>
		<variables name="A_MN_MS1" typeName="BOOL"/>
		<variables name="A_MN_N" typeName="BOOL"/>
		<variables name="A_MN_NM" typeName="BOOL"/>
		<variables name="A_MN_OTM" typeName="BOOL"/>
		<variables name="A_MN_S" typeName="BOOL"/>
		<variables name="A_MN_S1" typeName="BOOL"/>
		<variables name="A_MN_SnL_IM" typeName="BOOL"/>
		<variables name="A_MN_UK" typeName="BOOL"/>
		<variables name="A_MN_VNM" typeName="BOOL"/>
		<variables name="A_MNKM" typeName="BOOL"/>
		<variables name="A_MNMS_K" typeName="EBOOL"/>
		<variables name="A_MNMS_U" typeName="EBOOL"/>
		<variables name="A_MNMS_U_OBR" typeName="BOOL"/>
		<variables name="A_MNMS_U_PR" typeName="BOOL"/>
		<variables name="A_MNMS_UK" typeName="EBOOL"/>
		<variables name="A_MNO_K" typeName="EBOOL"/>
		<variables name="A_MON" typeName="BOOL"/>
		<variables name="A_MOPV" typeName="BOOL"/>
		<variables name="A_MOT" typeName="BOOL"/>
		<variables name="A_MPK_K" typeName="EBOOL">
			<comment>???????? ???? ???</comment>
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_MPK_U" typeName="EBOOL">
			<comment>?????????? ???? ???</comment>
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_MPP" typeName="BOOL"/>
		<variables name="A_MPP_K" typeName="EBOOL">
			<comment>???????? ???? ???</comment>
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_MPP_U" typeName="EBOOL">
			<comment>?????????? ???? ???</comment>
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_MPS" typeName="BOOL"/>
		<variables name="A_MPS_1" typeName="BOOL"/>
		<variables name="A_MPS_2" typeName="BOOL"/>
		<variables name="A_MPS_PN" typeName="BOOL"/>
		<variables name="A_MPS_PP" typeName="BOOL"/>
		<variables name="A_MPV" typeName="BOOL"/>
		<variables name="A_MU_1_3" typeName="BOOL"/>
		<variables name="A_MU_4" typeName="BOOL"/>
		<variables name="A_MU_5" typeName="BOOL"/>
		<variables name="A_MU_6" typeName="BOOL"/>
		<variables name="A_MU_8" typeName="BOOL"/>
		<variables name="A_MU_9" typeName="BOOL"/>
		<variables name="A_MU_11_13" typeName="BOOL"/>
		<variables name="A_MU_12" typeName="BOOL"/>
		<variables name="A_MU_14_16" typeName="BOOL"/>
		<variables name="A_MU_15" typeName="BOOL"/>
		<variables name="A_MU_17" typeName="BOOL"/>
		<variables name="A_MU_18" typeName="BOOL"/>
		<variables name="A_MU_19" typeName="BOOL"/>
		<variables name="A_MU_20" typeName="BOOL"/>
		<variables name="A_MU_22_24" typeName="BOOL"/>
		<variables name="A_MU_25" typeName="BOOL"/>
		<variables name="A_MU_26_28" typeName="BOOL"/>
		<variables name="A_MU_27" typeName="BOOL"/>
		<variables name="A_MU_29_31" typeName="BOOL"/>
		<variables name="A_MU_30_34" typeName="BOOL"/>
		<variables name="A_MU_32" typeName="BOOL"/>
		<variables name="A_MU_33_35" typeName="BOOL"/>
		<variables name="A_MU_36" typeName="BOOL"/>
		<variables name="A_MU_37" typeName="BOOL"/>
		<variables name="A_MU_38" typeName="BOOL"/>
		<variables name="A_MU_39" typeName="BOOL"/>
		<variables name="A_MU_41" typeName="BOOL"/>
		<variables name="A_MU_43" typeName="BOOL"/>
		<variables name="A_MU_47" typeName="BOOL"/>
		<variables name="A_MU_49" typeName="BOOL"/>
		<variables name="A_MU_51" typeName="BOOL"/>
		<variables name="A_MU_53" typeName="BOOL"/>
		<variables name="A_MU_84" typeName="BOOL"/>
		<variables name="A_MU_98" typeName="BOOL"/>
		<variables name="A_MU_100" typeName="BOOL"/>
		<variables name="A_MU_101" typeName="BOOL"/>
		<variables name="A_MU_102" typeName="BOOL"/>
		<variables name="A_MU_103" typeName="BOOL"/>
		<variables name="A_MU_104" typeName="BOOL"/>
		<variables name="A_MU_105" typeName="BOOL"/>
		<variables name="A_MU_106" typeName="BOOL"/>
		<variables name="A_MU_107" typeName="BOOL"/>
		<variables name="A_MU_108" typeName="BOOL"/>
		<variables name="A_MU_109" typeName="BOOL"/>
		<variables name="A_MU_110" typeName="BOOL"/>
		<variables name="A_MU_111" typeName="BOOL"/>
		<variables name="A_MU_112" typeName="BOOL"/>
		<variables name="A_MU_113" typeName="BOOL"/>
		<variables name="A_MU_114" typeName="BOOL"/>
		<variables name="A_MU_115" typeName="BOOL"/>
		<variables name="A_MU_116" typeName="BOOL"/>
		<variables name="A_MU_117_119" typeName="BOOL"/>
		<variables name="A_MU_120" typeName="BOOL"/>
		<variables name="A_MU_121" typeName="BOOL"/>
		<variables name="A_MU_122" typeName="BOOL"/>
		<variables name="A_MU_124" typeName="BOOL"/>
		<variables name="A_MU_128" typeName="BOOL"/>
		<variables name="A_MU_130" typeName="BOOL"/>
		<variables name="A_MU_132" typeName="BOOL"/>
		<variables name="A_MU_134" typeName="BOOL"/>
		<variables name="A_MU_136" typeName="BOOL"/>
		<variables name="A_MU_138_140" typeName="BOOL"/>
		<variables name="A_MU_142" typeName="BOOL"/>
		<variables name="A_MU_144" typeName="BOOL"/>
		<variables name="A_MU_146" typeName="BOOL"/>
		<variables name="A_MU_148" typeName="BOOL"/>
		<variables name="A_MU_160" typeName="BOOL"/>
		<variables name="A_MU_162" typeName="BOOL"/>
		<variables name="A_MU_164" typeName="BOOL"/>
		<variables name="A_MU_166" typeName="BOOL"/>
		<variables name="A_MU_201" typeName="BOOL"/>
		<variables name="A_MU_202" typeName="BOOL"/>
		<variables name="A_MU_203" typeName="BOOL"/>
		<variables name="A_MU_204_206" typeName="BOOL"/>
		<variables name="A_MU_208" typeName="BOOL"/>
		<variables name="A_MU_212" typeName="BOOL"/>
		<variables name="A_MU_302" typeName="BOOL"/>
		<variables name="A_MV" typeName="BOOL"/>
		<variables name="A_MV1" typeName="BOOL"/>
		<variables name="A_MZ_ARM_DSP" typeName="BOOL"/>
		<variables name="A_MZ_BL_IM" typeName="BOOL"/>
		<variables name="A_MZ_iK" typeName="BOOL"/>
		<variables name="A_MZ_IND" typeName="BYTE"/>
		<variables name="A_MZ_KL_IM" typeName="BOOL"/>
		<variables name="A_MZ_KM" typeName="BOOL"/>
		<variables name="A_MZ_MS" typeName="BOOL"/>
		<variables name="A_MZ_MS1" typeName="BOOL"/>
		<variables name="A_MZ_N" typeName="BOOL"/>
		<variables name="A_MZ_NM" typeName="BOOL"/>
		<variables name="A_MZ_OTM" typeName="BOOL"/>
		<variables name="A_MZ_S" typeName="BOOL"/>
		<variables name="A_MZ_S1" typeName="BOOL"/>
		<variables name="A_MZ_SnL_IM" typeName="BOOL"/>
		<variables name="A_MZ_UK" typeName="BOOL"/>
		<variables name="A_MZ_VCHM" typeName="BOOL"/>
		<variables name="A_MZ_VN" typeName="BOOL"/>
		<variables name="A_MZKM" typeName="BOOL"/>
		<variables name="A_MZMS_K" typeName="BOOL"/>
		<variables name="A_MZMS_U" typeName="EBOOL">
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_MZMS_U_OBR" typeName="BOOL"/>
		<variables name="A_MZMS_U_PR" typeName="BOOL"/>
		<variables name="A_MZMS_UK" typeName="EBOOL"/>
		<variables name="A_MZO_K" typeName="EBOOL"/>
		<variables name="A_N" typeName="BOOL"/>
		<variables name="A_n_ch" typeName="BOOL"/>
		<variables name="A_n_ch_chm" typeName="BOOL"/>
		<variables name="A_n_ch_nm" typeName="BOOL"/>
		<variables name="A_N_NM" typeName="BOOL"/>
		<variables name="A_NAPR_CH_BL" typeName="BOOL"/>
		<variables name="A_NAPR_CH_FonL" typeName="BOOL"/>
		<variables name="A_NAPR_CH_ZL" typeName="BOOL"/>
		<variables name="A_NAPR_N_BL" typeName="BOOL"/>
		<variables name="A_NAPR_N_FonL" typeName="BOOL"/>
		<variables name="A_NAPR_N_ZL" typeName="BOOL"/>
		<variables name="A_NDFDS_K" typeName="EBOOL">
			<comment>???????? ???? ????</comment>
			<attribute name="TimeStampCapability" value="0"/>
		</variables>
		<variables name="A_ndifp_schet_online" typeName="INT"/>
		<variables name="A_NDKMG_K" typeName="EBOOL">
			<comment>???????? ???? ?????</comment>
			<attribute name="TimeStampCapability" value="0"/>
		</variables>
		<variables name="A_NDMG_NEISPR_IM" typeName="BOOL"/>
		<variables name="A_NDOP_K" typeName="EBOOL">
			<comment>???????? ???? ????</comment>
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_NDPP_IND" typeName="BOOL"/>
		<variables name="A_NDPr_K" typeName="EBOOL">
			<comment>???????? ???? ????</comment>
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_NDPS_IND" typeName="BOOL"/>
		<variables name="A_ndps_schet_online" typeName="INT"/>
		<variables name="A_NDZV_IND" typeName="BOOL"/>
		<variables name="A_NEISPR_SIST" typeName="BOOL"/>
		<variables name="A_NEISPR_SIST_ARM_DSP" typeName="BOOL"/>
		<variables name="A_NFDS_K" typeName="EBOOL">
			<comment>???????? ???? ????</comment>
			<attribute name="TimeStampCapability" value="0"/>
		</variables>
		<variables name="A_NG_CH101_STR103" typeName="WORD"/>
		<variables name="A_NG_CH102_STR115" typeName="WORD"/>
		<variables name="A_NG_CH103_STR115" typeName="WORD"/>
		<variables name="A_NG_CH104_STR113" typeName="WORD"/>
		<variables name="A_NG_CH105_STR111" typeName="WORD"/>
		<variables name="A_NG_CH106_STR109" typeName="WORD"/>
		<variables name="A_NG_CH107_STR109" typeName="WORD"/>
		<variables name="A_NG_CH108_STR107" typeName="WORD"/>
		<variables name="A_NG_CH201_STR203" typeName="WORD"/>
		<variables name="A_NG_CH202_STR203" typeName="WORD"/>
		<variables name="A_NG_CH203_STR201" typeName="WORD"/>
		<variables name="A_NG_M1_MN" typeName="WORD"/>
		<variables name="A_NG_M3_STR1_3" typeName="WORD"/>
		<variables name="A_NG_M5_STR5" typeName="WORD"/>
		<variables name="A_NG_M7_M3" typeName="WORD"/>
		<variables name="A_NG_M7_MZ" typeName="WORD"/>
		<variables name="A_NG_M9_STR9" typeName="WORD"/>
		<variables name="A_NG_M10_M12" typeName="WORD"/>
		<variables name="A_NG_M10_STR6" typeName="WORD"/>
		<variables name="A_NG_M10_STR49" typeName="WORD"/>
		<variables name="A_NG_M11_M5" typeName="WORD"/>
		<variables name="A_NG_M12_STR84" typeName="WORD"/>
		<variables name="A_NG_M13_M3" typeName="WORD"/>
		<variables name="A_NG_M14_M104" typeName="WORD"/>
		<variables name="A_NG_M15_STR19" typeName="WORD"/>
		<variables name="A_NG_M16_M124" typeName="WORD"/>
		<variables name="A_NG_M17_STR19" typeName="WORD"/>
		<variables name="A_NG_M18_STR6" typeName="WORD"/>
		<variables name="A_NG_M19_O1_STR17" typeName="WORD"/>
		<variables name="A_NG_M20_M22" typeName="WORD"/>
		<variables name="A_NG_M21_O1_STR25" typeName="WORD"/>
		<variables name="A_NG_M22_STR8" typeName="WORD"/>
		<variables name="A_NG_M23_M19" typeName="WORD"/>
		<variables name="A_NG_M25_M21" typeName="WORD"/>
		<variables name="A_NG_M26_M30" typeName="WORD"/>
		<variables name="A_NG_M27_STR_29_31" typeName="WORD"/>
		<variables name="A_NG_M28_M34" typeName="WORD"/>
		<variables name="A_NG_M29_M27" typeName="WORD"/>
		<variables name="A_NG_M30_STR12" typeName="WORD"/>
		<variables name="A_NG_M32_M36" typeName="WORD"/>
		<variables name="A_NG_M33_STR41" typeName="WORD"/>
		<variables name="A_NG_M34_STR14_16" typeName="WORD"/>
		<variables name="A_NG_M35_STR37" typeName="WORD"/>
		<variables name="A_NG_M36_STR14_16" typeName="WORD"/>
		<variables name="A_NG_M37_STR43" typeName="WORD"/>
		<variables name="A_NG_M37_STR47" typeName="WORD"/>
		<variables name="A_NG_M38_STR102" typeName="WORD"/>
		<variables name="A_NG_M39_STR43" typeName="WORD"/>
		<variables name="A_NG_M40_STR18" typeName="WORD"/>
		<variables name="A_NG_M41_STR39" typeName="WORD"/>
		<variables name="A_NG_M42_STR20" typeName="WORD"/>
		<variables name="A_NG_M43_STR39" typeName="WORD"/>
		<variables name="A_NG_M44_M42" typeName="WORD"/>
		<variables name="A_NG_M45_STR47" typeName="WORD"/>
		<variables name="A_NG_M46_STR36" typeName="WORD"/>
		<variables name="A_NG_M47_STR49" typeName="WORD"/>
		<variables name="A_NG_M48_STR36" typeName="WORD"/>
		<variables name="A_NG_M49_STR49" typeName="WORD"/>
		<variables name="A_NG_M50_M68" typeName="WORD"/>
		<variables name="A_NG_M50_STR26_28" typeName="WORD"/>
		<variables name="A_NG_M52_M70" typeName="WORD"/>
		<variables name="A_NG_M52_STR22_24" typeName="WORD"/>
		<variables name="A_NG_M53_M41" typeName="WORD"/>
		<variables name="A_NG_M54_M56" typeName="WORD"/>
		<variables name="A_NG_M54_STR22_24" typeName="WORD"/>
		<variables name="A_NG_M55_STR51" typeName="WORD"/>
		<variables name="A_NG_M56_STR32" typeName="WORD"/>
		<variables name="A_NG_M57_M55" typeName="WORD"/>
		<variables name="A_NG_M58_STR32" typeName="WORD"/>
		<variables name="A_NG_M59_STR53" typeName="WORD"/>
		<variables name="A_NG_M60_M58" typeName="WORD"/>
		<variables name="A_NG_M61_STR53" typeName="WORD"/>
		<variables name="A_NG_M62_STR26_28" typeName="WORD"/>
		<variables name="A_NG_M63_STR33_35" typeName="WORD"/>
		<variables name="A_NG_M64_STR38" typeName="WORD"/>
		<variables name="A_NG_M65_STR15" typeName="WORD"/>
		<variables name="A_NG_M66_M64" typeName="WORD"/>
		<variables name="A_NG_M66_STR26_28" typeName="WORD"/>
		<variables name="A_NG_M67_M65" typeName="WORD"/>
		<variables name="A_NG_M68_M76" typeName="WORD"/>
		<variables name="A_NG_M69_STR27" typeName="WORD"/>
		<variables name="A_NG_M70_M78" typeName="WORD"/>
		<variables name="A_NG_M72_M61" typeName="WORD"/>
		<variables name="A_NG_M72_STR38" typeName="WORD"/>
		<variables name="A_NG_M74_M43" typeName="WORD"/>
		<variables name="A_NG_M76_M35" typeName="WORD"/>
		<variables name="A_NG_M78_M33" typeName="WORD"/>
		<variables name="A_NG_M80_STR100" typeName="WORD"/>
		<variables name="A_NG_M82_STR148" typeName="WORD"/>
		<variables name="A_NG_M97_M107" typeName="WORD"/>
		<variables name="A_NG_M100_M37" typeName="WORD"/>
		<variables name="A_NG_M101_STR107" typeName="WORD"/>
		<variables name="A_NG_M102_STR110" typeName="WORD"/>
		<variables name="A_NG_M103_STR117_119" typeName="WORD"/>
		<variables name="A_NG_M104_STR138_140" typeName="WORD"/>
		<variables name="A_NG_M105_M103" typeName="WORD"/>
		<variables name="A_NG_M105_STR103" typeName="WORD"/>
		<variables name="A_NG_M106_STR110" typeName="WORD"/>
		<variables name="A_NG_M106_STR112" typeName="WORD"/>
		<variables name="A_NG_M107_STR117_119" typeName="WORD"/>
		<variables name="A_NG_M109_M69" typeName="WORD"/>
		<variables name="A_NG_M120_STR162" typeName="WORD"/>
		<variables name="A_NG_M121_M105" typeName="WORD"/>
		<variables name="A_NG_M122_STR120" typeName="WORD"/>
		<variables name="A_NG_M124_STR124" typeName="WORD"/>
		<variables name="A_NG_M126_M130" typeName="WORD"/>
		<variables name="A_NG_M126_STR124" typeName="WORD"/>
		<variables name="A_NG_M128_M132" typeName="WORD"/>
		<variables name="A_NG_M128_O1_STR138_140" typeName="WORD"/>
		<variables name="A_NG_M130_STR116" typeName="WORD"/>
		<variables name="A_NG_M132_STR116" typeName="WORD"/>
		<variables name="A_NG_M134_STR136" typeName="WORD"/>
		<variables name="A_NG_M136_M172" typeName="WORD"/>
		<variables name="A_NG_M142_M158" typeName="WORD"/>
		<variables name="A_NG_M144_M162" typeName="WORD"/>
		<variables name="A_NG_M146_M80" typeName="WORD"/>
		<variables name="A_NG_M148_M82" typeName="WORD"/>
		<variables name="A_NG_M150_M160" typeName="WORD"/>
		<variables name="A_NG_M152_STR160" typeName="WORD"/>
		<variables name="A_NG_M154_M152" typeName="WORD"/>
		<variables name="A_NG_M154_STR122" typeName="WORD"/>
		<variables name="A_NG_M160_STR142" typeName="WORD"/>
		<variables name="A_NG_M162_STR166" typeName="WORD"/>
		<variables name="A_NG_M164_STR100" typeName="WORD"/>
		<variables name="A_NG_M164_STR136" typeName="WORD"/>
		<variables name="A_NG_M166_M168" typeName="WORD"/>
		<variables name="A_NG_M168_STR162" typeName="WORD"/>
		<variables name="A_NG_M172_STR160" typeName="WORD"/>
		<variables name="A_NG_M174_M176" typeName="WORD"/>
		<variables name="A_NG_M176_STR162" typeName="WORD"/>
		<variables name="A_NG_M201_STR5" typeName="WORD"/>
		<variables name="A_NG_M202_STR202" typeName="WORD"/>
		<variables name="A_NG_M203_M201" typeName="WORD"/>
		<variables name="A_NG_M204_STR202" typeName="WORD"/>
		<variables name="A_NG_M206_M210" typeName="WORD"/>
		<variables name="A_NG_M208_STR204_206" typeName="WORD"/>
		<variables name="A_NG_M210_STR212" typeName="WORD"/>
		<variables name="A_NG_M218_STR212" typeName="WORD"/>
		<variables name="A_NG_M302_STR302" typeName="WORD"/>
		<variables name="A_NG_M304_M62" typeName="WORD"/>
		<variables name="A_NG_M306_M308" typeName="WORD"/>
		<variables name="A_NG_M308_STR12" typeName="WORD"/>
		<variables name="A_NG_MK_M302" typeName="WORD"/>
		<variables name="A_NG_STR1_3_M1" typeName="WORD"/>
		<variables name="A_NG_STR1_3_M7" typeName="WORD"/>
		<variables name="A_NG_STR4_M16" typeName="WORD"/>
		<variables name="A_NG_STR4_STR120" typeName="WORD"/>
		<variables name="A_NG_STR5_STR1_3" typeName="WORD"/>
		<variables name="A_NG_STR6_M10" typeName="WORD"/>
		<variables name="A_NG_STR6_M20" typeName="WORD"/>
		<variables name="A_NG_STR8_M26" typeName="WORD"/>
		<variables name="A_NG_STR8_M28" typeName="WORD"/>
		<variables name="A_NG_STR9_M11" typeName="WORD"/>
		<variables name="A_NG_STR11_13_STR9" typeName="WORD"/>
		<variables name="A_NG_STR11_13_STR15" typeName="WORD"/>
		<variables name="A_NG_STR12_M32" typeName="WORD"/>
		<variables name="A_NG_STR12_M308" typeName="WORD"/>
		<variables name="A_NG_STR14_16_M40" typeName="WORD"/>
		<variables name="A_NG_STR14_16_M42" typeName="WORD"/>
		<variables name="A_NG_STR17_M19" typeName="WORD"/>
		<variables name="A_NG_STR17_STR11_13" typeName="WORD"/>
		<variables name="A_NG_STR18_STR22_24" typeName="WORD"/>
		<variables name="A_NG_STR18_STR26_28" typeName="WORD"/>
		<variables name="A_NG_STR19_STR17" typeName="WORD"/>
		<variables name="A_NG_STR20_STR22_24" typeName="WORD"/>
		<variables name="A_NG_STR20_STR30_34" typeName="WORD"/>
		<variables name="A_NG_STR22_24_M52" typeName="WORD"/>
		<variables name="A_NG_STR22_24_M54" typeName="WORD"/>
		<variables name="A_NG_STR25_M13" typeName="WORD"/>
		<variables name="A_NG_STR25_M21" typeName="WORD"/>
		<variables name="A_NG_STR25_STR11_13" typeName="WORD"/>
		<variables name="A_NG_STR26_28_M50" typeName="WORD"/>
		<variables name="A_NG_STR27_STR25" typeName="WORD"/>
		<variables name="A_NG_STR29_31_M23" typeName="WORD"/>
		<variables name="A_NG_STR29_31_M25" typeName="WORD"/>
		<variables name="A_NG_STR30_34_M60" typeName="WORD"/>
		<variables name="A_NG_STR32_M58" typeName="WORD"/>
		<variables name="A_NG_STR32_M100" typeName="WORD"/>
		<variables name="A_NG_STR33_35_M29" typeName="WORD"/>
		<variables name="A_NG_STR33_35_STR_29_31" typeName="WORD"/>
		<variables name="A_NG_STR36_STR30_34" typeName="WORD"/>
		<variables name="A_NG_STR37_M63" typeName="WORD"/>
		<variables name="A_NG_STR38_M64" typeName="WORD"/>
		<variables name="A_NG_STR38_M72" typeName="WORD"/>
		<variables name="A_NG_STR38_M74" typeName="WORD"/>
		<variables name="A_NG_STR39_STR37" typeName="WORD"/>
		<variables name="A_NG_STR41_STR33_35" typeName="WORD"/>
		<variables name="A_NG_STR43_STR41" typeName="WORD"/>
		<variables name="A_NG_STR47_M39" typeName="WORD"/>
		<variables name="A_NG_STR49_STR47" typeName="WORD"/>
		<variables name="A_NG_STR51_M51" typeName="WORD"/>
		<variables name="A_NG_STR51_M53" typeName="WORD"/>
		<variables name="A_NG_STR53_M57" typeName="WORD"/>
		<variables name="A_NG_STR84_M84" typeName="WORD"/>
		<variables name="A_NG_STR84_STR4" typeName="WORD"/>
		<variables name="A_NG_STR98_M142" typeName="WORD"/>
		<variables name="A_NG_STR98_M156" typeName="WORD"/>
		<variables name="A_NG_STR100_M118" typeName="WORD"/>
		<variables name="A_NG_STR101_M109" typeName="WORD"/>
		<variables name="A_NG_STR101_STR103" typeName="WORD"/>
		<variables name="A_NG_STR101_STR121" typeName="WORD"/>
		<variables name="A_NG_STR102_M14" typeName="WORD"/>
		<variables name="A_NG_STR102_M38" typeName="WORD"/>
		<variables name="A_NG_STR103_STR101" typeName="WORD"/>
		<variables name="A_NG_STR104_STR98" typeName="WORD"/>
		<variables name="A_NG_STR104_STR106" typeName="WORD"/>
		<variables name="A_NG_STR106_M134" typeName="WORD"/>
		<variables name="A_NG_STR106_STR108" typeName="WORD"/>
		<variables name="A_NG_STR107_STR121" typeName="WORD"/>
		<variables name="A_NG_STR108_M138" typeName="WORD"/>
		<variables name="A_NG_STR108_STR122" typeName="WORD"/>
		<variables name="A_NG_STR109_STR105" typeName="WORD"/>
		<variables name="A_NG_STR110_STR112" typeName="WORD"/>
		<variables name="A_NG_STR110_STR114" typeName="WORD"/>
		<variables name="A_NG_STR111_STR105" typeName="WORD"/>
		<variables name="A_NG_STR112_M106" typeName="WORD"/>
		<variables name="A_NG_STR112_M108" typeName="WORD"/>
		<variables name="A_NG_STR112_STR110" typeName="WORD"/>
		<variables name="A_NG_STR113_STR111" typeName="WORD"/>
		<variables name="A_NG_STR114_M110" typeName="WORD"/>
		<variables name="A_NG_STR114_STR142" typeName="WORD"/>
		<variables name="A_NG_STR115_STR113" typeName="WORD"/>
		<variables name="A_NG_STR116_M102" typeName="WORD"/>
		<variables name="A_NG_STR117_119_M67" typeName="WORD"/>
		<variables name="A_NG_STR117_119_STR27" typeName="WORD"/>
		<variables name="A_NG_STR120_STR102" typeName="WORD"/>
		<variables name="A_NG_STR122_M154" typeName="WORD"/>
		<variables name="A_NG_STR124_M126" typeName="WORD"/>
		<variables name="A_NG_STR124_STR138_140" typeName="WORD"/>
		<variables name="A_NG_STR128_M148" typeName="WORD"/>
		<variables name="A_NG_STR128_STR134" typeName="WORD"/>
		<variables name="A_NG_STR130_STR128" typeName="WORD"/>
		<variables name="A_NG_STR130_STR132" typeName="WORD"/>
		<variables name="A_NG_STR130_STR138_140" typeName="WORD"/>
		<variables name="A_NG_STR132_M144" typeName="WORD"/>
		<variables name="A_NG_STR132_STR164" typeName="WORD"/>
		<variables name="A_NG_STR134_M148" typeName="WORD"/>
		<variables name="A_NG_STR134_M150" typeName="WORD"/>
		<variables name="A_NG_STR136_M140" typeName="WORD"/>
		<variables name="A_NG_STR138_140_M128" typeName="WORD"/>
		<variables name="A_NG_STR138_140_STR130" typeName="WORD"/>
		<variables name="A_NG_STR142_STR144" typeName="WORD"/>
		<variables name="A_NG_STR144_M112" typeName="WORD"/>
		<variables name="A_NG_STR144_STR146" typeName="WORD"/>
		<variables name="A_NG_STR146_M114" typeName="WORD"/>
		<variables name="A_NG_STR146_STR148" typeName="WORD"/>
		<variables name="A_NG_STR148_M116" typeName="WORD"/>
		<variables name="A_NG_STR160_M174" typeName="WORD"/>
		<variables name="A_NG_STR162_M120" typeName="WORD"/>
		<variables name="A_NG_STR164_M164" typeName="WORD"/>
		<variables name="A_NG_STR166_M164" typeName="WORD"/>
		<variables name="A_NG_STR166_M166" typeName="WORD"/>
		<variables name="A_NG_STR201_M203" typeName="WORD"/>
		<variables name="A_NG_STR202_M206" typeName="WORD"/>
		<variables name="A_NG_STR203_STR201" typeName="WORD"/>
		<variables name="A_NG_STR204_206_M212" typeName="WORD"/>
		<variables name="A_NG_STR204_206_STR208" typeName="WORD"/>
		<variables name="A_NG_STR208_M214" typeName="WORD"/>
		<variables name="A_NG_STR208_M216" typeName="WORD"/>
		<variables name="A_NG_STR212_STR204_206" typeName="WORD"/>
		<variables name="A_NG_STR302_M304" typeName="WORD"/>
		<variables name="A_NG_STR302_M306" typeName="WORD"/>
		<variables name="A_nifp_schet_online" typeName="INT"/>
		<variables name="A_NIR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_NKJH_K" typeName="EBOOL">
			<comment>???????? ?????-????? ????</comment>
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_NKJT_K" typeName="EBOOL">
			<comment>???????? ?????-????? ???T</comment>
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_NKMG_K" typeName="EBOOL">
			<comment>???????? ???? ????</comment>
			<attribute name="TimeStampCapability" value="0"/>
		</variables>
		<variables name="A_NKS" typeName="BOOL"/>
		<variables name="A_nm" typeName="BOOL"/>
		<variables name="A_NMG_NEISPR_IM" typeName="BOOL"/>
		<variables name="A_NN_213" typeName="BOOL"/>
		<variables name="A_NN_JL" typeName="BOOL"/>
		<variables name="A_NOP_K" typeName="EBOOL">
			<comment>???????? ???? ???</comment>
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_NP_IND" typeName="BYTE"/>
		<variables name="A_NP_K" typeName="EBOOL"/>
		<variables name="A_NPI13_K" typeName="EBOOL"/>
		<variables name="A_NPI13_U" typeName="EBOOL"/>
		<variables name="A_NPI13_U_OBR" typeName="BOOL"/>
		<variables name="A_NPI13_U_PR" typeName="BOOL"/>
		<variables name="A_NPI13_UK" typeName="EBOOL"/>
		<variables name="A_NPP_IND" typeName="BOOL"/>
		<variables name="A_NPr_K" typeName="EBOOL">
			<comment>???????? ???? ???</comment>
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_NPS_IND" typeName="BOOL"/>
		<variables name="A_nps_schet_online" typeName="INT"/>
		<variables name="A_NRI" typeName="BOOL"/>
		<variables name="A_NVK" typeName="BOOL"/>
		<variables name="A_NZ" typeName="BOOL"/>
		<variables name="A_NZ108_1JL_IM" typeName="BOOL"/>
		<variables name="A_NZ108_2JL_IM" typeName="BOOL"/>
		<variables name="A_NZ108_BL_IM" typeName="BOOL"/>
		<variables name="A_NZ108_KL_IM" typeName="BOOL"/>
		<variables name="A_NZ108_ZL_IM" typeName="BOOL"/>
		<variables name="A_NZ108A_1JL_IM" typeName="BOOL"/>
		<variables name="A_NZ108A_2JL_IM" typeName="BOOL"/>
		<variables name="A_NZ108A_BL_IM" typeName="BOOL"/>
		<variables name="A_NZ108A_KL_IM" typeName="BOOL"/>
		<variables name="A_NZ108A_ZL_IM" typeName="BOOL"/>
		<variables name="A_NZ108AMS_K" typeName="EBOOL"/>
		<variables name="A_NZ108AMS_U" typeName="EBOOL"/>
		<variables name="A_NZ108AMS_U_OBR" typeName="BOOL"/>
		<variables name="A_NZ108AMS_U_PR" typeName="BOOL"/>
		<variables name="A_NZ108AMS_UK" typeName="EBOOL"/>
		<variables name="A_NZ108MS_K" typeName="EBOOL"/>
		<variables name="A_NZ108MS_U" typeName="EBOOL"/>
		<variables name="A_NZ108MS_U_OBR" typeName="BOOL"/>
		<variables name="A_NZ108MS_U_PR" typeName="BOOL"/>
		<variables name="A_NZ108MS_UK" typeName="EBOOL"/>
		<variables name="A_NZ109A_1JL_IM" typeName="BOOL"/>
		<variables name="A_NZ109A_2JL_IM" typeName="BOOL"/>
		<variables name="A_NZ109A_BL_IM" typeName="BOOL"/>
		<variables name="A_NZ109A_KL_IM" typeName="BOOL"/>
		<variables name="A_NZ109A_ZL_IM" typeName="BOOL"/>
		<variables name="A_NZ109MS_K" typeName="EBOOL"/>
		<variables name="A_NZ109MS_U" typeName="EBOOL"/>
		<variables name="A_NZ109MS_U_OBR" typeName="BOOL"/>
		<variables name="A_NZ109MS_U_PR" typeName="BOOL"/>
		<variables name="A_NZ109MS_UK" typeName="EBOOL"/>
		<variables name="A_NZKJH_K" typeName="EBOOL">
			<comment>???????? ???? ?????</comment>
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_NZV_IND" typeName="BOOL"/>
		<variables name="A_O_KL" typeName="BOOL"/>
		<variables name="A_O_MIG_KL" typeName="BOOL"/>
		<variables name="A_O_SrL" typeName="BOOL"/>
		<variables name="A_obnulenie_gir" typeName="BOOL"/>
		<variables name="A_obnulenie_grk" typeName="BOOL"/>
		<variables name="A_OG" typeName="BOOL"/>
		<variables name="A_OG_ARM_DSP" typeName="BOOL"/>
		<variables name="A_OG_KN" typeName="BOOL"/>
		<variables name="A_OK" typeName="BOOL"/>
		<variables name="A_OK_S_U" typeName="EBOOL"/>
		<variables name="A_OK_S_U_OBR" typeName="BOOL"/>
		<variables name="A_OK_S_U_PR" typeName="BOOL"/>
		<variables name="A_OK_S_UK" typeName="EBOOL"/>
		<variables name="A_om" typeName="BOOL"/>
		<variables name="A_OM_KL" typeName="BOOL"/>
		<variables name="A_OM_MIG_KL" typeName="BOOL"/>
		<variables name="A_OM_SrL" typeName="BOOL"/>
		<variables name="A_ON1" typeName="BOOL"/>
		<variables name="A_ON_ARM_DSP" typeName="BOOL"/>
		<variables name="A_ONM" typeName="BOOL"/>
		<variables name="A_OP_K" typeName="EBOOL">
			<comment>???????? ??</comment>
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_OP_KL" typeName="BOOL"/>
		<variables name="A_OP_MIG_KL" typeName="BOOL"/>
		<variables name="A_OP_SrL" typeName="BOOL"/>
		<variables name="A_OS_KL" typeName="BOOL"/>
		<variables name="A_OS_MIG_KL" typeName="BOOL"/>
		<variables name="A_OS_SrL" typeName="BOOL"/>
		<variables name="A_OUT_SHU4_43_8_RESET_CTU" typeName="BOOL"/>
		<variables name="A_OV" typeName="BOOL"/>
		<variables name="A_P" typeName="BOOL"/>
		<variables name="A_P101_AKN_MAN" typeName="BOOL"/>
		<variables name="A_P101_CHI" typeName="BOOL"/>
		<variables name="A_P101_CHIP" typeName="BOOL"/>
		<variables name="A_P101_NI" typeName="BOOL"/>
		<variables name="A_P101_NIP" typeName="BOOL"/>
		<variables name="A_P102_AKN_MAN" typeName="BOOL"/>
		<variables name="A_P102_CHI" typeName="BOOL"/>
		<variables name="A_P102_CHIP" typeName="BOOL"/>
		<variables name="A_P102_NI" typeName="BOOL"/>
		<variables name="A_P102_NIP" typeName="BOOL"/>
		<variables name="A_P103_AKN_MAN" typeName="BOOL"/>
		<variables name="A_P103_CHI" typeName="BOOL"/>
		<variables name="A_P103_CHIP" typeName="BOOL"/>
		<variables name="A_P103_NI" typeName="BOOL"/>
		<variables name="A_P103_NIP" typeName="BOOL"/>
		<variables name="A_P104_AKN_MAN" typeName="BOOL"/>
		<variables name="A_P104_CHI" typeName="BOOL"/>
		<variables name="A_P104_CHIP" typeName="BOOL"/>
		<variables name="A_P104_NI" typeName="BOOL"/>
		<variables name="A_P104_NIP" typeName="BOOL"/>
		<variables name="A_P105_AKN_MAN" typeName="BOOL"/>
		<variables name="A_P105_CHI" typeName="BOOL"/>
		<variables name="A_P105_CHIP" typeName="BOOL"/>
		<variables name="A_P105_NI" typeName="BOOL"/>
		<variables name="A_P105_NIP" typeName="BOOL"/>
		<variables name="A_P106_AKN_MAN" typeName="BOOL"/>
		<variables name="A_P106_CHI" typeName="BOOL"/>
		<variables name="A_P106_CHIP" typeName="BOOL"/>
		<variables name="A_P106_NI" typeName="BOOL"/>
		<variables name="A_P106_NIP" typeName="BOOL"/>
		<variables name="A_P107_CHI" typeName="BOOL"/>
		<variables name="A_P107_CHIP" typeName="BOOL"/>
		<variables name="A_P107_NI" typeName="BOOL"/>
		<variables name="A_P107_NIP" typeName="BOOL"/>
		<variables name="A_P108_AKN_MAN" typeName="BOOL"/>
		<variables name="A_P108_CHI" typeName="BOOL"/>
		<variables name="A_P108_CHIP" typeName="BOOL"/>
		<variables name="A_P108_NI" typeName="BOOL"/>
		<variables name="A_P108_NIP" typeName="BOOL"/>
		<variables name="A_P201_AKN_MAN" typeName="BOOL"/>
		<variables name="A_P201_CHI" typeName="BOOL"/>
		<variables name="A_P201_CHIP" typeName="BOOL"/>
		<variables name="A_P201_NI" typeName="BOOL"/>
		<variables name="A_P201_NIP" typeName="BOOL"/>
		<variables name="A_P202_AKN_MAN" typeName="BOOL"/>
		<variables name="A_P202_CHI" typeName="BOOL"/>
		<variables name="A_P202_CHIP" typeName="BOOL"/>
		<variables name="A_P202_NI" typeName="BOOL"/>
		<variables name="A_P202_NIP" typeName="BOOL"/>
		<variables name="A_P203_AKN_MAN" typeName="BOOL"/>
		<variables name="A_P203_CHI" typeName="BOOL"/>
		<variables name="A_P203_CHIP" typeName="BOOL"/>
		<variables name="A_P203_NI" typeName="BOOL"/>
		<variables name="A_P203_NIP" typeName="BOOL"/>
		<variables name="A_pb" typeName="BOOL"/>
		<variables name="A_PCHDKMG_K" typeName="EBOOL">
			<comment>???????? ???? ??????</comment>
			<attribute name="TimeStampCapability" value="0"/>
		</variables>
		<variables name="A_PCHDMG_NEISPR_IM" typeName="BOOL"/>
		<variables name="A_PCHKMG_K" typeName="EBOOL">
			<comment>???????? ???? ?????</comment>
			<attribute name="TimeStampCapability" value="0"/>
		</variables>
		<variables name="A_PCHMG_NEISPR_IM" typeName="BOOL"/>
		<variables name="A_PER_IZ1" typeName="BOOL"/>
		<variables name="A_PER_IZ2" typeName="BOOL"/>
		<variables name="A_PER_IZ3" typeName="BOOL"/>
		<variables name="A_PER_IZ4" typeName="BOOL"/>
		<variables name="A_PER_IZ5" typeName="BOOL"/>
		<variables name="A_PER_IZ6" typeName="BOOL"/>
		<variables name="A_PER_IZ7" typeName="BOOL"/>
		<variables name="A_PER_PR_KL" typeName="BOOL"/>
		<variables name="A_PER_PR_SrL" typeName="BOOL"/>
		<variables name="A_PER_PR_ST11P10_KL" typeName="BOOL"/>
		<variables name="A_PER_PR_ST11P10_SrL" typeName="BOOL"/>
		<variables name="A_PER_PR_ST11P11_KL" typeName="BOOL"/>
		<variables name="A_PER_PR_ST11P11_SrL" typeName="BOOL"/>
		<variables name="A_PER_PR_ST12P7_KL" typeName="BOOL"/>
		<variables name="A_PER_PR_ST12P7_SrL" typeName="BOOL"/>
		<variables name="A_PER_PR_ST12P8_KL" typeName="BOOL"/>
		<variables name="A_PER_PR_ST12P8_SrL" typeName="BOOL"/>
		<variables name="A_PER_PR_ST12P9_KL" typeName="BOOL"/>
		<variables name="A_PER_PR_ST12P9_SrL" typeName="BOOL"/>
		<variables name="A_PER_PR_ST12P10_KL" typeName="BOOL"/>
		<variables name="A_PER_PR_ST12P10_SrL" typeName="BOOL"/>
		<variables name="A_PER_PR_ST12P11_KL" typeName="BOOL"/>
		<variables name="A_PER_PR_ST12P11_SrL" typeName="BOOL"/>
		<variables name="A_PER_PR_ST12P12_KL" typeName="BOOL"/>
		<variables name="A_PER_PR_ST12P12_SrL" typeName="BOOL"/>
		<variables name="A_PER_PR_ST12P13_KL" typeName="BOOL"/>
		<variables name="A_PER_PR_ST12P13_SrL" typeName="BOOL"/>
		<variables name="A_PER_PR_ST13P10_KL" typeName="BOOL"/>
		<variables name="A_PER_PR_ST13P10_SrL" typeName="BOOL"/>
		<variables name="A_PER_PR_ST13P11_KL" typeName="BOOL"/>
		<variables name="A_PER_PR_ST13P11_SrL" typeName="BOOL"/>
		<variables name="A_PER_PR_ST22P9_KL" typeName="BOOL"/>
		<variables name="A_PER_PR_ST22P9_SrL" typeName="BOOL"/>
		<variables name="A_PER_PR_ST22P10_KL" typeName="BOOL"/>
		<variables name="A_PER_PR_ST22P10_SrL" typeName="BOOL"/>
		<variables name="A_pg" typeName="BOOL"/>
		<variables name="A_PI10_K" typeName="EBOOL"/>
		<variables name="A_PI10_U" typeName="EBOOL"/>
		<variables name="A_PI10_U_OBR" typeName="BOOL"/>
		<variables name="A_PI10_U_PR" typeName="BOOL"/>
		<variables name="A_PI10_UK" typeName="EBOOL"/>
		<variables name="A_PIT" typeName="WORD"/>
		<variables name="A_PIT_KONTR_PR_KL" typeName="BOOL"/>
		<variables name="A_PIT_KONTR_PR_SrL" typeName="BOOL"/>
		<variables name="A_PIV" typeName="BOOL"/>
		<variables name="A_pk" typeName="BOOL"/>
		<variables name="A_PLC_TIME_RUN" typeName="DINT"/>
		<variables name="A_PM" typeName="BOOL"/>
		<variables name="A_PMV" typeName="BOOL"/>
		<variables name="A_pn" typeName="BOOL"/>
		<variables name="A_PNDKMG_K" typeName="EBOOL">
			<comment>???????? ???? ??????</comment>
			<attribute name="TimeStampCapability" value="0"/>
		</variables>
		<variables name="A_PNDMG_NEISPR_IM" typeName="BOOL"/>
		<variables name="A_PNKMG_K" typeName="EBOOL">
			<comment>???????? ???? ?????</comment>
			<attribute name="TimeStampCapability" value="0"/>
		</variables>
		<variables name="A_PNMG_NEISPR_IM" typeName="BOOL"/>
		<variables name="A_PODSV_STR_N_ARM_DSP" typeName="BOOL"/>
		<variables name="A_POEZD" typeName="BOOL"/>
		<variables name="A_POLKA1_SH_DIAGN" typeName="WORD"/>
		<variables name="A_POLKA2_SH_DIAGN" typeName="WORD"/>
		<variables name="A_POLKA3_SH_DIAGN" typeName="WORD"/>
		<variables name="A_POLKA4_SH_DIAGN" typeName="WORD"/>
		<variables name="A_POV" typeName="BOOL"/>
		<variables name="A_POWER_K" typeName="EBOOL"/>
		<variables name="A_POWER_U" typeName="EBOOL"/>
		<variables name="A_POWER_U_OBR" typeName="BOOL"/>
		<variables name="A_POWER_U_PR" typeName="BOOL"/>
		<variables name="A_POWER_UK" typeName="EBOOL"/>
		<variables name="A_PPV" typeName="BOOL"/>
		<variables name="A_PU_1_3" typeName="BOOL"/>
		<variables name="A_PU_4" typeName="BOOL"/>
		<variables name="A_PU_5" typeName="BOOL"/>
		<variables name="A_PU_6" typeName="BOOL"/>
		<variables name="A_PU_8" typeName="BOOL"/>
		<variables name="A_PU_9" typeName="BOOL"/>
		<variables name="A_PU_11_13" typeName="BOOL"/>
		<variables name="A_PU_12" typeName="BOOL"/>
		<variables name="A_PU_14_16" typeName="BOOL"/>
		<variables name="A_PU_15" typeName="BOOL"/>
		<variables name="A_PU_17" typeName="BOOL"/>
		<variables name="A_PU_18" typeName="BOOL"/>
		<variables name="A_PU_19" typeName="BOOL"/>
		<variables name="A_PU_20" typeName="BOOL"/>
		<variables name="A_PU_22_24" typeName="BOOL"/>
		<variables name="A_PU_25" typeName="BOOL"/>
		<variables name="A_PU_26_28" typeName="BOOL"/>
		<variables name="A_PU_27" typeName="BOOL"/>
		<variables name="A_PU_29_31" typeName="BOOL"/>
		<variables name="A_PU_30_34" typeName="BOOL"/>
		<variables name="A_PU_32" typeName="BOOL"/>
		<variables name="A_PU_33_35" typeName="BOOL"/>
		<variables name="A_PU_36" typeName="BOOL"/>
		<variables name="A_PU_37" typeName="BOOL"/>
		<variables name="A_PU_38" typeName="BOOL"/>
		<variables name="A_PU_39" typeName="BOOL"/>
		<variables name="A_PU_41" typeName="BOOL"/>
		<variables name="A_PU_43" typeName="BOOL"/>
		<variables name="A_PU_47" typeName="BOOL"/>
		<variables name="A_PU_49" typeName="BOOL"/>
		<variables name="A_PU_51" typeName="BOOL"/>
		<variables name="A_PU_53" typeName="BOOL"/>
		<variables name="A_PU_84" typeName="BOOL"/>
		<variables name="A_PU_98" typeName="BOOL"/>
		<variables name="A_PU_100" typeName="BOOL"/>
		<variables name="A_PU_101" typeName="BOOL"/>
		<variables name="A_PU_102" typeName="BOOL"/>
		<variables name="A_PU_103" typeName="BOOL"/>
		<variables name="A_PU_104" typeName="BOOL"/>
		<variables name="A_PU_105" typeName="BOOL"/>
		<variables name="A_PU_106" typeName="BOOL"/>
		<variables name="A_PU_107" typeName="BOOL"/>
		<variables name="A_PU_108" typeName="BOOL"/>
		<variables name="A_PU_109" typeName="BOOL"/>
		<variables name="A_PU_110" typeName="BOOL"/>
		<variables name="A_PU_111" typeName="BOOL"/>
		<variables name="A_PU_112" typeName="BOOL"/>
		<variables name="A_PU_113" typeName="BOOL"/>
		<variables name="A_PU_114" typeName="BOOL"/>
		<variables name="A_PU_115" typeName="BOOL"/>
		<variables name="A_PU_116" typeName="BOOL"/>
		<variables name="A_PU_117_119" typeName="BOOL"/>
		<variables name="A_PU_120" typeName="BOOL"/>
		<variables name="A_PU_121" typeName="BOOL"/>
		<variables name="A_PU_122" typeName="BOOL"/>
		<variables name="A_PU_124" typeName="BOOL"/>
		<variables name="A_PU_128" typeName="BOOL"/>
		<variables name="A_PU_130" typeName="BOOL"/>
		<variables name="A_PU_132" typeName="BOOL"/>
		<variables name="A_PU_134" typeName="BOOL"/>
		<variables name="A_PU_136" typeName="BOOL"/>
		<variables name="A_PU_138_140" typeName="BOOL"/>
		<variables name="A_PU_142" typeName="BOOL"/>
		<variables name="A_PU_144" typeName="BOOL"/>
		<variables name="A_PU_146" typeName="BOOL"/>
		<variables name="A_PU_148" typeName="BOOL"/>
		<variables name="A_PU_160" typeName="BOOL"/>
		<variables name="A_PU_162" typeName="BOOL"/>
		<variables name="A_PU_164" typeName="BOOL"/>
		<variables name="A_PU_166" typeName="BOOL"/>
		<variables name="A_PU_201" typeName="BOOL"/>
		<variables name="A_PU_202" typeName="BOOL"/>
		<variables name="A_PU_203" typeName="BOOL"/>
		<variables name="A_PU_204_206" typeName="EBOOL"/>
		<variables name="A_PU_208" typeName="BOOL"/>
		<variables name="A_PU_212" typeName="BOOL"/>
		<variables name="A_PU_302" typeName="BOOL"/>
		<variables name="A_PV" typeName="BOOL"/>
		<variables name="A_PV1" typeName="BOOL"/>
		<variables name="A_PZ10_K" typeName="EBOOL"/>
		<variables name="A_RACK1" typeName="INT">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_RACK2" typeName="INT">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_RACK3" typeName="INT">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_RACK4" typeName="INT">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_RES1_K" typeName="EBOOL"/>
		<variables name="A_RES2_K" typeName="EBOOL"/>
		<variables name="A_RES3_K" typeName="EBOOL"/>
		<variables name="A_RES4_K" typeName="EBOOL"/>
		<variables name="A_RES5_K" typeName="EBOOL"/>
		<variables name="A_RES6_K" typeName="EBOOL"/>
		<variables name="A_RES7_K" typeName="EBOOL"/>
		<variables name="A_RES8_K" typeName="EBOOL"/>
		<variables name="A_RES9_K" typeName="EBOOL"/>
		<variables name="A_RES10_K" typeName="EBOOL"/>
		<variables name="A_RES11_K" typeName="EBOOL"/>
		<variables name="A_RES12_K" typeName="EBOOL"/>
		<variables name="A_RES13_K" typeName="EBOOL"/>
		<variables name="A_RES14_K" typeName="EBOOL"/>
		<variables name="A_RES15_K" typeName="EBOOL"/>
		<variables name="A_RES16_K" typeName="EBOOL"/>
		<variables name="A_RESET_DIAG" typeName="BOOL"/>
		<variables name="A_RI" typeName="BOOL"/>
		<variables name="A_RM" typeName="BOOL"/>
		<variables name="A_RP" typeName="BOOL"/>
		<variables name="A_schet_iv" typeName="DINT"/>
		<variables name="A_schet_mv" typeName="DINT"/>
		<variables name="A_schet_ov" typeName="DINT"/>
		<variables name="A_schet_pv" typeName="DINT"/>
		<variables name="A_SH44_NOE54_ETH_SUCCESS" typeName="BOOL"/>
		<variables name="A_SH44_NOE75_ETH_SUCCESS" typeName="BOOL"/>
		<variables name="A_SH_NOE45_ETH_SUCCESS" typeName="BOOL"/>
		<variables name="A_SHBK_K" typeName="EBOOL">
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_SHN_ETH_SUCCESS" typeName="BOOL"/>
		<variables name="A_SHSSO1_NOE64_ETH_SUCCESS" typeName="BOOL"/>
		<variables name="A_SHSSO1_NOE74_ETH_SUCCESS" typeName="BOOL"/>
		<variables name="A_SHSSO1_NOE84_ETH_SUCCESS" typeName="BOOL"/>
		<variables name="A_SHSSO1BK_K" typeName="EBOOL">
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_SHSSO2_BK_K" typeName="BOOL"/>
		<variables name="A_SHSSO2_NOE84_ETH_SUCCESS" typeName="BOOL"/>
		<variables name="A_SHSSO2_NOE812_ETH_SUCCESS" typeName="BOOL"/>
		<variables name="A_SOG" typeName="BOOL"/>
		<variables name="A_ST11P10_IM" typeName="BOOL"/>
		<variables name="A_ST11P10_K" typeName="EBOOL">
			<comment>???????? ???????. ? 11 ??????? ?? ????? 10</comment>
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_ST11P11_IM" typeName="BOOL"/>
		<variables name="A_ST11P11_K" typeName="EBOOL">
			<comment>???????? ???????. ? 11 ??????? ?? ????? 11</comment>
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_ST12P7_IM" typeName="BOOL"/>
		<variables name="A_ST12P7_K" typeName="EBOOL">
			<comment>???????? ???????. ? 12 ??????? ?? ????? 7</comment>
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_ST12P8_IM" typeName="BOOL"/>
		<variables name="A_ST12P8_K" typeName="EBOOL">
			<comment>???????? ???????. ? 12 ??????? ?? ????? 8</comment>
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_ST12P9_IM" typeName="BOOL"/>
		<variables name="A_ST12P9_K" typeName="EBOOL">
			<comment>???????? ???????. ? 12 ??????? ?? ????? 9</comment>
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_ST12P10_IM" typeName="BOOL"/>
		<variables name="A_ST12P10_K" typeName="EBOOL">
			<comment>???????? ???????. ? 12 ??????? ?? ????? 10</comment>
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_ST12P11_IM" typeName="BOOL"/>
		<variables name="A_ST12P11_K" typeName="EBOOL">
			<comment>???????? ???????. ? 12 ??????? ?? ????? 11</comment>
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_ST12P12_IM" typeName="BOOL"/>
		<variables name="A_ST12P12_K" typeName="EBOOL">
			<comment>???????? ???????. ? 12 ??????? ?? ????? 12</comment>
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_ST12P13_IM" typeName="BOOL"/>
		<variables name="A_ST12P13_K" typeName="EBOOL">
			<comment>???????? ???????. ? 12 ??????? ?? ????? 13</comment>
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_ST13P10_IM" typeName="BOOL"/>
		<variables name="A_ST13P10_K" typeName="EBOOL">
			<comment>???????? ???????. ? 13 ??????? ?? ????? 10</comment>
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_ST13P11_IM" typeName="BOOL"/>
		<variables name="A_ST13P11_K" typeName="EBOOL">
			<comment>???????? ???????. ? 13 ??????? ?? ????? 11</comment>
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_ST22P9_IM" typeName="BOOL"/>
		<variables name="A_ST22P9_K" typeName="EBOOL">
			<comment>???????? ???????. ? 22 ??????? ?? ????? 9</comment>
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_ST22P10_IM" typeName="BOOL"/>
		<variables name="A_ST22P10_K" typeName="EBOOL">
			<comment>???????? ???????. ? 22 ??????? ?? ????? 10</comment>
			<attribute name="TimeStampCapability" value="0"/>
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="A_t_ch" typeName="BOOL"/>
		<variables name="A_t_chm" typeName="BOOL"/>
		<variables name="A_T_IV" typeName="TIME"/>
		<variables name="A_T_MV" typeName="TIME"/>
		<variables name="A_t_n" typeName="BOOL"/>
		<variables name="A_t_nm" typeName="BOOL"/>
		<variables name="A_T_OUT_IV" typeName="TIME"/>
		<variables name="A_T_OUT_MV" typeName="TIME"/>
		<variables name="A_T_OUT_OV" typeName="TIME"/>
		<variables name="A_T_OUT_PV" typeName="TIME"/>
		<variables name="A_T_OV" typeName="TIME"/>
		<variables name="A_T_PV" typeName="TIME"/>
		<variables name="A_Tkn" typeName="TIME"/>
		<variables name="A_Tsv" typeName="TIME"/>
		<variables name="A_TVDIR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_TVDKS" typeName="BOOL"/>
		<variables name="A_TVDP_IND" typeName="BYTE"/>
		<variables name="A_TVDP_K" typeName="EBOOL"/>
		<variables name="A_TVDRI" typeName="BOOL"/>
		<variables name="A_TVDZ" typeName="BOOL"/>
		<variables name="A_Tz" typeName="TIME">
			<attribute name="IsConstant" value="-1"/>
			<variableInit value="t#500ms"/>
		</variables>
		<variables name="A_UK_1_3" typeName="BOOL"/>
		<variables name="A_UK_11_13" typeName="BOOL"/>
		<variables name="A_UK_14_16" typeName="BOOL"/>
		<variables name="A_UK_22_24" typeName="BOOL"/>
		<variables name="A_UK_26_28" typeName="BOOL"/>
		<variables name="A_UK_29_31" typeName="BOOL"/>
		<variables name="A_UK_30_34" typeName="BOOL"/>
		<variables name="A_UK_33_35" typeName="BOOL"/>
		<variables name="A_UK_117_119" typeName="BOOL"/>
		<variables name="A_UK_138_140" typeName="BOOL"/>
		<variables name="A_UK_204_206" typeName="BOOL"/>
		<variables name="A_UPR_KOMP" typeName="BOOL"/>
		<variables name="A_UPR_KOMP_ARM_DSP" typeName="BOOL"/>
		<variables name="A_VCH" typeName="BOOL"/>
		<variables name="A_VCHM" typeName="BOOL"/>
		<variables name="A_VN" typeName="BOOL"/>
		<variables name="A_VN_1" typeName="BOOL"/>
		<variables name="A_VN_1SP" typeName="BOOL"/>
		<variables name="A_VN_3" typeName="BOOL"/>
		<variables name="A_VN_3_5SP" typeName="BOOL"/>
		<variables name="A_VN_4" typeName="BOOL"/>
		<variables name="A_VN_4_84SP" typeName="BOOL"/>
		<variables name="A_VN_5" typeName="BOOL"/>
		<variables name="A_VN_6" typeName="BOOL"/>
		<variables name="A_VN_6SP" typeName="BOOL"/>
		<variables name="A_VN_8" typeName="BOOL"/>
		<variables name="A_VN_8SP" typeName="BOOL"/>
		<variables name="A_VN_9" typeName="BOOL"/>
		<variables name="A_VN_9SP" typeName="BOOL"/>
		<variables name="A_VN_11" typeName="BOOL"/>
		<variables name="A_VN_11_19SP" typeName="BOOL"/>
		<variables name="A_VN_12" typeName="BOOL"/>
		<variables name="A_VN_12SP" typeName="BOOL"/>
		<variables name="A_VN_13" typeName="BOOL"/>
		<variables name="A_VN_13SP" typeName="BOOL"/>
		<variables name="A_VN_14" typeName="BOOL"/>
		<variables name="A_VN_14SP" typeName="BOOL"/>
		<variables name="A_VN_15" typeName="BOOL"/>
		<variables name="A_VN_15SP" typeName="BOOL"/>
		<variables name="A_VN_16" typeName="BOOL"/>
		<variables name="A_VN_16SP" typeName="BOOL"/>
		<variables name="A_VN_17" typeName="BOOL"/>
		<variables name="A_VN_18" typeName="BOOL"/>
		<variables name="A_VN_18_26SP" typeName="BOOL"/>
		<variables name="A_VN_19" typeName="BOOL"/>
		<variables name="A_VN_20" typeName="BOOL"/>
		<variables name="A_VN_20_22SP" typeName="BOOL"/>
		<variables name="A_VN_22" typeName="BOOL"/>
		<variables name="A_VN_23SP" typeName="BOOL"/>
		<variables name="A_VN_24" typeName="BOOL"/>
		<variables name="A_VN_24SP" typeName="BOOL"/>
		<variables name="A_VN_25" typeName="BOOL"/>
		<variables name="A_VN_25_27SP" typeName="BOOL"/>
		<variables name="A_VN_26" typeName="BOOL"/>
		<variables name="A_VN_27" typeName="BOOL"/>
		<variables name="A_VN_28" typeName="BOOL"/>
		<variables name="A_VN_28SP" typeName="BOOL"/>
		<variables name="A_VN_29" typeName="BOOL"/>
		<variables name="A_VN_29SP" typeName="BOOL"/>
		<variables name="A_VN_30" typeName="BOOL"/>
		<variables name="A_VN_30SP" typeName="BOOL"/>
		<variables name="A_VN_31" typeName="BOOL"/>
		<variables name="A_VN_31SP" typeName="BOOL"/>
		<variables name="A_VN_32" typeName="BOOL"/>
		<variables name="A_VN_32SP" typeName="BOOL"/>
		<variables name="A_VN_33" typeName="BOOL"/>
		<variables name="A_VN_33_43SP" typeName="BOOL"/>
		<variables name="A_VN_34" typeName="BOOL"/>
		<variables name="A_VN_34_36SP" typeName="BOOL"/>
		<variables name="A_VN_35" typeName="BOOL"/>
		<variables name="A_VN_35SP" typeName="BOOL"/>
		<variables name="A_VN_36" typeName="BOOL"/>
		<variables name="A_VN_37" typeName="BOOL"/>
		<variables name="A_VN_37_39SP" typeName="BOOL"/>
		<variables name="A_VN_38" typeName="BOOL"/>
		<variables name="A_VN_38SP" typeName="BOOL"/>
		<variables name="A_VN_39" typeName="BOOL"/>
		<variables name="A_VN_41" typeName="BOOL"/>
		<variables name="A_VN_43" typeName="BOOL"/>
		<variables name="A_VN_47" typeName="BOOL"/>
		<variables name="A_VN_47_49SP" typeName="BOOL"/>
		<variables name="A_VN_49" typeName="BOOL"/>
		<variables name="A_VN_51" typeName="BOOL"/>
		<variables name="A_VN_51SP" typeName="BOOL"/>
		<variables name="A_VN_53" typeName="BOOL"/>
		<variables name="A_VN_53SP" typeName="BOOL"/>
		<variables name="A_VN_84" typeName="BOOL"/>
		<variables name="A_VN_98" typeName="BOOL"/>
		<variables name="A_VN_98_104SP" typeName="BOOL"/>
		<variables name="A_VN_100" typeName="BOOL"/>
		<variables name="A_VN_100SP" typeName="BOOL"/>
		<variables name="A_VN_101" typeName="BOOL"/>
		<variables name="A_VN_101_103SP" typeName="BOOL"/>
		<variables name="A_VN_102" typeName="BOOL"/>
		<variables name="A_VN_102_120SP" typeName="BOOL"/>
		<variables name="A_VN_103" typeName="BOOL"/>
		<variables name="A_VN_104" typeName="BOOL"/>
		<variables name="A_VN_105" typeName="BOOL"/>
		<variables name="A_VN_105SP" typeName="BOOL"/>
		<variables name="A_VN_106" typeName="BOOL"/>
		<variables name="A_VN_106_108SP" typeName="BOOL"/>
		<variables name="A_VN_107" typeName="BOOL"/>
		<variables name="A_VN_107_121SP" typeName="BOOL"/>
		<variables name="A_VN_108" typeName="BOOL"/>
		<variables name="A_VN_109" typeName="BOOL"/>
		<variables name="A_VN_109SP" typeName="BOOL"/>
		<variables name="A_VN_110" typeName="BOOL"/>
		<variables name="A_VN_110_112SP" typeName="BOOL"/>
		<variables name="A_VN_111" typeName="BOOL"/>
		<variables name="A_VN_111_113SP" typeName="BOOL"/>
		<variables name="A_VN_112" typeName="BOOL"/>
		<variables name="A_VN_113" typeName="BOOL"/>
		<variables name="A_VN_114" typeName="BOOL"/>
		<variables name="A_VN_114SP" typeName="BOOL"/>
		<variables name="A_VN_115" typeName="BOOL"/>
		<variables name="A_VN_115SP" typeName="BOOL"/>
		<variables name="A_VN_116" typeName="BOOL"/>
		<variables name="A_VN_116SP" typeName="BOOL"/>
		<variables name="A_VN_117" typeName="BOOL"/>
		<variables name="A_VN_117SP" typeName="BOOL"/>
		<variables name="A_VN_119" typeName="BOOL"/>
		<variables name="A_VN_119SP" typeName="BOOL"/>
		<variables name="A_VN_120" typeName="BOOL"/>
		<variables name="A_VN_121" typeName="BOOL"/>
		<variables name="A_VN_122" typeName="BOOL"/>
		<variables name="A_VN_122SP" typeName="BOOL"/>
		<variables name="A_VN_124" typeName="BOOL"/>
		<variables name="A_VN_124_138SP" typeName="BOOL"/>
		<variables name="A_VN_128" typeName="BOOL"/>
		<variables name="A_VN_128_134SP" typeName="BOOL"/>
		<variables name="A_VN_130" typeName="BOOL"/>
		<variables name="A_VN_130_164SP" typeName="BOOL"/>
		<variables name="A_VN_132" typeName="BOOL"/>
		<variables name="A_VN_134" typeName="BOOL"/>
		<variables name="A_VN_136" typeName="BOOL"/>
		<variables name="A_VN_136SP" typeName="BOOL"/>
		<variables name="A_VN_138" typeName="BOOL"/>
		<variables name="A_VN_140" typeName="BOOL"/>
		<variables name="A_VN_140SP" typeName="BOOL"/>
		<variables name="A_VN_142" typeName="BOOL"/>
		<variables name="A_VN_142SP" typeName="BOOL"/>
		<variables name="A_VN_144" typeName="BOOL"/>
		<variables name="A_VN_144_146SP" typeName="BOOL"/>
		<variables name="A_VN_146" typeName="BOOL"/>
		<variables name="A_VN_148" typeName="BOOL"/>
		<variables name="A_VN_148SP" typeName="BOOL"/>
		<variables name="A_VN_160" typeName="BOOL"/>
		<variables name="A_VN_160SP" typeName="BOOL"/>
		<variables name="A_VN_162" typeName="BOOL"/>
		<variables name="A_VN_162SP" typeName="BOOL"/>
		<variables name="A_VN_164" typeName="BOOL"/>
		<variables name="A_VN_166" typeName="BOOL"/>
		<variables name="A_VN_166SP" typeName="BOOL"/>
		<variables name="A_VN_201" typeName="BOOL"/>
		<variables name="A_VN_201_203SP" typeName="BOOL"/>
		<variables name="A_VN_202" typeName="BOOL"/>
		<variables name="A_VN_202SP" typeName="BOOL"/>
		<variables name="A_VN_203" typeName="BOOL"/>
		<variables name="A_VN_204" typeName="BOOL"/>
		<variables name="A_VN_204_212SP" typeName="BOOL"/>
		<variables name="A_VN_206" typeName="BOOL"/>
		<variables name="A_VN_206SP" typeName="BOOL"/>
		<variables name="A_VN_208" typeName="BOOL"/>
		<variables name="A_VN_212" typeName="BOOL"/>
		<variables name="A_VN_302" typeName="BOOL"/>
		<variables name="A_VN_302SP" typeName="BOOL"/>
		<variables name="A_VNM" typeName="BOOL"/>
		<variables name="A_VO" typeName="BOOL"/>
		<variables name="A_VO1" typeName="BOOL"/>
		<variables name="A_VO2" typeName="BOOL"/>
		<variables name="A_VOG" typeName="BOOL"/>
		<variables name="A_VP_K" typeName="EBOOL"/>
		<variables name="A_VZ" typeName="BOOL"/>
		<variables name="A_VZ_STR" typeName="BOOL"/>
		<variables name="A_VZ_STR_ARM_DSP" typeName="BOOL"/>
		<variables name="A_VZ_STR_KL" typeName="BOOL"/>
		<variables name="A_VZ_STR_MIG_KL" typeName="BOOL"/>
		<variables name="A_VZ_STR_SrL" typeName="BOOL"/>
		<variables name="A_Z_K" typeName="EBOOL"/>
		<variables name="A_ZEMLIA_IM" typeName="BOOL"/>
		<variables name="A_ZEMLIA_KL" typeName="BOOL"/>
		<variables name="A_ZEMLIA_SrL" typeName="BOOL"/>
		<variables name="A_ZG13_K" typeName="EBOOL"/>
		<variables name="A_ZS10_K" typeName="EBOOL"/>
		<variables name="A_ZV1_U" typeName="EBOOL"/>
		<variables name="A_ZV1_U_OBR" typeName="BOOL"/>
		<variables name="A_ZV1_U_PR" typeName="BOOL"/>
		<variables name="A_ZV1_UK" typeName="EBOOL"/>
		<variables name="A_ZV2_K" typeName="EBOOL"/>
		<variables name="A_ZV2_U" typeName="EBOOL"/>
		<variables name="A_ZV2_U_OBR" typeName="BOOL"/>
		<variables name="A_ZV2_U_PR" typeName="BOOL"/>
		<variables name="A_ZV2_UK" typeName="EBOOL"/>
		<variables name="A_ZV5_K" typeName="EBOOL"/>
		<variables name="A_ZV5_U" typeName="EBOOL"/>
		<variables name="A_ZV5_U_OBR" typeName="BOOL"/>
		<variables name="A_ZV5_U_PR" typeName="BOOL"/>
		<variables name="A_ZV5_UK" typeName="EBOOL"/>
		<variables name="A_ZV6_K" typeName="EBOOL"/>
		<variables name="A_ZV6_U" typeName="EBOOL"/>
		<variables name="A_ZV6_U_OBR" typeName="BOOL"/>
		<variables name="A_ZV6_U_PR" typeName="BOOL"/>
		<variables name="A_ZV6_UK" typeName="EBOOL"/>
		<variables name="A_ZV8_U" typeName="EBOOL"/>
		<variables name="A_ZV8_U_OBR" typeName="BOOL"/>
		<variables name="A_ZV8_U_PR" typeName="BOOL"/>
		<variables name="A_ZV8_UK" typeName="EBOOL"/>
		<variables name="A_ZV9_K" typeName="EBOOL"/>
		<variables name="A_ZV9_U" typeName="EBOOL"/>
		<variables name="A_ZV9_U_OBR" typeName="BOOL"/>
		<variables name="A_ZV9_U_PR" typeName="BOOL"/>
		<variables name="A_ZV9_UK" typeName="EBOOL"/>
		<variables name="A_ZV16_K" typeName="EBOOL"/>
		<variables name="A_ZV16_U" typeName="EBOOL"/>
		<variables name="A_ZV16_U_OBR" typeName="BOOL"/>
		<variables name="A_ZV16_U_PR" typeName="BOOL"/>
		<variables name="A_ZV16_UK" typeName="EBOOL"/>
		<variables name="A_ZV38A_K" typeName="EBOOL"/>
		<variables name="A_ZV38A_U" typeName="EBOOL"/>
		<variables name="A_ZV38A_U_OBR" typeName="BOOL"/>
		<variables name="A_ZV38A_U_PR" typeName="BOOL"/>
		<variables name="A_ZV38A_UK" typeName="EBOOL"/>
		<variables name="A_ZV38B_K" typeName="EBOOL"/>
		<variables name="A_ZV38B_U" typeName="EBOOL"/>
		<variables name="A_ZV38B_U_OBR" typeName="BOOL"/>
		<variables name="A_ZV38B_U_PR" typeName="BOOL"/>
		<variables name="A_ZV38B_UK" typeName="EBOOL"/>
		<variables name="A_ZV39A_K" typeName="EBOOL"/>
		<variables name="A_ZV39A_U" typeName="EBOOL"/>
		<variables name="A_ZV39A_U_OBR" typeName="BOOL"/>
		<variables name="A_ZV39A_U_PR" typeName="BOOL"/>
		<variables name="A_ZV39A_UK" typeName="EBOOL"/>
		<variables name="A_ZV39B2_K" typeName="EBOOL"/>
		<variables name="A_ZV39B_U" typeName="EBOOL"/>
		<variables name="A_ZV39B_U_OBR" typeName="BOOL"/>
		<variables name="A_ZV39B_U_PR" typeName="BOOL"/>
		<variables name="A_ZV39B_UK" typeName="EBOOL"/>
		<variables name="A_ZVONOK_FID" typeName="BOOL"/>
		<variables name="A_ZVONOK_NEISPR_SIST" typeName="BOOL"/>
		<variables name="A_ZVONOK_VZ_STR" typeName="BOOL"/>
		<variables name="FU_1_1_K" typeName="EBOOL"/>
		<variables name="FU_1_2_K" typeName="EBOOL"/>
		<variables name="FU_2_1_K" typeName="EBOOL"/>
		<variables name="FU_2_2_K" typeName="EBOOL"/>
		<variables name="FU_3_1_K" typeName="EBOOL"/>
		<variables name="FU_3_2_K" typeName="EBOOL"/>
		<variables name="FU_3_3_K" typeName="EBOOL"/>
		<variables name="FU_4_1_K" typeName="EBOOL"/>
		<variables name="FU_4_2_K" typeName="EBOOL"/>
		<variables name="FU_4_3_K" typeName="EBOOL"/>
		<variables name="FU_5_1_K" typeName="EBOOL"/>
		<variables name="FU_5_2_K" typeName="EBOOL"/>
		<variables name="FU_6_1_K" typeName="EBOOL"/>
		<variables name="FU_6_2_K" typeName="EBOOL"/>
		<variables name="FU_6_3_K" typeName="EBOOL"/>
		<variables name="FU_8_1_K" typeName="EBOOL"/>
		<variables name="FUSUP4_3_K" typeName="EBOOL"/>
		<variables name="FUSUP_1_1_K" typeName="EBOOL"/>
		<variables name="FUSUP_1_2_K" typeName="EBOOL"/>
		<variables name="FUSUP_2_1_K" typeName="EBOOL"/>
		<variables name="FUSUP_2_2_K" typeName="EBOOL"/>
		<variables name="FUSUP_3_1_K" typeName="EBOOL"/>
		<variables name="FUSUP_3_2_K" typeName="EBOOL"/>
		<variables name="FUSUP_3_3_K" typeName="EBOOL"/>
		<variables name="FUSUP_4_1_K" typeName="EBOOL"/>
		<variables name="FUSUP_4_2_K" typeName="EBOOL"/>
		<variables name="FUSUP_5_1_K" typeName="EBOOL"/>
		<variables name="FUSUP_5_2_K" typeName="EBOOL"/>
		<variables name="FUSUP_6_1_K" typeName="EBOOL"/>
		<variables name="FUSUP_6_2_K" typeName="EBOOL"/>
		<variables name="FUSUP_6_3_K" typeName="EBOOL"/>
		<variables name="FUSUP_8_1_K" typeName="EBOOL"/>
		<variables name="K_PODSV_STR_CH" typeName="BOOL"/>
		<variables name="K_PODSV_STR_CH_ARM_DSP" typeName="BOOL"/>
		<variables name="K_PODSV_STR_N" typeName="BOOL"/>
		<variables name="K_PODSV_STR_N_ARM_DSP" typeName="BOOL"/>
		<variables name="NAPR_MAN" typeName="BOOL"/>
		<variables name="NAPR_POEZD" typeName="BOOL"/>
		<variables name="OTKL_IMIT" typeName="BOOL"/>
		<variables name="R2_STR1_ARM" typeName="WORD" topologicalAddress="%MW501">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R2_STR3_ARM" typeName="WORD" topologicalAddress="%MW503">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R2_STR4_ARM" typeName="WORD" topologicalAddress="%MW597"/>
		<variables name="R2_STR5_ARM" typeName="WORD" topologicalAddress="%MW505"/>
		<variables name="R2_STR6_ARM" typeName="WORD" topologicalAddress="%MW599"/>
		<variables name="R2_STR8_ARM" typeName="WORD" topologicalAddress="%MW601"/>
		<variables name="R2_STR9_ARM" typeName="WORD" topologicalAddress="%MW507"/>
		<variables name="R2_STR11_ARM" typeName="WORD" topologicalAddress="%MW509"/>
		<variables name="R2_STR12_ARM" typeName="WORD" topologicalAddress="%MW603"/>
		<variables name="R2_STR13_ARM" typeName="WORD" topologicalAddress="%MW511"/>
		<variables name="R2_STR14_ARM" typeName="WORD" topologicalAddress="%MW513"/>
		<variables name="R2_STR15_ARM" typeName="WORD" topologicalAddress="%MW515"/>
		<variables name="R2_STR16_ARM" typeName="WORD" topologicalAddress="%MW517"/>
		<variables name="R2_STR17_ARM" typeName="WORD" topologicalAddress="%MW519"/>
		<variables name="R2_STR18_ARM" typeName="WORD" topologicalAddress="%MW521"/>
		<variables name="R2_STR19_ARM" typeName="WORD" topologicalAddress="%MW523"/>
		<variables name="R2_STR20_ARM" typeName="WORD" topologicalAddress="%MW525"/>
		<variables name="R2_STR22_ARM" typeName="WORD" topologicalAddress="%MW527"/>
		<variables name="R2_STR24_ARM" typeName="WORD" topologicalAddress="%MW529"/>
		<variables name="R2_STR25_ARM" typeName="WORD" topologicalAddress="%MW531"/>
		<variables name="R2_STR26_ARM" typeName="WORD" topologicalAddress="%MW533"/>
		<variables name="R2_STR27_ARM" typeName="WORD" topologicalAddress="%MW535"/>
		<variables name="R2_STR28_ARM" typeName="WORD" topologicalAddress="%MW537"/>
		<variables name="R2_STR29_ARM" typeName="WORD" topologicalAddress="%MW539"/>
		<variables name="R2_STR30_ARM" typeName="WORD" topologicalAddress="%MW541"/>
		<variables name="R2_STR31_ARM" typeName="WORD" topologicalAddress="%MW543"/>
		<variables name="R2_STR32_ARM" typeName="WORD" topologicalAddress="%MW545"/>
		<variables name="R2_STR33_ARM" typeName="WORD" topologicalAddress="%MW547"/>
		<variables name="R2_STR34_ARM" typeName="WORD" topologicalAddress="%MW549"/>
		<variables name="R2_STR35_ARM" typeName="WORD" topologicalAddress="%MW551"/>
		<variables name="R2_STR36_ARM" typeName="WORD" topologicalAddress="%MW553"/>
		<variables name="R2_STR37_ARM" typeName="WORD" topologicalAddress="%MW555"/>
		<variables name="R2_STR38_ARM" typeName="WORD" topologicalAddress="%MW557"/>
		<variables name="R2_STR39_ARM" typeName="WORD" topologicalAddress="%MW559"/>
		<variables name="R2_STR41_ARM" typeName="WORD" topologicalAddress="%MW561"/>
		<variables name="R2_STR43_ARM" typeName="WORD" topologicalAddress="%MW563">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R2_STR47_ARM" typeName="WORD" topologicalAddress="%MW565"/>
		<variables name="R2_STR49_ARM" typeName="WORD" topologicalAddress="%MW567"/>
		<variables name="R2_STR51_ARM" typeName="WORD" topologicalAddress="%MW569"/>
		<variables name="R2_STR53_ARM" typeName="WORD" topologicalAddress="%MW571"/>
		<variables name="R2_STR84_ARM" typeName="WORD" topologicalAddress="%MW605"/>
		<variables name="R2_STR98_ARM" typeName="WORD" topologicalAddress="%MW607"/>
		<variables name="R2_STR100_ARM" typeName="WORD" topologicalAddress="%MW609"/>
		<variables name="R2_STR101_ARM" typeName="WORD" topologicalAddress="%MW573">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R2_STR102_ARM" typeName="WORD" topologicalAddress="%MW611"/>
		<variables name="R2_STR103_ARM" typeName="WORD" topologicalAddress="%MW613"/>
		<variables name="R2_STR104_ARM" typeName="WORD" topologicalAddress="%MW615"/>
		<variables name="R2_STR105_ARM" typeName="WORD" topologicalAddress="%MW617"/>
		<variables name="R2_STR106_ARM" typeName="WORD" topologicalAddress="%MW619"/>
		<variables name="R2_STR107_ARM" typeName="WORD" topologicalAddress="%MW575"/>
		<variables name="R2_STR108_ARM" typeName="WORD" topologicalAddress="%MW621"/>
		<variables name="R2_STR109_ARM" typeName="WORD" topologicalAddress="%MW623"/>
		<variables name="R2_STR110_ARM" typeName="WORD" topologicalAddress="%MW625"/>
		<variables name="R2_STR111_ARM" typeName="WORD" topologicalAddress="%MW627"/>
		<variables name="R2_STR112_ARM" typeName="WORD" topologicalAddress="%MW629"/>
		<variables name="R2_STR113_ARM" typeName="WORD" topologicalAddress="%MW631"/>
		<variables name="R2_STR114_ARM" typeName="WORD" topologicalAddress="%MW633"/>
		<variables name="R2_STR115_ARM" typeName="WORD" topologicalAddress="%MW635"/>
		<variables name="R2_STR116_ARM" typeName="WORD" topologicalAddress="%MW637"/>
		<variables name="R2_STR117_ARM" typeName="WORD" topologicalAddress="%MW577"/>
		<variables name="R2_STR119_ARM" typeName="WORD" topologicalAddress="%MW579"/>
		<variables name="R2_STR120_ARM" typeName="WORD" topologicalAddress="%MW639"/>
		<variables name="R2_STR121_ARM" typeName="WORD" topologicalAddress="%MW581">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R2_STR122_ARM" typeName="WORD" topologicalAddress="%MW641"/>
		<variables name="R2_STR124_ARM" typeName="WORD" topologicalAddress="%MW643"/>
		<variables name="R2_STR128_ARM" typeName="WORD" topologicalAddress="%MW645"/>
		<variables name="R2_STR130_ARM" typeName="WORD" topologicalAddress="%MW647"/>
		<variables name="R2_STR132_ARM" typeName="WORD" topologicalAddress="%MW649">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R2_STR134_ARM" typeName="WORD" topologicalAddress="%MW651"/>
		<variables name="R2_STR136_ARM" typeName="WORD" topologicalAddress="%MW653"/>
		<variables name="R2_STR138_ARM" typeName="WORD" topologicalAddress="%MW655"/>
		<variables name="R2_STR140_ARM" typeName="WORD" topologicalAddress="%MW657"/>
		<variables name="R2_STR142_ARM" typeName="WORD" topologicalAddress="%MW659"/>
		<variables name="R2_STR144_ARM" typeName="WORD" topologicalAddress="%MW661"/>
		<variables name="R2_STR146_ARM" typeName="WORD" topologicalAddress="%MW663"/>
		<variables name="R2_STR148_ARM" typeName="WORD" topologicalAddress="%MW665"/>
		<variables name="R2_STR160_ARM" typeName="WORD" topologicalAddress="%MW667"/>
		<variables name="R2_STR162_ARM" typeName="WORD" topologicalAddress="%MW669"/>
		<variables name="R2_STR164_ARM" typeName="WORD" topologicalAddress="%MW671"/>
		<variables name="R2_STR166_ARM" typeName="WORD" topologicalAddress="%MW673"/>
		<variables name="R2_STR201_ARM" typeName="WORD" topologicalAddress="%MW583"/>
		<variables name="R2_STR202_ARM" typeName="WORD" topologicalAddress="%MW585"/>
		<variables name="R2_STR203_ARM" typeName="WORD" topologicalAddress="%MW587"/>
		<variables name="R2_STR204_ARM" typeName="WORD" topologicalAddress="%MW589"/>
		<variables name="R2_STR206_ARM" typeName="WORD" topologicalAddress="%MW591"/>
		<variables name="R2_STR208_ARM" typeName="WORD" topologicalAddress="%MW593"/>
		<variables name="R2_STR212_ARM" typeName="WORD" topologicalAddress="%MW595"/>
		<variables name="R2_STR302_ARM" typeName="WORD" topologicalAddress="%MW675">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R3_STR1_ARM" typeName="WORD" topologicalAddress="%MW502">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R3_STR3_ARM" typeName="WORD" topologicalAddress="%MW504">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R3_STR4_ARM" typeName="WORD" topologicalAddress="%MW598"/>
		<variables name="R3_STR5_ARM" typeName="WORD" topologicalAddress="%MW506"/>
		<variables name="R3_STR6_ARM" typeName="WORD" topologicalAddress="%MW600"/>
		<variables name="R3_STR8_ARM" typeName="WORD" topologicalAddress="%MW602">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R3_STR9_ARM" typeName="WORD" topologicalAddress="%MW508"/>
		<variables name="R3_STR11_ARM" typeName="WORD" topologicalAddress="%MW510"/>
		<variables name="R3_STR12_ARM" typeName="WORD" topologicalAddress="%MW604"/>
		<variables name="R3_STR13_ARM" typeName="WORD" topologicalAddress="%MW512"/>
		<variables name="R3_STR14_ARM" typeName="WORD" topologicalAddress="%MW514"/>
		<variables name="R3_STR15_ARM" typeName="WORD" topologicalAddress="%MW516"/>
		<variables name="R3_STR16_ARM" typeName="WORD" topologicalAddress="%MW518"/>
		<variables name="R3_STR17_ARM" typeName="WORD" topologicalAddress="%MW520"/>
		<variables name="R3_STR18_ARM" typeName="WORD" topologicalAddress="%MW522"/>
		<variables name="R3_STR19_ARM" typeName="WORD" topologicalAddress="%MW524"/>
		<variables name="R3_STR20_ARM" typeName="WORD" topologicalAddress="%MW526"/>
		<variables name="R3_STR22_ARM" typeName="WORD" topologicalAddress="%MW528"/>
		<variables name="R3_STR24_ARM" typeName="WORD" topologicalAddress="%MW530"/>
		<variables name="R3_STR25_ARM" typeName="WORD" topologicalAddress="%MW532"/>
		<variables name="R3_STR26_ARM" typeName="WORD" topologicalAddress="%MW534"/>
		<variables name="R3_STR27_ARM" typeName="WORD" topologicalAddress="%MW536"/>
		<variables name="R3_STR28_ARM" typeName="WORD" topologicalAddress="%MW538"/>
		<variables name="R3_STR29_ARM" typeName="WORD" topologicalAddress="%MW540"/>
		<variables name="R3_STR30_ARM" typeName="WORD" topologicalAddress="%MW542"/>
		<variables name="R3_STR31_ARM" typeName="WORD" topologicalAddress="%MW544"/>
		<variables name="R3_STR32_ARM" typeName="WORD" topologicalAddress="%MW546"/>
		<variables name="R3_STR33_ARM" typeName="WORD" topologicalAddress="%MW548"/>
		<variables name="R3_STR34_ARM" typeName="WORD" topologicalAddress="%MW550"/>
		<variables name="R3_STR35_ARM" typeName="WORD" topologicalAddress="%MW552">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R3_STR36_ARM" typeName="WORD" topologicalAddress="%MW554"/>
		<variables name="R3_STR37_ARM" typeName="WORD" topologicalAddress="%MW556"/>
		<variables name="R3_STR38_ARM" typeName="WORD" topologicalAddress="%MW558"/>
		<variables name="R3_STR39_ARM" typeName="WORD" topologicalAddress="%MW560"/>
		<variables name="R3_STR41_ARM" typeName="WORD" topologicalAddress="%MW562"/>
		<variables name="R3_STR43_ARM" typeName="WORD" topologicalAddress="%MW564"/>
		<variables name="R3_STR47_ARM" typeName="WORD" topologicalAddress="%MW566"/>
		<variables name="R3_STR49_ARM" typeName="WORD" topologicalAddress="%MW568"/>
		<variables name="R3_STR51_ARM" typeName="WORD" topologicalAddress="%MW570"/>
		<variables name="R3_STR53_ARM" typeName="WORD" topologicalAddress="%MW572"/>
		<variables name="R3_STR84_ARM" typeName="WORD" topologicalAddress="%MW606"/>
		<variables name="R3_STR98_ARM" typeName="WORD" topologicalAddress="%MW608"/>
		<variables name="R3_STR100_ARM" typeName="WORD" topologicalAddress="%MW610"/>
		<variables name="R3_STR101_ARM" typeName="WORD" topologicalAddress="%MW574">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R3_STR102_ARM" typeName="WORD" topologicalAddress="%MW612"/>
		<variables name="R3_STR103_ARM" typeName="WORD" topologicalAddress="%MW614">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R3_STR104_ARM" typeName="WORD" topologicalAddress="%MW616"/>
		<variables name="R3_STR105_ARM" typeName="WORD" topologicalAddress="%MW618"/>
		<variables name="R3_STR106_ARM" typeName="WORD" topologicalAddress="%MW620"/>
		<variables name="R3_STR107_ARM" typeName="WORD" topologicalAddress="%MW576">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R3_STR108_ARM" typeName="WORD" topologicalAddress="%MW622"/>
		<variables name="R3_STR109_ARM" typeName="WORD" topologicalAddress="%MW624"/>
		<variables name="R3_STR110_ARM" typeName="WORD" topologicalAddress="%MW626"/>
		<variables name="R3_STR111_ARM" typeName="WORD" topologicalAddress="%MW628"/>
		<variables name="R3_STR112_ARM" typeName="WORD" topologicalAddress="%MW630"/>
		<variables name="R3_STR113_ARM" typeName="WORD" topologicalAddress="%MW632"/>
		<variables name="R3_STR114_ARM" typeName="WORD" topologicalAddress="%MW634"/>
		<variables name="R3_STR115_ARM" typeName="WORD" topologicalAddress="%MW636"/>
		<variables name="R3_STR116_ARM" typeName="WORD" topologicalAddress="%MW638"/>
		<variables name="R3_STR117_ARM" typeName="WORD" topologicalAddress="%MW578"/>
		<variables name="R3_STR119_ARM" typeName="WORD" topologicalAddress="%MW580"/>
		<variables name="R3_STR120_ARM" typeName="WORD" topologicalAddress="%MW640"/>
		<variables name="R3_STR121_ARM" typeName="WORD" topologicalAddress="%MW582">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R3_STR122_ARM" typeName="WORD" topologicalAddress="%MW642"/>
		<variables name="R3_STR124_ARM" typeName="WORD" topologicalAddress="%MW644"/>
		<variables name="R3_STR128_ARM" typeName="WORD" topologicalAddress="%MW646"/>
		<variables name="R3_STR130_ARM" typeName="WORD" topologicalAddress="%MW648"/>
		<variables name="R3_STR132_ARM" typeName="WORD" topologicalAddress="%MW650"/>
		<variables name="R3_STR134_ARM" typeName="WORD" topologicalAddress="%MW652"/>
		<variables name="R3_STR136_ARM" typeName="WORD" topologicalAddress="%MW654"/>
		<variables name="R3_STR138_ARM" typeName="WORD" topologicalAddress="%MW656"/>
		<variables name="R3_STR140_ARM" typeName="WORD" topologicalAddress="%MW658"/>
		<variables name="R3_STR142_ARM" typeName="WORD" topologicalAddress="%MW660"/>
		<variables name="R3_STR144_ARM" typeName="WORD" topologicalAddress="%MW662"/>
		<variables name="R3_STR146_ARM" typeName="WORD" topologicalAddress="%MW664"/>
		<variables name="R3_STR148_ARM" typeName="WORD" topologicalAddress="%MW666"/>
		<variables name="R3_STR160_ARM" typeName="WORD" topologicalAddress="%MW668"/>
		<variables name="R3_STR162_ARM" typeName="WORD" topologicalAddress="%MW670"/>
		<variables name="R3_STR164_ARM" typeName="WORD" topologicalAddress="%MW672"/>
		<variables name="R3_STR166_ARM" typeName="WORD" topologicalAddress="%MW674"/>
		<variables name="R3_STR201_ARM" typeName="WORD" topologicalAddress="%MW584"/>
		<variables name="R3_STR202_ARM" typeName="WORD" topologicalAddress="%MW586"/>
		<variables name="R3_STR203_ARM" typeName="WORD" topologicalAddress="%MW588"/>
		<variables name="R3_STR204_ARM" typeName="WORD" topologicalAddress="%MW590"/>
		<variables name="R3_STR206_ARM" typeName="WORD" topologicalAddress="%MW592"/>
		<variables name="R3_STR208_ARM" typeName="WORD" topologicalAddress="%MW594"/>
		<variables name="R3_STR212_ARM" typeName="WORD" topologicalAddress="%MW596"/>
		<variables name="R3_STR302_ARM" typeName="WORD" topologicalAddress="%MW676"/>
		<variables name="R_IMIT1_ARM" typeName="WORD" topologicalAddress="%MW1">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT2_ARM" typeName="WORD" topologicalAddress="%MW2">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT3_ARM" typeName="WORD" topologicalAddress="%MW3">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT4_ARM" typeName="WORD" topologicalAddress="%MW4">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT6_ARM" typeName="WORD" topologicalAddress="%MW6">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT7_ARM" typeName="WORD" topologicalAddress="%MW7">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT8_ARM" typeName="WORD" topologicalAddress="%MW8">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT9_ARM" typeName="WORD" topologicalAddress="%MW9">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT10_ARM" typeName="WORD" topologicalAddress="%MW10">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT11_ARM" typeName="WORD" topologicalAddress="%MW11">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT12_ARM" typeName="WORD" topologicalAddress="%MW12">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT13_ARM" typeName="WORD" topologicalAddress="%MW13">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT14_ARM" typeName="WORD" topologicalAddress="%MW14">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT15_ARM" typeName="WORD" topologicalAddress="%MW15">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT16_ARM" typeName="WORD" topologicalAddress="%MW16">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT17_ARM" typeName="WORD" topologicalAddress="%MW17">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT18_ARM" typeName="WORD" topologicalAddress="%MW18">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT19_ARM" typeName="WORD" topologicalAddress="%MW19">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT20_ARM" typeName="WORD" topologicalAddress="%MW20">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT21_ARM" typeName="WORD" topologicalAddress="%MW21">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT22_ARM" typeName="WORD" topologicalAddress="%MW22">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT23_ARM" typeName="WORD" topologicalAddress="%MW23">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT24_ARM" typeName="WORD" topologicalAddress="%MW24">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT25_ARM" typeName="WORD" topologicalAddress="%MW25">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT26_ARM" typeName="WORD" topologicalAddress="%MW26">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT27_ARM" typeName="WORD" topologicalAddress="%MW27">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT28_ARM" typeName="WORD" topologicalAddress="%MW28">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT30_ARM" typeName="WORD" topologicalAddress="%MW30">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT31_ARM" typeName="WORD" topologicalAddress="%MW31">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT32_ARM" typeName="WORD" topologicalAddress="%MW32">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT33_ARM" typeName="WORD" topologicalAddress="%MW33">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT34_ARM" typeName="WORD" topologicalAddress="%MW34">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT35_ARM" typeName="WORD" topologicalAddress="%MW35">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT36_ARM" typeName="WORD" topologicalAddress="%MW36">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT37_ARM" typeName="WORD" topologicalAddress="%MW37">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT38_ARM" typeName="WORD" topologicalAddress="%MW38">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT39_ARM" typeName="WORD" topologicalAddress="%MW39">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT40_ARM" typeName="WORD" topologicalAddress="%MW40">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT41_ARM" typeName="WORD" topologicalAddress="%MW41">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT42_ARM" typeName="WORD" topologicalAddress="%MW42">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT43_ARM" typeName="WORD" topologicalAddress="%MW43">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT44_ARM" typeName="WORD" topologicalAddress="%MW44">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT45_ARM" typeName="WORD" topologicalAddress="%MW45">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT46_ARM" typeName="WORD" topologicalAddress="%MW46">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="R_IMIT47_ARM" typeName="WORD" topologicalAddress="%MW47">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="RASP1_ARM" typeName="WORD" topologicalAddress="%MW401"/>
		<variables name="RASP2_ARM" typeName="WORD" topologicalAddress="%MW402">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="RASP3_ARM" typeName="WORD" topologicalAddress="%MW403">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="RASP4_ARM" typeName="WORD" topologicalAddress="%MW404">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="RASP5_ARM" typeName="WORD" topologicalAddress="%MW405">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="RASP6_ARM" typeName="WORD" topologicalAddress="%MW406">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="RASP7_ARM" typeName="WORD" topologicalAddress="%MW407">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="RASP8_ARM" typeName="WORD" topologicalAddress="%MW408">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="RASP9_ARM" typeName="WORD" topologicalAddress="%MW409">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="RASP10_ARM" typeName="WORD" topologicalAddress="%MW410">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="RASP11_ARM" typeName="WORD" topologicalAddress="%MW411">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="RASP12_ARM" typeName="WORD" topologicalAddress="%MW412">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="RASP13_ARM" typeName="WORD" topologicalAddress="%MW413">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="RASP14_ARM" typeName="WORD" topologicalAddress="%MW414">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="RASP15_ARM" typeName="WORD" topologicalAddress="%MW415">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="RASP16_ARM" typeName="WORD" topologicalAddress="%MW416">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="RASP18_ARM" typeName="WORD" topologicalAddress="%MW418"/>
		<variables name="RASP19_ARM" typeName="WORD" topologicalAddress="%MW419"/>
		<variables name="RASP20_ARM" typeName="WORD" topologicalAddress="%MW420"/>
		<variables name="RASP21_ARM" typeName="WORD" topologicalAddress="%MW421"/>
		<variables name="RASP22_ARM" typeName="WORD" topologicalAddress="%MW422"/>
		<variables name="RASP24_ARM" typeName="WORD" topologicalAddress="%MW424"/>
		<variables name="RASP25_ARM" typeName="WORD" topologicalAddress="%MW425"/>
		<variables name="RASP26_ARM" typeName="WORD" topologicalAddress="%MW426"/>
		<variables name="RASP27_ARM" typeName="WORD" topologicalAddress="%MW427"/>
		<variables name="RASP28_ARM" typeName="WORD" topologicalAddress="%MW428">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="RASP30_ARM" typeName="WORD" topologicalAddress="%MW430">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="RASP31_ARM" typeName="WORD" topologicalAddress="%MW431">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="RASP32_ARM" typeName="WORD" topologicalAddress="%MW432">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="RASP33_ARM" typeName="WORD" topologicalAddress="%MW433">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="RASP34_ARM" typeName="WORD" topologicalAddress="%MW434">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="RASP36_ARM" typeName="WORD" topologicalAddress="%MW436">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="RASP37_ARM" typeName="WORD" topologicalAddress="%MW437">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="RASP38_ARM" typeName="WORD" topologicalAddress="%MW438">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="RASP39_ARM" typeName="WORD" topologicalAddress="%MW439">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="RASP40_ARM" typeName="WORD" topologicalAddress="%MW440">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="RASP42_ARM" typeName="WORD" topologicalAddress="%MW442"/>
		<variables name="RASP43_ARM" typeName="WORD" topologicalAddress="%MW443">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="RASP44_ARM" typeName="WORD" topologicalAddress="%MW444">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="RASP45_ARM" typeName="WORD" topologicalAddress="%MW445">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="RASP47_ARM" typeName="WORD" topologicalAddress="%MW447">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="RASP48_ARM" typeName="WORD" topologicalAddress="%MW448">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="RASP49_ARM" typeName="WORD" topologicalAddress="%MW449">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="RASP50_ARM" typeName="WORD" topologicalAddress="%MW450">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="RASP52_ARM" typeName="WORD" topologicalAddress="%MW452">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="SEL_TIME_OTM" typeName="BOOL"/>
		<variables name="test" typeName="BOOL"/>
		<variables name="UPAK_REG701_ARM" typeName="WORD" topologicalAddress="%MW701">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG702_ARM" typeName="WORD" topologicalAddress="%MW702">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG703_ARM" typeName="WORD" topologicalAddress="%MW703">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG704_ARM" typeName="WORD" topologicalAddress="%MW704">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG705_ARM" typeName="WORD" topologicalAddress="%MW705">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG706_ARM" typeName="WORD" topologicalAddress="%MW706">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG707_ARM" typeName="WORD" topologicalAddress="%MW707">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG708_ARM" typeName="WORD" topologicalAddress="%MW708"/>
		<variables name="UPAK_REG709_ARM" typeName="WORD" topologicalAddress="%MW709">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG710_ARM" typeName="WORD" topologicalAddress="%MW710">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG711_ARM" typeName="WORD" topologicalAddress="%MW711">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG712_ARM" typeName="WORD" topologicalAddress="%MW712">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG713_ARM" typeName="WORD" topologicalAddress="%MW713">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG714_ARM" typeName="WORD" topologicalAddress="%MW714">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG715_ARM" typeName="WORD" topologicalAddress="%MW715">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG716_ARM" typeName="WORD" topologicalAddress="%MW716">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG717_ARM" typeName="WORD" topologicalAddress="%MW717">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG718_ARM" typeName="WORD" topologicalAddress="%MW718">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG719_ARM" typeName="WORD" topologicalAddress="%MW719">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG720_ARM" typeName="WORD" topologicalAddress="%MW720">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG721_ARM" typeName="WORD" topologicalAddress="%MW721">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG722_ARM" typeName="WORD" topologicalAddress="%MW722">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG723_ARM" typeName="WORD" topologicalAddress="%MW723">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG724_ARM" typeName="WORD" topologicalAddress="%MW724">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG725_ARM" typeName="WORD" topologicalAddress="%MW725">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG726_ARM" typeName="WORD" topologicalAddress="%MW726">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG727_ARM" typeName="WORD" topologicalAddress="%MW727">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG728_ARM" typeName="WORD" topologicalAddress="%MW728">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG729_ARM" typeName="WORD" topologicalAddress="%MW729">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG730_ARM" typeName="WORD" topologicalAddress="%MW730">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG731_ARM" typeName="WORD" topologicalAddress="%MW731">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG732_ARM" typeName="WORD" topologicalAddress="%MW732">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG733_ARM" typeName="WORD" topologicalAddress="%MW733"/>
		<variables name="UPAK_REG734_ARM" typeName="WORD" topologicalAddress="%MW734">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG735_ARM" typeName="WORD" topologicalAddress="%MW735">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG736_ARM" typeName="WORD" topologicalAddress="%MW736">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG737_ARM" typeName="WORD" topologicalAddress="%MW737">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG738_ARM" typeName="WORD" topologicalAddress="%MW738">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG739_ARM" typeName="WORD" topologicalAddress="%MW739">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG740_ARM" typeName="WORD" topologicalAddress="%MW740">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG741_ARM" typeName="WORD" topologicalAddress="%MW741">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG742_ARM" typeName="WORD" topologicalAddress="%MW742">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG743_ARM" typeName="WORD" topologicalAddress="%MW743">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG744_ARM" typeName="WORD" topologicalAddress="%MW744">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG745_ARM" typeName="WORD" topologicalAddress="%MW745">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG746_ARM" typeName="WORD" topologicalAddress="%MW746">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG747_ARM" typeName="WORD" topologicalAddress="%MW747">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG748_ARM" typeName="WORD" topologicalAddress="%MW748">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG749_ARM" typeName="WORD" topologicalAddress="%MW749">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG750_ARM" typeName="WORD" topologicalAddress="%MW750">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG751_ARM" typeName="WORD" topologicalAddress="%MW751">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG752_ARM" typeName="WORD" topologicalAddress="%MW752">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG753_ARM" typeName="WORD" topologicalAddress="%MW753">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG754_ARM" typeName="WORD" topologicalAddress="%MW754"/>
		<variables name="UPAK_REG755_ARM" typeName="WORD" topologicalAddress="%MW755">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG756_ARM" typeName="WORD" topologicalAddress="%MW756">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG757_ARM" typeName="WORD" topologicalAddress="%MW757"/>
		<variables name="UPAK_REG758_ARM" typeName="WORD" topologicalAddress="%MW758">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG759_ARM" typeName="WORD" topologicalAddress="%MW759">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG760_ARM" typeName="WORD" topologicalAddress="%MW760">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG761_ARM" typeName="WORD" topologicalAddress="%MW761"/>
		<variables name="UPAK_REG762_ARM" typeName="WORD" topologicalAddress="%MW762">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG763_ARM" typeName="WORD" topologicalAddress="%MW763">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG764_ARM" typeName="WORD" topologicalAddress="%MW764">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG765_ARM" typeName="WORD" topologicalAddress="%MW765">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG766_ARM" typeName="WORD" topologicalAddress="%MW766">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG767_ARM" typeName="WORD" topologicalAddress="%MW767">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG768_ARM" typeName="WORD" topologicalAddress="%MW768">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG769_ARM" typeName="WORD" topologicalAddress="%MW769">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG770_ARM" typeName="WORD" topologicalAddress="%MW770">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG771_ARM" typeName="WORD" topologicalAddress="%MW771">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG772_ARM" typeName="WORD" topologicalAddress="%MW772">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG773_ARM" typeName="WORD" topologicalAddress="%MW773">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG774_ARM" typeName="WORD" topologicalAddress="%MW774">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG775_ARM" typeName="WORD" topologicalAddress="%MW775">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG776_ARM" typeName="WORD" topologicalAddress="%MW776">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG777_ARM" typeName="WORD" topologicalAddress="%MW777">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG778_ARM" typeName="WORD" topologicalAddress="%MW778">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG779_ARM" typeName="WORD" topologicalAddress="%MW779">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG780_ARM" typeName="WORD" topologicalAddress="%MW780">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG781_ARM" typeName="WORD" topologicalAddress="%MW781">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG782_ARM" typeName="WORD" topologicalAddress="%MW782">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG783_ARM" typeName="WORD" topologicalAddress="%MW783">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG784_ARM" typeName="WORD" topologicalAddress="%MW784">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG785_ARM" typeName="WORD" topologicalAddress="%MW785">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG786_ARM" typeName="WORD" topologicalAddress="%MW786">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG787_ARM" typeName="WORD" topologicalAddress="%MW787">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG788_ARM" typeName="WORD" topologicalAddress="%MW788">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG801_ARM" typeName="WORD" topologicalAddress="%MW801">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG802_ARM" typeName="WORD" topologicalAddress="%MW802">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG803_ARM" typeName="WORD" topologicalAddress="%MW803">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG804_ARM" typeName="WORD" topologicalAddress="%MW804">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG805_ARM" typeName="WORD" topologicalAddress="%MW805">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG806_ARM" typeName="WORD" topologicalAddress="%MW806">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG807_ARM" typeName="WORD" topologicalAddress="%MW807">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG808_ARM" typeName="WORD" topologicalAddress="%MW808">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG809_ARM" typeName="WORD" topologicalAddress="%MW809">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG810_ARM" typeName="WORD" topologicalAddress="%MW810">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG811_ARM" typeName="WORD" topologicalAddress="%MW811">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG812_ARM" typeName="WORD" topologicalAddress="%MW812">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG813_ARM" typeName="WORD" topologicalAddress="%MW813">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG814_ARM" typeName="WORD" topologicalAddress="%MW814">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG815_ARM" typeName="WORD" topologicalAddress="%MW815">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG816_ARM" typeName="WORD" topologicalAddress="%MW816"/>
		<variables name="UPAK_REG817_ARM" typeName="WORD" topologicalAddress="%MW817">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG818_ARM" typeName="WORD" topologicalAddress="%MW818">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG819_ARM" typeName="WORD" topologicalAddress="%MW819">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG820_ARM" typeName="WORD" topologicalAddress="%MW820">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG821_ARM" typeName="WORD" topologicalAddress="%MW821">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG822_ARM" typeName="WORD" topologicalAddress="%MW822">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG823_ARM" typeName="WORD" topologicalAddress="%MW823">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG824_ARM" typeName="WORD" topologicalAddress="%MW824">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG825_ARM" typeName="WORD" topologicalAddress="%MW825">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG826_ARM" typeName="WORD" topologicalAddress="%MW826">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG827_ARM" typeName="WORD" topologicalAddress="%MW827">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG831_ARM" typeName="WORD" topologicalAddress="%MW831">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG832_ARM" typeName="WORD" topologicalAddress="%MW832">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG833_ARM" typeName="WORD" topologicalAddress="%MW833">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG834_ARM" typeName="WORD" topologicalAddress="%MW834">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG835_ARM" typeName="WORD" topologicalAddress="%MW835">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG836_ARM" typeName="WORD" topologicalAddress="%MW836">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG843_ARM" typeName="WORD" topologicalAddress="%MW843">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG844_ARM" typeName="WORD" topologicalAddress="%MW844">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG845_ARM" typeName="WORD" topologicalAddress="%MW845">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG846_ARM" typeName="WORD" topologicalAddress="%MW846">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG851_ARM" typeName="WORD" topologicalAddress="%MW851">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG852_ARM" typeName="WORD" topologicalAddress="%MW852">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG853_ARM" typeName="WORD" topologicalAddress="%MW853">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG854_ARM" typeName="WORD" topologicalAddress="%MW854">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG855_ARM" typeName="WORD" topologicalAddress="%MW855">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG856_ARM" typeName="WORD" topologicalAddress="%MW856">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG857_ARM" typeName="WORD" topologicalAddress="%MW857">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG858_ARM" typeName="WORD" topologicalAddress="%MW858">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG859_ARM" typeName="WORD" topologicalAddress="%MW859">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG860_ARM" typeName="WORD" topologicalAddress="%MW860">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG861_ARM" typeName="WORD" topologicalAddress="%MW861">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG862_ARM" typeName="WORD" topologicalAddress="%MW862">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG863_ARM" typeName="WORD" topologicalAddress="%MW863">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG864_ARM" typeName="WORD" topologicalAddress="%MW864">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG865_ARM" typeName="WORD" topologicalAddress="%MW865">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG866_ARM" typeName="WORD" topologicalAddress="%MW866">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG867_ARM" typeName="WORD" topologicalAddress="%MW867"/>
		<variables name="UPAK_REG868_ARM" typeName="WORD" topologicalAddress="%MW868"/>
		<variables name="UPAK_REG869_ARM" typeName="WORD" topologicalAddress="%MW869"/>
		<variables name="UPAK_REG870_ARM" typeName="WORD" topologicalAddress="%MW870"/>
		<variables name="UPAK_REG871_ARM" typeName="WORD" topologicalAddress="%MW871"/>
		<variables name="UPAK_REG872_ARM" typeName="WORD" topologicalAddress="%MW872"/>
		<variables name="UPAK_REG873_ARM" typeName="WORD" topologicalAddress="%MW873"/>
		<variables name="UPAK_REG874_ARM" typeName="WORD" topologicalAddress="%MW874">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG875_ARM" typeName="WORD" topologicalAddress="%MW875">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG876_ARM" typeName="WORD" topologicalAddress="%MW876">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG877_ARM" typeName="WORD" topologicalAddress="%MW877">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG878_ARM" typeName="WORD" topologicalAddress="%MW878">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG879_ARM" typeName="WORD" topologicalAddress="%MW879">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG880_ARM" typeName="WORD" topologicalAddress="%MW880">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG881_ARM" typeName="WORD" topologicalAddress="%MW881">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG882_ARM" typeName="WORD" topologicalAddress="%MW882">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG883_ARM" typeName="WORD" topologicalAddress="%MW883">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG884_ARM" typeName="WORD" topologicalAddress="%MW884">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG885_ARM" typeName="WORD" topologicalAddress="%MW885">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG886_ARM" typeName="WORD" topologicalAddress="%MW886">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG887_ARM" typeName="WORD" topologicalAddress="%MW887">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG888_ARM" typeName="WORD" topologicalAddress="%MW888">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG889_ARM" typeName="WORD" topologicalAddress="%MW889">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG890_ARM" typeName="WORD" topologicalAddress="%MW890">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG891_ARM" typeName="WORD" topologicalAddress="%MW891">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG892_ARM" typeName="WORD" topologicalAddress="%MW892">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG893_ARM" typeName="WORD" topologicalAddress="%MW893">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG894_ARM" typeName="WORD" topologicalAddress="%MW894">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG895_ARM" typeName="WORD" topologicalAddress="%MW895">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG896_ARM" typeName="WORD" topologicalAddress="%MW896">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG897_ARM" typeName="WORD" topologicalAddress="%MW897">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG898_ARM" typeName="WORD" topologicalAddress="%MW898">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG899_ARM" typeName="WORD" topologicalAddress="%MW899">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG900_ARM" typeName="WORD" topologicalAddress="%MW900">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG901_ARM" typeName="WORD" topologicalAddress="%MW901">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG902_ARM" typeName="WORD" topologicalAddress="%MW902">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG903_ARM" typeName="WORD" topologicalAddress="%MW903">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG904_ARM" typeName="WORD" topologicalAddress="%MW904">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG905_ARM" typeName="WORD" topologicalAddress="%MW905">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG906_ARM" typeName="WORD" topologicalAddress="%MW906">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG907_ARM" typeName="WORD" topologicalAddress="%MW907">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG908_ARM" typeName="WORD" topologicalAddress="%MW908">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG909_ARM" typeName="WORD" topologicalAddress="%MW909">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG910_ARM" typeName="WORD" topologicalAddress="%MW910">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG911_ARM" typeName="WORD" topologicalAddress="%MW911">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG912_ARM" typeName="WORD" topologicalAddress="%MW912">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG913_ARM" typeName="WORD" topologicalAddress="%MW913">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG914_ARM" typeName="WORD" topologicalAddress="%MW914">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG915_ARM" typeName="WORD" topologicalAddress="%MW915">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG916_ARM" typeName="WORD" topologicalAddress="%MW916">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG917_ARM" typeName="WORD" topologicalAddress="%MW917">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG918_ARM" typeName="WORD" topologicalAddress="%MW918">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG919_ARM" typeName="WORD" topologicalAddress="%MW919">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG920_ARM" typeName="WORD" topologicalAddress="%MW920">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG921_ARM" typeName="WORD" topologicalAddress="%MW921">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG922_ARM" typeName="WORD" topologicalAddress="%MW922">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG923_ARM" typeName="WORD" topologicalAddress="%MW923">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG924_ARM" typeName="WORD" topologicalAddress="%MW924">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG925_ARM" typeName="WORD" topologicalAddress="%MW925">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG926_ARM" typeName="WORD" topologicalAddress="%MW926">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG927_ARM" typeName="WORD" topologicalAddress="%MW927">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG928_ARM" typeName="WORD" topologicalAddress="%MW928">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG929_ARM" typeName="WORD" topologicalAddress="%MW929">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG930_ARM" typeName="WORD" topologicalAddress="%MW930">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG931_ARM" typeName="WORD" topologicalAddress="%MW931">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG932_ARM" typeName="WORD" topologicalAddress="%MW932">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG933_ARM" typeName="WORD" topologicalAddress="%MW933">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG934_ARM" typeName="WORD" topologicalAddress="%MW934">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG935_ARM" typeName="WORD" topologicalAddress="%MW935">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG936_ARM" typeName="WORD" topologicalAddress="%MW936">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG937_ARM" typeName="WORD" topologicalAddress="%MW937">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG938_ARM" typeName="WORD" topologicalAddress="%MW938">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG939_ARM" typeName="WORD" topologicalAddress="%MW939">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG940_ARM" typeName="WORD" topologicalAddress="%MW940">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG941_ARM" typeName="WORD" topologicalAddress="%MW941">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG942_ARM" typeName="WORD" topologicalAddress="%MW942">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG943_ARM" typeName="WORD" topologicalAddress="%MW943">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG944_ARM" typeName="WORD" topologicalAddress="%MW944">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG945_ARM" typeName="WORD" topologicalAddress="%MW945">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG946_ARM" typeName="WORD" topologicalAddress="%MW946">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG947_ARM" typeName="WORD" topologicalAddress="%MW947">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG948_ARM" typeName="WORD" topologicalAddress="%MW948">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG949_ARM" typeName="WORD" topologicalAddress="%MW949">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG950_ARM" typeName="WORD" topologicalAddress="%MW950">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG951_ARM" typeName="WORD" topologicalAddress="%MW951">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG952_ARM" typeName="WORD" topologicalAddress="%MW952">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG953_ARM" typeName="WORD" topologicalAddress="%MW953">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG954_ARM" typeName="WORD" topologicalAddress="%MW954">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG955_ARM" typeName="WORD" topologicalAddress="%MW955">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG956_ARM" typeName="WORD" topologicalAddress="%MW956">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG957_ARM" typeName="WORD" topologicalAddress="%MW957">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG958_ARM" typeName="WORD" topologicalAddress="%MW958">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG959_ARM" typeName="WORD" topologicalAddress="%MW959">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG960_ARM" typeName="WORD" topologicalAddress="%MW960">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG961_ARM" typeName="WORD" topologicalAddress="%MW961">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG962_ARM" typeName="WORD" topologicalAddress="%MW962">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG963_ARM" typeName="WORD" topologicalAddress="%MW963">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG964_ARM" typeName="WORD" topologicalAddress="%MW964">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG965_ARM" typeName="WORD" topologicalAddress="%MW965">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG966_ARM" typeName="WORD" topologicalAddress="%MW966">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG967_ARM" typeName="WORD" topologicalAddress="%MW967">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG968_ARM" typeName="WORD" topologicalAddress="%MW968">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG969_ARM" typeName="WORD" topologicalAddress="%MW969">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG970_ARM" typeName="WORD" topologicalAddress="%MW970">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG971_ARM" typeName="WORD" topologicalAddress="%MW971">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG972_ARM" typeName="WORD" topologicalAddress="%MW972">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG973_ARM" typeName="WORD" topologicalAddress="%MW973">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG974_ARM" typeName="WORD" topologicalAddress="%MW974">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG975_ARM" typeName="WORD" topologicalAddress="%MW975">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG976_ARM" typeName="WORD" topologicalAddress="%MW976">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG977_ARM" typeName="WORD" topologicalAddress="%MW977">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG978_ARM" typeName="WORD" topologicalAddress="%MW978">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG979_ARM" typeName="WORD" topologicalAddress="%MW979">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG980_ARM" typeName="WORD" topologicalAddress="%MW980">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG981_ARM" typeName="WORD" topologicalAddress="%MW981">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG982_ARM" typeName="WORD" topologicalAddress="%MW982">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG983_ARM" typeName="WORD" topologicalAddress="%MW983">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG1001_ARM" typeName="WORD" topologicalAddress="%MW1001">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG1002_ARM" typeName="WORD" topologicalAddress="%MW1002">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG1003_ARM" typeName="WORD" topologicalAddress="%MW1003">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG1004_ARM" typeName="WORD" topologicalAddress="%MW1004">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG1005_ARM" typeName="WORD" topologicalAddress="%MW1005">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG1006_ARM" typeName="WORD" topologicalAddress="%MW1006">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG1007_ARM" typeName="WORD" topologicalAddress="%MW1007">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG1008_ARM" typeName="WORD" topologicalAddress="%MW1008">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG1009_ARM" typeName="WORD" topologicalAddress="%MW1009">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG1010_ARM" typeName="WORD" topologicalAddress="%MW1010">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG1011_ARM" typeName="WORD" topologicalAddress="%MW1011">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG1026_ARM" typeName="WORD" topologicalAddress="%MW1026">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG1027_ARM" typeName="WORD" topologicalAddress="%MW1027">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG1028_ARM" typeName="WORD" topologicalAddress="%MW1028">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG1029_ARM" typeName="WORD" topologicalAddress="%MW1029">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG1030_ARM" typeName="WORD" topologicalAddress="%MW1030">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG1050_ARM" typeName="WORD" topologicalAddress="%MW1050">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="UPAK_REG1054_ARM" typeName="WORD" topologicalAddress="%MW1054">
			<attribute name="TimeStampSource" value="0"/>
		</variables>
		<variables name="VKL_IMIT" typeName="BOOL"/>
'''

In [11]:
s2 = '''
<variables name="A_1_3MAK_M_IM" typeName="BOOL"/>
		<variables name="A_1_3MAK_P_IM" typeName="BOOL"/>
'''

In [16]:
s22 = s2.split('<variables name=')
s22

['\n',
 '"A_1_3MAK_M_IM" typeName="BOOL"/>\n\t\t',
 '"A_1_3MAK_P_IM" typeName="BOOL"/>\n']

In [18]:
s22[1].split('"')

['', 'A_1_3MAK_M_IM', ' typeName=', 'BOOL', '/>\n\t\t']

In [14]:
s3 = s1.split('<variables name=')

In [15]:
s3 = s3[1:]



In [19]:
ls = [x.split('"')[1] for x in s3]


In [20]:
ls

['A_1_3MAK_KN',
 'A_1_3MAK_M_IM',
 'A_1_3MAK_P_IM',
 'A_1_3MAK_UST_IM',
 'A_1_3MK_K',
 'A_1_3MKStr',
 'A_1_3MKSTR_ARM_DSP',
 'A_1_3MU_ARM_DSP',
 'A_1_3MU_U',
 'A_1_3MU_U_OBR',
 'A_1_3MU_U_PR',
 'A_1_3MU_UK',
 'A_1_3NPS_U',
 'A_1_3PK_K',
 'A_1_3PU_ARM_DSP',
 'A_1_3PU_U',
 'A_1_3PU_U_OBR',
 'A_1_3PU_U_PR',
 'A_1_3PU_UK',
 'A_1_3VK_ARM_DSP',
 'A_1_3VK_U',
 'A_1_3VK_U_OBR',
 'A_1_3VK_U_PR',
 'A_1_3VK_UK',
 'A_1_3VZ',
 'A_1_3VZREZ_IM',
 'A_1_3ZSTR',
 'A_1_3ZSTR_ARM_DSP',
 'A_1_15IR_ARM_DSP',
 'A_1_15KS',
 'A_1_15P_IND',
 'A_1_15P_K',
 'A_1_15RI',
 'A_1_15Z',
 'A_1AIR_ARM_DSP',
 'A_1AKS',
 'A_1AP_IND',
 'A_1AP_K',
 'A_1ARI',
 'A_1AZ',
 'A_1F_ACTIV_IM',
 'A_1F_JK',
 'A_1F_JL',
 'A_1F_KL',
 'A_1F_NEISPR_IM',
 'A_1FA_K',
 'A_1FB_K',
 'A_1FC_K',
 'A_1FK_K',
 'A_1GIR_ARM_DSP',
 'A_1GKS',
 'A_1GP_IND',
 'A_1GP_K',
 'A_1GRI',
 'A_1GZ',
 'A_1IR_ARM_DSP',
 'A_1P1MS_U',
 'A_1P1MS_U_OBR',
 'A_1P1MS_U_PR',
 'A_1P1MS_UK',
 'A_1P2MS_U',
 'A_1P2MS_U_OBR',
 'A_1P2MS_U_PR',
 'A_1P2MS_UK',
 'A_1RI',
 'A_1SP_I

In [28]:
with open('var_unity.txt', 'w') as f:
    for x in ls:
        f.writelines(x + '\n')

In [ ]:
# project = Project('test61850_2')
project = Project('Vostochnaya_v3')
project.copy_libfile(['libDevice.xlsx', 'setProject.xlsx'])
# project.create_dbf_from_xls([('boards', 'NAME'), ('ports', 'NAME')])

project.create_eqtype()
project.create_io()
# project.create_equip()
project.create_equip(isCreateTag=True, typeCreateTag='IEC61850_104')

In [ ]:
# project = Project('test61850_2')
project = Project('Vostochnaya_v3')
project.base.base_io.clear_dbf()
project.base.base_equip.clear_dbf()
project.base.base_tag.clear_dbf()

In [ ]:
sheet_name = 'P3F30_line6_61850'
df_dev = pd.read_excel(project.config_libDevice_xls, sheet_name=sheet_name)
df_dev

In [ ]:
project = Project('test61850_1')
# project.copy_libfile(['libDevice.xlsx', 'setProject.xlsx'])
project.create_dbf_from_xls([('boards', 'NAME'), ('ports', 'NAME')])

In [ ]:
df1 = pd.DataFrame([{'x1': 1, 'x2': True}])
df1

In [ ]:
# project = Project('Vostochnaya_v3')
project = Project('tmp1')
df1 = project.base.base_pgdynobj.dataframe()
df1

In [ ]:
# df1['TXT_STR'].iloc[-1] = r'Cell_2_TH6\fMMXU8\Hz\mag\f'
# df1[df1['TXT_STR'] == r'Cell_2_TH6\UV2PTUV2\Str\general']
df1[df1['TXT_STR'] == r'Cell_2_TH_6kV\VO3GGIO99\Ind\stVal']


In [ ]:
df1.loc[10740, 'TXT_STR'] = r'Cell_2_TH_6kV\VO4GGIO100\Ind\stVal'

In [ ]:
project.base.base_pgdynobj.clear_dbf()
project.base.base_pgdynobj.to_dataframe_dbf(df1)

In [ ]:
sr = df1[df1['NAME'].str.contains(r'stVal|general|ctlVal', regex=True)]

In [ ]:
sr

In [ ]:
df1['NAME'].str.extract(r"(\S+$)")

In [ ]:
s1 = sr['NAME'].values[1]
s1

In [ ]:
'\\mag' in r'Cell_2_TH6\\U3ppMMXU5\\PPV\\phsAB\\cVal\\mag\\f'


In [ ]:
project = Project('Vostochnaya_v5')
df1 = project.base.base_tag.dataframe()
df1

In [ ]:
ls = list(df1[df1['TYPE'] == 'DIGITAL']['NAME'].values)
ls = [f'Rand_dig("{nm}");' for nm in ls]
s5 = '\n'.join(ls)
print(s5)

In [ ]:
ls = list(df1[df1['TYPE'] == 'INT']['NAME'].values)
ls = [f'Rand_dig3("{nm}");' for nm in ls]
s5 = '\n'.join(ls)
print(s5)

In [ ]:
sr = df1[df1['ITEM'].str.contains(r'mag')]
ls = []
for i, tag in sr.iterrows():
    if '\\A\\' in tag['ITEM']:
        ls.append(f'Rand_analg("{tag.NAME}", 10, 9);')
    if '\\B\\' in tag['ITEM']:
        ls.append(f'Rand_analg("{tag.NAME}", 10, 9);')
    if '\\C\\' in tag['ITEM']:
        ls.append(f'Rand_analg("{tag.NAME}", 10, 9);')
    if '\\PPV\\' in tag['ITEM']:
        ls.append(f'Rand_analg_int("{tag.NAME}", 1000, 5500);')
    if '\\PhV\\' in tag['ITEM']:
        ls.append(f'Rand_analg_int("{tag.NAME}", 1000, 5500);')
    if '\\Hz\\' in tag['ITEM']:
        ls.append(f'Rand_analg("{tag.NAME}", 20, 40);')
s5 = '\n'.join(ls)
print(s5)

In [ ]:
project = Project('Vostochnaya_v3')
df1 = project.base.base_tag.dataframe()
sr = df1[df1['ITEM'].str.contains(r'stVal|general|ctlVal', regex=True)]
ls = []
for tag in sr['NAME'].values:
    ls.append(f'Rand_dig("{tag}");')
s5 = '\n'.join(ls)
print(s5)

In [ ]:
s1 = 'Cell_4_LN6_P3F30_line6'
match = re.search(r'(\S+)_(\S+)_(\S+)_(\S+)_(\S+)', s1)
x1 = match.group(1)
x2 = match.group(2)
x3 = match.group(3)

In [ ]:
sr

In [ ]:
project = Project('Vostochnaya_v3')
df1 = project.base.base_tag.dataframe()
# sr = df1[~((df1['NAME'].str.contains(r'\\t')) | (df1['NAME'].str.contains(r'\\q'))| (df1['NAME'].str.contains(r'\\T')))]
sr = df1[df1['ITEM'].str.contains(r'MMXU')]
sr = sr[~((sr['ITEM'].str.contains(r'\\t')) | (sr['ITEM'].str.contains(r'\\q')) | (sr['ITEM'].str.contains(r'\\T')))]

ds = {
    'TH6': 'tn',
    'LN6': 'ln',
    'IN6': 'in',
    'CB6': 'cb',
    'T1': 't1',
    'T2': 't2',
    'CB35': 'cb',
    'RPN1': 'rpn1',
    'RPN2': 'rpn2',
}

ls4 = []
cols = []
for i, tag in sr.iterrows():
    # match = re.search(r'(\S+)_(\S+)_(\S+?)\\', tag)
    match = re.search(r'(\S+)_(\S+)_(\S+)_(\S+)_(\S+)', tag['EQUIP'])
    match1 = re.search(r'(\S+)\\(\S+$)', tag['ITEM'])
    x1 = match.group(1)
    x2 = match.group(2)
    x3 = match.group(3)
    x4 = match1.group(2)

    shkaf_name = 'c'
    if x1 == 'RZA': shkaf_name = 'r'

    val = ''
    if 'phsAB\\' in tag['ITEM']: val = 'uab'
    if 'phsBC\\' in tag['ITEM']: val = 'ubc'
    if 'phsCA\\' in tag['ITEM']: val = 'uca'
    if 'phsA\\' in tag['ITEM']: val = 'ia'
    if 'phsB\\' in tag['ITEM']: val = 'ib'
    if 'phsC\\' in tag['ITEM']: val = 'ic'
    if 'A\\neut' in tag['ITEM']: val = '3io'
    if 'PhV\\neut' in tag['ITEM']: val = '3uo'
    if 'Hz' in tag['ITEM']: val = 'hz'

    col_base = f'{shkaf_name}{x2}_{ds[x3]}_{val}_'
    col = f'{shkaf_name}{x2}_{ds[x3]}_{val}_{x4}'
    cols.append(col)

    s1 = rf'MapValueSet(sm1, "{col}",  RealToStr({tag.NAME}, 5, 2, "."));'
    # s11 = rf'MapValueSet(sm1, "{col_base}t", TagGetValue("{tag.NAME}.Field", 5));'
    s11 = rf'MapValueSet(sm1, "{col_base}t", getTimeTag("{tag.NAME}.Field"));'
    s12 = rf'MapValueSet(sm1, "{col_base}q", getQualityTag("{tag.NAME}.Field"));'

    ls4.append(s1)
    ls4.append(s11)
    ls4.append(s12)
s5 = '\n'.join(ls4)
print(s5)

In [ ]:
match = re.search(r'(\S+)_(\S+)_(\S+?)\\', s1)

In [ ]:
print(match.group(1))
print(match.group(2))
print(match.group(3))

In [ ]:
project = Project('test61850_2')
df1 = project.base.base_tag.dataframe()
sr = df1[df1['NAME'].str.contains(r'MMXU')]

ds = {
    'TH6': 'tn',
    'LN6': 'ln',
    'IN6': 'in',
    'CB6': 'cb',
    'T1': 't1',
    'T2': 't2',
    'CB35': 'cb',
}

ls = []
ls1 = []
ls2 = []
ls3 = []
cols = []
for tag in sr['NAME'].values:
    # match = re.search(r'(\S+)_(\S+)_(\S+?)\\', tag)
    match = re.search(r'(\S+)_(\S+)_(\S+?)\\', tag)
    match1 = re.search(r'(\S+)\\(\S+$)', tag)
    x1 = match.group(1)
    x2 = match.group(2)
    x3 = match.group(3)
    x4 = match1.group(2)

    shkaf_name = 'c'
    if x1 == 'RZA': shkaf_name = 'r'

    val = ''
    if 'phsAB\\' in tag: val = 'uab'
    if 'phsBC\\' in tag: val = 'ubc'
    if 'phsCA\\' in tag: val = 'uca'
    if 'phsA\\' in tag: val = 'ia'
    if 'phsB\\' in tag: val = 'ib'
    if 'phsC\\' in tag: val = 'ic'
    if 'A\\neut' in tag: val = '3io'
    if 'PhV\\neut' in tag: val = '3uo'
    if 'Hz' in tag: val = 'hz'

    col = f'{shkaf_name}{x2}_{ds[x3]}_{val}_{x4}'
    cols.append(col)
    s1 = rf'MapValueSet(sm1, "{col}", {tag});'
    s2 = f'SQLAppend(hSQL, MapValueGet(smap1 , "{col}") + ",");'
    if x4 in ['t', 'q']:
        s2 = f'''SQLAppend(hSQL, "'" + MapValueGet(smap1 , "{col}") + "',");'''
    s3 = f'SQLAppend(hSQL, "{col},");'
    ls.append(s1)
    ls1.append(s2)
    ls2.append(s3)

    typ = 'real'
    if x4.lower() == 't': typ = 'datetime'
    if x4 == 'q': typ = 'nvarchar(30)'
    ls3.append(f'{col} {typ},')
ls
r'MapValueSet(sm1, "c3l_ia", Cell_3_LN6_P3F30_line6.I3pMMXU1\A\phsA\cVal\mag\f);'

In [ ]:
s5 = '\n'.join(ls)
print(s5)

In [ ]:
sr = df1[df1['NAME'].str.contains(r'stVal|general|ctlVal', regex=True)]

In [ ]:
ls = []

for tag in sr['NAME'].values:
    # s1 = f'''
    #     subsHandle = TagSubscribe("{tag}");
    #     Sub
    #     scriptionAddCallback(subsHandle, "OnValueChanged");
    # ''' 
    print(f'subsHandle = TagSubscribe("{tag}");')
    print('SubscriptionAddCallback(subsHandle, "OnValueChanged");')
    # ls.append(s1)

# s5 = '\n'.join(ls)
# print(s5)

In [ ]:
project = Project('Vostochnaya_v3')
df1 = project.base.base_tag.dataframe()
# sr = df1[~((df1['NAME'].str.contains(r'\\t')) | (df1['NAME'].str.contains(r'\\q'))| (df1['NAME'].str.contains(r'\\T')))]
sr = df1[df1['NAME'].str.contains(r'MMXU')]
sr = sr[~((sr['NAME'].str.contains(r'\\t')) | (sr['NAME'].str.contains(r'\\q')) | (sr['NAME'].str.contains(r'\\T')))]

ds = {
    'TH6': 'tn',
    'LN6': 'ln',
    'IN6': 'in',
    'CB6': 'cb',
    'T1': 't1',
    'T2': 't2',
    'CB35': 'cb',
}

ls4 = []
cols = []
for tag in sr['NAME'].values:
    # match = re.search(r'(\S+)_(\S+)_(\S+?)\\', tag)
    match = re.search(r'(\S+)_(\S+)_(\S+?)\\', tag)
    match1 = re.search(r'(\S+)\\(\S+$)', tag)
    x1 = match.group(1)
    x2 = match.group(2)
    x3 = match.group(3)
    x4 = match1.group(2)

    shkaf_name = 'c'
    if x1 == 'RZA': shkaf_name = 'r'

    val = ''
    if 'phsAB\\' in tag: val = 'uab'
    if 'phsBC\\' in tag: val = 'ubc'
    if 'phsCA\\' in tag: val = 'uca'
    if 'phsA\\' in tag: val = 'ia'
    if 'phsB\\' in tag: val = 'ib'
    if 'phsC\\' in tag: val = 'ic'
    if 'A\\neut' in tag: val = '3io'
    if 'PhV\\neut' in tag: val = '3uo'
    if 'Hz' in tag: val = 'hz'

    col_base = f'{shkaf_name}{x2}_{ds[x3]}_{val}_'
    col = f'{shkaf_name}{x2}_{ds[x3]}_{val}_{x4}'
    cols.append(col)

    s1 = rf'MapValueSet(sm1, "{col}", {tag});'
    s11 = rf'MapValueSet(sm1, "{col_base}t", TagGetValue("{tag}.Field", 5));'
    s12 = rf'MapValueSet(sm1, "{col_base}q", TagGetValue("{tag}.Field", 3));'

    ls4.append(s1)
    ls4.append(s11)
    ls4.append(s12)


In [ ]:
s5 = '\n'.join(ls4)
print(s5)

In [ ]:
project = Project('test61850_2')
df1 = project.base.base_tag.dataframe()

ds = {
    'TH6': 'tn',
    'LN6': 'ln',
    'IN6': 'in',
    'CB6': 'cb',
    'T1': 't1'
}

sr = df1[df1['NAME'].str.contains(r'mag')]

ls = []
ls1 = []
ls2 = []
ls3 = []
cols = []
for tag in sr['NAME'].values:
    match = re.search(r'(\S+)_(\S+)_(\S+?)\\', tag)
    x1 = match.group(1)
    x2 = match.group(2)
    x3 = match.group(3)

    if x1 == 'Cell' or x1 == 'RZA':
        val = ''
        if 'phsAB\\' in tag: val = 'uab'
        if 'phsBC\\' in tag: val = 'ubc'
        if 'phsCA\\' in tag: val = 'uca'
        if 'phsA\\' in tag: val = 'ia'
        if 'phsB\\' in tag: val = 'ib'
        if 'phsC\\' in tag: val = 'ic'
        if 'A\\neut' in tag: val = '3io'
        if 'PhV\\neut' in tag: val = '3uo'
        if 'Hz' in tag: val = 'f'

        col = f'c{x2}_{ds[x3]}_{val}'
        cols.append(col)
        s1 = rf'MapValueSet(sm1, "{col}", {tag});'
        s2 = f'SQLAppend(hSQL, MapValueGet(smap1 , "{col}") + ",");'
        s3 = f'SQLAppend(hSQL, "{col},");'
        ls.append(s1)
        ls1.append(s2)
        ls2.append(s3)

        ls3.append(f'{col} real,')
        ls3.append(f'{col}_t datetime,')
        ls3.append(f'{col}_q NVARCHAR(20),')
ls
r'MapValueSet(sm1, "c3l_ia", Cell_3_LN6_P3F30_line6.I3pMMXU1\A\phsA\cVal\mag\f);'

In [ ]:
s5 = '\n'.join(ls)
print(s5)

In [ ]:
print('\n'.join(ls3))

In [ ]:
file_path = r'c:\libCitectScada\libDevice.xlsx'
xll = pd.ExcelFile(file_path)
xll.sheet_names
for sheet_name in xll.sheet_names:
    df1 = pd.read_excel(file_path, sheet_name=sheet_name)
    ls = df1.values.tolist()
    ls_new = []
    for ls1 in ls:
        ls_new.append(ls1)
        s1 = ls1[-1]
        match = re.search(r'(час)', s1)
        if match:
            ls2 = [ls1[0]]
            s2 = ls1[1]
            s2 = s2.replace('/t', '/q')
            s2 = s2.replace('/T', '/q')
            ls2.append(s2)
            ls2.append(ls1[2])
            ls2.append(ls1[3])
            s3 = ls1[-1]
            ls2.append(s3.replace('(час)', '(якість)'))
            ls_new.append(ls2)

    ls_new

    df2 = pd.DataFrame(ls_new, columns=df1.columns)

    file_path1 = r'c:\libCitectScada\libDevice1.xlsx'
    with ExcelWriter(file_path1, mode="a" if os.path.exists(file_path1) else "w") as writer:
        df2.to_excel(writer, sheet_name=sheet_name, index=False)



In [ ]:
project = Project('test61850_2')
project.copy_libfile(['libDevice.xlsx', 'setProject.xlsx'])
project.create_dbf_from_xls([('boards', 'NAME'), ('ports', 'NAME')])

project.create_eqtype()
project.create_io()
# project.create_equip()
project.create_equip(isCreateTag=True)

In [ ]:
df_tag = pd.read_excel(project.config_setProject_xls, sheet_name='Tag')
project.create_tag(df_tag)

In [ ]:
project = Project('test61850_2')
project.base.base_io.clear_dbf()
project.base.base_equip.clear_dbf()
project.base.base_tag.clear_dbf()

In [ ]:
project = Project('tmp1')
df_tag = pd.read_excel(project.config_xls, sheet_name='Tag')
# project.create_tag_simple(df_tag)
project.create_tag(df_tag, replace=True)

In [ ]:
project = Project('Proj16850_17112021')
df2 = project.base.base_tag.dataframe()
df2

In [ ]:
ls = [1, 2, 3, 4]
ls.append(5)
ls

In [ ]:
df1 = project.base.base_tag.dataframe()
df1

In [ ]:
values = {'NAME': '454588',
          'TYPE': '',
          'UNIT': '',
          'ADDR': "",
          'RAW_ZERO': '',
          'RAW_FULL': '',
          'ENG_ZERO': '',
          'ENG_FULL': '',
          'ENG_UNITS': '',
          'FORMAT': '',
          'COMMENT': '',
          'EDITCODE': '',
          'LINKED': '',
          'OID': '',
          'REF1': '',
          'REF2': '',
          'DEADBAND': '',
          'CUSTOM': '',
          'TAGGENLINK': '',
          'CLUSTER': '',
          'HISTORIAN': 'TRUE',
          'EQUIP': '',
          'CUSTOM1': '',
          'CUSTOM2': '',
          'CUSTOM3': '',
          'CUSTOM4': '',
          'CUSTOM5': '',
          'CUSTOM6': '',
          'CUSTOM7': '',
          'CUSTOM8': '',
          'ITEM': ''}

# pd.concat([df1, pd.DataFrame([values])], ignore_index=True)


In [ ]:
df3 = pd.DataFrame()
df3

In [ ]:
pd.concat([df3, pd.DataFrame([values])], ignore_index=True)

In [ ]:
df1.loc[df1['NAME'] == 'd5', :] = list(values.values())

In [ ]:
list(values.values())

In [ ]:
df1

In [ ]:
pd.DataFrame([values])

In [ ]:
df1 = project.base.base_tag.dataframe()
df1

In [ ]:
df1[df1['NAME'] == 'd5'].loc[0, 'ADDR'] = '112'

In [ ]:
df1['ADDR'].where(~(df1['NAME'] == 'd5'), other='1', inplace=True)

In [ ]:
df1.iloc[-1]

In [ ]:
df1

In [ ]:
df1[df1['NAME'] == 'd5']

In [ ]:
[col for col in df1.columns]

In [ ]:
project.base.base_tag.to_dbf(df1.to_dict('records'))

In [ ]:
project.base.base_tag

In [ ]:
df1.to_dict('records')

In [ ]:
project.create_equip()

In [ ]:
class ItemIEC61850:
    def __init__(self, name_device, node, attribs, type_attr, fs):
        self.node = node
        self.attribs = attribs
        self.attribs_str = '\\'.join(self.attribs)
        self.type_attr = type_attr
        self.fs = fs

        self.name_tag = fr"{self.node}\{self.attribs_str}"
        self.addr = ''



In [ ]:
project = Project('tmp1')
plc = 'PLC_C264_TU'

df_dev = pd.read_excel('ListDevice.xlsx', sheet_name='LD')

df_dev['io'] = plc
# df_dev['io'] = df_dev.apply(lambda x: f"{x['name']}_{x['number']}", axis=1)

protocol = 'IEC61850N'
df_io = pd.DataFrame([{'name': plc, 'addr': '', 'protocol': protocol, 'port': 'PORT1_BOARD1'}])
if protocol:
    df_io['addr'] = f"[RUN]:conf61850_{plc}"

project.create_io(df_io)
project.create_equip(df_dev)


In [ ]:
project = Project('tmp1')
plc = 'PLC_C264_TU'

df_dev = pd.read_excel('ListDevice.xlsx', sheet_name='P3F30_61850')
df_dev

In [ ]:
xll = pd.ExcelFile('ListDevice.xlsx')

In [ ]:
project = Project('tmp1')
df = project.base.base_tag.dataframe()
df

In [ ]:
project = Project('Proj16850_02122021')
df = project.base.base_tag.dataframe()
df


In [ ]:
project = Project('Proj16850_02122021')
df = project.base.base_tag.dataframe()

df_eqps = project.base.base_equip.dataframe()
eqps = df_eqps.TYPE.values
# eqps = ['RZA5_C264', 'RZA3_C264', 'Cell_3_P3F30', 'Cell_2_P3U30', 'Cell_11_P3F30', 'Cell_5_P3F30']

for item in eqps:
    eqp = df_eqps[df_eqps['TYPE'] == item]['NAME'].values[0]

    d1 = (df[df['EQUIP'] == eqp]['ITEM'].apply(lambda x: x.split('\\')[0])).values.tolist()
    sr = df[df['EQUIP'] == eqp]['ITEM'].apply(lambda x: x.split('\\')[1:])
    d2 = sr.apply(lambda x: '/'.join(x)).values.tolist()
    d3 = df[df['EQUIP'] == eqp]['TYPE'].values.tolist()
    d4 = df[df['EQUIP'] == eqp]['ADDR'].apply(lambda x: x.split('$')[1] if x else '').values.tolist()
    d5 = df[df['EQUIP'] == eqp]['COMMENT'].values.tolist()

    data = {'node': d1, 'attribs': d2, 'type': d3, 'fc': d4, 'comment': d5}
    df1 = pd.DataFrame(data)

    with ExcelWriter(project.config_xls, mode="a" if os.path.exists(project.config_xls) else "w") as writer:
        df1.to_excel(writer, sheet_name=item + '_61850', index=False)

In [ ]:
df_eqps = project.base.base_equip.dataframe()
df_eqps[df_eqps['TYPE'] == 'C264reserve']['NAME'].values[0]

In [ ]:
df2 = project.base.base_equip.dataframe()
df2

In [ ]:
df3 = project.base.base_equip.dataframe()
df3

In [ ]:
project.create_io('Cell', 5)

In [ ]:
df1 = pd.DataFrame([{'name': 'C264', 'addr': '[RUN]:conf61850_C264', 'protocol': 'IEC61850N', 'port': 'PORT1_BOARD1'}])
df1

In [ ]:
df = pd.read_excel('ListDevice.xlsx')
df['io'] = 'C264'
# df['io'] = df.apply(lambda x: f"{x['name']}_{x['number']}", axis=1)

In [ ]:
df['io'] = 'C264'
df

In [ ]:
project = Project('ExampleSA')
project.base.base_tag.dataframe()

In [ ]:
df1 = pd.DataFrame(columns=['node', 'attribs', 'type', 'fc', 'comment'])
df1


In [ ]:
df = project.base.base_io.dataframe()

In [ ]:
import re

In [ ]:
s = 'Cell_2_P3U30_45'
re.findall(r'.*_(\S+)', s)[-1]

In [ ]:
x = BaseProject(os.path.join(r"c:\ProgramData\AVEVA\Citect SCADA 2018 R2\User", 'tmp1', 'equip.DBF'))
x.dataframe()

In [ ]:
dd = dbf.Table(os.path.join(r"c:\ProgramData\AVEVA\Citect SCADA 2018 R2\User", 'tmp1', 'variable.DBF'),
               codepage='cp1251')
dd.open(mode=dbf.READ_ONLY)
x = dict(zip(dd.field_names, ['' for _ in range(len(dd.field_names))]))
dd.close()
x

In [ ]:
st = ''
for t in dd.field_names:
    st += f"self.{t} = ''\n"
print(st)

In [ ]:
class BaseProject:

    def __init__(self, file_name):
        self.filename = filename


In [ ]:
x = Clss()

In [ ]:
x.filename = 'jfjdklj'
x.__dict__

In [ ]:
dd.open(mode=dbf.READ_ONLY)

In [ ]:
dd.close()

In [ ]:
project.show_io()

In [ ]:
project = Project('tmp1')
project.write_io()

In [ ]:
# Удаление строки
rec = table[-1]
dbf.delete(rec)
table.pack()

In [ ]:
name_file = r"units.DBF"
filename = os.path.join(path_projects, name_projects, name_file)

db_in_dbf = DBF(filename, encoding='cp1251')
df = pd.DataFrame(db_in_dbf)
df


In [ ]:
name_file = r"variable.DBF"
filename = os.path.join(path_projects, name_projects, name_file)

db_in_dbf = DBF(filename, encoding='cp1251')
df = pd.DataFrame(db_in_dbf)
df

In [ ]:
name_file = r"equip.DBF"
filename = os.path.join(path_projects, name_projects, name_file)

db_in_dbf = DBF(filename, encoding='cp1251')
df = pd.DataFrame(db_in_dbf)
df

In [ ]:
path_projects = r"c:\ProgramData\AVEVA\Citect SCADA 2018 R2\User"
name_projects = "tmp1"


# name_file = r"equip.DBF"
# name_file = r"pgdynobj.DBF"
# filename = os.path.join(path_projects, name_projects, name_file)


class TypeDevice:
    P3U30 = 'P3U30'
    P3F30 = 'P3F30'
    P3T32 = 'P3T32'


class ProjectBases:

    def __init__(self, path_project):
        self.path_project = path_project
        self.base_equip = BaseProject(os.path.join(self.path_project, 'equip.DBF'))
        self.base_io = BaseProject(os.path.join(self.path_project, 'units.DBF'))
        self.base_tag = BaseProject(os.path.join(self.path_project, 'variable.DBF'))


class BaseProject:

    def __init__(self, file_name):
        self.filename = file_name

    def get_dbf(self):
        db = None
        with dbf.Table(self.filename, codepage='cp1251') as tb:
            db = tb.open(mode=dbf.READ_ONLY)
        return db

    def to_dbf(self, params_list):
        with dbf.Table(self.filename, codepage='cp1251') as tb:
            tb.open(mode=dbf.READ_WRITE)
            for params in params_list:
                tb.append(params)

    def clear_dbf(self):
        with dbf.Table(self.filename, codepage='cp1251') as tb:
            tb.open(mode=dbf.READ_WRITE)
            tb.zap()

    def dataframe(self):
        return pd.DataFrame(DBF(self.filename, encoding='cp1251'))


class P3U30:
    def __init__(self, name_project, name_cluster='Cluster1', name_server='IOServer1'):
        self.name = 'P3U30'
        self.name_projects = name_project
        self.path_project = os.path.join(self.path_projects, self.name_projects)

        self.name_files = {'Оборудование': 'equip.DBF', 'Теги': 'variable.DBF', 'IO': 'units.DBF'}
        self.type_devices = TypeDevice()
        self.type_device_current = ''
        self.cluster = name_cluster
        self.server = name_server
        self.base = ProjectBases(self.path_project)


class Project:
    def __init__(self, name_project, name_cluster='Cluster1', name_server='IOServer1'):
        self.path_projects = r"c:\ProgramData\AVEVA\Citect SCADA 2018 R2\User"
        self.name_projects = name_project
        self.path_project = os.path.join(self.path_projects, self.name_projects)

        self.config_xls = self.get_path_file('ListDevice.xlsx')

        self.name_files = {'Оборудование': 'equip.DBF', 'Теги': 'variable.DBF', 'IO': 'units.DBF'}
        self.type_devices = TypeDevice()
        self.type_device_current = ''
        self.cluster = name_cluster
        self.server = name_server
        self.base = ProjectBases(self.path_project)

        self.df_io = None
        try:
            self.df_io = pd.read_excel(self.config_xls, sheet_name='IO')
        except:
            pass

        self.df_equips = None
        try:
            self.df_equips = pd.read_excel(self.config_xls, sheet_name='LD')
        except:
            pass

    def create_io(self):

        if self.df_io is not None:

            self.df_io['port'] = 'PORT1_BOARD1'
            addr = ''

            params_list = []

            df_tmp = self.base.base_io.dataframe()
            try:
                numb = int(df_tmp.iloc[-1]['NUMBER'])
            except:
                numb = 1

            df_io_exist = self.base.base_io.dataframe()

            for i in range(len(self.df_io)):
                io = self.df_io.iloc[i]

                if io['name'] not in df_io_exist['NAME'].values:
                    numb += 1

                    if io['protocol'] == 'IEC61850N':
                        addr = f"[RUN]:conf61850_{io['name']}"
                        self.create_config61850(f"conf61850_{io['name']}.xml")

                    values = {'SERVER': self.server,
                              'NAME': io['name'],
                              'NUMBER': f'{numb}',
                              'ADDR': addr,
                              'PROTOCOL': io['protocol'],
                              'PORT': io['port'],
                              'MODE': 'Primary',
                              'LOGWRITE': '',
                              'LOGREAD': '',
                              'CACHE': '',
                              'CACHETIME': '',
                              'COMMENT': '',
                              'PROTOCOLID': '',
                              'LINKED': '',
                              'EXTERNDB': '',
                              'DRIVERID': '',
                              'CONNSTRING': '',
                              'REFRESH': '',
                              'TAGPREFIX': '',
                              'LASTUPDATE': '',
                              'REMOTE': '',
                              'REMOTEWRIT': '',
                              'TIME': '',
                              'PERIOD': '',
                              'PHONE': '',
                              'CALLERID': '',
                              'ONCONNECT': '',
                              'ONHANGUP': '',
                              'ONBROWSE': '',
                              'LIVEUPDATE': '',
                              'LASTVARMOD': '',
                              'MEMORY': 'FALSE',
                              'PERSIST': '',
                              'TAGGEN': '',
                              'TAGGENTEMP': '',
                              'PRIORITY': '1',
                              'BGPOLL': '',
                              'BGPOLLRATE': '',
                              'PERSFREQ': '',
                              'PERSPATH': '',
                              'MINUPDATE': '',
                              'STALEPERIO': '',
                              'READONLY': '',
                              'EXCLUSIVE': ''}
                    params_list.append(values)

            self.base.base_io.to_dbf(params_list)

    def create_equip(self):
        if self.df_equips is not None:

            self.df_equips['io'] = self.df_io.iloc[0]['name']  #TODO

            params_list = []

            df_equip_exist = self.base.base_equip.dataframe()

            for i in range(len(self.df_equips)):
                equip = self.df_equips.iloc[i]

                values = {'NAME': f"{equip['name']}_{equip['number']}_{equip['spec']}_{equip['device']}",
                          'CLUSTER': self.cluster,
                          'TYPE': equip['device'],
                          'AREA': '',
                          'LOCATION': '',
                          'COMMENT': equip['comment'],
                          'CUSTOM1': f"{equip['io']}\\",
                          'CUSTOM2': '',
                          'CUSTOM3': '',
                          'CUSTOM4': '',
                          'CUSTOM5': '',
                          'CUSTOM6': '',
                          'CUSTOM7': '',
                          'CUSTOM8': '',
                          'IODEVICE': equip['io'],
                          'PAGE': '',
                          'HELP': '',
                          'DEFSTATE': '',
                          'SCHEDULED': '',
                          'TAGPREFIX': f"{equip['name']}_{equip['number']}/",
                          'TAGGENLINK': '',
                          'LINKED': '',
                          'EDITCODE': '',
                          'PARAM': '',
                          'COMPOSITE': '',
                          'REFERENCE': '',
                          'DEVSCHED': '',
                          'SCHEDID': '',
                          'ALIAS': '',
                          'CONTENT': '',
                          'HIDDEN': ''}

                if len(df_equip_exist) == 0:
                    params_list.append(values)
                else:
                    if values['NAME'] not in df_equip_exist['NAME'].values:
                        params_list.append(values)

                sheet_name = equip['device'] + '_61850'
                df_dev = pd.read_excel(self.config_xls, sheet_name=sheet_name)
                df_dev['io'] = equip['io']
                df_dev['LD'] = f"{equip['name']}_{equip['number']}"
                df_dev['equip'] = values['NAME']
                self.create_tag(df_dev, type='IEC61850')

            self.base.base_equip.to_dbf(params_list)

    def create_tag_61850(self, df, replace=None):
        df_tags_exist = self.base.base_tag.dataframe()

        names = []
        if len(df_tags_exist) != 0:
            names = list(df_tags_exist['NAME'].values)

        df_tags = pd.DataFrame()
        if replace and len(df_tags_exist) != 0:
            df_tags = df_tags_exist

        if replace:
            self.base.base_tag.clear_dbf()

        for i in range(len(df)):
            dev = df.iloc[i]

            attribs = dev['attribs'].split('/')
            name_tag_attribs = '\\'.join(attribs)
            values = {'NAME': f"{dev['LD']}\\{dev['node']}\\{name_tag_attribs}",
                      'TYPE': dev['type'],
                      'UNIT': dev['io'],
                      'ADDR': f"{dev['LD']}/{dev['node']}${dev['fc']}\\{'$'.join(attribs)}",
                      'RAW_ZERO': '',
                      'RAW_FULL': '',
                      'ENG_ZERO': '',
                      'ENG_FULL': '',
                      'ENG_UNITS': '',
                      'FORMAT': '',
                      'COMMENT': dev['comment'],
                      'EDITCODE': '',
                      'LINKED': '',
                      'OID': '',
                      'REF1': '',
                      'REF2': '',
                      'DEADBAND': '',
                      'CUSTOM': '',
                      'TAGGENLINK': '',
                      'CLUSTER': self.cluster,
                      'HISTORIAN': 'TRUE',
                      'EQUIP': dev['equip'],
                      'CUSTOM1': '',
                      'CUSTOM2': '',
                      'CUSTOM3': '',
                      'CUSTOM4': '',
                      'CUSTOM5': '',
                      'CUSTOM6': '',
                      'CUSTOM7': '',
                      'CUSTOM8': '',
                      'ITEM': f"{dev['node']}\\{name_tag_attribs}"}

            if values['NAME'] not in names:
                df_tags = pd.concat([df_tags, pd.DataFrame([values])], ignore_index=True)
                names.append(values['NAME'])
            else:
                if replace:
                    df_tags.loc[df_tags['NAME'] == values['NAME'], :] = list(values.values())

        # if len(df_tags_exist)==0:
        #     # params_list.append(values)
        #     df_tags = pd.DataFrame([values])
        # else:
        #     if values['NAME'] not in df_tags_exist['NAME'].values:
        #         # params_list.append(values)
        #         df_tags = pd.concat([df_tags, pd.DataFrame([values])], ignore_index=True)
        #     else:
        #         if replace:
        #             df_tags.loc[df_tags['NAME']==values['NAME'], :] = list(values.values())
        #             # dev_repl = df_tags_exist[df_tags_exist['NAME'] == dev['name']]
        #             # for col in df_tags_exist.columns:
        #             #     if col in dev.index.values:
        #             #         dev_repl[col] = dev[col]
        #             #     else:
        #             #         dev_repl[col] = ''

        self.base.base_tag.to_dbf(df_tags.to_dict('records'))

    def create_tag_simple(self, df, replace=None):
        params_list = []

        df_tags_exist = self.base.base_tag.dataframe()

        names = []
        if len(df_tags_exist) != 0:
            names = list(df_tags_exist['NAME'].values)

        df_tags = pd.DataFrame()
        if replace and len(df_tags_exist) != 0:
            df_tags = df_tags_exist

        if replace:
            # df_old = self.base.base_tag.dataframe()
            self.base.base_tag.clear_dbf()
            # params_list = df_old.to_dict('records')

        for i in range(len(df)):
            dev = df.iloc[i]
            values = {'NAME': dev['name'],
                      'TYPE': dev['type'],
                      'UNIT': dev['io'],
                      'ADDR': "",
                      'RAW_ZERO': '',
                      'RAW_FULL': '',
                      'ENG_ZERO': '',
                      'ENG_FULL': '',
                      'ENG_UNITS': '',
                      'FORMAT': '',
                      'COMMENT': dev['comment'],
                      'EDITCODE': '',
                      'LINKED': '',
                      'OID': '',
                      'REF1': '',
                      'REF2': '',
                      'DEADBAND': '',
                      'CUSTOM': '',
                      'TAGGENLINK': '',
                      'CLUSTER': self.cluster,
                      'HISTORIAN': 'TRUE',
                      'EQUIP': '',
                      'CUSTOM1': '',
                      'CUSTOM2': '',
                      'CUSTOM3': '',
                      'CUSTOM4': '',
                      'CUSTOM5': '',
                      'CUSTOM6': '',
                      'CUSTOM7': '',
                      'CUSTOM8': '',
                      'ITEM': ''}

            if values['NAME'] not in names:
                df_tags = pd.concat([df_tags, pd.DataFrame([values])], ignore_index=True)
                names.append(values['NAME'])
            else:
                if replace:
                    df_tags.loc[df_tags['NAME'] == values['NAME'], :] = list(values.values())

            # if len(df_tags_exist) == 0:
            #     # params_list.append(values)
            #     df_tags = pd.DataFrame([values])
            # else:
            #     if values['NAME'] not in df_tags_exist['NAME'].values:
            #         # params_list.append(values)
            #         df_tags = pd.concat([df_tags_exist, pd.DataFrame([values])], ignore_index=True)
            #     else:
            #         if replace:
            #             df_tags.loc[df_tags['NAME'] == values['NAME'], :] = list(values.values())
            #             # dev_repl = df_tags_exist[df_tags_exist['NAME'] == dev['name']]
            #             # for col in df_tags_exist.columns:
            #             #     if values[col] != '':
            #             #         # df_tags_exist[df_tags_exist['NAME'] == dev['name']].loc[:,col] = dev[col]
            #             #         df_tags_exist[col].where(~(df_tags_exist['NAME'] == dev['name']), other=values[col], inplace=True)
            #             #
            #             #     else:
            #             #         # df_tags_exist[df_tags_exist['NAME'] == dev['name']].loc[:,col] = ''
            #             #         df_tags_exist[col].where(~(df_tags_exist['NAME'] == dev['name']), other='', inplace=True)

        # self.base.base_tag.to_dbf(params_list)
        self.base.base_tag.to_dbf(df_tags.to_dict('records'))

    def create_tag_values(self, df, type='simple'):
        '''

        :param df:
        :param type: 'simple', 'IEC61850'
        :return:
        '''
        dev = df

        values = {}

        if type == 'simple':
            values = {'NAME': dev['name'],
                      'TYPE': dev['type'],
                      'UNIT': dev['io'],
                      'ADDR': "",
                      'RAW_ZERO': '',
                      'RAW_FULL': '',
                      'ENG_ZERO': '',
                      'ENG_FULL': '',
                      'ENG_UNITS': '',
                      'FORMAT': '',
                      'COMMENT': dev['comment'],
                      'EDITCODE': '',
                      'LINKED': '',
                      'OID': '',
                      'REF1': '',
                      'REF2': '',
                      'DEADBAND': '',
                      'CUSTOM': '',
                      'TAGGENLINK': '',
                      'CLUSTER': self.cluster,
                      'HISTORIAN': 'TRUE',
                      'EQUIP': '',
                      'CUSTOM1': '',
                      'CUSTOM2': '',
                      'CUSTOM3': '',
                      'CUSTOM4': '',
                      'CUSTOM5': '',
                      'CUSTOM6': '',
                      'CUSTOM7': '',
                      'CUSTOM8': '',
                      'ITEM': ''}

        if type == 'IEC61850':
            attribs = dev['attribs'].split('/')
            name_tag_attribs = '\\'.join(attribs)
            values = {'NAME': f"{dev['LD']}\\{dev['node']}\\{name_tag_attribs}",
                      'TYPE': dev['type'],
                      'UNIT': dev['io'],
                      'ADDR': f"{dev['LD']}/{dev['node']}${dev['fc']}\\{'$'.join(attribs)}",
                      'RAW_ZERO': '',
                      'RAW_FULL': '',
                      'ENG_ZERO': '',
                      'ENG_FULL': '',
                      'ENG_UNITS': '',
                      'FORMAT': '',
                      'COMMENT': dev['comment'],
                      'EDITCODE': '',
                      'LINKED': '',
                      'OID': '',
                      'REF1': '',
                      'REF2': '',
                      'DEADBAND': '',
                      'CUSTOM': '',
                      'TAGGENLINK': '',
                      'CLUSTER': self.cluster,
                      'HISTORIAN': 'TRUE',
                      'EQUIP': dev['equip'],
                      'CUSTOM1': '',
                      'CUSTOM2': '',
                      'CUSTOM3': '',
                      'CUSTOM4': '',
                      'CUSTOM5': '',
                      'CUSTOM6': '',
                      'CUSTOM7': '',
                      'CUSTOM8': '',
                      'ITEM': f"{dev['node']}\\{name_tag_attribs}"}

        return values

    def create_tag(self, df, type='simple', replace=None):
        '''

        :param df:
        :param type: 'simple', 'IEC61850'
        :param replace:
        :return:
        '''
        df_tags_exist = self.base.base_tag.dataframe()

        names = []
        if len(df_tags_exist) != 0:
            names = list(df_tags_exist['NAME'].values)

        df_tags = pd.DataFrame()
        if replace and len(df_tags_exist) != 0:
            df_tags = df_tags_exist

        if replace:
            self.base.base_tag.clear_dbf()

        for i in range(len(df)):
            dev = df.iloc[i]
            values = self.create_tag_values(dev, type=type)

            if values['NAME'] not in names:
                df_tags = pd.concat([df_tags, pd.DataFrame([values])], ignore_index=True)
                names.append(values['NAME'])
            else:
                if replace:
                    df_tags.loc[df_tags['NAME'] == values['NAME'], :] = list(values.values())

        self.base.base_tag.to_dbf(df_tags.to_dict('records'))

    def create_config61850(self, name_file):
        xml_doc = f"""
            <?xml version="1.0" encoding="utf-8"?>
            <ScadaDevice xmlns="http://www.schneider-electric.com/SCADA/Drivers/IEC61850/DeviceConfig/v1/">
               <IPConnection>
                  <IP>127.0.0.1</IP>
               </IPConnection>
               <IED>TEMPLATE</IED>
               <LogicalDevice Name="Relay">
                  <BRCB>brcbEV1</BRCB>
                  <BRCB>brcbMX01</BRCB>
                  <URCB>urcbMX01</URCB>
               </LogicalDevice>
            </ScadaDevice>
            """

        soup = BeautifulSoup(xml_doc, 'lxml-xml')
        xml = soup.prettify()
        with open(self.get_path_file(name_file), "w") as file:
            file.write(xml)

    def get_path_file(self, name_file):
        return os.path.join(self.path_project, name_file)



In [137]:
dd = dbf.Table(os.path.join(r"c:\ProgramData\AVEVA\Citect SCADA 2018 R2\User", 'tmp1', 'variable.DBF'),
               codepage='cp1251')
dd.open(mode=dbf.READ_ONLY)
x = dict(zip(dd.field_names, ['' for _ in range(len(dd.field_names))]))
dd.close()
x

{'NAME': '',
 'TYPE': '',
 'UNIT': '',
 'ADDR': '',
 'RAW_ZERO': '',
 'RAW_FULL': '',
 'ENG_ZERO': '',
 'ENG_FULL': '',
 'ENG_UNITS': '',
 'FORMAT': '',
 'COMMENT': '',
 'EDITCODE': '',
 'LINKED': '',
 'OID': '',
 'REF1': '',
 'REF2': '',
 'DEADBAND': '',
 'CUSTOM': '',
 'TAGGENLINK': '',
 'CLUSTER': '',
 'HISTORIAN': '',
 'EQUIP': '',
 'CUSTOM1': '',
 'CUSTOM2': '',
 'CUSTOM3': '',
 'CUSTOM4': '',
 'CUSTOM5': '',
 'CUSTOM6': '',
 'CUSTOM7': '',
 'CUSTOM8': '',
 'ITEM': ''}

In [27]:
st = ''
for t in dd.field_names:
    st += f"self.{t} = ''\n"
print(st)

self.NAME = ''
self.CLUSTER = ''
self.TYPE = ''
self.AREA = ''
self.LOCATION = ''
self.COMMENT = ''
self.CUSTOM1 = ''
self.CUSTOM2 = ''
self.CUSTOM3 = ''
self.CUSTOM4 = ''
self.CUSTOM5 = ''
self.CUSTOM6 = ''
self.CUSTOM7 = ''
self.CUSTOM8 = ''
self.IODEVICE = ''
self.PAGE = ''
self.HELP = ''
self.DEFSTATE = ''
self.SCHEDULED = ''
self.TAGPREFIX = ''
self.TAGGENLINK = ''
self.LINKED = ''
self.EDITCODE = ''
self.PARAM = ''
self.COMPOSITE = ''
self.REFERENCE = ''
self.DEVSCHED = ''
self.SCHEDID = ''
self.ALIAS = ''
self.CONTENT = ''
self.HIDDEN = ''



In [31]:
class BaseProject:

    def __init__(self, file_name):
        self.filename = filename


In [32]:
x = Clss()

In [34]:
x.filename = 'jfjdklj'
x.__dict__

{'NAME': '',
 'CLUSTER': '',
 'TYPE': '',
 'AREA': '',
 'LOCATION': '',
 'COMMENT': '',
 'CUSTOM1': '',
 'CUSTOM2': '',
 'CUSTOM3': '',
 'CUSTOM4': '',
 'CUSTOM5': '',
 'CUSTOM6': '',
 'CUSTOM7': '',
 'CUSTOM8': '',
 'IODEVICE': '',
 'PAGE': '',
 'HELP': '',
 'DEFSTATE': '',
 'SCHEDULED': '',
 'TAGPREFIX': '',
 'TAGGENLINK': '',
 'LINKED': '',
 'EDITCODE': '',
 'PARAM': '',
 'COMPOSITE': '',
 'REFERENCE': '',
 'DEVSCHED': '',
 'SCHEDID': '',
 'ALIAS': '',
 'CONTENT': '',
 'HIDDEN': '',
 'filename': 'jfjdklj'}

In [77]:
dd.open(mode=dbf.READ_ONLY)

dbf.Table('c:\\ProgramData\\AVEVA\\Citect SCADA 2018 R2\\User\\tmp1\\units.DBF', status=<DbfStatus.READ_ONLY: 1>)

In [12]:
dd.close()

In [68]:
project.show_io()

,SERVER,NAME,NUMBER,ADDR,PROTOCOL,PORT,MODE,LOGWRITE,LOGREAD,CACHE,...,TAGGENTEMP,PRIORITY,BGPOLL,BGPOLLRATE,PERSFREQ,PERSPATH,MINUPDATE,STALEPERIO,READONLY,EXCLUSIVE
0,IOServer1,Internal,1,,OFSOPC,,Primary,,,,...,,1,,,,,,,,
1,IOServer1,Cicode,2,,CICODE,,Primary,,,,...,,1,,,,,,,,


In [57]:
project = Project('tmp1')
project.write_io()

ValueError: invalid literal for int() with base 10: '        '

Traceback (most recent call last):
  File "_pydevd_bundle\pydevd_cython_win32_39_64.pyx", line 1034, in _pydevd_bundle.pydevd_cython_win32_39_64.PyDBFrame.trace_dispatch
  File "C:\Program Files\JetBrains\PyCharm 213.4631.9\plugins\python\helpers-pro\jupyter_debug\pydev_jupyter_plugin.py", line 144, in cmd_step_over
    if _is_inside_jupyter_cell(frame, pydb):
  File "C:\Program Files\JetBrains\PyCharm 213.4631.9\plugins\python\helpers-pro\jupyter_debug\pydev_jupyter_plugin.py", line 209, in _is_inside_jupyter_cell
    if is_cell_filename(filename):
  File "C:\Program Files\JetBrains\PyCharm 213.4631.9\plugins\python\helpers-pro\jupyter_debug\pydev_jupyter_plugin.py", line 220, in is_cell_filename
    ipython_shell = get_ipython()
NameError: name 'get_ipython' is not defined


In [20]:
# Удаление строки
rec = table[-1]
dbf.delete(rec)
table.pack()

In [11]:
name_file = r"units.DBF"
filename = os.path.join(path_projects, name_projects, name_file)

db_in_dbf = DBF(filename, encoding='cp1251')
df = pd.DataFrame(db_in_dbf)
df


,SERVER,NAME,NUMBER,ADDR,PROTOCOL,PORT,MODE,LOGWRITE,LOGREAD,CACHE,...,TAGGENTEMP,PRIORITY,BGPOLL,BGPOLLRATE,PERSFREQ,PERSPATH,MINUPDATE,STALEPERIO,READONLY,EXCLUSIVE
0,IOServer1,Internal,1,,OFSOPC,,Primary,,,,...,,1,,,,,,,,
1,IOServer1,Cicode,2,,CICODE,,Primary,,,,...,,1,,,,,,,,


In [10]:
name_file = r"variable.DBF"
filename = os.path.join(path_projects, name_projects, name_file)

db_in_dbf = DBF(filename, encoding='cp1251')
df = pd.DataFrame(db_in_dbf)
df

,NAME,TYPE,UNIT,ADDR,RAW_ZERO,RAW_FULL,ENG_ZERO,ENG_FULL,ENG_UNITS,FORMAT,...,EQUIP,CUSTOM1,CUSTOM2,CUSTOM3,CUSTOM4,CUSTOM5,CUSTOM6,CUSTOM7,CUSTOM8,ITEM
0,Cell_2\VO3GGIO99\Ind\t,STRING,Cell_2,Relay/VO3GGIO99$ST$Ind$t,,,,,,,...,Cell_2_P3U30,,,,,,,,,VO3GGIO99\Ind\t
1,Cell_2\VO3GGIO99\Ind\stVal,STRING,Cell_2,Relay/VO3GGIO99$ST$Ind$stVal,,,,,,,...,Cell_2_P3U30,,,,,,,,,VO3GGIO99\Ind\stVal
2,Cell_2\VO4GGIO100\Ind\stVal,STRING,Cell_2,Relay/VO4GGIO100$ST$Ind$stVal,,,,,,,...,Cell_2_P3U30,,,,,,,,,VO4GGIO100\Ind\stVal
3,Cell_2\VO4GGIO100\Ind\t,STRING,Cell_2,Relay/VO4GGIO100$ST$Ind$t,,,,,,,...,Cell_2_P3U30,,,,,,,,,VO4GGIO100\Ind\t
4,Cell_2\Obj2CSWI2\Pos\stVal,STRING,Cell_2,Relay/Obj2CSWI2$ST$Pos$stVal,,,,,,,...,Cell_2_P3U30,,,,,,,,,Obj2CSWI2\Pos\stVal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698,Cell_20\U3ppMMXU5\PPV\phsBC\t,STRING,Cell_20,Relay/U3ppMMXU5$MX$PPV$phsBC$t,,,,,,,...,Cell_20_P3U30,,,,,,,,,U3ppMMXU5\PPV\phsBC\t
699,Cell_20\U3ppMMXU5\PPV\phsCA\cVal,STRING,Cell_20,Relay/U3ppMMXU5$MX$PPV$phsCA$cVal,,,,,,,...,Cell_20_P3U30,,,,,,,,,U3ppMMXU5\PPV\phsCA\cVal
700,Cell_20\U3ppMMXU5\PPV\phsCA\t,STRING,Cell_20,Relay/U3ppMMXU5$MX$PPV$phsCA$t,,,,,,,...,Cell_20_P3U30,,,,,,,,,U3ppMMXU5\PPV\phsCA\t
701,Cell_20\UovMMXU1\PhV\neut\cVal,STRING,Cell_20,Relay/UovMMXU1$MX$PhV$neut$cVal,,,,,,,...,Cell_20_P3U30,,,,,,,,,UovMMXU1\PhV\neut\cVal


In [7]:
name_file = r"equip.DBF"
filename = os.path.join(path_projects, name_projects, name_file)

db_in_dbf = DBF(filename, encoding='cp1251')
df = pd.DataFrame(db_in_dbf)
df

,NAME,CLUSTER,TYPE,AREA,LOCATION,COMMENT,CUSTOM1,CUSTOM2,CUSTOM3,CUSTOM4,...,LINKED,EDITCODE,PARAM,COMPOSITE,REFERENCE,DEVSCHED,SCHEDID,ALIAS,CONTENT,HIDDEN
0,Cell_2_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_2\,,,,...,,,,,,,,,,
1,Cell_3_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_3\,,,,...,,,,,,,,,,
2,Cell_4_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_4\,,,,...,,,,,,,,,,
3,Cell_5_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_5\,,,,...,,,,,,,,,,
4,Cell_6_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_6\,,,,...,,,,,,,,,,
5,Cell_7_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_7\,,,,...,,,,,,,,,,
6,Cell_8_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_8\,,,,...,,,,,,,,,,
7,Cell_9_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_9\,,,,...,,,,,,,,,,
8,Cell_10_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_10\,,,,...,,,,,,,,,,
9,Cell_11_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_11\,,,,...,,,,,,,,,,


In [ ]:
path_projects = r"c:\ProgramData\AVEVA\Citect SCADA 2018 R2\User"
name_projects = "tmp1"


# name_file = r"equip.DBF"
# name_file = r"pgdynobj.DBF"
# filename = os.path.join(path_projects, name_projects, name_file)


class TypeDevice:
    P3U30 = 'P3U30'
    P3F30 = 'P3F30'
    P3T32 = 'P3T32'


class ProjectBases:

    def __init__(self, path_project):
        self.path_project = path_project
        self.base_equip = BaseProject(os.path.join(self.path_project, 'equip.DBF'))
        self.base_io = BaseProject(os.path.join(self.path_project, 'units.DBF'))
        self.base_tag = BaseProject(os.path.join(self.path_project, 'variable.DBF'))


class BaseProject:

    def __init__(self, file_name):
        self.filename = file_name

    def get_dbf(self):
        db = None
        with dbf.Table(self.filename, codepage='cp1251') as tb:
            db = tb.open(mode=dbf.READ_ONLY)
        return db

    def to_dbf(self, params_list):
        with dbf.Table(self.filename, codepage='cp1251') as tb:
            tb.open(mode=dbf.READ_WRITE)
            for params in params_list:
                tb.append(params)

    def clear_dbf(self):
        with dbf.Table(self.filename, codepage='cp1251') as tb:
            tb.open(mode=dbf.READ_WRITE)
            tb.zap()

    def dataframe(self):
        return pd.DataFrame(DBF(self.filename, encoding='cp1251'))


class P3U30:
    def __init__(self, name_project, name_cluster='Cluster1', name_server='IOServer1'):
        self.name = 'P3U30'
        self.name_projects = name_project
        self.path_project = os.path.join(self.path_projects, self.name_projects)

        self.name_files = {'Оборудование': 'equip.DBF', 'Теги': 'variable.DBF', 'IO': 'units.DBF'}
        self.type_devices = TypeDevice()
        self.type_device_current = ''
        self.cluster = name_cluster
        self.server = name_server
        self.base = ProjectBases(self.path_project)


class Project:
    def __init__(self, name_project, name_cluster='Cluster1', name_server='IOServer1'):
        self.path_projects = r"c:\ProgramData\AVEVA\Citect SCADA 2018 R2\User"
        self.name_projects = name_project
        self.path_project = os.path.join(self.path_projects, self.name_projects)

        self.config_xls = self.get_path_file('ListDevice.xlsx')

        self.name_files = {'Оборудование': 'equip.DBF', 'Теги': 'variable.DBF', 'IO': 'units.DBF'}
        self.type_devices = TypeDevice()
        self.type_device_current = ''
        self.cluster = name_cluster
        self.server = name_server
        self.base = ProjectBases(self.path_project)

        self.df_io = None
        try:
            self.df_io = pd.read_excel(self.config_xls, sheet_name='IO')
        except:
            pass

        self.df_equips = None
        try:
            self.df_equips = pd.read_excel(self.config_xls, sheet_name='LD')
        except:
            pass

    def create_io(self):

        if self.df_io is not None:

            self.df_io['port'] = 'PORT1_BOARD1'
            addr = ''

            params_list = []

            df_tmp = self.base.base_io.dataframe()
            try:
                numb = int(df_tmp.iloc[-1]['NUMBER'])
            except:
                numb = 1

            df_io_exist = self.base.base_io.dataframe()

            for i in range(len(self.df_io)):
                io = self.df_io.iloc[i]

                if io['name'] not in df_io_exist['NAME'].values:
                    numb += 1

                    if io['protocol'] == 'IEC61850N':
                        addr = f"[RUN]:conf61850_{io['name']}"
                        self.create_config61850(f"conf61850_{io['name']}.xml")

                    values = {'SERVER': self.server,
                              'NAME': io['name'],
                              'NUMBER': f'{numb}',
                              'ADDR': addr,
                              'PROTOCOL': io['protocol'],
                              'PORT': io['port'],
                              'MODE': 'Primary',
                              'LOGWRITE': '',
                              'LOGREAD': '',
                              'CACHE': '',
                              'CACHETIME': '',
                              'COMMENT': '',
                              'PROTOCOLID': '',
                              'LINKED': '',
                              'EXTERNDB': '',
                              'DRIVERID': '',
                              'CONNSTRING': '',
                              'REFRESH': '',
                              'TAGPREFIX': '',
                              'LASTUPDATE': '',
                              'REMOTE': '',
                              'REMOTEWRIT': '',
                              'TIME': '',
                              'PERIOD': '',
                              'PHONE': '',
                              'CALLERID': '',
                              'ONCONNECT': '',
                              'ONHANGUP': '',
                              'ONBROWSE': '',
                              'LIVEUPDATE': '',
                              'LASTVARMOD': '',
                              'MEMORY': 'FALSE',
                              'PERSIST': '',
                              'TAGGEN': '',
                              'TAGGENTEMP': '',
                              'PRIORITY': '1',
                              'BGPOLL': '',
                              'BGPOLLRATE': '',
                              'PERSFREQ': '',
                              'PERSPATH': '',
                              'MINUPDATE': '',
                              'STALEPERIO': '',
                              'READONLY': '',
                              'EXCLUSIVE': ''}
                    params_list.append(values)

            self.base.base_io.to_dbf(params_list)

    def create_equip(self):
        if self.df_equips is not None:

            self.df_equips['io'] = self.df_io.iloc[0]['name']  #TODO

            params_list = []

            df_equip_exist = self.base.base_equip.dataframe()

            for i in range(len(self.df_equips)):
                equip = self.df_equips.iloc[i]

                values = {'NAME': f"{equip['name']}_{equip['number']}_{equip['spec']}_{equip['device']}",
                          'CLUSTER': self.cluster,
                          'TYPE': equip['device'],
                          'AREA': '',
                          'LOCATION': '',
                          'COMMENT': equip['comment'],
                          'CUSTOM1': f"{equip['io']}\\",
                          'CUSTOM2': '',
                          'CUSTOM3': '',
                          'CUSTOM4': '',
                          'CUSTOM5': '',
                          'CUSTOM6': '',
                          'CUSTOM7': '',
                          'CUSTOM8': '',
                          'IODEVICE': equip['io'],
                          'PAGE': '',
                          'HELP': '',
                          'DEFSTATE': '',
                          'SCHEDULED': '',
                          'TAGPREFIX': f"{equip['name']}_{equip['number']}/",
                          'TAGGENLINK': '',
                          'LINKED': '',
                          'EDITCODE': '',
                          'PARAM': '',
                          'COMPOSITE': '',
                          'REFERENCE': '',
                          'DEVSCHED': '',
                          'SCHEDID': '',
                          'ALIAS': '',
                          'CONTENT': '',
                          'HIDDEN': ''}

                if len(df_equip_exist) == 0:
                    params_list.append(values)
                else:
                    if values['NAME'] not in df_equip_exist['NAME'].values:
                        params_list.append(values)

                sheet_name = equip['device'] + '_61850'
                df_dev = pd.read_excel(self.config_xls, sheet_name=sheet_name)
                df_dev['io'] = equip['io']
                df_dev['LD'] = f"{equip['name']}_{equip['number']}"
                df_dev['equip'] = values['NAME']
                self.create_tag(df_dev, type='IEC61850')

            self.base.base_equip.to_dbf(params_list)

    def create_tag_61850(self, df, replace=None):
        df_tags_exist = self.base.base_tag.dataframe()

        names = []
        if len(df_tags_exist) != 0:
            names = list(df_tags_exist['NAME'].values)

        df_tags = pd.DataFrame()
        if replace and len(df_tags_exist) != 0:
            df_tags = df_tags_exist

        if replace:
            self.base.base_tag.clear_dbf()

        for i in range(len(df)):
            dev = df.iloc[i]

            attribs = dev['attribs'].split('/')
            name_tag_attribs = '\\'.join(attribs)
            values = {'NAME': f"{dev['LD']}\\{dev['node']}\\{name_tag_attribs}",
                      'TYPE': dev['type'],
                      'UNIT': dev['io'],
                      'ADDR': f"{dev['LD']}/{dev['node']}${dev['fc']}\\{'$'.join(attribs)}",
                      'RAW_ZERO': '',
                      'RAW_FULL': '',
                      'ENG_ZERO': '',
                      'ENG_FULL': '',
                      'ENG_UNITS': '',
                      'FORMAT': '',
                      'COMMENT': dev['comment'],
                      'EDITCODE': '',
                      'LINKED': '',
                      'OID': '',
                      'REF1': '',
                      'REF2': '',
                      'DEADBAND': '',
                      'CUSTOM': '',
                      'TAGGENLINK': '',
                      'CLUSTER': self.cluster,
                      'HISTORIAN': 'TRUE',
                      'EQUIP': dev['equip'],
                      'CUSTOM1': '',
                      'CUSTOM2': '',
                      'CUSTOM3': '',
                      'CUSTOM4': '',
                      'CUSTOM5': '',
                      'CUSTOM6': '',
                      'CUSTOM7': '',
                      'CUSTOM8': '',
                      'ITEM': f"{dev['node']}\\{name_tag_attribs}"}

            if values['NAME'] not in names:
                df_tags = pd.concat([df_tags, pd.DataFrame([values])], ignore_index=True)
                names.append(values['NAME'])
            else:
                if replace:
                    df_tags.loc[df_tags['NAME'] == values['NAME'], :] = list(values.values())

        # if len(df_tags_exist)==0:
        #     # params_list.append(values)
        #     df_tags = pd.DataFrame([values])
        # else:
        #     if values['NAME'] not in df_tags_exist['NAME'].values:
        #         # params_list.append(values)
        #         df_tags = pd.concat([df_tags, pd.DataFrame([values])], ignore_index=True)
        #     else:
        #         if replace:
        #             df_tags.loc[df_tags['NAME']==values['NAME'], :] = list(values.values())
        #             # dev_repl = df_tags_exist[df_tags_exist['NAME'] == dev['name']]
        #             # for col in df_tags_exist.columns:
        #             #     if col in dev.index.values:
        #             #         dev_repl[col] = dev[col]
        #             #     else:
        #             #         dev_repl[col] = ''

        self.base.base_tag.to_dbf(df_tags.to_dict('records'))

    def create_tag_simple(self, df, replace=None):
        params_list = []

        df_tags_exist = self.base.base_tag.dataframe()

        names = []
        if len(df_tags_exist) != 0:
            names = list(df_tags_exist['NAME'].values)

        df_tags = pd.DataFrame()
        if replace and len(df_tags_exist) != 0:
            df_tags = df_tags_exist

        if replace:
            # df_old = self.base.base_tag.dataframe()
            self.base.base_tag.clear_dbf()
            # params_list = df_old.to_dict('records')

        for i in range(len(df)):
            dev = df.iloc[i]
            values = {'NAME': dev['name'],
                      'TYPE': dev['type'],
                      'UNIT': dev['io'],
                      'ADDR': "",
                      'RAW_ZERO': '',
                      'RAW_FULL': '',
                      'ENG_ZERO': '',
                      'ENG_FULL': '',
                      'ENG_UNITS': '',
                      'FORMAT': '',
                      'COMMENT': dev['comment'],
                      'EDITCODE': '',
                      'LINKED': '',
                      'OID': '',
                      'REF1': '',
                      'REF2': '',
                      'DEADBAND': '',
                      'CUSTOM': '',
                      'TAGGENLINK': '',
                      'CLUSTER': self.cluster,
                      'HISTORIAN': 'TRUE',
                      'EQUIP': '',
                      'CUSTOM1': '',
                      'CUSTOM2': '',
                      'CUSTOM3': '',
                      'CUSTOM4': '',
                      'CUSTOM5': '',
                      'CUSTOM6': '',
                      'CUSTOM7': '',
                      'CUSTOM8': '',
                      'ITEM': ''}

            if values['NAME'] not in names:
                df_tags = pd.concat([df_tags, pd.DataFrame([values])], ignore_index=True)
                names.append(values['NAME'])
            else:
                if replace:
                    df_tags.loc[df_tags['NAME'] == values['NAME'], :] = list(values.values())

            # if len(df_tags_exist) == 0:
            #     # params_list.append(values)
            #     df_tags = pd.DataFrame([values])
            # else:
            #     if values['NAME'] not in df_tags_exist['NAME'].values:
            #         # params_list.append(values)
            #         df_tags = pd.concat([df_tags_exist, pd.DataFrame([values])], ignore_index=True)
            #     else:
            #         if replace:
            #             df_tags.loc[df_tags['NAME'] == values['NAME'], :] = list(values.values())
            #             # dev_repl = df_tags_exist[df_tags_exist['NAME'] == dev['name']]
            #             # for col in df_tags_exist.columns:
            #             #     if values[col] != '':
            #             #         # df_tags_exist[df_tags_exist['NAME'] == dev['name']].loc[:,col] = dev[col]
            #             #         df_tags_exist[col].where(~(df_tags_exist['NAME'] == dev['name']), other=values[col], inplace=True)
            #             #
            #             #     else:
            #             #         # df_tags_exist[df_tags_exist['NAME'] == dev['name']].loc[:,col] = ''
            #             #         df_tags_exist[col].where(~(df_tags_exist['NAME'] == dev['name']), other='', inplace=True)

        # self.base.base_tag.to_dbf(params_list)
        self.base.base_tag.to_dbf(df_tags.to_dict('records'))

    def create_tag_values(self, df, type='simple'):
        '''

        :param df:
        :param type: 'simple', 'IEC61850'
        :return:
        '''
        dev = df

        values = {}

        if type == 'simple':
            values = {'NAME': dev['name'],
                      'TYPE': dev['type'],
                      'UNIT': dev['io'],
                      'ADDR': "",
                      'RAW_ZERO': '',
                      'RAW_FULL': '',
                      'ENG_ZERO': '',
                      'ENG_FULL': '',
                      'ENG_UNITS': '',
                      'FORMAT': '',
                      'COMMENT': dev['comment'],
                      'EDITCODE': '',
                      'LINKED': '',
                      'OID': '',
                      'REF1': '',
                      'REF2': '',
                      'DEADBAND': '',
                      'CUSTOM': '',
                      'TAGGENLINK': '',
                      'CLUSTER': self.cluster,
                      'HISTORIAN': 'TRUE',
                      'EQUIP': '',
                      'CUSTOM1': '',
                      'CUSTOM2': '',
                      'CUSTOM3': '',
                      'CUSTOM4': '',
                      'CUSTOM5': '',
                      'CUSTOM6': '',
                      'CUSTOM7': '',
                      'CUSTOM8': '',
                      'ITEM': ''}

        if type == 'IEC61850':
            attribs = dev['attribs'].split('/')
            name_tag_attribs = '\\'.join(attribs)
            values = {'NAME': f"{dev['LD']}\\{dev['node']}\\{name_tag_attribs}",
                      'TYPE': dev['type'],
                      'UNIT': dev['io'],
                      'ADDR': f"{dev['LD']}/{dev['node']}${dev['fc']}\\{'$'.join(attribs)}",
                      'RAW_ZERO': '',
                      'RAW_FULL': '',
                      'ENG_ZERO': '',
                      'ENG_FULL': '',
                      'ENG_UNITS': '',
                      'FORMAT': '',
                      'COMMENT': dev['comment'],
                      'EDITCODE': '',
                      'LINKED': '',
                      'OID': '',
                      'REF1': '',
                      'REF2': '',
                      'DEADBAND': '',
                      'CUSTOM': '',
                      'TAGGENLINK': '',
                      'CLUSTER': self.cluster,
                      'HISTORIAN': 'TRUE',
                      'EQUIP': dev['equip'],
                      'CUSTOM1': '',
                      'CUSTOM2': '',
                      'CUSTOM3': '',
                      'CUSTOM4': '',
                      'CUSTOM5': '',
                      'CUSTOM6': '',
                      'CUSTOM7': '',
                      'CUSTOM8': '',
                      'ITEM': f"{dev['node']}\\{name_tag_attribs}"}

        return values

    def create_tag(self, df, type='simple', replace=None):
        '''

        :param df:
        :param type: 'simple', 'IEC61850'
        :param replace:
        :return:
        '''
        df_tags_exist = self.base.base_tag.dataframe()

        names = []
        if len(df_tags_exist) != 0:
            names = list(df_tags_exist['NAME'].values)

        df_tags = pd.DataFrame()
        if replace and len(df_tags_exist) != 0:
            df_tags = df_tags_exist

        if replace:
            self.base.base_tag.clear_dbf()

        for i in range(len(df)):
            dev = df.iloc[i]
            values = self.create_tag_values(dev, type=type)

            if values['NAME'] not in names:
                df_tags = pd.concat([df_tags, pd.DataFrame([values])], ignore_index=True)
                names.append(values['NAME'])
            else:
                if replace:
                    df_tags.loc[df_tags['NAME'] == values['NAME'], :] = list(values.values())

        self.base.base_tag.to_dbf(df_tags.to_dict('records'))

    def create_config61850(self, name_file):
        xml_doc = f"""
            <?xml version="1.0" encoding="utf-8"?>
            <ScadaDevice xmlns="http://www.schneider-electric.com/SCADA/Drivers/IEC61850/DeviceConfig/v1/">
               <IPConnection>
                  <IP>127.0.0.1</IP>
               </IPConnection>
               <IED>TEMPLATE</IED>
               <LogicalDevice Name="Relay">
                  <BRCB>brcbEV1</BRCB>
                  <BRCB>brcbMX01</BRCB>
                  <URCB>urcbMX01</URCB>
               </LogicalDevice>
            </ScadaDevice>
            """

        soup = BeautifulSoup(xml_doc, 'lxml-xml')
        xml = soup.prettify()
        with open(self.get_path_file(name_file), "w") as file:
            file.write(xml)

    def get_path_file(self, name_file):
        return os.path.join(self.path_project, name_file)

